In [1]:
```python
import os
import json
import requests
import time
from datetime import datetime
from typing import Dict, List, Optional, Union, Any

from src.utils.logger import Logger
from src.utils.api_utils import rate_limiter, handle_api_errors, APIResponse, load_config

class EbayAPIClient:
    """
    Client for interacting with eBay API for data collection
    """
    def __init__(self, config_path: str = "/data/chats/p6wyr/workspace/config/config.json"):
        """
        Initialize the eBay API client with configuration
        
        Args:
            config_path (str): Path to the configuration file
        """
        self.logger = Logger().get_logger()
        self.logger.info("Initializing eBay API client")
        
        # Load configuration
        self.config = self._load_config(config_path)
        self.api_config = self.config.get("api", {}).get("ebay", {})
        
        # API credentials
        self.app_id = self.api_config.get("app_id", "")
        self.cert_id = self.api_config.get("cert_id", "")
        self.dev_id = self.api_config.get("dev_id", "")
        self.client_secret = self.api_config.get("client_secret", "")
        self.ru_name = self.api_config.get("ru_name", "")
        
        # API settings
        self.sandbox_mode = self.api_config.get("sandbox_mode", True)
        self.rate_limits = self.api_config.get("rate_limits", {
            "calls_per_second": 5,
            "calls_per_day": 5000
        })
        
        # Set base URLs based on sandbox mode
        if self.sandbox_mode:
            self.base_url = "https://api.sandbox.ebay.com"
            self.auth_url = "https://api.sandbox.ebay.com/identity/v1/oauth2/token"
        else:
            self.base_url = "https://api.ebay.com"
            self.auth_url = "https://api.ebay.com/identity/v1/oauth2/token"
            
        # Initialize access token
        self.access_token = None
        self.token_expiry = None
        
        # Cache directory for API responses
        self.cache_dir = "/data/chats/p6wyr/workspace/data/cache/api_responses"
        os.makedirs(self.cache_dir, exist_ok=True)
        
    def _load_config(self, config_path: str) -> Dict:
        """
        Load configuration from file
        
        Args:
            config_path (str): Path to the configuration file
            
        Returns:
            Dict: Configuration settings
        """
        if os.path.exists(config_path):
            with open(config_path, 'r') as f:
                return json.load(f)
        else:
            self.logger.warning(f"Configuration file not found at {config_path}, using defaults")
            return {}

    @handle_api_errors
    async def authenticate(self) -> bool:
        """
        Authenticate with eBay API and get access token
        
        Returns:
            bool: True if authentication successful, False otherwise
        """
        # Check if we already have a valid token
        if self.access_token and self.token_expiry and datetime.now() < self.token_expiry:
            return True
            
        self.logger.info("Authenticating with eBay API")
        
        # Prepare authentication request
        headers = {
            'Content-Type': 'application/x-www-form-urlencoded',
            'Authorization': f'Basic {self._get_basic_auth_header()}'
        }
        
        data = {
            'grant_type': 'client_credentials',
            'scope': 'https://api.ebay.com/oauth/api_scope'
        }
        
        try:
            response = requests.post(self.auth_url, headers=headers, data=data)
            response.raise_for_status()
            
            # Parse token response
            token_data = response.json()
            self.access_token = token_data.get('access_token')
            expires_in = token_data.get('expires_in', 7200)  # Default 2 hours
            
            # Set token expiry time
            self.token_expiry = datetime.now() + timedelta(seconds=expires_in - 300)  # 5 min buffer
            
            self.logger.info(f"Successfully authenticated with eBay API, token valid until {self.token_expiry}")
            return True
            
        except requests.exceptions.RequestException as e:
            self.logger.error(f"Authentication failed: {str(e)}")
            return False
    
    def _get_basic_auth_header(self) -> str:
        """
        Generate the Base64 encoded Basic auth header
        
        Returns:
            str: Base64 encoded auth string
        """
        import base64
        credentials = f"{self.app_id}:{self.client_secret}"
        encoded_credentials = base64.b64encode(credentials.encode('utf-8')).decode('utf-8')
        return encoded_credentials
    
    def _get_auth_headers(self) -> Dict[str, str]:
        """
        Get headers with authorization token
        
        Returns:
            Dict[str, str]: Headers for API requests
        """
        return {
            'Authorization': f'Bearer {self.access_token}',
            'Content-Type': 'application/json',
            'X-EBAY-C-MARKETPLACE-ID': 'EBAY_US'  # Default to US marketplace
        }
    
    def _get_cache_key(self, endpoint: str, params: Dict) -> str:
        """
        Generate a cache key for the API request
        
        Args:
            endpoint (str): API endpoint
            params (Dict): Request parameters
            
        Returns:
            str: Cache key
        """
        param_str = json.dumps(params, sort_keys=True)
        return f"{endpoint}_{hash(param_str)}"
    
    def _get_cached_response(self, cache_key: str) -> Optional[Dict]:
        """
        Get cached API response
        
        Args:
            cache_key (str): Cache key
            
        Returns:
            Optional[Dict]: Cached response or None
        """
        cache_file = os.path.join(self.cache_dir, f"{cache_key}.json")
        if os.path.exists(cache_file):
            try:
                with open(cache_file, 'r') as f:
                    cached_data = json.load(f)
                
                # Check if cache is still valid
                if datetime.now().timestamp() < cached_data.get('expiry', 0):
                    self.logger.info(f"Using cached response for {cache_key}")
                    return cached_data.get('data')
            except Exception as e:
                self.logger.warning(f"Error reading cache: {str(e)}")
        
        return None
    
    def _cache_response(self, cache_key: str, data: Dict, cache_ttl: int = 3600) -> None:
        """
        Cache API response
        
        Args:
            cache_key (str): Cache key
            data (Dict): Response data
            cache_ttl (int): Cache TTL in seconds
        """
        cache_file = os.path.join(self.cache_dir, f"{cache_key}.json")
        try:
            cached_data = {
                'data': data,
                'expiry': datetime.now().timestamp() + cache_ttl
            }
            with open(cache_file, 'w') as f:
                json.dump(cached_data, f)
            self.logger.info(f"Cached response for {cache_key}")
        except Exception as e:
            self.logger.warning(f"Error caching response: {str(e)}")
    
    @rate_limiter(max_calls=5, time_frame=1)  # 5 calls per second
    @handle_api_errors
    async def make_request(self, method: str, endpoint: str, params: Optional[Dict] = None, 
                          data: Optional[Dict] = None, use_cache: bool = True,
                          cache_ttl: int = 3600) -> APIResponse:
        """
        Make an API request to eBay
        
        Args:
            method (str): HTTP method (GET, POST, etc.)
            endpoint (str): API endpoint
            params (Optional[Dict]): Query parameters
            data (Optional[Dict]): Request body for POST requests
            use_cache (bool): Whether to use cache
            cache_ttl (int): Cache TTL in seconds
            
        Returns:
            APIResponse: Response object
        """
        # Ensure authentication
        if not self.access_token or not self.token_expiry or datetime.now() > self.token_expiry:
            if not await self.authenticate():
                return APIResponse(
                    success=False, 
                    error="Authentication failed", 
                    status_code=401
                )
        
        # Check cache if enabled
        if use_cache and method.upper() == 'GET':
            cache_key = self._get_cache_key(endpoint, params or {})
            cached_response = self._get_cached_response(cache_key)
            if cached_response:
                return APIResponse(success=True, data=cached_response, status_code=200)
        
        # Prepare request
        url = f"{self.base_url}/{endpoint}"
        headers = self._get_auth_headers()
        
        try:
            # Make request
            response = requests.request(
                method=method.upper(),
                url=url,
                headers=headers,
                params=params,
                json=data
            )
            response.raise_for_status()
            
            # Parse response
            response_data = response.json()
            
            # Cache response if needed
            if use_cache and method.upper() == 'GET':
                cache_key = self._get_cache_key(endpoint, params or {})
                self._cache_response(cache_key, response_data, cache_ttl)
            
            return APIResponse(
                success=True,
                data=response_data,
                status_code=response.status_code
            )
            
        except requests.exceptions.HTTPError as e:
            error_msg = f"HTTP error: {str(e)}"
            try:
                error_data = e.response.json()
                if 'errors' in error_data:
                    error_msg = f"{error_msg} - {error_data['errors']}"
            except:
                pass
            
            return APIResponse(
                success=False,
                error=error_msg,
                status_code=e.response.status_code if e.response else 500
            )
            
        except Exception as e:
            return APIResponse(
                success=False,
                error=f"Request failed: {str(e)}",
                status_code=500
            )
    
    async def search_items(self, keywords: str, category_id: Optional[str] = None, 
                          item_condition: Optional[List[str]] = None, 
                          price_range: Optional[Dict[str, float]] = None,
                          sort_order: str = "endingSoonest",
                          limit: int = 100) -> APIResponse:
        """
        Search for items on eBay
        
        Args:
            keywords (str): Search keywords
            category_id (Optional[str]): Category ID
            item_condition (Optional[List[str]]): List of condition IDs
            price_range (Optional[Dict[str, float]]): Price range with min and max keys
            sort_order (str): Sort order (endingSoonest, newlyListed, etc.)
            limit (int): Maximum number of items to return
            
        Returns:
            APIResponse: Response with search results
        """
        self.logger.info(f"Searching for '{keywords}' in category {category_id}")
        
        # Build query parameters
        params = {
            'q': keywords,
            'limit': min(limit, 200),  # API maximum is 200
            'sort': sort_order,
            'filter': []
        }
        
        # Add category filter if provided
        if category_id:
            params['category_ids'] = category_id
        
        # Add condition filter if provided
        if item_condition:
            condition_filter = {
                'itemFilter': {
                    'name': 'Condition',
                    'value': item_condition
                }
            }
            params['filter'].append(condition_filter)
        
        # Add price range filter if provided
        if price_range:
            if 'min' in price_range:
                min_price_filter = {
                    'itemFilter': {
                        'name': 'MinPrice',
                        'value': price_range['min'],
                        'paramName': 'Currency',
                        'paramValue': 'USD'
                    }
                }
                params['filter'].append(min_price_filter)
                
            if 'max' in price_range:
                max_price_filter = {
                    'itemFilter': {
                        'name': 'MaxPrice',
                        'value': price_range['max'],
                        'paramName': 'Currency',
                        'paramValue': 'USD'
                    }
                }
                params['filter'].append(max_price_filter)
        
        # Convert filters to string format
        if params['filter']:
            params['filter'] = json.dumps(params['filter'])
        else:
            del params['filter']
        
        # Make API request
        return await self.make_request(
            method='GET',
            endpoint='buy/browse/v1/item_summary/search',
            params=params,
            use_cache=True,
            cache_ttl=3600  # Cache for 1 hour
        )

    async def get_item_details(self, item_id: str, use_cache: bool = True) -> APIResponse:
        """
        Get detailed information about a specific item
        
        Args:
            item_id (str): eBay item ID
            use_cache (bool): Whether to use cache
            
        Returns:
            APIResponse: Response with item details
        """
        self.logger.info(f"Getting details for item {item_id}")
        
        return await self.make_request(
            method='GET',
            endpoint=f'buy/browse/v1/item/{item_id}',
            use_cache=use_cache,
            cache_ttl=3600  # Cache for 1 hour
        )

    async def get_item_analytics(self, item_ids: List[str]) -> APIResponse:
        """
        Get analytics for multiple items
        
        Args:
            item_ids (List[str]): List of eBay item IDs
            
        Returns:
            APIResponse: Response with item analytics
        """
        self.logger.info(f"Getting analytics for {len(item_ids)} items")
        
        # eBay Analytics API has a limit on the number of items
        if len(item_ids) > 20:
            self.logger.warning(f"Too many item IDs provided ({len(item_ids)}), using first 20 only")
            item_ids = item_ids[:20]
        
        return await self.make_request(
            method='POST',
            endpoint='commerce/analytics/v1/item_analytics',
            data={'itemIds': item_ids},
            use_cache=False  # Analytics data should be fresh
        )

    async def get_sold_items(self, keywords: str, category_id: Optional[str] = None,
                            days_back: int = 30, limit: int = 100) -> APIResponse:
        """
        Search for completed/sold items
        
        Args:
            keywords (str): Search keywords
            category_id (Optional[str]): Category ID
            days_back (int): Number of days to look back
            limit (int): Maximum number of items to return
            
        Returns:
            APIResponse: Response with completed/sold items
        """
        self.logger.info(f"Searching for sold items: '{keywords}' in category {category_id}")
        
        # This endpoint requires the Finding API, which has a different structure
        # We'll adapt our client to handle this special case
        
        params = {
            'keywords': keywords,
            'itemFilter': [
                {'name': 'SoldItemsOnly', 'value': 'true'},
                {'name': 'ListingType', 'value': 'FixedPrice'}
            ],
            'sortOrder': 'EndTimeSoonest',
            'paginationInput': {
                'entriesPerPage': min(limit, 100),
                'pageNumber': 1
            }
        }
        
        if category_id:
            params['categoryId'] = category_id
        
        # The Finding API has a different endpoint structure
        endpoint = 'finding/v1/services'
        
        return await self.make_request(
            method='GET',
            endpoint=endpoint,
            params={
                'OPERATION-NAME': 'findCompletedItems',
                'SERVICE-VERSION': '1.13.0',
                'SECURITY-APPNAME': self.app_id,
                'RESPONSE-DATA-FORMAT': 'JSON',
                'REST-PAYLOAD': 'true',
                'keywords': keywords,
                'categoryId': category_id,
                'itemFilter(0).name': 'SoldItemsOnly',
                'itemFilter(0).value': 'true',
                'sortOrder': 'EndTimeSoonest',
                'paginationInput.entriesPerPage': min(limit, 100),
            },
            use_cache=True,
            cache_ttl=3600 * 6  # Cache for 6 hours
        )

    async def get_categories(self, parent_id: Optional[str] = None) -> APIResponse:
        """
        Get eBay categories
        
        Args:
            parent_id (Optional[str]): Parent category ID for subcategories
            
        Returns:
            APIResponse: Response with categories
        """
        self.logger.info(f"Getting categories with parent ID: {parent_id}")
        
        params = {}
        if parent_id:
            params['category_id'] = parent_id
        
        return await self.make_request(
            method='GET',
            endpoint='commerce/taxonomy/v1/category_tree/0',  # 0 is for US marketplace
            params=params,
            use_cache=True,
            cache_ttl=86400 * 7  # Cache for 7 days as categories don't change often
        )

    async def get_category_aspects(self, category_id: str) -> APIResponse:
        """
        Get category aspects (item specifics)
        
        Args:
            category_id (str): Category ID
            
        Returns:
            APIResponse: Response with category aspects
        """
        self.logger.info(f"Getting aspects for category {category_id}")
        
        return await self.make_request(
            method='GET',
            endpoint=f'commerce/taxonomy/v1/category_tree/0/get_item_aspects_for_category',
            params={'category_id': category_id},
            use_cache=True,
            cache_ttl=86400 * 7  # Cache for 7 days
        )

# Fix missing import
from datetime import datetime, timedelta

# Example usage
if __name__ == "__main__":
    import asyncio
    
    async def test_ebay_api():
        # Initialize the client
        client = EbayAPIClient()
        
        # Authenticate
        auth_result = await client.authenticate()
        print(f"Authentication result: {auth_result}")
        
        # Search for items
        search_result = await client.search_items(
            keywords="iPhone 12",
            category_id="9355",
            item_condition=["USED_EXCELLENT", "USED_GOOD"],
            limit=10
        )
        
        if search_result.success:
            items = search_result.data.get("itemSummaries", [])
            print(f"Found {len(items)} items")
            
            # Get details of the first item
            if items:
                item_id = items[0].get("itemId")
                details_result = await client.get_item_details(item_id)
                
                if details_result.success:
                    print(f"Item details: {details_result.data}")
                else:
                    print(f"Failed to get item details: {details_result.error}")
        else:
            print(f"Search failed: {search_result.error}")
    
    # asyncio.run(test_ebay_api())  # Commented out as we're in a notebook environment

# Save the code to the specific file
import os
file_path = "/data/chats/p6wyr/workspace/src/api/ebay_api_client.py"

with open(file_path, "w") as f:
    f.write("""import os
import json
import requests
import time
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Union, Any

from src.utils.logger import Logger
from src.utils.api_utils import rate_limiter, handle_api_errors, APIResponse, load_config

class EbayAPIClient:
    """\"
    Client for interacting with eBay API for data collection
    \"""
    def __init__(self, config_path: str = "/data/chats/p6wyr/workspace/config/config.json"):
        """\"
        Initialize the eBay API client with configuration
        
        Args:
            config_path (str): Path to the configuration file
        \"""
        self.logger = Logger().get_logger()
        self.logger.info("Initializing eBay API client")
        
        # Load configuration
        self.config = self._load_config(config_path)
        self.api_config = self.config.get("api", {}).get("ebay", {})
        
        # API credentials
        self.app_id = self.api_config.get("app_id", "")
        self.cert_id = self.api_config.get("cert_id", "")
        self.dev_id = self.api_config.get("dev_id", "")
        self.client_secret = self.api_config.get("client_secret", "")
        self.ru_name = self.api_config.get("ru_name", "")
        
        # API settings
        self.sandbox_mode = self.api_config.get("sandbox_mode", True)
        self.rate_limits = self.api_config.get("rate_limits", {
            "calls_per_second": 5,
            "calls_per_day": 5000
        })
        
        # Set base URLs based on sandbox mode
        if self.sandbox_mode:
            self.base_url = "https://api.sandbox.ebay.com"
            self.auth_url = "https://api.sandbox.ebay.com/identity/v1/oauth2/token"
        else:
            self.base_url = "https://api.ebay.com"
            self.auth_url = "https://api.ebay.com/identity/v1/oauth2/token"
            
        # Initialize access token
        self.access_token = None
        self.token_expiry = None
        
        # Cache directory for API responses
        self.cache_dir = "/data/chats/p6wyr/workspace/data/cache/api_responses"
        os.makedirs(self.cache_dir, exist_ok=True)
        
    def _load_config(self, config_path: str) -> Dict:
        """\"
        Load configuration from file
        
        Args:
            config_path (str): Path to the configuration file
            
        Returns:
            Dict: Configuration settings
        \"""
        if os.path.exists(config_path):
            with open(config_path, 'r') as f:
                return json.load(f)
        else:
            self.logger.warning(f"Configuration file not found at {config_path}, using defaults")
            return {}

    @handle_api_errors
    async def authenticate(self) -> bool:
        """\"
        Authenticate with eBay API and get access token
        
        Returns:
            bool: True if authentication successful, False otherwise
        \"""
        # Check if we already have a valid token
        if self.access_token and self.token_expiry and datetime.now() < self.token_expiry:
            return True
            
        self.logger.info("Authenticating with eBay API")
        
        # Prepare authentication request
        headers = {
            'Content-Type': 'application/x-www-form-urlencoded',
            'Authorization': f'Basic {self._get_basic_auth_header()}'
        }
        
        data = {
            'grant_type': 'client_credentials',
            'scope': 'https://api.ebay.com/oauth/api_scope'
        }
        
        try:
            response = requests.post(self.auth_url, headers=headers, data=data)
            response.raise_for_status()
            
            # Parse token response
            token_data = response.json()
            self.access_token = token_data.get('access_token')
            expires_in = token_data.get('expires_in', 7200)  # Default 2 hours
            
            # Set token expiry time
            self.token_expiry = datetime.now() + timedelta(seconds=expires_in - 300)  # 5 min buffer
            
            self.logger.info(f"Successfully authenticated with eBay API, token valid until {self.token_expiry}")
            return True
            
        except requests.exceptions.RequestException as e:
            self.logger.error(f"Authentication failed: {str(e)}")
            return False
    
    def _get_basic_auth_header(self) -> str:
        """\"
        Generate the Base64 encoded Basic auth header
        
        Returns:
            str: Base64 encoded auth string
        \"""
        import base64
        credentials = f"{self.app_id}:{self.client_secret}"
        encoded_credentials = base64.b64encode(credentials.encode('utf-8')).decode('utf-8')
        return encoded_credentials
    
    def _get_auth_headers(self) -> Dict[str, str]:
        """\"
        Get headers with authorization token
        
        Returns:
            Dict[str, str]: Headers for API requests
        \"""
        return {
            'Authorization': f'Bearer {self.access_token}',
            'Content-Type': 'application/json',
            'X-EBAY-C-MARKETPLACE-ID': 'EBAY_US'  # Default to US marketplace
        }
    
    def _get_cache_key(self, endpoint: str, params: Dict) -> str:
        """\"
        Generate a cache key for the API request
        
        Args:
            endpoint (str): API endpoint
            params (Dict): Request parameters
            
        Returns:
            str: Cache key
        \"""
        param_str = json.dumps(params, sort_keys=True)
        return f"{endpoint}_{hash(param_str)}"
    
    def _get_cached_response(self, cache_key: str) -> Optional[Dict]:
        """\"
        Get cached API response
        
        Args:
            cache_key (str): Cache key
            
        Returns:
            Optional[Dict]: Cached response or None
        \"""
        cache_file = os.path.join(self.cache_dir, f"{cache_key}.json")
        if os.path.exists(cache_file):
            try:
                with open(cache_file, 'r') as f:
                    cached_data = json.load(f)
                
                # Check if cache is still valid
                if datetime.now().timestamp() < cached_data.get('expiry', 0):
                    self.logger.info(f"Using cached response for {cache_key}")
                    return cached_data.get('data')
            except Exception as e:
                self.logger.warning(f"Error reading cache: {str(e)}")
        
        return None
    
    def _cache_response(self, cache_key: str, data: Dict, cache_ttl: int = 3600) -> None:
        """\"
        Cache API response
        
        Args:
            cache_key (str): Cache key
            data (Dict): Response data
            cache_ttl (int): Cache TTL in seconds
        \"""
        cache_file = os.path.join(self.cache_dir, f"{cache_key}.json")
        try:
            cached_data = {
                'data': data,
                'expiry': datetime.now().timestamp() + cache_ttl
            }
            with open(cache_file, 'w') as f:
                json.dump(cached_data, f)
            self.logger.info(f"Cached response for {cache_key}")
        except Exception as e:
            self.logger.warning(f"Error caching response: {str(e)}")
    
    @rate_limiter(max_calls=5, time_frame=1)  # 5 calls per second
    @handle_api_errors
    async def make_request(self, method: str, endpoint: str, params: Optional[Dict] = None, 
                          data: Optional[Dict] = None, use_cache: bool = True,
                          cache_ttl: int = 3600) -> APIResponse:
        """\"
        Make an API request to eBay
        
        Args:
            method (str): HTTP method (GET, POST, etc.)
            endpoint (str): API endpoint
            params (Optional[Dict]): Query parameters
            data (Optional[Dict]): Request body for POST requests
            use_cache (bool): Whether to use cache
            cache_ttl (int): Cache TTL in seconds
            
        Returns:
            APIResponse: Response object
        \"""
        # Ensure authentication
        if not self.access_token or not self.token_expiry or datetime.now() > self.token_expiry:
            if not await self.authenticate():
                return APIResponse(
                    success=False, 
                    error="Authentication failed", 
                    status_code=401
                )
        
        # Check cache if enabled
        if use_cache and method.upper() == 'GET':
            cache_key = self._get_cache_key(endpoint, params or {})
            cached_response = self._get_cached_response(cache_key)
            if cached_response:
                return APIResponse(success=True, data=cached_response, status_code=200)
        
        # Prepare request
        url = f"{self.base_url}/{endpoint}"
        headers = self._get_auth_headers()
        
        try:
            # Make request
            response = requests.request(
                method=method.upper(),
                url=url,
                headers=headers,
                params=params,
                json=data
            )
            response.raise_for_status()
            
            # Parse response
            response_data = response.json()
            
            # Cache response if needed
            if use_cache and method.upper() == 'GET':
                cache_key = self._get_cache_key(endpoint, params or {})
                self._cache_response(cache_key, response_data, cache_ttl)
            
            return APIResponse(
                success=True,
                data=response_data,
                status_code=response.status_code
            )
            
        except requests.exceptions.HTTPError as e:
            error_msg = f"HTTP error: {str(e)}"
            try:
                error_data = e.response.json()
                if 'errors' in error_data:
                    error_msg = f"{error_msg} - {error_data['errors']}"
            except:
                pass
            
            return APIResponse(
                success=False,
                error=error_msg,
                status_code=e.response.status_code if e.response else 500
            )
            
        except Exception as e:
            return APIResponse(
                success=False,
                error=f"Request failed: {str(e)}",
                status_code=500
            )
    
    async def search_items(self, keywords: str, category_id: Optional[str] = None, 
                          item_condition: Optional[List[str]] = None, 
                          price_range: Optional[Dict[str, float]] = None,
                          sort

SyntaxError: invalid syntax (1824683791.py, line 1)

In [2]:
from metagpt.tools.libs.terminal import Terminal
terminal = Terminal()

ebay_api_client_code = """
import os
import json
import requests
import time
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Union, Any

from src.utils.logger import Logger
from src.utils.api_utils import rate_limiter, handle_api_errors, APIResponse, load_config

class EbayAPIClient:
    \"\"\"
    Client for interacting with eBay API for data collection
    \"\"\"
    def __init__(self, config_path: str = "/data/chats/p6wyr/workspace/config/config.json"):
        \"\"\"
        Initialize the eBay API client with configuration
        
        Args:
            config_path (str): Path to the configuration file
        \"\"\"
        self.logger = Logger().get_logger()
        self.logger.info("Initializing eBay API client")
        
        # Load configuration
        self.config = self._load_config(config_path)
        self.api_config = self.config.get("api", {}).get("ebay", {})
        
        # API credentials
        self.app_id = self.api_config.get("app_id", "")
        self.cert_id = self.api_config.get("cert_id", "")
        self.dev_id = self.api_config.get("dev_id", "")
        self.client_secret = self.api_config.get("client_secret", "")
        self.ru_name = self.api_config.get("ru_name", "")
        
        # API settings
        self.sandbox_mode = self.api_config.get("sandbox_mode", True)
        self.rate_limits = self.api_config.get("rate_limits", {
            "calls_per_second": 5,
            "calls_per_day": 5000
        })
        
        # Set base URLs based on sandbox mode
        if self.sandbox_mode:
            self.base_url = "https://api.sandbox.ebay.com"
            self.auth_url = "https://api.sandbox.ebay.com/identity/v1/oauth2/token"
        else:
            self.base_url = "https://api.ebay.com"
            self.auth_url = "https://api.ebay.com/identity/v1/oauth2/token"
            
        # Initialize access token
        self.access_token = None
        self.token_expiry = None
        
        # Cache directory for API responses
        self.cache_dir = "/data/chats/p6wyr/workspace/data/cache/api_responses"
        os.makedirs(self.cache_dir, exist_ok=True)
        
    def _load_config(self, config_path: str) -> Dict:
        \"\"\"
        Load configuration from file
        
        Args:
            config_path (str): Path to the configuration file
            
        Returns:
            Dict: Configuration settings
        \"\"\"
        if os.path.exists(config_path):
            with open(config_path, 'r') as f:
                return json.load(f)
        else:
            self.logger.warning(f"Configuration file not found at {config_path}, using defaults")
            return {}

    @handle_api_errors
    async def authenticate(self) -> bool:
        \"\"\"
        Authenticate with eBay API and get access token
        
        Returns:
            bool: True if authentication successful, False otherwise
        \"\"\"
        # Check if we already have a valid token
        if self.access_token and self.token_expiry and datetime.now() < self.token_expiry:
            return True
            
        self.logger.info("Authenticating with eBay API")
        
        # Prepare authentication request
        headers = {
            'Content-Type': 'application/x-www-form-urlencoded',
            'Authorization': f'Basic {self._get_basic_auth_header()}'
        }
        
        data = {
            'grant_type': 'client_credentials',
            'scope': 'https://api.ebay.com/oauth/api_scope'
        }
        
        try:
            response = requests.post(self.auth_url, headers=headers, data=data)
            response.raise_for_status()
            
            # Parse token response
            token_data = response.json()
            self.access_token = token_data.get('access_token')
            expires_in = token_data.get('expires_in', 7200)  # Default 2 hours
            
            # Set token expiry time
            self.token_expiry = datetime.now() + timedelta(seconds=expires_in - 300)  # 5 min buffer
            
            self.logger.info(f"Successfully authenticated with eBay API, token valid until {self.token_expiry}")
            return True
            
        except requests.exceptions.RequestException as e:
            self.logger.error(f"Authentication failed: {str(e)}")
            return False
    
    def _get_basic_auth_header(self) -> str:
        \"\"\"
        Generate the Base64 encoded Basic auth header
        
        Returns:
            str: Base64 encoded auth string
        \"\"\"
        import base64
        credentials = f"{self.app_id}:{self.client_secret}"
        encoded_credentials = base64.b64encode(credentials.encode('utf-8')).decode('utf-8')
        return encoded_credentials
    
    def _get_auth_headers(self) -> Dict[str, str]:
        \"\"\"
        Get headers with authorization token
        
        Returns:
            Dict[str, str]: Headers for API requests
        \"\"\"
        return {
            'Authorization': f'Bearer {self.access_token}',
            'Content-Type': 'application/json',
            'X-EBAY-C-MARKETPLACE-ID': 'EBAY_US'  # Default to US marketplace
        }
    
    def _get_cache_key(self, endpoint: str, params: Dict) -> str:
        \"\"\"
        Generate a cache key for the API request
        
        Args:
            endpoint (str): API endpoint
            params (Dict): Request parameters
            
        Returns:
            str: Cache key
        \"\"\"
        param_str = json.dumps(params, sort_keys=True)
        return f"{endpoint}_{hash(param_str)}"
    
    def _get_cached_response(self, cache_key: str) -> Optional[Dict]:
        \"\"\"
        Get cached API response
        
        Args:
            cache_key (str): Cache key
            
        Returns:
            Optional[Dict]: Cached response or None
        \"\"\"
        cache_file = os.path.join(self.cache_dir, f"{cache_key}.json")
        if os.path.exists(cache_file):
            try:
                with open(cache_file, 'r') as f:
                    cached_data = json.load(f)
                
                # Check if cache is still valid
                if datetime.now().timestamp() < cached_data.get('expiry', 0):
                    self.logger.info(f"Using cached response for {cache_key}")
                    return cached_data.get('data')
            except Exception as e:
                self.logger.warning(f"Error reading cache: {str(e)}")
        
        return None
    
    def _cache_response(self, cache_key: str, data: Dict, cache_ttl: int = 3600) -> None:
        \"\"\"
        Cache API response
        
        Args:
            cache_key (str): Cache key
            data (Dict): Response data
            cache_ttl (int): Cache TTL in seconds
        \"\"\"
        cache_file = os.path.join(self.cache_dir, f"{cache_key}.json")
        try:
            cached_data = {
                'data': data,
                'expiry': datetime.now().timestamp() + cache_ttl
            }
            with open(cache_file, 'w') as f:
                json.dump(cached_data, f)
            self.logger.info(f"Cached response for {cache_key}")
        except Exception as e:
            self.logger.warning(f"Error caching response: {str(e)}")
    
    @rate_limiter(max_calls=5, time_frame=1)  # 5 calls per second
    @handle_api_errors
    async def make_request(self, method: str, endpoint: str, params: Optional[Dict] = None, 
                          data: Optional[Dict] = None, use_cache: bool = True,
                          cache_ttl: int = 3600) -> APIResponse:
        \"\"\"
        Make an API request to eBay
        
        Args:
            method (str): HTTP method (GET, POST, etc.)
            endpoint (str): API endpoint
            params (Optional[Dict]): Query parameters
            data (Optional[Dict]): Request body for POST requests
            use_cache (bool): Whether to use cache
            cache_ttl (int): Cache TTL in seconds
            
        Returns:
            APIResponse: Response object
        \"\"\"
        # Ensure authentication
        if not self.access_token or not self.token_expiry or datetime.now() > self.token_expiry:
            if not await self.authenticate():
                return APIResponse(
                    success=False, 
                    error="Authentication failed", 
                    status_code=401
                )
        
        # Check cache if enabled
        if use_cache and method.upper() == 'GET':
            cache_key = self._get_cache_key(endpoint, params or {})
            cached_response = self._get_cached_response(cache_key)
            if cached_response:
                return APIResponse(success=True, data=cached_response, status_code=200)
        
        # Prepare request
        url = f"{self.base_url}/{endpoint}"
        headers = self._get_auth_headers()
        
        try:
            # Make request
            response = requests.request(
                method=method.upper(),
                url=url,
                headers=headers,
                params=params,
                json=data
            )
            response.raise_for_status()
            
            # Parse response
            response_data = response.json()
            
            # Cache response if needed
            if use_cache and method.upper() == 'GET':
                cache_key = self._get_cache_key(endpoint, params or {})
                self._cache_response(cache_key, response_data, cache_ttl)
            
            return APIResponse(
                success=True,
                data=response_data,
                status_code=response.status_code
            )
            
        except requests.exceptions.HTTPError as e:
            error_msg = f"HTTP error: {str(e)}"
            try:
                error_data = e.response.json()
                if 'errors' in error_data:
                    error_msg = f"{error_msg} - {error_data['errors']}"
            except:
                pass
            
            return APIResponse(
                success=False,
                error=error_msg,
                status_code=e.response.status_code if e.response else 500
            )
            
        except Exception as e:
            return APIResponse(
                success=False,
                error=f"Request failed: {str(e)}",
                status_code=500
            )
    
    async def search_items(self, keywords: str, category_id: Optional[str] = None, 
                          item_condition: Optional[List[str]] = None, 
                          price_range: Optional[Dict[str, float]] = None,
                          sort_order: str = "endingSoonest",
                          limit: int = 100) -> APIResponse:
        \"\"\"
        Search for items on eBay
        
        Args:
            keywords (str): Search keywords
            category_id (Optional[str]): Category ID
            item_condition (Optional[List[str]]): List of condition IDs
            price_range (Optional[Dict[str, float]]): Price range with min and max keys
            sort_order (str): Sort order (endingSoonest, newlyListed, etc.)
            limit (int): Maximum number of items to return
            
        Returns:
            APIResponse: Response with search results
        \"\"\"
        self.logger.info(f"Searching for '{keywords}' in category {category_id}")
        
        # Build query parameters
        params = {
            'q': keywords,
            'limit': min(limit, 200),  # API maximum is 200
            'sort': sort_order,
            'filter': []
        }
        
        # Add category filter if provided
        if category_id:
            params['category_ids'] = category_id
        
        # Add condition filter if provided
        if item_condition:
            condition_filter = {
                'itemFilter': {
                    'name': 'Condition',
                    'value': item_condition
                }
            }
            params['filter'].append(condition_filter)
        
        # Add price range filter if provided
        if price_range:
            if 'min' in price_range:
                min_price_filter = {
                    'itemFilter': {
                        'name': 'MinPrice',
                        'value': price_range['min'],
                        'paramName': 'Currency',
                        'paramValue': 'USD'
                    }
                }
                params['filter'].append(min_price_filter)
                
            if 'max' in price_range:
                max_price_filter = {
                    'itemFilter': {
                        'name': 'MaxPrice',
                        'value': price_range['max'],
                        'paramName': 'Currency',
                        'paramValue': 'USD'
                    }
                }
                params['filter'].append(max_price_filter)
        
        # Convert filters to string format
        if params['filter']:
            params['filter'] = json.dumps(params['filter'])
        else:
            del params['filter']
        
        # Make API request
        return await self.make_request(
            method='GET',
            endpoint='buy/browse/v1/item_summary/search',
            params=params,
            use_cache=True,
            cache_ttl=3600  # Cache for 1 hour
        )

    async def get_item_details(self, item_id: str, use_cache: bool = True) -> APIResponse:
        \"\"\"
        Get detailed information about a specific item
        
        Args:
            item_id (str): eBay item ID
            use_cache (bool): Whether to use cache
            
        Returns:
            APIResponse: Response with item details
        \"\"\"
        self.logger.info(f"Getting details for item {item_id}")
        
        return await self.make_request(
            method='GET',
            endpoint=f'buy/browse/v1/item/{item_id}',
            use_cache=use_cache,
            cache_ttl=3600  # Cache for 1 hour
        )

    async def get_item_analytics(self, item_ids: List[str]) -> APIResponse:
        \"\"\"
        Get analytics for multiple items
        
        Args:
            item_ids (List[str]): List of eBay item IDs
            
        Returns:
            APIResponse: Response with item analytics
        \"\"\"
        self.logger.info(f"Getting analytics for {len(item_ids)} items")
        
        # eBay Analytics API has a limit on the number of items
        if len(item_ids) > 20:
            self.logger.warning(f"Too many item IDs provided ({len(item_ids)}), using first 20 only")
            item_ids = item_ids[:20]
        
        return await self.make_request(
            method='POST',
            endpoint='commerce/analytics/v1/item_analytics',
            data={'itemIds': item_ids},
            use_cache=False  # Analytics data should be fresh
        )

    async def get_sold_items(self, keywords: str, category_id: Optional[str] = None,
                            days_back: int = 30, limit: int = 100) -> APIResponse:
        \"\"\"
        Search for completed/sold items
        
        Args:
            keywords (str): Search keywords
            category_id (Optional[str]): Category ID
            days_back (int): Number of days to look back
            limit (int): Maximum number of items to return
            
        Returns:
            APIResponse: Response with completed/sold items
        \"\"\"
        self.logger.info(f"Searching for sold items: '{keywords}' in category {category_id}")
        
        # This endpoint requires the Finding API, which has a different structure
        # We'll adapt our client to handle this special case
        
        params = {
            'keywords': keywords,
            'itemFilter': [
                {'name': 'SoldItemsOnly', 'value': 'true'},
                {'name': 'ListingType', 'value': 'FixedPrice'}
            ],
            'sortOrder': 'EndTimeSoonest',
            'paginationInput': {
                'entriesPerPage': min(limit, 100),
                'pageNumber': 1
            }
        }
        
        if category_id:
            params['categoryId'] = category_id
        
        # The Finding API has a different endpoint structure
        endpoint = 'finding/v1/services'
        
        return await self.make_request(
            method='GET',
            endpoint=endpoint,
            params={
                'OPERATION-NAME': 'findCompletedItems',
                'SERVICE-VERSION': '1.13.0',
                'SECURITY-APPNAME': self.app_id,
                'RESPONSE-DATA-FORMAT': 'JSON',
                'REST-PAYLOAD': 'true',
                'keywords': keywords,
                'categoryId': category_id,
                'itemFilter(0).name': 'SoldItemsOnly',
                'itemFilter(0).value': 'true',
                'sortOrder': 'EndTimeSoonest',
                'paginationInput.entriesPerPage': min(limit, 100),
            },
            use_cache=True,
            cache_ttl=3600 * 6  # Cache for 6 hours
        )

    async def get_categories(self, parent_id: Optional[str] = None) -> APIResponse:
        \"\"\"
        Get eBay categories
        
        Args:
            parent_id (Optional[str]): Parent category ID for subcategories
            
        Returns:
            APIResponse: Response with categories
        \"\"\"
        self.logger.info(f"Getting categories with parent ID: {parent_id}")
        
        params = {}
        if parent_id:
            params['category_id'] = parent_id
        
        return await self.make_request(
            method='GET',
            endpoint='commerce/taxonomy/v1/category_tree/0',  # 0 is for US marketplace
            params=params,
            use_cache=True,
            cache_ttl=86400 * 7  # Cache for 7 days as categories don't change often
        )

    async def get_category_aspects(self, category_id: str) -> APIResponse:
        \"\"\"
        Get category aspects (item specifics)
        
        Args:
            category_id (str): Category ID
            
        Returns:
            APIResponse: Response with category aspects
        \"\"\"
        self.logger.info(f"Getting aspects for category {category_id}")
        
        return await self.make_request(
            method='GET',
            endpoint=f'commerce/taxonomy/v1/category_tree/0/get_item_aspects_for_category',
            params={'category_id': category_id},
            use_cache=True,
            cache_ttl=86400 * 7  # Cache for 7 days
        )
"""

# Create two files using the Terminal tool - this avoids issues with string truncation
file_path = "/data/chats/p6wyr/workspace/src/api/ebay_api_client.py"

# Use the Terminal to write the file in parts for better reliability
await terminal.run(f"rm -f {file_path}")
await terminal.run(f"touch {file_path}")

# Write the code to the file in smaller chunks to avoid truncation
with open(file_path, 'w') as f:
    f.write(ebay_api_client_code.strip())

# Create the cache directory for API responses
await terminal.run("mkdir -p /data/chats/p6wyr/workspace/data/cache/api_responses")

# Print confirmation message
print("Successfully created the eBay API client implementation.")
print(f"Wrote file to: {file_path}")

Successfully created the eBay API client implementation.
Wrote file to: /data/chats/p6wyr/workspace/src/api/ebay_api_client.py


In [3]:
import os
import json
import pandas as pd
import asyncio
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Union, Any, Tuple
import time
import random
from tqdm import tqdm

from src.api.ebay_api_client import EbayAPIClient
from src.utils.logger import Logger
from src.utils.data_utils import generate_item_id, save_json, load_json, get_category_folder, save_items_to_csv, save_item_history
from src.utils.scheduled_task_manager import ScheduledTaskManager


class DataCollector:
    """
    Data collection module for retrieving second-hand item data from eBay
    """
    def __init__(self, config_path: str = "/data/chats/p6wyr/workspace/config/config.json"):
        """
        Initialize the data collector
        
        Args:
            config_path (str): Path to configuration file
        """
        self.logger = Logger().get_logger()
        self.logger.info("Initializing DataCollector module")
        
        # Load configuration
        self.config_path = config_path
        self._load_config()
        
        # Initialize eBay API client
        self.ebay_client = EbayAPIClient(config_path=config_path)
        
        # Set up data directories
        self.raw_data_dir = "/data/chats/p6wyr/workspace/data/raw"
        os.makedirs(self.raw_data_dir, exist_ok=True)
        
        # Track collection statistics
        self.stats = {
            "items_collected": 0,
            "categories_processed": 0,
            "errors": 0,
            "start_time": None,
            "end_time": None
        }
    
    def _load_config(self) -> None:
        """
        Load configuration from file
        """
        try:
            with open(self.config_path, 'r') as f:
                self.config = json.load(f)
            
            # Extract relevant configuration
            self.categories = self.config.get("data", {}).get("categories", [])
            self.sample_size_per_category = self.config.get("data", {}).get("sample_size_per_category", 100)
            self.history_window = self.config.get("data", {}).get("history_window", 30)
            self.update_frequency = self.config.get("data", {}).get("update_frequency", 24)
            
            self.logger.info(f"Configuration loaded: {len(self.categories)} categories, "
                             f"{self.sample_size_per_category} items per category")
        except Exception as e:
            self.logger.error(f"Error loading configuration: {str(e)}")
            # Use default values
            self.categories = [
                {"id": "9355", "name": "Laptops & Netbooks"},
                {"id": "15032", "name": "Cell Phones & Smartphones"}
            ]
            self.sample_size_per_category = 100
            self.history_window = 30
            self.update_frequency = 24
    
    async def collect_data(self) -> Dict:
        """
        Collect data for all configured categories
        
        Returns:
            Dict: Collection statistics
        """
        self.stats["start_time"] = datetime.now()
        self.logger.info(f"Starting data collection for {len(self.categories)} categories")
        
        for category in self.categories:
            try:
                await self.collect_category_data(category)
                self.stats["categories_processed"] += 1
            except Exception as e:
                self.logger.error(f"Error collecting data for category {category['name']}: {str(e)}")
                self.stats["errors"] += 1
        
        self.stats["end_time"] = datetime.now()
        collection_time = (self.stats["end_time"] - self.stats["start_time"]).total_seconds()
        self.logger.info(f"Data collection completed: {self.stats['items_collected']} items collected "
                         f"from {self.stats['categories_processed']} categories in {collection_time:.2f} seconds")
        
        # Save collection stats
        stats_file = os.path.join(self.raw_data_dir, f"collection_stats_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json")
        save_json(self.stats, stats_file)
        
        return self.stats
    
    async def collect_category_data(self, category: Dict) -> None:
        """
        Collect data for a specific category
        
        Args:
            category (Dict): Category information with id and name
        """
        category_id = category["id"]
        category_name = category["name"]
        self.logger.info(f"Collecting data for category: {category_name} (ID: {category_id})")
        
        # Create category folder
        category_folder = get_category_folder(category_id)
        os.makedirs(category_folder, exist_ok=True)
        
        # Define search terms for this category
        search_terms = self._get_search_terms_for_category(category)
        
        # Collect active listings
        active_items = await self._collect_active_items(category, search_terms)
        
        # Collect sold items for price history
        sold_items = await self._collect_sold_items(category, search_terms)
        
        # Save category data
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        active_file = os.path.join(category_folder, f"active_items_{timestamp}.csv")
        sold_file = os.path.join(category_folder, f"sold_items_{timestamp}.csv")
        
        save_items_to_csv(active_items, active_file)
        save_items_to_csv(sold_items, sold_file)
        
        # Create a metadata file to track the collection
        metadata = {
            "category_id": category_id,
            "category_name": category_name,
            "timestamp": datetime.now().isoformat(),
            "active_items_count": len(active_items),
            "sold_items_count": len(sold_items),
            "active_items_file": active_file,
            "sold_items_file": sold_file
        }
        
        metadata_file = os.path.join(category_folder, f"metadata_{timestamp}.json")
        save_json(metadata, metadata_file)
        
        self.logger.info(f"Collected {len(active_items)} active items and {len(sold_items)} sold items "
                         f"for category {category_name}")
        
        # Update stats
        self.stats["items_collected"] += len(active_items) + len(sold_items)
    
    def _get_search_terms_for_category(self, category: Dict) -> List[str]:
        """
        Get appropriate search terms for a category
        
        Args:
            category (Dict): Category information
            
        Returns:
            List[str]: List of search terms
        """
        category_name = category["name"]
        # This could be expanded with more specific search terms per category
        base_terms = [category_name, f"used {category_name}", f"second hand {category_name}"]
        
        # Add category-specific terms
        if category["id"] == "9355":  # Laptops
            return base_terms + ["refurbished laptop", "used notebook", "macbook", "thinkpad", "gaming laptop"]
        elif category["id"] == "15032":  # Cell Phones
            return base_terms + ["used iphone", "used samsung galaxy", "refurbished phone", "unlocked phone"]
        elif category["id"] == "11450":  # Watches
            return base_terms + ["used rolex", "used omega", "vintage watch", "pre-owned watch"]
        elif category["id"] == "261007":  # Cameras
            return base_terms + ["used dslr", "second hand mirrorless", "vintage camera", "used sony camera"]
        else:
            return base_terms
    
    async def _collect_active_items(self, category: Dict, search_terms: List[str]) -> List[Dict]:
        """
        Collect active listings for a category using multiple search terms
        
        Args:
            category (Dict): Category information
            search_terms (List[str]): List of search terms
            
        Returns:
            List[Dict]: Collected items data
        """
        category_id = category["id"]
        collected_items = []
        
        for search_term in tqdm(search_terms, desc=f"Collecting active items for {category['name']}"):
            try:
                # Make the API request with reasonable limits
                response = await self.ebay_client.search_items(
                    keywords=search_term,
                    category_id=category_id,
                    limit=min(50, self.sample_size_per_category // len(search_terms))
                )
                
                if not response.success:
                    self.logger.warning(f"Error searching for '{search_term}' in category {category_id}: {response.error}")
                    continue
                
                # Extract items from response
                items = response.data.get("itemSummaries", [])
                self.logger.info(f"Found {len(items)} items for search term '{search_term}'")
                
                # Process each item
                for item in items:
                    try:
                        # Generate a consistent item ID
                        item_id = item.get("itemId") or generate_item_id(item)
                        
                        # Get more detailed information about the item
                        details_response = await self.ebay_client.get_item_details(item_id, use_cache=True)
                        
                        if details_response.success:
                            item_details = details_response.data
                            
                            # Merge summary and details
                            processed_item = self._process_item_data(item, item_details)
                            collected_items.append(processed_item)
                            
                            # Add a small delay to avoid overwhelming the API
                            await asyncio.sleep(random.uniform(0.1, 0.5))
                    except Exception as e:
                        self.logger.error(f"Error processing item {item.get('itemId', 'unknown')}: {str(e)}")
                
                # Add a delay between search terms
                await asyncio.sleep(random.uniform(1.0, 2.0))
                
            except Exception as e:
                self.logger.error(f"Error collecting items for search term '{search_term}': {str(e)}")
                self.stats["errors"] += 1
        
        # Deduplicate items based on itemId
        unique_items = []
        seen_ids = set()
        for item in collected_items:
            if item["itemId"] not in seen_ids:
                unique_items.append(item)
                seen_ids.add(item["itemId"])
        
        self.logger.info(f"Collected {len(unique_items)} unique active items for category {category['name']}")
        return unique_items
    
    async def _collect_sold_items(self, category: Dict, search_terms: List[str]) -> List[Dict]:
        """
        Collect sold items data for price history
        
        Args:
            category (Dict): Category information
            search_terms (List[str]): List of search terms
            
        Returns:
            List[Dict]: Collected sold items data
        """
        category_id = category["id"]
        sold_items = []
        
        for search_term in tqdm(search_terms[:2], desc=f"Collecting sold items for {category['name']}"):
            try:
                # Make the API request for sold items
                response = await self.ebay_client.get_sold_items(
                    keywords=search_term,
                    category_id=category_id,
                    days_back=self.history_window,
                    limit=min(50, self.sample_size_per_category // len(search_terms))
                )
                
                if not response.success:
                    self.logger.warning(f"Error searching for sold items with '{search_term}' in category {category_id}: {response.error}")
                    continue
                
                # The structure of the sold items response is different
                items = []
                try:
                    # The response structure might vary depending on the API version
                    search_result = response.data.get("findCompletedItemsResponse", [{}])[0]
                    result_items = search_result.get("searchResult", [{}])[0].get("item", [])
                    items.extend(result_items)
                except (KeyError, IndexError, TypeError) as e:
                    self.logger.warning(f"Error parsing sold items response: {str(e)}")
                
                self.logger.info(f"Found {len(items)} sold items for search term '{search_term}'")
                
                # Process sold items
                for item in items:
                    try:
                        processed_item = self._process_sold_item_data(item)
                        sold_items.append(processed_item)
                    except Exception as e:
                        self.logger.error(f"Error processing sold item: {str(e)}")
                
                # Add a delay between search terms
                await asyncio.sleep(random.uniform(1.0, 2.0))
                
            except Exception as e:
                self.logger.error(f"Error collecting sold items for search term '{search_term}': {str(e)}")
                self.stats["errors"] += 1
        
        # Deduplicate sold items
        unique_sold_items = []
        seen_ids = set()
        for item in sold_items:
            if item["itemId"] not in seen_ids:
                unique_sold_items.append(item)
                seen_ids.add(item["itemId"])
        
        self.logger.info(f"Collected {len(unique_sold_items)} unique sold items for category {category['name']}")
        return unique_sold_items
    
    def _process_item_data(self, item_summary: Dict, item_details: Dict) -> Dict:
        """
        Process and normalize item data from API responses
        
        Args:
            item_summary (Dict): Item summary from search results
            item_details (Dict): Detailed item information
            
        Returns:
            Dict: Processed item data
        """
        # Extract the base information from summary
        processed_item = {
            "itemId": item_summary.get("itemId", ""),
            "title": item_summary.get("title", ""),
            "condition": item_summary.get("condition", ""),
            "conditionId": item_summary.get("conditionId", ""),
            "price": self._extract_price(item_summary.get("price", {})),
            "currency": self._extract_currency(item_summary.get("price", {})),
            "category_id": item_details.get("categoryId", ""),
            "category_path": item_details.get("categoryPath", ""),
            "listing_date": item_summary.get("itemCreationDate", ""),
            "end_date": item_summary.get("itemEndDate", ""),
            "collection_date": datetime.now().isoformat(),
            "url": item_summary.get("itemWebUrl", ""),
            "item_location": self._extract_location(item_details),
            "shipping_options": self._extract_shipping(item_details),
            "item_specifics": self._extract_item_specifics(item_details),
            "image_urls": self._extract_image_urls(item_summary, item_details)
        }
        
        return processed_item
    
    def _process_sold_item_data(self, item: Dict) -> Dict:
        """
        Process sold item data which has a different structure
        
        Args:
            item (Dict): Sold item data from API
            
        Returns:
            Dict: Processed sold item data
        """
        # Extract listing info
        listing_info = item.get("listingInfo", {})
        selling_status = item.get("sellingStatus", [{}])[0]
        
        # Extract location
        location = item.get("location", "")
        country = item.get("country", "")
        
        # Build the processed item
        processed_item = {
            "itemId": item.get("itemId", ""),
            "title": item.get("title", [""])[0] if isinstance(item.get("title"), list) else item.get("title", ""),
            "condition": item.get("condition", {}).get("conditionDisplayName", ""),
            "conditionId": item.get("condition", {}).get("conditionId", ""),
            "price": float(selling_status.get("currentPrice", [{}])[0].get("__value__", 0)),
            "currency": selling_status.get("currentPrice", [{}])[0].get("@currencyId", "USD"),
            "category_id": item.get("primaryCategory", [{}])[0].get("categoryId", ""),
            "category_path": item.get("primaryCategory", [{}])[0].get("categoryName", ""),
            "listing_date": listing_info.get("startTime", ""),
            "end_date": listing_info.get("endTime", ""),
            "collection_date": datetime.now().isoformat(),
            "url": item.get("viewItemURL", [""])[0] if isinstance(item.get("viewItemURL"), list) else item.get("viewItemURL", ""),
            "item_location": f"{location}, {country}",
            "is_sold": True,
            "sold_date": listing_info.get("endTime", "")
        }
        
        return processed_item
    
    def _extract_price(self, price_data: Dict) -> float:
        """
        Extract price value from price object
        
        Args:
            price_data (Dict): Price object from API
            
        Returns:
            float: Price value
        """
        # The price structure might be different based on API version
        if isinstance(price_data, dict):
            value = price_data.get("value")
            if value is not None:
                try:
                    return float(value)
                except (ValueError, TypeError):
                    pass
        return 0.0
    
    def _extract_currency(self, price_data: Dict) -> str:
        """
        Extract currency from price object
        
        Args:
            price_data (Dict): Price object from API
            
        Returns:
            str: Currency code
        """
        if isinstance(price_data, dict):
            return price_data.get("currency", "USD")
        return "USD"
    
    def _extract_location(self, item_details: Dict) -> Dict:
        """
        Extract location information from item details
        
        Args:
            item_details (Dict): Item details
            
        Returns:
            Dict: Location information
        """
        location = {}
        
        # Try to extract from itemLocation
        item_location = item_details.get("itemLocation", {})
        if item_location:
            location["country"] = item_location.get("country", "")
            location["postal_code"] = item_location.get("postalCode", "")
            location["city"] = item_location.get("city", "")
            location["state_or_province"] = item_location.get("stateOrProvince", "")
        
        return location
    
    def _extract_shipping(self, item_details: Dict) -> List[Dict]:
        """
        Extract shipping options from item details
        
        Args:
            item_details (Dict): Item details
            
        Returns:
            List[Dict]: Shipping options
        """
        shipping_options = []
        
        # Extract from shippingOptions
        options = item_details.get("shippingOptions", [])
        for option in options:
            shipping_option = {
                "shipping_cost": self._extract_price(option.get("shippingCost", {})),
                "shipping_carrier": option.get("shippingServiceCode", ""),
                "shipping_type": option.get("type", "")
            }
            shipping_options.append(shipping_option)
        
        return shipping_options
    
    def _extract_item_specifics(self, item_details: Dict) -> Dict:
        """
        Extract item specifics (attributes) from item details
        
        Args:
            item_details (Dict): Item details
            
        Returns:
            Dict: Item specifics
        """
        specifics = {}
        
        # Extract from localizedAspects
        aspects = item_details.get("localizedAspects", [])
        for aspect in aspects:
            name = aspect.get("name", "")
            value = aspect.get("value", "")
            if name and value:
                specifics[name] = value
        
        return specifics
    
    def _extract_image_urls(self, item_summary: Dict, item_details: Dict) -> List[str]:
        """
        Extract image URLs from item data
        
        Args:
            item_summary (Dict): Item summary
            item_details (Dict): Item details
            
        Returns:
            List[str]: List of image URLs
        """
        image_urls = []
        
        # Try from item_summary first
        if "thumbnailImages" in item_summary:
            for image in item_summary.get("thumbnailImages", []):
                image_url = image.get("imageUrl", "")
                if image_url:
                    image_urls.append(image_url)
        
        # Try from item_details if no images found
        if not image_urls and "images" in item_details:
            for image in item_details.get("images", []):
                image_url = image.get("imageUrl", "")
                if image_url:
                    image_urls.append(image_url)
        
        return image_urls
    
    async def schedule_data_collection(self) -> None:
        """
        Schedule regular data collection
        """
        task_manager = ScheduledTaskManager()
        
        # Define the data collection task
        async def scheduled_collection():
            try:
                self.logger.info("Running scheduled data collection")
                await self.collect_data()
                self.logger.info("Scheduled data collection completed successfully")
            except Exception as e:
                self.logger.error(f"Error in scheduled data collection: {str(e)}")
        
        # Schedule the task to run every 'update_frequency' hours
        task_manager.schedule_interval_task(scheduled_collection, interval_hours=self.update_frequency)
        
        # Start the task manager
        task_manager.start()
        
        self.logger.info(f"Data collection scheduled to run every {self.update_frequency} hours")


# Example usage (commented out as we're implementing the module)
"""
async def main():
    collector = DataCollector()
    stats = await collector.collect_data()
    print(f"Data collection completed: {stats['items_collected']} items collected")
    
    # Schedule regular collection
    await collector.schedule_data_collection()

if __name__ == "__main__":
    asyncio.run(main())
"""

ModuleNotFoundError: No module named 'schedule'

In [4]:
from metagpt.tools.libs.terminal import Terminal
terminal = Terminal()
# Install the schedule package which is missing
await terminal.run('pip install schedule')

import os
import json
import pandas as pd
import asyncio
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Union, Any, Tuple
import time
import random
from tqdm import tqdm

from src.api.ebay_api_client import EbayAPIClient
from src.utils.logger import Logger
from src.utils.data_utils import generate_item_id, save_json, load_json, get_category_folder, save_items_to_csv, save_item_history
from src.utils.scheduled_task_manager import ScheduledTaskManager


class DataCollector:
    """
    Data collection module for retrieving second-hand item data from eBay
    """
    def __init__(self, config_path: str = "/data/chats/p6wyr/workspace/config/config.json"):
        """
        Initialize the data collector
        
        Args:
            config_path (str): Path to configuration file
        """
        self.logger = Logger().get_logger()
        self.logger.info("Initializing DataCollector module")
        
        # Load configuration
        self.config_path = config_path
        self._load_config()
        
        # Initialize eBay API client
        self.ebay_client = EbayAPIClient(config_path=config_path)
        
        # Set up data directories
        self.raw_data_dir = "/data/chats/p6wyr/workspace/data/raw"
        os.makedirs(self.raw_data_dir, exist_ok=True)
        
        # Track collection statistics
        self.stats = {
            "items_collected": 0,
            "categories_processed": 0,
            "errors": 0,
            "start_time": None,
            "end_time": None
        }
    
    def _load_config(self) -> None:
        """
        Load configuration from file
        """
        try:
            with open(self.config_path, 'r') as f:
                self.config = json.load(f)
            
            # Extract relevant configuration
            self.categories = self.config.get("data", {}).get("categories", [])
            self.sample_size_per_category = self.config.get("data", {}).get("sample_size_per_category", 100)
            self.history_window = self.config.get("data", {}).get("history_window", 30)
            self.update_frequency = self.config.get("data", {}).get("update_frequency", 24)
            
            self.logger.info(f"Configuration loaded: {len(self.categories)} categories, "
                             f"{self.sample_size_per_category} items per category")
        except Exception as e:
            self.logger.error(f"Error loading configuration: {str(e)}")
            # Use default values
            self.categories = [
                {"id": "9355", "name": "Laptops & Netbooks"},
                {"id": "15032", "name": "Cell Phones & Smartphones"}
            ]
            self.sample_size_per_category = 100
            self.history_window = 30
            self.update_frequency = 24
    
    async def collect_data(self) -> Dict:
        """
        Collect data for all configured categories
        
        Returns:
            Dict: Collection statistics
        """
        self.stats["start_time"] = datetime.now()
        self.logger.info(f"Starting data collection for {len(self.categories)} categories")
        
        for category in self.categories:
            try:
                await self.collect_category_data(category)
                self.stats["categories_processed"] += 1
            except Exception as e:
                self.logger.error(f"Error collecting data for category {category['name']}: {str(e)}")
                self.stats["errors"] += 1
        
        self.stats["end_time"] = datetime.now()
        collection_time = (self.stats["end_time"] - self.stats["start_time"]).total_seconds()
        self.logger.info(f"Data collection completed: {self.stats['items_collected']} items collected "
                         f"from {self.stats['categories_processed']} categories in {collection_time:.2f} seconds")
        
        # Save collection stats
        stats_file = os.path.join(self.raw_data_dir, f"collection_stats_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json")
        save_json(self.stats, stats_file)
        
        return self.stats
    
    async def collect_category_data(self, category: Dict) -> None:
        """
        Collect data for a specific category
        
        Args:
            category (Dict): Category information with id and name
        """
        category_id = category["id"]
        category_name = category["name"]
        self.logger.info(f"Collecting data for category: {category_name} (ID: {category_id})")
        
        # Create category folder
        category_folder = get_category_folder(category_id)
        os.makedirs(category_folder, exist_ok=True)
        
        # Define search terms for this category
        search_terms = self._get_search_terms_for_category(category)
        
        # Collect active listings
        active_items = await self._collect_active_items(category, search_terms)
        
        # Collect sold items for price history
        sold_items = await self._collect_sold_items(category, search_terms)
        
        # Save category data
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        active_file = os.path.join(category_folder, f"active_items_{timestamp}.csv")
        sold_file = os.path.join(category_folder, f"sold_items_{timestamp}.csv")
        
        save_items_to_csv(active_items, active_file)
        save_items_to_csv(sold_items, sold_file)
        
        # Create a metadata file to track the collection
        metadata = {
            "category_id": category_id,
            "category_name": category_name,
            "timestamp": datetime.now().isoformat(),
            "active_items_count": len(active_items),
            "sold_items_count": len(sold_items),
            "active_items_file": active_file,
            "sold_items_file": sold_file
        }
        
        metadata_file = os.path.join(category_folder, f"metadata_{timestamp}.json")
        save_json(metadata, metadata_file)
        
        self.logger.info(f"Collected {len(active_items)} active items and {len(sold_items)} sold items "
                         f"for category {category_name}")
        
        # Update stats
        self.stats["items_collected"] += len(active_items) + len(sold_items)
    
    def _get_search_terms_for_category(self, category: Dict) -> List[str]:
        """
        Get appropriate search terms for a category
        
        Args:
            category (Dict): Category information
            
        Returns:
            List[str]: List of search terms
        """
        category_name = category["name"]
        # This could be expanded with more specific search terms per category
        base_terms = [category_name, f"used {category_name}", f"second hand {category_name}"]
        
        # Add category-specific terms
        if category["id"] == "9355":  # Laptops
            return base_terms + ["refurbished laptop", "used notebook", "macbook", "thinkpad", "gaming laptop"]
        elif category["id"] == "15032":  # Cell Phones
            return base_terms + ["used iphone", "used samsung galaxy", "refurbished phone", "unlocked phone"]
        elif category["id"] == "11450":  # Watches
            return base_terms + ["used rolex", "used omega", "vintage watch", "pre-owned watch"]
        elif category["id"] == "261007":  # Cameras
            return base_terms + ["used dslr", "second hand mirrorless", "vintage camera", "used sony camera"]
        else:
            return base_terms
    
    async def _collect_active_items(self, category: Dict, search_terms: List[str]) -> List[Dict]:
        """
        Collect active listings for a category using multiple search terms
        
        Args:
            category (Dict): Category information
            search_terms (List[str]): List of search terms
            
        Returns:
            List[Dict]: Collected items data
        """
        category_id = category["id"]
        collected_items = []
        
        for search_term in tqdm(search_terms, desc=f"Collecting active items for {category['name']}"):
            try:
                # Make the API request with reasonable limits
                response = await self.ebay_client.search_items(
                    keywords=search_term,
                    category_id=category_id,
                    limit=min(50, self.sample_size_per_category // len(search_terms))
                )
                
                if not response.success:
                    self.logger.warning(f"Error searching for '{search_term}' in category {category_id}: {response.error}")
                    continue
                
                # Extract items from response
                items = response.data.get("itemSummaries", [])
                self.logger.info(f"Found {len(items)} items for search term '{search_term}'")
                
                # Process each item
                for item in items:
                    try:
                        # Generate a consistent item ID
                        item_id = item.get("itemId") or generate_item_id(item)
                        
                        # Get more detailed information about the item
                        details_response = await self.ebay_client.get_item_details(item_id, use_cache=True)
                        
                        if details_response.success:
                            item_details = details_response.data
                            
                            # Merge summary and details
                            processed_item = self._process_item_data(item, item_details)
                            collected_items.append(processed_item)
                            
                            # Add a small delay to avoid overwhelming the API
                            await asyncio.sleep(random.uniform(0.1, 0.5))
                    except Exception as e:
                        self.logger.error(f"Error processing item {item.get('itemId', 'unknown')}: {str(e)}")
                
                # Add a delay between search terms
                await asyncio.sleep(random.uniform(1.0, 2.0))
                
            except Exception as e:
                self.logger.error(f"Error collecting items for search term '{search_term}': {str(e)}")
                self.stats["errors"] += 1
        
        # Deduplicate items based on itemId
        unique_items = []
        seen_ids = set()
        for item in collected_items:
            if item["itemId"] not in seen_ids:
                unique_items.append(item)
                seen_ids.add(item["itemId"])
        
        self.logger.info(f"Collected {len(unique_items)} unique active items for category {category['name']}")
        return unique_items
    
    async def _collect_sold_items(self, category: Dict, search_terms: List[str]) -> List[Dict]:
        """
        Collect sold items data for price history
        
        Args:
            category (Dict): Category information
            search_terms (List[str]): List of search terms
            
        Returns:
            List[Dict]: Collected sold items data
        """
        category_id = category["id"]
        sold_items = []
        
        for search_term in tqdm(search_terms[:2], desc=f"Collecting sold items for {category['name']}"):
            try:
                # Make the API request for sold items
                response = await self.ebay_client.get_sold_items(
                    keywords=search_term,
                    category_id=category_id,
                    days_back=self.history_window,
                    limit=min(50, self.sample_size_per_category // len(search_terms))
                )
                
                if not response.success:
                    self.logger.warning(f"Error searching for sold items with '{search_term}' in category {category_id}: {response.error}")
                    continue
                
                # The structure of the sold items response is different
                items = []
                try:
                    # The response structure might vary depending on the API version
                    search_result = response.data.get("findCompletedItemsResponse", [{}])[0]
                    result_items = search_result.get("searchResult", [{}])[0].get("item", [])
                    items.extend(result_items)
                except (KeyError, IndexError, TypeError) as e:
                    self.logger.warning(f"Error parsing sold items response: {str(e)}")
                
                self.logger.info(f"Found {len(items)} sold items for search term '{search_term}'")
                
                # Process sold items
                for item in items:
                    try:
                        processed_item = self._process_sold_item_data(item)
                        sold_items.append(processed_item)
                    except Exception as e:
                        self.logger.error(f"Error processing sold item: {str(e)}")
                
                # Add a delay between search terms
                await asyncio.sleep(random.uniform(1.0, 2.0))
                
            except Exception as e:
                self.logger.error(f"Error collecting sold items for search term '{search_term}': {str(e)}")
                self.stats["errors"] += 1
        
        # Deduplicate sold items
        unique_sold_items = []
        seen_ids = set()
        for item in sold_items:
            if item["itemId"] not in seen_ids:
                unique_sold_items.append(item)
                seen_ids.add(item["itemId"])
        
        self.logger.info(f"Collected {len(unique_sold_items)} unique sold items for category {category['name']}")
        return unique_sold_items
    
    def _process_item_data(self, item_summary: Dict, item_details: Dict) -> Dict:
        """
        Process and normalize item data from API responses
        
        Args:
            item_summary (Dict): Item summary from search results
            item_details (Dict): Detailed item information
            
        Returns:
            Dict: Processed item data
        """
        # Extract the base information from summary
        processed_item = {
            "itemId": item_summary.get("itemId", ""),
            "title": item_summary.get("title", ""),
            "condition": item_summary.get("condition", ""),
            "conditionId": item_summary.get("conditionId", ""),
            "price": self._extract_price(item_summary.get("price", {})),
            "currency": self._extract_currency(item_summary.get("price", {})),
            "category_id": item_details.get("categoryId", ""),
            "category_path": item_details.get("categoryPath", ""),
            "listing_date": item_summary.get("itemCreationDate", ""),
            "end_date": item_summary.get("itemEndDate", ""),
            "collection_date": datetime.now().isoformat(),
            "url": item_summary.get("itemWebUrl", ""),
            "item_location": self._extract_location(item_details),
            "shipping_options": self._extract_shipping(item_details),
            "item_specifics": self._extract_item_specifics(item_details),
            "image_urls": self._extract_image_urls(item_summary, item_details)
        }
        
        return processed_item
    
    def _process_sold_item_data(self, item: Dict) -> Dict:
        """
        Process sold item data which has a different structure
        
        Args:
            item (Dict): Sold item data from API
            
        Returns:
            Dict: Processed sold item data
        """
        # Extract listing info
        listing_info = item.get("listingInfo", {})
        selling_status = item.get("sellingStatus", [{}])[0]
        
        # Extract location
        location = item.get("location", "")
        country = item.get("country", "")
        
        # Build the processed item
        processed_item = {
            "itemId": item.get("itemId", ""),
            "title": item.get("title", [""])[0] if isinstance(item.get("title"), list) else item.get("title", ""),
            "condition": item.get("condition", {}).get("conditionDisplayName", ""),
            "conditionId": item.get("condition", {}).get("conditionId", ""),
            "price": float(selling_status.get("currentPrice", [{}])[0].get("__value__", 0)),
            "currency": selling_status.get("currentPrice", [{}])[0].get("@currencyId", "USD"),
            "category_id": item.get("primaryCategory", [{}])[0].get("categoryId", ""),
            "category_path": item.get("primaryCategory", [{}])[0].get("categoryName", ""),
            "listing_date": listing_info.get("startTime", ""),
            "end_date": listing_info.get("endTime", ""),
            "collection_date": datetime.now().isoformat(),
            "url": item.get("viewItemURL", [""])[0] if isinstance(item.get("viewItemURL"), list) else item.get("viewItemURL", ""),
            "item_location": f"{location}, {country}",
            "is_sold": True,
            "sold_date": listing_info.get("endTime", "")
        }
        
        return processed_item
    
    def _extract_price(self, price_data: Dict) -> float:
        """
        Extract price value from price object
        
        Args:
            price_data (Dict): Price object from API
            
        Returns:
            float: Price value
        """
        # The price structure might be different based on API version
        if isinstance(price_data, dict):
            value = price_data.get("value")
            if value is not None:
                try:
                    return float(value)
                except (ValueError, TypeError):
                    pass
        return 0.0
    
    def _extract_currency(self, price_data: Dict) -> str:
        """
        Extract currency from price object
        
        Args:
            price_data (Dict): Price object from API
            
        Returns:
            str: Currency code
        """
        if isinstance(price_data, dict):
            return price_data.get("currency", "USD")
        return "USD"
    
    def _extract_location(self, item_details: Dict) -> Dict:
        """
        Extract location information from item details
        
        Args:
            item_details (Dict): Item details
            
        Returns:
            Dict: Location information
        """
        location = {}
        
        # Try to extract from itemLocation
        item_location = item_details.get("itemLocation", {})
        if item_location:
            location["country"] = item_location.get("country", "")
            location["postal_code"] = item_location.get("postalCode", "")
            location["city"] = item_location.get("city", "")
            location["state_or_province"] = item_location.get("stateOrProvince", "")
        
        return location
    
    def _extract_shipping(self, item_details: Dict) -> List[Dict]:
        """
        Extract shipping options from item details
        
        Args:
            item_details (Dict): Item details
            
        Returns:
            List[Dict]: Shipping options
        """
        shipping_options = []
        
        # Extract from shippingOptions
        options = item_details.get("shippingOptions", [])
        for option in options:
            shipping_option = {
                "shipping_cost": self._extract_price(option.get("shippingCost", {})),
                "shipping_carrier": option.get("shippingServiceCode", ""),
                "shipping_type": option.get("type", "")
            }
            shipping_options.append(shipping_option)
        
        return shipping_options
    
    def _extract_item_specifics(self, item_details: Dict) -> Dict:
        """
        Extract item specifics (attributes) from item details
        
        Args:
            item_details (Dict): Item details
            
        Returns:
            Dict: Item specifics
        """
        specifics = {}
        
        # Extract from localizedAspects
        aspects = item_details.get("localizedAspects", [])
        for aspect in aspects:
            name = aspect.get("name", "")
            value = aspect.get("value", "")
            if name and value:
                specifics[name] = value
        
        return specifics
    
    def _extract_image_urls(self, item_summary: Dict, item_details: Dict) -> List[str]:
        """
        Extract image URLs from item data
        
        Args:
            item_summary (Dict): Item summary
            item_details (Dict): Item details
            
        Returns:
            List[str]: List of image URLs
        """
        image_urls = []
        
        # Try from item_summary first
        if "thumbnailImages" in item_summary:
            for image in item_summary.get("thumbnailImages", []):
                image_url = image.get("imageUrl", "")
                if image_url:
                    image_urls.append(image_url)
        
        # Try from item_details if no images found
        if not image_urls and "images" in item_details:
            for image in item_details.get("images", []):
                image_url = image.get("imageUrl", "")
                if image_url:
                    image_urls.append(image_url)
        
        return image_urls
    
    async def schedule_data_collection(self) -> None:
        """
        Schedule regular data collection
        """
        task_manager = ScheduledTaskManager()
        
        # Define the data collection task
        async def scheduled_collection():
            try:
                self.logger.info("Running scheduled data collection")
                await self.collect_data()
                self.logger.info("Scheduled data collection completed successfully")
            except Exception as e:
                self.logger.error(f"Error in scheduled data collection: {str(e)}")
        
        # Schedule the task to run every 'update_frequency' hours
        task_manager.schedule_interval_task(scheduled_collection, interval_hours=self.update_frequency)
        
        # Start the task manager
        task_manager.start()
        
        self.logger.info(f"Data collection scheduled to run every {self.update_frequency} hours")


# Example usage (commented out as we're implementing the module)
"""
async def main():
    collector = DataCollector()
    stats = await collector.collect_data()
    print(f"Data collection completed: {stats['items_collected']} items collected")
    
    # Schedule regular collection
    await collector.schedule_data_collection()

if __name__ == "__main__":
    asyncio.run(main())
"""

ModuleNotFoundError: No module named 'schedule'

In [5]:
from metagpt.tools.libs.terminal import Terminal
terminal = Terminal()
# Install required package for future use
await terminal.run('pip install schedule')

import os
import json
import pandas as pd
import asyncio
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Union, Any, Tuple
import time
import random
from tqdm import tqdm

from src.api.ebay_api_client import EbayAPIClient
from src.utils.logger import Logger
from src.utils.data_utils import generate_item_id, save_json, load_json, get_category_folder, save_items_to_csv, save_item_history

# Temporarily removing this import to avoid the ModuleNotFoundError
# from src.utils.scheduled_task_manager import ScheduledTaskManager


class DataCollector:
    """
    Data collection module for retrieving second-hand item data from eBay
    """
    def __init__(self, config_path: str = "/data/chats/p6wyr/workspace/config/config.json"):
        """
        Initialize the data collector
        
        Args:
            config_path (str): Path to configuration file
        """
        self.logger = Logger().get_logger()
        self.logger.info("Initializing DataCollector module")
        
        # Load configuration
        self.config_path = config_path
        self._load_config()
        
        # Initialize eBay API client
        self.ebay_client = EbayAPIClient(config_path=config_path)
        
        # Set up data directories
        self.raw_data_dir = "/data/chats/p6wyr/workspace/data/raw"
        os.makedirs(self.raw_data_dir, exist_ok=True)
        
        # Track collection statistics
        self.stats = {
            "items_collected": 0,
            "categories_processed": 0,
            "errors": 0,
            "start_time": None,
            "end_time": None
        }
    
    def _load_config(self) -> None:
        """
        Load configuration from file
        """
        try:
            with open(self.config_path, 'r') as f:
                self.config = json.load(f)
            
            # Extract relevant configuration
            self.categories = self.config.get("data", {}).get("categories", [])
            self.sample_size_per_category = self.config.get("data", {}).get("sample_size_per_category", 100)
            self.history_window = self.config.get("data", {}).get("history_window", 30)
            self.update_frequency = self.config.get("data", {}).get("update_frequency", 24)
            
            self.logger.info(f"Configuration loaded: {len(self.categories)} categories, "
                             f"{self.sample_size_per_category} items per category")
        except Exception as e:
            self.logger.error(f"Error loading configuration: {str(e)}")
            # Use default values
            self.categories = [
                {"id": "9355", "name": "Laptops & Netbooks"},
                {"id": "15032", "name": "Cell Phones & Smartphones"}
            ]
            self.sample_size_per_category = 100
            self.history_window = 30
            self.update_frequency = 24
    
    async def collect_data(self) -> Dict:
        """
        Collect data for all configured categories
        
        Returns:
            Dict: Collection statistics
        """
        self.stats["start_time"] = datetime.now()
        self.logger.info(f"Starting data collection for {len(self.categories)} categories")
        
        for category in self.categories:
            try:
                await self.collect_category_data(category)
                self.stats["categories_processed"] += 1
            except Exception as e:
                self.logger.error(f"Error collecting data for category {category['name']}: {str(e)}")
                self.stats["errors"] += 1
        
        self.stats["end_time"] = datetime.now()
        collection_time = (self.stats["end_time"] - self.stats["start_time"]).total_seconds()
        self.logger.info(f"Data collection completed: {self.stats['items_collected']} items collected "
                         f"from {self.stats['categories_processed']} categories in {collection_time:.2f} seconds")
        
        # Save collection stats
        stats_file = os.path.join(self.raw_data_dir, f"collection_stats_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json")
        save_json(self.stats, stats_file)
        
        return self.stats
    
    async def collect_category_data(self, category: Dict) -> None:
        """
        Collect data for a specific category
        
        Args:
            category (Dict): Category information with id and name
        """
        category_id = category["id"]
        category_name = category["name"]
        self.logger.info(f"Collecting data for category: {category_name} (ID: {category_id})")
        
        # Create category folder
        category_folder = get_category_folder(category_id)
        os.makedirs(category_folder, exist_ok=True)
        
        # Define search terms for this category
        search_terms = self._get_search_terms_for_category(category)
        
        # Collect active listings
        active_items = await self._collect_active_items(category, search_terms)
        
        # Collect sold items for price history
        sold_items = await self._collect_sold_items(category, search_terms)
        
        # Save category data
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        active_file = os.path.join(category_folder, f"active_items_{timestamp}.csv")
        sold_file = os.path.join(category_folder, f"sold_items_{timestamp}.csv")
        
        save_items_to_csv(active_items, active_file)
        save_items_to_csv(sold_items, sold_file)
        
        # Create a metadata file to track the collection
        metadata = {
            "category_id": category_id,
            "category_name": category_name,
            "timestamp": datetime.now().isoformat(),
            "active_items_count": len(active_items),
            "sold_items_count": len(sold_items),
            "active_items_file": active_file,
            "sold_items_file": sold_file
        }
        
        metadata_file = os.path.join(category_folder, f"metadata_{timestamp}.json")
        save_json(metadata, metadata_file)
        
        self.logger.info(f"Collected {len(active_items)} active items and {len(sold_items)} sold items "
                         f"for category {category_name}")
        
        # Update stats
        self.stats["items_collected"] += len(active_items) + len(sold_items)
    
    def _get_search_terms_for_category(self, category: Dict) -> List[str]:
        """
        Get appropriate search terms for a category
        
        Args:
            category (Dict): Category information
            
        Returns:
            List[str]: List of search terms
        """
        category_name = category["name"]
        # This could be expanded with more specific search terms per category
        base_terms = [category_name, f"used {category_name}", f"second hand {category_name}"]
        
        # Add category-specific terms
        if category["id"] == "9355":  # Laptops
            return base_terms + ["refurbished laptop", "used notebook", "macbook", "thinkpad", "gaming laptop"]
        elif category["id"] == "15032":  # Cell Phones
            return base_terms + ["used iphone", "used samsung galaxy", "refurbished phone", "unlocked phone"]
        elif category["id"] == "11450":  # Watches
            return base_terms + ["used rolex", "used omega", "vintage watch", "pre-owned watch"]
        elif category["id"] == "261007":  # Cameras
            return base_terms + ["used dslr", "second hand mirrorless", "vintage camera", "used sony camera"]
        else:
            return base_terms
    
    async def _collect_active_items(self, category: Dict, search_terms: List[str]) -> List[Dict]:
        """
        Collect active listings for a category using multiple search terms
        
        Args:
            category (Dict): Category information
            search_terms (List[str]): List of search terms
            
        Returns:
            List[Dict]: Collected items data
        """
        category_id = category["id"]
        collected_items = []
        
        for search_term in tqdm(search_terms, desc=f"Collecting active items for {category['name']}"):
            try:
                # Make the API request with reasonable limits
                response = await self.ebay_client.search_items(
                    keywords=search_term,
                    category_id=category_id,
                    limit=min(50, self.sample_size_per_category // len(search_terms))
                )
                
                if not response.success:
                    self.logger.warning(f"Error searching for '{search_term}' in category {category_id}: {response.error}")
                    continue
                
                # Extract items from response
                items = response.data.get("itemSummaries", [])
                self.logger.info(f"Found {len(items)} items for search term '{search_term}'")
                
                # Process each item
                for item in items:
                    try:
                        # Generate a consistent item ID
                        item_id = item.get("itemId") or generate_item_id(item)
                        
                        # Get more detailed information about the item
                        details_response = await self.ebay_client.get_item_details(item_id, use_cache=True)
                        
                        if details_response.success:
                            item_details = details_response.data
                            
                            # Merge summary and details
                            processed_item = self._process_item_data(item, item_details)
                            collected_items.append(processed_item)
                            
                            # Add a small delay to avoid overwhelming the API
                            await asyncio.sleep(random.uniform(0.1, 0.5))
                    except Exception as e:
                        self.logger.error(f"Error processing item {item.get('itemId', 'unknown')}: {str(e)}")
                
                # Add a delay between search terms
                await asyncio.sleep(random.uniform(1.0, 2.0))
                
            except Exception as e:
                self.logger.error(f"Error collecting items for search term '{search_term}': {str(e)}")
                self.stats["errors"] += 1
        
        # Deduplicate items based on itemId
        unique_items = []
        seen_ids = set()
        for item in collected_items:
            if item["itemId"] not in seen_ids:
                unique_items.append(item)
                seen_ids.add(item["itemId"])
        
        self.logger.info(f"Collected {len(unique_items)} unique active items for category {category['name']}")
        return unique_items
    
    async def _collect_sold_items(self, category: Dict, search_terms: List[str]) -> List[Dict]:
        """
        Collect sold items data for price history
        
        Args:
            category (Dict): Category information
            search_terms (List[str]): List of search terms
            
        Returns:
            List[Dict]: Collected sold items data
        """
        category_id = category["id"]
        sold_items = []
        
        for search_term in tqdm(search_terms[:2], desc=f"Collecting sold items for {category['name']}"):
            try:
                # Make the API request for sold items
                response = await self.ebay_client.get_sold_items(
                    keywords=search_term,
                    category_id=category_id,
                    days_back=self.history_window,
                    limit=min(50, self.sample_size_per_category // len(search_terms))
                )
                
                if not response.success:
                    self.logger.warning(f"Error searching for sold items with '{search_term}' in category {category_id}: {response.error}")
                    continue
                
                # The structure of the sold items response is different
                items = []
                try:
                    # The response structure might vary depending on the API version
                    search_result = response.data.get("findCompletedItemsResponse", [{}])[0]
                    result_items = search_result.get("searchResult", [{}])[0].get("item", [])
                    items.extend(result_items)
                except (KeyError, IndexError, TypeError) as e:
                    self.logger.warning(f"Error parsing sold items response: {str(e)}")
                
                self.logger.info(f"Found {len(items)} sold items for search term '{search_term}'")
                
                # Process sold items
                for item in items:
                    try:
                        processed_item = self._process_sold_item_data(item)
                        sold_items.append(processed_item)
                    except Exception as e:
                        self.logger.error(f"Error processing sold item: {str(e)}")
                
                # Add a delay between search terms
                await asyncio.sleep(random.uniform(1.0, 2.0))
                
            except Exception as e:
                self.logger.error(f"Error collecting sold items for search term '{search_term}': {str(e)}")
                self.stats["errors"] += 1
        
        # Deduplicate sold items
        unique_sold_items = []
        seen_ids = set()
        for item in sold_items:
            if item["itemId"] not in seen_ids:
                unique_sold_items.append(item)
                seen_ids.add(item["itemId"])
        
        self.logger.info(f"Collected {len(unique_sold_items)} unique sold items for category {category['name']}")
        return unique_sold_items
    
    def _process_item_data(self, item_summary: Dict, item_details: Dict) -> Dict:
        """
        Process and normalize item data from API responses
        
        Args:
            item_summary (Dict): Item summary from search results
            item_details (Dict): Detailed item information
            
        Returns:
            Dict: Processed item data
        """
        # Extract the base information from summary
        processed_item = {
            "itemId": item_summary.get("itemId", ""),
            "title": item_summary.get("title", ""),
            "condition": item_summary.get("condition", ""),
            "conditionId": item_summary.get("conditionId", ""),
            "price": self._extract_price(item_summary.get("price", {})),
            "currency": self._extract_currency(item_summary.get("price", {})),
            "category_id": item_details.get("categoryId", ""),
            "category_path": item_details.get("categoryPath", ""),
            "listing_date": item_summary.get("itemCreationDate", ""),
            "end_date": item_summary.get("itemEndDate", ""),
            "collection_date": datetime.now().isoformat(),
            "url": item_summary.get("itemWebUrl", ""),
            "item_location": self._extract_location(item_details),
            "shipping_options": self._extract_shipping(item_details),
            "item_specifics": self._extract_item_specifics(item_details),
            "image_urls": self._extract_image_urls(item_summary, item_details)
        }
        
        return processed_item
    
    def _process_sold_item_data(self, item: Dict) -> Dict:
        """
        Process sold item data which has a different structure
        
        Args:
            item (Dict): Sold item data from API
            
        Returns:
            Dict: Processed sold item data
        """
        # Extract listing info
        listing_info = item.get("listingInfo", {})
        selling_status = item.get("sellingStatus", [{}])[0]
        
        # Extract location
        location = item.get("location", "")
        country = item.get("country", "")
        
        # Build the processed item
        processed_item = {
            "itemId": item.get("itemId", ""),
            "title": item.get("title", [""])[0] if isinstance(item.get("title"), list) else item.get("title", ""),
            "condition": item.get("condition", {}).get("conditionDisplayName", ""),
            "conditionId": item.get("condition", {}).get("conditionId", ""),
            "price": float(selling_status.get("currentPrice", [{}])[0].get("__value__", 0)),
            "currency": selling_status.get("currentPrice", [{}])[0].get("@currencyId", "USD"),
            "category_id": item.get("primaryCategory", [{}])[0].get("categoryId", ""),
            "category_path": item.get("primaryCategory", [{}])[0].get("categoryName", ""),
            "listing_date": listing_info.get("startTime", ""),
            "end_date": listing_info.get("endTime", ""),
            "collection_date": datetime.now().isoformat(),
            "url": item.get("viewItemURL", [""])[0] if isinstance(item.get("viewItemURL"), list) else item.get("viewItemURL", ""),
            "item_location": f"{location}, {country}",
            "is_sold": True,
            "sold_date": listing_info.get("endTime", "")
        }
        
        return processed_item
    
    def _extract_price(self, price_data: Dict) -> float:
        """
        Extract price value from price object
        
        Args:
            price_data (Dict): Price object from API
            
        Returns:
            float: Price value
        """
        # The price structure might be different based on API version
        if isinstance(price_data, dict):
            value = price_data.get("value")
            if value is not None:
                try:
                    return float(value)
                except (ValueError, TypeError):
                    pass
        return 0.0
    
    def _extract_currency(self, price_data: Dict) -> str:
        """
        Extract currency from price object
        
        Args:
            price_data (Dict): Price object from API
            
        Returns:
            str: Currency code
        """
        if isinstance(price_data, dict):
            return price_data.get("currency", "USD")
        return "USD"
    
    def _extract_location(self, item_details: Dict) -> Dict:
        """
        Extract location information from item details
        
        Args:
            item_details (Dict): Item details
            
        Returns:
            Dict: Location information
        """
        location = {}
        
        # Try to extract from itemLocation
        item_location = item_details.get("itemLocation", {})
        if item_location:
            location["country"] = item_location.get("country", "")
            location["postal_code"] = item_location.get("postalCode", "")
            location["city"] = item_location.get("city", "")
            location["state_or_province"] = item_location.get("stateOrProvince", "")
        
        return location
    
    def _extract_shipping(self, item_details: Dict) -> List[Dict]:
        """
        Extract shipping options from item details
        
        Args:
            item_details (Dict): Item details
            
        Returns:
            List[Dict]: Shipping options
        """
        shipping_options = []
        
        # Extract from shippingOptions
        options = item_details.get("shippingOptions", [])
        for option in options:
            shipping_option = {
                "shipping_cost": self._extract_price(option.get("shippingCost", {})),
                "shipping_carrier": option.get("shippingServiceCode", ""),
                "shipping_type": option.get("type", "")
            }
            shipping_options.append(shipping_option)
        
        return shipping_options
    
    def _extract_item_specifics(self, item_details: Dict) -> Dict:
        """
        Extract item specifics (attributes) from item details
        
        Args:
            item_details (Dict): Item details
            
        Returns:
            Dict: Item specifics
        """
        specifics = {}
        
        # Extract from localizedAspects
        aspects = item_details.get("localizedAspects", [])
        for aspect in aspects:
            name = aspect.get("name", "")
            value = aspect.get("value", "")
            if name and value:
                specifics[name] = value
        
        return specifics
    
    def _extract_image_urls(self, item_summary: Dict, item_details: Dict) -> List[str]:
        """
        Extract image URLs from item data
        
        Args:
            item_summary (Dict): Item summary
            item_details (Dict): Item details
            
        Returns:
            List[str]: List of image URLs
        """
        image_urls = []
        
        # Try from item_summary first
        if "thumbnailImages" in item_summary:
            for image in item_summary.get("thumbnailImages", []):
                image_url = image.get("imageUrl", "")
                if image_url:
                    image_urls.append(image_url)
        
        # Try from item_details if no images found
        if not image_urls and "images" in item_details:
            for image in item_details.get("images", []):
                image_url = image.get("imageUrl", "")
                if image_url:
                    image_urls.append(image_url)
        
        return image_urls
    
    # Simple scheduling function instead of using ScheduledTaskManager
    async def schedule_data_collection_simple(self, interval_hours: int = None) -> None:
        """
        Simple scheduling function for data collection
        
        Args:
            interval_hours (int): Interval between collections in hours
        """
        if interval_hours is None:
            interval_hours = self.update_frequency
            
        self.logger.info(f"Starting simple scheduler to collect data every {interval_hours} hours")
        
        while True:
            try:
                self.logger.info("Running scheduled data collection")
                await self.collect_data()
                self.logger.info(f"Data collection complete. Next collection in {interval_hours} hours")
            except Exception as e:
                self.logger.error(f"Error in scheduled data collection: {str(e)}")
            
            # Sleep for the specified interval
            await asyncio.sleep(interval_hours * 3600)  # Convert to seconds


# Example usage (commented out as we're implementing the module)
"""
async def main():
    collector = DataCollector()
    stats = await collector.collect_data()
    print(f"Data collection completed: {stats['items_collected']} items collected")
    
    # For continuous collection:
    # asyncio.create_task(collector.schedule_data_collection_simple())
    # await asyncio.sleep(86400)  # Run for a day

if __name__ == "__main__":
    asyncio.run(main())
"""

'\nasync def main():\n    collector = DataCollector()\n    stats = await collector.collect_data()\n    print(f"Data collection completed: {stats[\'items_collected\']} items collected")\n    \n    # For continuous collection:\n    # asyncio.create_task(collector.schedule_data_collection_simple())\n    # await asyncio.sleep(86400)  # Run for a day\n\nif __name__ == "__main__":\n    asyncio.run(main())\n'

In [6]:
I'll now implement the data preprocessor module and main application entry point as requested. This will integrate our previously created components into a cohesive system.

```python
from metagpt.tools.libs.terminal import Terminal
terminal = Terminal()

# First, let's implement the data preprocessor module
import os
import json
import pandas as pd
import numpy as np
from typing import Dict, List, Optional, Union, Any, Tuple
from datetime import datetime
import asyncio
from tqdm import tqdm
import logging

from src.utils.logger import Logger
from src.utils.data_utils import save_json, load_json, timestamp_to_features, normalize_price

class DataPreprocessor:
    """
    Module for preprocessing raw eBay data for the price prediction model
    """
    def __init__(self, config_path: str = "/data/chats/p6wyr/workspace/config/config.json"):
        """
        Initialize the data preprocessor
        
        Args:
            config_path (str): Path to configuration file
        """
        self.logger = Logger().get_logger()
        self.logger.info("Initializing DataPreprocessor module")
        
        # Load configuration
        self.config_path = config_path
        self._load_config()
        
        # Set up data directories
        self.raw_data_dir = "/data/chats/p6wyr/workspace/data/raw"
        self.processed_data_dir = "/data/chats/p6wyr/workspace/data/processed"
        self.features_data_dir = "/data/chats/p6wyr/workspace/data/features"
        
        os.makedirs(self.processed_data_dir, exist_ok=True)
        os.makedirs(self.features_data_dir, exist_ok=True)
        
        # Initialize cache for category statistics
        self.category_stats = {}
        
    def _load_config(self) -> None:
        """
        Load configuration from file
        """
        try:
            with open(self.config_path, 'r') as f:
                self.config = json.load(f)
            
            # Extract relevant configuration
            self.categories = self.config.get("data", {}).get("categories", [])
            self.min_items_for_training = self.config.get("data", {}).get("min_items_for_training", 500)
            
            self.model_config = self.config.get("model", {})
            self.state_features = self.model_config.get("state_features", [
                "item_condition", 
                "brand_popularity", 
                "season_demand", 
                "market_saturation",
                "avg_price", 
                "price_std", 
                "time_on_market",
                "conversion_rate"
            ])
            
            self.logger.info(f"Configuration loaded: {len(self.categories)} categories, "
                             f"{self.min_items_for_training} minimum items for training")
        except Exception as e:
            self.logger.error(f"Error loading configuration: {str(e)}")
            # Use default values
            self.categories = [
                {"id": "9355", "name": "Laptops & Netbooks"},
                {"id": "15032", "name": "Cell Phones & Smartphones"}
            ]
            self.min_items_for_training = 500
            self.state_features = [
                "item_condition", 
                "brand_popularity", 
                "season_demand", 
                "market_saturation",
                "avg_price", 
                "price_std", 
                "time_on_market",
                "conversion_rate"
            ]
    
    async def process_all_categories(self) -> Dict:
        """
        Process data for all configured categories
        
        Returns:
            Dict: Processing statistics
        """
        stats = {
            "categories_processed": 0,
            "items_processed": 0,
            "start_time": datetime.now(),
            "end_time": None
        }
        
        self.logger.info(f"Starting data preprocessing for {len(self.categories)} categories")
        
        for category in self.categories:
            try:
                category_stats = await self.process_category(category)
                stats["items_processed"] += category_stats.get("total_items", 0)
                stats["categories_processed"] += 1
            except Exception as e:
                self.logger.error(f"Error processing category {category['name']}: {str(e)}")
        
        stats["end_time"] = datetime.now()
        processing_time = (stats["end_time"] - stats["start_time"]).total_seconds()
        
        self.logger.info(f"Data preprocessing completed: {stats['items_processed']} items processed "
                         f"from {stats['categories_processed']} categories in {processing_time:.2f} seconds")
        
        # Save processing stats
        stats_file = os.path.join(self.processed_data_dir, 
                                 f"preprocessing_stats_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json")
        save_json(stats, stats_file)
        
        return stats
    
    async def process_category(self, category: Dict) -> Dict:
        """
        Process data for a specific category
        
        Args:
            category (Dict): Category information with id and name
            
        Returns:
            Dict: Category processing statistics
        """
        category_id = category["id"]
        category_name = category["name"]
        
        self.logger.info(f"Processing data for category: {category_name} (ID: {category_id})")
        
        # Find the latest data files for this category
        category_folder = os.path.join(self.raw_data_dir, f"category_{category_id}")
        
        if not os.path.exists(category_folder):
            self.logger.warning(f"No data folder found for category {category_name}")
            return {"category_id": category_id, "total_items": 0, "error": "No data folder found"}
        
        # Find the latest metadata file
        metadata_files = [f for f in os.listdir(category_folder) if f.startswith("metadata_")]
        
        if not metadata_files:
            self.logger.warning(f"No metadata files found for category {category_name}")
            return {"category_id": category_id, "total_items": 0, "error": "No metadata files found"}
        
        # Sort by timestamp (newest first)
        latest_metadata_file = sorted(metadata_files, reverse=True)[0]
        metadata_path = os.path.join(category_folder, latest_metadata_file)
        
        try:
            metadata = load_json(metadata_path)
            active_items_file = metadata.get("active_items_file")
            sold_items_file = metadata.get("sold_items_file")
            
            if not active_items_file or not sold_items_file:
                self.logger.warning(f"Missing data file paths in metadata for category {category_name}")
                return {"category_id": category_id, "total_items": 0, "error": "Missing data file paths"}
            
            # Load the data
            active_items_df = pd.read_csv(active_items_file)
            sold_items_df = pd.read_csv(sold_items_file)
            
            self.logger.info(f"Loaded {len(active_items_df)} active items and {len(sold_items_df)} sold items "
                            f"for category {category_name}")
            
            # Clean and normalize the data
            active_items_clean = self._clean_data(active_items_df)
            sold_items_clean = self._clean_data(sold_items_df)
            
            # Calculate category statistics for normalization
            category_stats = self._calculate_category_statistics(
                pd.concat([active_items_clean, sold_items_clean], ignore_index=True)
            )
            
            # Cache the category statistics for feature extraction
            self.category_stats[category_id] = category_stats
            
            # Apply basic transformations
            active_items_processed = self._apply_transformations(active_items_clean, category_stats)
            sold_items_processed = self._apply_transformations(sold_items_clean, category_stats)
            
            # Save processed data
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            processed_dir = os.path.join(self.processed_data_dir, f"category_{category_id}")
            os.makedirs(processed_dir, exist_ok=True)
            
            active_processed_file = os.path.join(processed_dir, f"active_processed_{timestamp}.csv")
            sold_processed_file = os.path.join(processed_dir, f"sold_processed_{timestamp}.csv")
            stats_file = os.path.join(processed_dir, f"category_stats_{timestamp}.json")
            
            active_items_processed.to_csv(active_processed_file, index=False)
            sold_items_processed.to_csv(sold_processed_file, index=False)
            save_json(category_stats, stats_file)
            
            # Create a metadata file for processed data
            processed_metadata = {
                "category_id": category_id,
                "category_name": category_name,
                "timestamp": datetime.now().isoformat(),
                "active_items_count": len(active_items_processed),
                "sold_items_count": len(sold_items_processed),
                "active_items_file": active_processed_file,
                "sold_items_file": sold_processed_file,
                "category_stats_file": stats_file,
                "raw_data_metadata_file": metadata_path
            }
            
            processed_metadata_file = os.path.join(processed_dir, f"processed_metadata_{timestamp}.json")
            save_json(processed_metadata, processed_metadata_file)
            
            self.logger.info(f"Saved processed data for category {category_name}: "
                            f"{len(active_items_processed)} active items, {len(sold_items_processed)} sold items")
            
            return {
                "category_id": category_id,
                "category_name": category_name,
                "total_items": len(active_items_processed) + len(sold_items_processed),
                "active_items": len(active_items_processed),
                "sold_items": len(sold_items_processed),
                "processed_metadata_file": processed_metadata_file
            }
            
        except Exception as e:
            self.logger.error(f"Error processing data for category {category_name}: {str(e)}")
            return {"category_id": category_id, "total_items": 0, "error": str(e)}
    
    def _clean_data(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Clean and validate the data
        
        Args:
            df (pd.DataFrame): Raw data frame
            
        Returns:
            pd.DataFrame: Cleaned data frame
        """
        # Make a copy to avoid modifying the original
        df_clean = df.copy()
        
        # Remove rows with missing critical values
        critical_columns = ["itemId", "title", "price"]
        df_clean = df_clean.dropna(subset=critical_columns)
        
        # Fix data types
        if "price" in df_clean.columns:
            df_clean["price"] = pd.to_numeric(df_clean["price"], errors="coerce")
            df_clean = df_clean[df_clean["price"] > 0]  # Remove items with invalid prices
        
        # Convert date strings to datetime objects
        date_columns = ["listing_date", "end_date", "collection_date", "sold_date"]
        for col in date_columns:
            if col in df_clean.columns:
                try:
                    df_clean[col] = pd.to_datetime(df_clean[col], errors="coerce")
                except Exception as e:
                    self.logger.warning(f"Error converting {col} to datetime: {str(e)}")
        
        # Handle item_specifics (assuming it's stored as a string representation of a dict)
        if "item_specifics" in df_clean.columns:
            try:
                df_clean["item_specifics"] = df_clean["item_specifics"].apply(
                    lambda x: json.loads(x) if isinstance(x, str) else x
                )
            except Exception as e:
                self.logger.warning(f"Error parsing item_specifics: {str(e)}")
                # If we can't parse, create an empty dict
                df_clean["item_specifics"] = df_clean["item_specifics"].apply(lambda x: {})
        
        return df_clean
    
    def _calculate_category_statistics(self, df: pd.DataFrame) -> Dict:
        """
        Calculate statistics for a category
        
        Args:
            df (pd.DataFrame): Data frame with items from a category
            
        Returns:
            Dict: Category statistics
        """
        stats = {}
        
        # Price statistics
        stats["price_mean"] = df["price"].mean()
        stats["price_median"] = df["price"].median()
        stats["price_std"] = df["price"].std()
        stats["price_min"] = df["price"].min()
        stats["price_max"] = df["price"].max()
        stats["price_q25"] = df["price"].quantile(0.25)
        stats["price_q75"] = df["price"].quantile(0.75)
        
        # Count by condition
        if "condition" in df.columns:
            condition_counts = df["condition"].value_counts().to_dict()
            stats["condition_counts"] = condition_counts
            # Calculate condition popularity scores (normalize to [0,1])
            total_items = len(df)
            condition_scores = {k: v / total_items for k, v in condition_counts.items()}
            stats["condition_scores"] = condition_scores
        
        # Extract and count brands (from item_specifics or title)
        brands = []
        if "item_specifics" in df.columns:
            for specifics in df["item_specifics"]:
                if isinstance(specifics, dict) and "Brand" in specifics:
                    brands.append(specifics["Brand"])
                elif isinstance(specifics, dict) and "brand" in specifics:
                    brands.append(specifics["brand"])
        
        # If we couldn't extract brands from item_specifics, try from the title
        if not brands and "title" in df.columns:
            # This is a simplistic approach - a more sophisticated model would use NER
            common_brands = ["Apple", "Samsung", "Sony", "Dell", "HP", "Lenovo", "Asus", 
                            "Acer", "Microsoft", "LG", "Toshiba", "IBM", "Rolex", "Omega",
                            "Canon", "Nikon", "Seiko", "Casio", "Motorola", "Google", "Nokia"]
            
            for title in df["title"]:
                found_brand = None
                for brand in common_brands:
                    if brand.lower() in title.lower():
                        found_brand = brand
                        break
                brands.append(found_brand)
        
        if brands:
            brand_counts = pd.Series(brands).value_counts().to_dict()
            stats["brand_counts"] = brand_counts
            # Calculate brand popularity scores (normalize to [0,1])
            total_brands = sum(brand_counts.values())
            brand_scores = {k: v / total_brands for k, v in brand_counts.items()}
            stats["brand_scores"] = brand_scores
        
        # Calculate time on market for sold items
        if "listing_date" in df.columns and "sold_date" in df.columns and "is_sold" in df.columns:
            sold_items = df[df["is_sold"] == True]
            if not sold_items.empty:
                try:
                    sold_items["time_on_market"] = (
                        sold_items["sold_date"] - sold_items["listing_date"]
                    ).dt.total_seconds() / (24 * 3600)  # Convert to days
                    
                    stats["time_on_market_mean"] = sold_items["time_on_market"].mean()
                    stats["time_on_market_median"] = sold_items["time_on_market"].median()
                    stats["time_on_market_std"] = sold_items["time_on_market"].std()
                except Exception as e:
                    self.logger.warning(f"Error calculating time on market: {str(e)}")
        
        return stats
    
    def _apply_transformations(self, df: pd.DataFrame, category_stats: Dict) -> pd.DataFrame:
        """
        Apply transformations to the data
        
        Args:
            df (pd.DataFrame): Clean data frame
            category_stats (Dict): Category statistics for normalization
            
        Returns:
            pd.DataFrame: Transformed data frame
        """
        df_transformed = df.copy()
        
        # Normalize price
        df_transformed["price_normalized"] = df_transformed["price"].apply(
            lambda x: (x - category_stats["price_mean"]) / (category_stats["price_std"] if category_stats["price_std"] > 0 else 1)
        )
        
        # Convert condition to numerical score
        if "condition" in df_transformed.columns and "condition_scores" in category_stats:
            df_transformed["condition_score"] = df_transformed["condition"].map(
                category_stats["condition_scores"]
            ).fillna(0.5)  # Default to middle score if unknown
        
        # Add brand popularity score
        if "item_specifics" in df_transformed.columns and "brand_scores" in category_stats:
            df_transformed["brand_score"] = df_transformed["item_specifics"].apply(
                lambda x: category_stats["brand_scores"].get(
                    x.get("Brand", x.get("brand", "")), 0
                )
            )
        
        # Extract temporal features
        if "listing_date" in df_transformed.columns:
            # Add listing time features
            df_transformed["listing_month"] = df_transformed["listing_date"].dt.month
            df_transformed["listing_day"] = df_transformed["listing_date"].dt.day
            df_transformed["listing_day_of_week"] = df_transformed["listing_date"].dt.dayofweek
            df_transformed["listing_is_weekend"] = df_transformed["listing_day_of_week"].apply(
                lambda x: 1 if x >= 5 else 0
            )
        
        # Add shipping cost to total price if available
        if "shipping_options" in df_transformed.columns:
            try:
                df_transformed["shipping_cost"] = df_transformed["shipping_options"].apply(
                    lambda x: min([option.get("shipping_cost", 0) for option in x]) if isinstance(x, list) and x else 0
                )
                df_transformed["total_price"] = df_transformed["price"] + df_transformed["shipping_cost"]
            except Exception as e:
                self.logger.warning(f"Error calculating shipping cost: {str(e)}")
                df_transformed["shipping_cost"] = 0
                df_transformed["total_price"] = df_transformed["price"]
        
        # Calculate time on market for active items (from listing to collection)
        if "listing_date" in df_transformed.columns and "collection_date" in df_transformed.columns:
            try:
                df_transformed["time_listed"] = (
                    df_transformed["collection_date"] - df_transformed["listing_date"]
                ).dt.total_seconds() / (24 * 3600)  # Convert to days
            except Exception as e:
                self.logger.warning(f"Error calculating time listed: {str(e)}")
        
        return df_transformed

    async def prepare_features_for_training(self) -> Dict:
        """
        Extract features from processed data for model training
        
        Returns:
            Dict: Feature extraction statistics
        """
        stats = {
            "categories_processed": 0,
            "items_processed": 0,
            "start_time": datetime.now(),
            "end_time": None
        }
        
        self.logger.info("Starting feature extraction for model training")
        
        # Process each category
        for category in self.categories:
            try:
                category_id = category["id"]
                category_name = category["name"]
                
                self.logger.info(f"Extracting features for category {category_name}")
                
                # Find the latest processed data
                processed_dir = os.path.join(self.processed_data_dir, f"category_{category_id}")
                
                if not os.path.exists(processed_dir):
                    self.logger.warning(f"No processed data directory found for category {category_name}")
                    continue
                
                # Find the latest metadata file
                metadata_files = [f for f in os.listdir(processed_dir) if f.startswith("processed_metadata_")]
                
                if not metadata_files:
                    self.logger.warning(f"No processed metadata files found for category {category_name}")
                    continue
                
                # Sort by timestamp (newest first)
                latest_metadata_file = sorted(metadata_files, reverse=True)[0]
                metadata_path = os.path.join(processed_dir, latest_metadata_file)
                
                # Load metadata and data files
                metadata = load_json(metadata_path)
                active_file = metadata.get("active_items_file")
                sold_file = metadata.get("sold_items_file")
                stats_file = metadata.get("category_stats_file")
                
                if not active_file or not sold_file or not stats_file:
                    self.logger.warning(f"Missing data files in metadata for category {category_name}")
                    continue
                
                active_df = pd.read_csv(active_file)
                sold_df = pd.read_csv(sold_file)
                category_stats = load_json(stats_file)
                
                # Extract features for active and sold items
                active_features = self._extract_features(active_df, category_stats, is_sold=False)
                sold_features = self._extract_features(sold_df, category_stats, is_sold=True)
                
                # Save features
                features_dir = os.path.join(self.features_data_dir, f"category_{category_id}")
                os.makedirs(features_dir, exist_ok=True)
                
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                active_features_file = os.path.join(features_dir, f"active_features_{timestamp}.csv")
                sold_features_file = os.path.join(features_dir, f"sold_features_{timestamp}.csv")
                
                active_features.to_csv(active_features_file, index=False)
                sold_features.to_csv(sold_features_file, index=False)
                
                # Create metadata for features
                features_metadata = {
                    "category_id": category_id,
                    "category_name": category_name,
                    "timestamp": datetime.now().isoformat(),
                    "active_features_count": len(active_features),
                    "sold_features_count": len(sold_features),
                    "active_features_file": active_features_file,
                    "sold_features_file": sold_features_file,
                    "processed_data_metadata": metadata_path
                }
                
                features_metadata_file = os.path.join(features_dir, f"features_metadata_{timestamp}.json")
                save_json(features_metadata, features_metadata_file)
                
                # Update statistics
                stats["categories_processed"] += 1
                stats["items_processed"] += len(active_features) + len(sold_features)
                
                self.logger.info(f"Extracted features for category {category_name}: "
                                f"{len(active_features)} active items, {len(sold_features)} sold items")
                
            except Exception as e:
                self.logger.error(f"Error extracting features for category {category.get('name', '')}: {str(e)}")
        
        stats["end_time"] = datetime.now()
        processing_time = (stats["end_time"] - stats["start_time"]).total_seconds()
        
        self.logger.info(f"Feature extraction completed: {stats['items_processed']} items processed "
                         f"from {stats['categories_processed']} categories in {processing_time:.2f} seconds")
        
        # Save extraction stats
        stats_file = os.path.join(self.features_data_dir, 
                                 f"feature_extraction_stats_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json")
        save_json(stats, stats_file)
        
        return stats
    
    def _extract_features(self, df: pd.DataFrame, category_stats: Dict, is_sold: bool = False) -> pd.DataFrame:
        """
        Extract features for model training
        
        Args:
            df (pd.DataFrame): Processed data frame
            category_stats (Dict): Category statistics
            is_sold (bool): Whether the items are sold
            
        Returns:
            pd.DataFrame: Data frame with extracted features
        """
        # Start with the base fields we want to keep
        features_df = df[["itemId", "price", "price_normalized", "condition_score"]].copy()
        
        # Add is_sold flag
        features_df["is_sold"] = int(is_sold)
        
        # Add time-related features if available
        if "time_listed" in df.columns:
            features_df["time_on_market"] = df["time_listed"]
        elif "time_on_market" in df.columns:
            features_df["time_on_market"] = df["time_on_market"]
        
        # Add listing time features if available
        temporal_features = ["listing_month", "listing_day", "listing_day_of_week", "listing_is_weekend"]
        for feature in temporal_features:
            if feature in df.columns:
                features_df[feature] = df[feature]
        
        # Add shipping and total price if available
        if "shipping_cost" in df.columns:
            features_df["shipping_cost"] = df["shipping_cost"]
        
        if "total_price" in df.columns:
            features_df["total_price"] = df["total_price"]
        
        # Add brand popularity if available
        if "brand_score" in df.columns:
            features_df["brand_popularity"] = df["brand_score"]
        
        # Add market saturation (relative count of similar items)
        # This is a simplified version - in a real system we'd use more sophisticated clustering
        if "condition" in df.columns:
            condition_counts = category_stats.get("condition_counts", {})
            features_df["market_saturation"] = df["condition"].apply(
                lambda x: condition_counts.get(x, 0) / sum(condition_counts.values()) 
                if condition_counts else 0.5
            )
        
        # Add season demand based on month (simplified)
        # In a real system, we would use historical seasonal trends
        if "listing_month" in features_df.columns:
            # Example demand pattern - higher demand in December (holidays), August (back to school), etc.
            season_demand = {
                1: 0.5,   # January
                2: 0.4,   # February
                3: 0.5,   # March
                4: 0.6,   # April
                5: 0.7,   # May
                6: 0.6,   # June
                7: 0.5,   # July
                8: 0.8,   # August (back to school)
                9: 0.7,   # September
                10: 0.6,  # October
                11: 0.8,  # November (Black Friday)
                12: 1.0   # December (holidays)
            }
            features_df["season_demand"] = features_df["listing_month"].map(season_demand)
        
        # Add category average and standard deviation
        features_df["avg_price"] = category_stats["price_mean"]
        features_df["price_std"] = category_stats["price_std"]
        
        # Add derived features
        features_df["price_to_avg_ratio"] = df["price"] / category_stats["price_mean"]
        
        # For sold items, we can calculate a conversion rate proxy
        # (in a real system, this would be based on view/sale data)
        if is_sold and "time_on_market" in features_df.columns:
            avg_time = category_stats.get("time_on_market_mean", 7)
            features_df["conversion_rate"] = 1.0 / (1.0 + features_df["time_on_market"] / avg_time)
        else:
            # For active items, use a default or estimated value
            features_df["conversion_rate"] = 0.5
        
        return features_df

# Now let's implement the feature extractor module
class FeatureExtractor:
    """
    Module for feature extraction from preprocessed data
    """
    def __init__(self, config_path: str = "/data/chats/p6wyr/workspace/config/config.json"):
        """
        Initialize the feature extractor
        
        Args:
            config_path (str): Path to configuration file
        """
        self.logger = Logger().get_logger()
        self.logger.info("Initializing FeatureExtractor module")
        
        # Load configuration
        self.config_path = config_path
        self._load_config()
        
        # Set up data directories
        self.processed_data_dir = "/data/chats/p6wyr/workspace/data/processed"
        self.features_data_dir = "/data/chats/p6wyr/workspace/data/features"
        
        os.makedirs(self.features_data_dir, exist_ok=True)
    
    def _load_config(self) -> None:
        """
        Load configuration from file
        """
        try:
            with open(self.config_path, 'r') as f:
                self.config = json.load(f)
            
            # Extract relevant configuration
            self.categories = self.config.get("data", {}).get("categories", [])
            
            self.model_config = self.config.get("model", {})
            self.state_features = self.model_config.get("state_features", [
                "item_condition", 
                "brand_popularity", 
                "season_demand", 
                "market_saturation",
                "avg_price", 
                "price_std", 
                "time_on_market",
                "conversion_rate"
            ])
            
            self.logger.info(f"Configuration loaded: {len(self.categories)} categories")
        except Exception as e:
            self.logger.error(f"Error loading configuration: {str(e)}")
            # Use default values
            self.categories = [
                {"id": "9355", "name": "Laptops & Netbooks"},
                {"id": "15032", "name": "Cell Phones & Smartphones"}
            ]
            self.state_features = [
                "item_condition", 
                "brand_popularity", 
                "season_demand", 
                "market_saturation",
                "avg_price", 
                "price_std", 
                "time_on_market",
                "conversion_rate"
            ]
    
    async def extract_features_for_item(self, item: Dict, category_id: str) -> Dict:
        """
        Extract features for a single item
        
        Args:
            item (Dict): Item data
            category_id (str): Category ID
            
        Returns:
            Dict: Extracted features
        """
        try:
            # Load the latest category statistics
            category_stats = self._get_latest_category_stats(category_id)
            
            if not category_stats:
                self.logger.warning(f"No category statistics found for category {category_id}")
                return {}
            
            # Create a DataFrame with the single item for consistent processing
            item_df = pd.DataFrame([item])
            
            # Basic preprocessing
            if "condition" in item and "condition_scores" in category_stats:
                condition_score = category_stats["condition_scores"].get(item["condition"], 0.5)
            

SyntaxError: unterminated string literal (detected at line 1) (2478879329.py, line 1)

In [7]:
[reflection on previous impl]
The error in my previous implementation is a SyntaxError caused by attempting to include a block of code with markdown formatting. The error message indicates an "unterminated string literal" at the beginning of the response, where I wrote "I'll now implement..." which is not valid Python code. 

The issue is that I formatted my response as if I were having a conversation, rather than directly providing the executable Python code that was requested. The proper approach is to simply provide the complete, valid Python code for the data preprocessor module and main application entry point without any conversational text.

[improved impl]:
```python
from metagpt.tools.libs.terminal import Terminal
terminal = Terminal()

# First, let's implement the data preprocessor module
import os
import json
import pandas as pd
import numpy as np
from typing import Dict, List, Optional, Union, Any, Tuple
from datetime import datetime
import asyncio
from tqdm import tqdm
import logging

from src.utils.logger import Logger
from src.utils.data_utils import save_json, load_json, timestamp_to_features, normalize_price

class DataPreprocessor:
    """
    Module for preprocessing raw eBay data for the price prediction model
    """
    def __init__(self, config_path: str = "/data/chats/p6wyr/workspace/config/config.json"):
        """
        Initialize the data preprocessor
        
        Args:
            config_path (str): Path to configuration file
        """
        self.logger = Logger().get_logger()
        self.logger.info("Initializing DataPreprocessor module")
        
        # Load configuration
        self.config_path = config_path
        self._load_config()
        
        # Set up data directories
        self.raw_data_dir = "/data/chats/p6wyr/workspace/data/raw"
        self.processed_data_dir = "/data/chats/p6wyr/workspace/data/processed"
        self.features_data_dir = "/data/chats/p6wyr/workspace/data/features"
        
        os.makedirs(self.processed_data_dir, exist_ok=True)
        os.makedirs(self.features_data_dir, exist_ok=True)
        
        # Initialize cache for category statistics
        self.category_stats = {}
        
    def _load_config(self) -> None:
        """
        Load configuration from file
        """
        try:
            with open(self.config_path, 'r') as f:
                self.config = json.load(f)
            
            # Extract relevant configuration
            self.categories = self.config.get("data", {}).get("categories", [])
            self.min_items_for_training = self.config.get("data", {}).get("min_items_for_training", 500)
            
            self.model_config = self.config.get("model", {})
            self.state_features = self.model_config.get("state_features", [
                "item_condition", 
                "brand_popularity", 
                "season_demand", 
                "market_saturation",
                "avg_price", 
                "price_std", 
                "time_on_market",
                "conversion_rate"
            ])
            
            self.logger.info(f"Configuration loaded: {len(self.categories)} categories, "
                             f"{self.min_items_for_training} minimum items for training")
        except Exception as e:
            self.logger.error(f"Error loading configuration: {str(e)}")
            # Use default values
            self.categories = [
                {"id": "9355", "name": "Laptops & Netbooks"},
                {"id": "15032", "name": "Cell Phones & Smartphones"}
            ]
            self.min_items_for_training = 500
            self.state_features = [
                "item_condition", 
                "brand_popularity", 
                "season_demand", 
                "market_saturation",
                "avg_price", 
                "price_std", 
                "time_on_market",
                "conversion_rate"
            ]
    
    async def process_all_categories(self) -> Dict:
        """
        Process data for all configured categories
        
        Returns:
            Dict: Processing statistics
        """
        stats = {
            "categories_processed": 0,
            "items_processed": 0,
            "start_time": datetime.now(),
            "end_time": None
        }
        
        self.logger.info(f"Starting data preprocessing for {len(self.categories)} categories")
        
        for category in self.categories:
            try:
                category_stats = await self.process_category(category)
                stats["items_processed"] += category_stats.get("total_items", 0)
                stats["categories_processed"] += 1
            except Exception as e:
                self.logger.error(f"Error processing category {category['name']}: {str(e)}")
        
        stats["end_time"] = datetime.now()
        processing_time = (stats["end_time"] - stats["start_time"]).total_seconds()
        
        self.logger.info(f"Data preprocessing completed: {stats['items_processed']} items processed "
                         f"from {stats['categories_processed']} categories in {processing_time:.2f} seconds")
        
        # Save processing stats
        stats_file = os.path.join(self.processed_data_dir, 
                                 f"preprocessing_stats_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json")
        save_json(stats, stats_file)
        
        return stats
    
    async def process_category(self, category: Dict) -> Dict:
        """
        Process data for a specific category
        
        Args:
            category (Dict): Category information with id and name
            
        Returns:
            Dict: Category processing statistics
        """
        category_id = category["id"]
        category_name = category["name"]
        
        self.logger.info(f"Processing data for category: {category_name} (ID: {category_id})")
        
        # Find the latest data files for this category
        category_folder = os.path.join(self.raw_data_dir, f"category_{category_id}")
        
        if not os.path.exists(category_folder):
            self.logger.warning(f"No data folder found for category {category_name}")
            return {"category_id": category_id, "total_items": 0, "error": "No data folder found"}
        
        # Find the latest metadata file
        metadata_files = [f for f in os.listdir(category_folder) if f.startswith("metadata_")]
        
        if not metadata_files:
            self.logger.warning(f"No metadata files found for category {category_name}")
            return {"category_id": category_id, "total_items": 0, "error": "No metadata files found"}
        
        # Sort by timestamp (newest first)
        latest_metadata_file = sorted(metadata_files, reverse=True)[0]
        metadata_path = os.path.join(category_folder, latest_metadata_file)
        
        try:
            metadata = load_json(metadata_path)
            active_items_file = metadata.get("active_items_file")
            sold_items_file = metadata.get("sold_items_file")
            
            if not active_items_file or not sold_items_file:
                self.logger.warning(f"Missing data file paths in metadata for category {category_name}")
                return {"category_id": category_id, "total_items": 0, "error": "Missing data file paths"}
            
            # Load the data
            active_items_df = pd.read_csv(active_items_file)
            sold_items_df = pd.read_csv(sold_items_file)
            
            self.logger.info(f"Loaded {len(active_items_df)} active items and {len(sold_items_df)} sold items "
                            f"for category {category_name}")
            
            # Clean and normalize the data
            active_items_clean = self._clean_data(active_items_df)
            sold_items_clean = self._clean_data(sold_items_df)
            
            # Calculate category statistics for normalization
            category_stats = self._calculate_category_statistics(
                pd.concat([active_items_clean, sold_items_clean], ignore_index=True)
            )
            
            # Cache the category statistics for feature extraction
            self.category_stats[category_id] = category_stats
            
            # Apply basic transformations
            active_items_processed = self._apply_transformations(active_items_clean, category_stats)
            sold_items_processed = self._apply_transformations(sold_items_clean, category_stats)
            
            # Save processed data
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            processed_dir = os.path.join(self.processed_data_dir, f"category_{category_id}")
            os.makedirs(processed_dir, exist_ok=True)
            
            active_processed_file = os.path.join(processed_dir, f"active_processed_{timestamp}.csv")
            sold_processed_file = os.path.join(processed_dir, f"sold_processed_{timestamp}.csv")
            stats_file = os.path.join(processed_dir, f"category_stats_{timestamp}.json")
            
            active_items_processed.to_csv(active_processed_file, index=False)
            sold_items_processed.to_csv(sold_processed_file, index=False)
            save_json(category_stats, stats_file)
            
            # Create a metadata file for processed data
            processed_metadata = {
                "category_id": category_id,
                "category_name": category_name,
                "timestamp": datetime.now().isoformat(),
                "active_items_count": len(active_items_processed),
                "sold_items_count": len(sold_items_processed),
                "active_items_file": active_processed_file,
                "sold_items_file": sold_processed_file,
                "category_stats_file": stats_file,
                "raw_data_metadata_file": metadata_path
            }
            
            processed_metadata_file = os.path.join(processed_dir, f"processed_metadata_{timestamp}.json")
            save_json(processed_metadata, processed_metadata_file)
            
            self.logger.info(f"Saved processed data for category {category_name}: "
                            f"{len(active_items_processed)} active items, {len(sold_items_processed)} sold items")
            
            return {
                "category_id": category_id,
                "category_name": category_name,
                "total_items": len(active_items_processed) + len(sold_items_processed),
                "active_items": len(active_items_processed),
                "sold_items": len(sold_items_processed),
                "processed_metadata_file": processed_metadata_file
            }
            
        except Exception as e:
            self.logger.error(f"Error processing data for category {category_name}: {str(e)}")
            return {"category_id": category_id, "total_items": 0, "error": str(e)}
    
    def _clean_data(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Clean and validate the data
        
        Args:
            df (pd.DataFrame): Raw data frame
            
        Returns:
            pd.DataFrame: Cleaned data frame
        """
        # Make a copy to avoid modifying the original
        df_clean = df.copy()
        
        # Remove rows with missing critical values
        critical_columns = ["itemId", "title", "price"]
        df_clean = df_clean.dropna(subset=critical_columns)
        
        # Fix data types
        if "price" in df_clean.columns:
            df_clean["price"] = pd.to_numeric(df_clean["price"], errors="coerce")
            df_clean = df_clean[df_clean["price"] > 0]  # Remove items with invalid prices
        
        # Convert date strings to datetime objects
        date_columns = ["listing_date", "end_date", "collection_date", "sold_date"]
        for col in date_columns:
            if col in df_clean.columns:
                try:
                    df_clean[col] = pd.to_datetime(df_clean[col], errors="coerce")
                except Exception as e:
                    self.logger.warning(f"Error converting {col} to datetime: {str(e)}")
        
        # Handle item_specifics (assuming it's stored as a string representation of a dict)
        if "item_specifics" in df_clean.columns:
            try:
                df_clean["item_specifics"] = df_clean["item_specifics"].apply(
                    lambda x: json.loads(x) if isinstance(x, str) else x
                )
            except Exception as e:
                self.logger.warning(f"Error parsing item_specifics: {str(e)}")
                # If we can't parse, create an empty dict
                df_clean["item_specifics"] = df_clean["item_specifics"].apply(lambda x: {})
        
        return df_clean
    
    def _calculate_category_statistics(self, df: pd.DataFrame) -> Dict:
        """
        Calculate statistics for a category
        
        Args:
            df (pd.DataFrame): Data frame with items from a category
            
        Returns:
            Dict: Category statistics
        """
        stats = {}
        
        # Price statistics
        stats["price_mean"] = df["price"].mean()
        stats["price_median"] = df["price"].median()
        stats["price_std"] = df["price"].std()
        stats["price_min"] = df["price"].min()
        stats["price_max"] = df["price"].max()
        stats["price_q25"] = df["price"].quantile(0.25)
        stats["price_q75"] = df["price"].quantile(0.75)
        
        # Count by condition
        if "condition" in df.columns:
            condition_counts = df["condition"].value_counts().to_dict()
            stats["condition_counts"] = condition_counts
            # Calculate condition popularity scores (normalize to [0,1])
            total_items = len(df)
            condition_scores = {k: v / total_items for k, v in condition_counts.items()}
            stats["condition_scores"] = condition_scores
        
        # Extract and count brands (from item_specifics or title)
        brands = []
        if "item_specifics" in df.columns:
            for specifics in df["item_specifics"]:
                if isinstance(specifics, dict) and "Brand" in specifics:
                    brands.append(specifics["Brand"])
                elif isinstance(specifics, dict) and "brand" in specifics:
                    brands.append(specifics["brand"])
        
        # If we couldn't extract brands from item_specifics, try from the title
        if not brands and "title" in df.columns:
            # This is a simplistic approach - a more sophisticated model would use NER
            common_brands = ["Apple", "Samsung", "Sony", "Dell", "HP", "Lenovo", "Asus", 
                            "Acer", "Microsoft", "LG", "Toshiba", "IBM", "Rolex", "Omega",
                            "Canon", "Nikon", "Seiko", "Casio", "Motorola", "Google", "Nokia"]
            
            for title in df["title"]:
                found_brand = None
                for brand in common_brands:
                    if brand.lower() in title.lower():
                        found_brand = brand
                        break
                brands.append(found_brand)
        
        if brands:
            brand_counts = pd.Series(brands).value_counts().to_dict()
            stats["brand_counts"] = brand_counts
            # Calculate brand popularity scores (normalize to [0,1])
            total_brands = sum(brand_counts.values())
            brand_scores = {k: v / total_brands for k, v in brand_counts.items()}
            stats["brand_scores"] = brand_scores
        
        # Calculate time on market for sold items
        if "listing_date" in df.columns and "sold_date" in df.columns and "is_sold" in df.columns:
            sold_items = df[df["is_sold"] == True]
            if not sold_items.empty:
                try:
                    sold_items["time_on_market"] = (
                        sold_items["sold_date"] - sold_items["listing_date"]
                    ).dt.total_seconds() / (24 * 3600)  # Convert to days
                    
                    stats["time_on_market_mean"] = sold_items["time_on_market"].mean()
                    stats["time_on_market_median"] = sold_items["time_on_market"].median()
                    stats["time_on_market_std"] = sold_items["time_on_market"].std()
                except Exception as e:
                    self.logger.warning(f"Error calculating time on market: {str(e)}")
        
        return stats
    
    def _apply_transformations(self, df: pd.DataFrame, category_stats: Dict) -> pd.DataFrame:
        """
        Apply transformations to the data
        
        Args:
            df (pd.DataFrame): Clean data frame
            category_stats (Dict): Category statistics for normalization
            
        Returns:
            pd.DataFrame: Transformed data frame
        """
        df_transformed = df.copy()
        
        # Normalize price
        df_transformed["price_normalized"] = df_transformed["price"].apply(
            lambda x: (x - category_stats["price_mean"]) / (category_stats["price_std"] if category_stats["price_std"] > 0 else 1)
        )
        
        # Convert condition to numerical score
        if "condition" in df_transformed.columns and "condition_scores" in category_stats:
            df_transformed["condition_score"] = df_transformed["condition"].map(
                category_stats["condition_scores"]
            ).fillna(0.5)  # Default to middle score if unknown
        
        # Add brand popularity score
        if "item_specifics" in df_transformed.columns and "brand_scores" in category_stats:
            df_transformed["brand_score"] = df_transformed["item_specifics"].apply(
                lambda x: category_stats["brand_scores"].get(
                    x.get("Brand", x.get("brand", "")), 0
                )
            )
        
        # Extract temporal features
        if "listing_date" in df_transformed.columns:
            # Add listing time features
            df_transformed["listing_month"] = df_transformed["listing_date"].dt.month
            df_transformed["listing_day"] = df_transformed["listing_date"].dt.day
            df_transformed["listing_day_of_week"] = df_transformed["listing_date"].dt.dayofweek
            df_transformed["listing_is_weekend"] = df_transformed["listing_day_of_week"].apply(
                lambda x: 1 if x >= 5 else 0
            )
        
        # Add shipping cost to total price if available
        if "shipping_options" in df_transformed.columns:
            try:
                df_transformed["shipping_cost"] = df_transformed["shipping_options"].apply(
                    lambda x: min([option.get("shipping_cost", 0) for option in x]) if isinstance(x, list) and x else 0
                )
                df_transformed["total_price"] = df_transformed["price"] + df_transformed["shipping_cost"]
            except Exception as e:
                self.logger.warning(f"Error calculating shipping cost: {str(e)}")
                df_transformed["shipping_cost"] = 0
                df_transformed["total_price"] = df_transformed["price"]
        
        # Calculate time on market for active items (from listing to collection)
        if "listing_date" in df_transformed.columns and "collection_date" in df_transformed.columns:
            try:
                df_transformed["time_listed"] = (
                    df_transformed["collection_date"] - df_transformed["listing_date"]
                ).dt.total_seconds() / (24 * 3600)  # Convert to days
            except Exception as e:
                self.logger.warning(f"Error calculating time listed: {str(e)}")
        
        return df_transformed

    async def prepare_features_for_training(self) -> Dict:
        """
        Extract features from processed data for model training
        
        Returns:
            Dict: Feature extraction statistics
        """
        stats = {
            "categories_processed": 0,
            "items_processed": 0,
            "start_time": datetime.now(),
            "end_time": None
        }
        
        self.logger.info("Starting feature extraction for model training")
        
        # Process each category
        for category in self.categories:
            try:
                category_id = category["id"]
                category_name = category["name"]
                
                self.logger.info(f"Extracting features for category {category_name}")
                
                # Find the latest processed data
                processed_dir = os.path.join(self.processed_data_dir, f"category_{category_id}")
                
                if not os.path.exists(processed_dir):
                    self.logger.warning(f"No processed data directory found for category {category_name}")
                    continue
                
                # Find the latest metadata file
                metadata_files = [f for f in os.listdir(processed_dir) if f.startswith("processed_metadata_")]
                
                if not metadata_files:
                    self.logger.warning(f"No processed metadata files found for category {category_name}")
                    continue
                
                # Sort by timestamp (newest first)
                latest_metadata_file = sorted(metadata_files, reverse=True)[0]
                metadata_path = os.path.join(processed_dir, latest_metadata_file)
                
                # Load metadata and data files
                metadata = load_json(metadata_path)
                active_file = metadata.get("active_items_file")
                sold_file = metadata.get("sold_items_file")
                stats_file = metadata.get("category_stats_file")
                
                if not active_file or not sold_file or not stats_file:
                    self.logger.warning(f"Missing data files in metadata for category {category_name}")
                    continue
                
                active_df = pd.read_csv(active_file)
                sold_df = pd.read_csv(sold_file)
                category_stats = load_json(stats_file)
                
                # Extract features for active and sold items
                active_features = self._extract_features(active_df, category_stats, is_sold=False)
                sold_features = self._extract_features(sold_df, category_stats, is_sold=True)
                
                # Save features
                features_dir = os.path.join(self.features_data_dir, f"category_{category_id}")
                os.makedirs(features_dir, exist_ok=True)
                
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                active_features_file = os.path.join(features_dir, f"active_features_{timestamp}.csv")
                sold_features_file = os.path.join(features_dir, f"sold_features_{timestamp}.csv")
                
                active_features.to_csv(active_features_file, index=False)
                sold_features.to_csv(sold_features_file, index=False)
                
                # Create metadata for features
                features_metadata = {
                    "category_id": category_id,
                    "category_name": category_name,
                    "timestamp": datetime.now().isoformat(),
                    "active_features_count": len(active_features),
                    "sold_features_count": len(sold_features),
                    "active_features_file": active_features_file,
                    "sold_features_file": sold_features_file,
                    "processed_data_metadata": metadata_path
                }
                
                features_metadata_file = os.path.join(features_dir, f"features_metadata_{timestamp}.json")
                save_json(features_metadata, features_metadata_file)
                
                # Update statistics
                stats["categories_processed"] += 1
                stats["items_processed"] += len(active_features) + len(sold_features)
                
                self.logger.info(f"Extracted features for category {category_name}: "
                                f"{len(active_features)} active items, {len(sold_features)} sold items")
                
            except Exception as e:
                self.logger.error(f"Error extracting features for category {category.get('name', '')}: {str(e)}")
        
        stats["end_time"] = datetime.now()
        processing_time = (stats["end_time"] - stats["start_time"]).total_seconds()
        
        self.logger.info(f"Feature extraction completed: {stats['items_processed']} items processed "
                         f"from {stats['categories_processed']} categories in {processing_time:.2f} seconds")
        
        # Save extraction stats
        stats_file = os.path.join(self.features_data_dir, 
                                 f"feature_extraction_stats_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json")
        save_json(stats, stats_file)
        
        return stats
    
    def _extract_features(self, df: pd.DataFrame, category_stats: Dict, is_sold: bool = False) -> pd.DataFrame:
        """
        Extract features for model training
        
        Args:
            df (pd.DataFrame): Processed data frame
            category_stats (Dict): Category statistics
            is_sold (bool): Whether the items are sold
            
        Returns:
            pd.DataFrame: Data frame with extracted features
        """
        # Start with the base fields we want to keep
        features_df = df[["itemId", "price", "price_normalized", "condition_score"]].copy()
        
        # Add is_sold flag
        features_df["is_sold"] = int(is_sold)
        
        # Add time-related features if available
        if "time_listed" in df.columns:
            features_df["time_on_market"] = df["time_listed"]
        elif "time_on_market" in df.columns:
            features_df["time_on_market"] = df["time_on_market"]
        
        # Add listing time features if available
        temporal_features = ["listing_month", "listing_day", "listing_day_of_week", "listing_is_weekend"]
        for feature in temporal_features:
            if feature in df.columns:
                features_df[feature] = df[feature]
        
        # Add shipping and total price if available
        if "shipping_cost" in df.columns:
            features_df["shipping_cost"] = df["shipping_cost"]
        
        if "total_price" in df.columns:
            features_df["total_price"] = df["total_price"]
        
        # Add brand popularity if available
        if "brand_score" in df.columns:
            features_df["brand_popularity"] = df["brand_score"]
        
        # Add market saturation (relative count of similar items)
        # This is a simplified version - in a real system we'd use more sophisticated clustering
        if "condition" in df.columns:
            condition_counts = category_stats.get("condition_counts", {})
            features_df["market_saturation"] = df["condition"].apply(
                lambda x: condition_counts.get(x, 0) / sum(condition_counts.values()) 
                if condition_counts else 0.5
            )
        
        # Add season demand based on month (simplified)
        # In a real system, we would use historical seasonal trends
        if "listing_month" in features_df.columns:
            # Example demand pattern - higher demand in December (holidays), August (back to school), etc.
            season_demand = {
                1: 0.5,   # January
                2: 0.4,   # February
                3: 0.5,   # March
                4: 0.6,   # April
                5: 0.7,   # May
                6: 0.6,   # June
                7: 0.5,   # July
                8: 0.8,   # August (back to school)
                9: 0.7,   # September
                10: 0.6,  # October
                11: 0.8,  # November (Black Friday)
                12: 1.0   # December (holidays)
            }
            features_df["season_demand"] = features_df["listing_month"].map(season_demand)
        
        # Add category average and standard deviation
        features_df["avg_price"] = category_stats["price_mean"]
        features_df["price_std"] = category_stats["price_std"]
        
        # Add derived features
        features_df["price_to_avg_ratio"] = df["price"] / category_stats["price_mean"]
        
        # For sold items, we can calculate a conversion rate proxy
        # (in a real system, this would be based on view/sale data)
        if is_sold and "time_on_market" in features_df.columns:
            avg_time = category_stats.get("time_on_market_mean", 7)
            features_df["conversion_rate"] = 1.0 / (1.0 + features_df["time_on_market"] / avg_time)
        else:
            # For active items, use a default or estimated value
            features_df["conversion_rate"] = 0.5
        
        return features_df

class FeatureExtractor:
    """
    Module for feature extraction from preprocessed data
    """
    def __init__(self, config_path: str = "/data/chats/p6wyr/workspace/config/config.json"):
        """
        Initialize the feature extractor
        
        Args:
            config_path (str): Path to configuration file
        """
        self.logger = Logger().get_logger()
        self.logger.info("Initializing FeatureExtractor module")
        
        # Load configuration
        self.config_path = config_path
        self._load_config()
        
        # Set up data directories
        self.processed_data_dir = "/data/chats/p6wyr/workspace/data/processed"
        self.features_data_dir = "/data/chats/p6wyr/workspace/data/features"
        
        os.makedirs(self.features_data_dir, exist_ok=True)
    
    def _load_config(self) -> None:
        """
        Load configuration from file
        """
        try:
            with open(self.config_path, 'r') as f:
                self.config = json.load(f)
            
            # Extract relevant configuration
            self.categories = self.config.get("data", {}).get("categories", [])
            
            self.model_config = self.config.get("model", {})
            self.state_features = self.model_config.get("state_features", [
                "item_condition", 
                "brand_popularity", 
                "season_demand", 
                "market_saturation",
                "avg_price", 
                "price_std", 
                "time_on_market",
                "conversion_rate"
            ])
            
            self.logger.info(f"Configuration loaded: {len(self.categories)} categories")
        except Exception as e:
            self.logger.error(f"Error loading configuration: {str(e)}")
            # Use default values
            self.categories = [
                {"id": "9355", "name": "Laptops & Netbooks"},
                {"id": "15032", "name": "Cell Phones & Smartphones"}
            ]
            self.state_features = [
                "item_condition", 
                "brand_popularity", 
                "season_demand", 
                "market_saturation",
                "avg_price", 
                "price_std", 
                "time_on_market",
                "conversion_rate"
            ]
    
    def _get_latest_category_stats(self, category_id: str) -> Dict:
        """
        Get the latest statistics for a category
        
        Args:
            category_id (str): Category ID
            
        Returns:
            Dict: Category statistics
        """
        # Check if we have cached statistics
        if category_id in self.category_stats:
            return self.category_stats[category_id]
        
        # Otherwise, load from file
        processed_dir = os.path.join(self.processed_data_dir, f"category_{

SyntaxError: unterminated string literal (detected at line 712) (120768559.py, line 712)

In [8]:
Cell In[7], line 712
    processed_dir = os.path.join(self.processed_data_dir, f"category_{
                                                          ^
SyntaxError: unterminated string literal (detected at line 712)

SyntaxError: unterminated string literal (detected at line 2) (856668197.py, line 2)

In [9]:
import os
import json
import pandas as pd
import numpy as np
from datetime import datetime
from typing import Dict, List, Optional, Union, Any, Tuple
import glob
from tqdm import tqdm

from src.utils.logger import Logger
from src.utils.data_utils import save_json, load_json, get_category_folder, save_items_to_csv

class DataPreprocessor:
    """
    Data preprocessing module for cleaning and standardizing collected eBay data
    """
    def __init__(self, config_path: str = "/data/chats/p6wyr/workspace/config/config.json"):
        """
        Initialize the data preprocessor
        
        Args:
            config_path (str): Path to configuration file
        """
        self.logger = Logger().get_logger()
        self.logger.info("Initializing DataPreprocessor module")
        
        # Load configuration
        self.config_path = config_path
        self._load_config()
        
        # Set up data directories
        self.raw_data_dir = "/data/chats/p6wyr/workspace/data/raw"
        self.processed_data_dir = "/data/chats/p6wyr/workspace/data/processed"
        os.makedirs(self.processed_data_dir, exist_ok=True)
        
        # Track processing statistics
        self.stats = {
            "items_processed": 0,
            "items_filtered_out": 0,
            "categories_processed": 0,
            "errors": 0,
            "start_time": None,
            "end_time": None
        }
        
        # Define normalization ranges
        self.condition_mapping = {
            "New": 1.0,
            "New with tags": 0.95,
            "New with box": 0.95,
            "New without tags": 0.9,
            "New other (see details)": 0.85,
            "New without box": 0.85,
            "Like New": 0.8,
            "Open box": 0.75,
            "Certified - Refurbished": 0.7,
            "Excellent - Refurbished": 0.65,
            "Excellent": 0.6,
            "Very Good": 0.5,
            "Good": 0.4,
            "Acceptable": 0.3,
            "For parts or not working": 0.1,
            "Unknown": 0.0
        }
    
    def _load_config(self) -> None:
        """
        Load configuration from file
        """
        try:
            with open(self.config_path, 'r') as f:
                self.config = json.load(f)
            
            # Extract relevant configuration
            self.categories = self.config.get("data", {}).get("categories", [])
            
            self.logger.info(f"Configuration loaded: {len(self.categories)} categories")
        except Exception as e:
            self.logger.error(f"Error loading configuration: {str(e)}")
            # Use default values
            self.categories = [
                {"id": "9355", "name": "Laptops & Netbooks"},
                {"id": "15032", "name": "Cell Phones & Smartphones"}
            ]
    
    def process_all_data(self) -> Dict:
        """
        Process all collected data in the raw data directory
        
        Returns:
            Dict: Processing statistics
        """
        self.stats["start_time"] = datetime.now()
        self.logger.info("Starting data preprocessing for all categories")
        
        for category in self.categories:
            try:
                self.process_category_data(category)
                self.stats["categories_processed"] += 1
            except Exception as e:
                self.logger.error(f"Error preprocessing data for category {category['name']}: {str(e)}")
                self.stats["errors"] += 1
        
        self.stats["end_time"] = datetime.now()
        processing_time = (self.stats["end_time"] - self.stats["start_time"]).total_seconds()
        self.logger.info(f"Data preprocessing completed: {self.stats['items_processed']} items processed "
                        f"({self.stats['items_filtered_out']} filtered out) "
                        f"from {self.stats['categories_processed']} categories "
                        f"in {processing_time:.2f} seconds")
        
        # Save processing stats
        stats_file = os.path.join(self.processed_data_dir, f"preprocessing_stats_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json")
        save_json(self.stats, stats_file)
        
        return self.stats
    
    def process_category_data(self, category: Dict) -> None:
        """
        Process data for a specific category
        
        Args:
            category (Dict): Category information with id and name
        """
        category_id = category["id"]
        category_name = category["name"]
        self.logger.info(f"Processing data for category: {category_name} (ID: {category_id})")
        
        # Create category folder in processed data directory
        processed_dir = os.path.join(self.processed_data_dir, f"category_{category_id}")
        os.makedirs(processed_dir, exist_ok=True)
        
        # Get raw data directory for this category
        raw_category_dir = get_category_folder(category_id)
        
        # Find all active and sold item CSV files
        active_files = glob.glob(os.path.join(raw_category_dir, "active_items_*.csv"))
        sold_files = glob.glob(os.path.join(raw_category_dir, "sold_items_*.csv"))
        
        # Process active items
        active_items_df = self._load_and_combine_files(active_files)
        if active_items_df is not None:
            processed_active_df = self._preprocess_items(active_items_df, is_sold=False)
            
            # Save processed active items
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            processed_active_file = os.path.join(processed_dir, f"processed_active_items_{timestamp}.csv")
            processed_active_df.to_csv(processed_active_file, index=False)
            self.logger.info(f"Saved {len(processed_active_df)} processed active items to {processed_active_file}")
        
        # Process sold items
        sold_items_df = self._load_and_combine_files(sold_files)
        if sold_items_df is not None:
            processed_sold_df = self._preprocess_items(sold_items_df, is_sold=True)
            
            # Save processed sold items
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            processed_sold_file = os.path.join(processed_dir, f"processed_sold_items_{timestamp}.csv")
            processed_sold_df.to_csv(processed_sold_file, index=False)
            self.logger.info(f"Saved {len(processed_sold_df)} processed sold items to {processed_sold_file}")
        
        # Calculate and save category statistics
        category_stats = self._calculate_category_stats(processed_active_df, processed_sold_df)
        stats_file = os.path.join(processed_dir, f"category_stats_{timestamp}.json")
        save_json(category_stats, stats_file)
        self.logger.info(f"Saved category statistics to {stats_file}")
        
        # Create a metadata file to track the processing
        metadata = {
            "category_id": category_id,
            "category_name": category_name,
            "timestamp": datetime.now().isoformat(),
            "active_items_count": len(processed_active_df) if processed_active_df is not None else 0,
            "sold_items_count": len(processed_sold_df) if processed_sold_df is not None else 0,
            "active_items_file": processed_active_file if 'processed_active_file' in locals() else None,
            "sold_items_file": processed_sold_file if 'processed_sold_file' in locals() else None,
            "stats_file": stats_file
        }
        
        metadata_file = os.path.join(processed_dir, f"metadata_{timestamp}.json")
        save_json(metadata, metadata_file)
    
    def _load_and_combine_files(self, file_paths: List[str]) -> Optional[pd.DataFrame]:
        """
        Load and combine multiple CSV files into a single DataFrame
        
        Args:
            file_paths (List[str]): List of file paths
            
        Returns:
            Optional[pd.DataFrame]: Combined DataFrame or None if no files
        """
        if not file_paths:
            return None
        
        dfs = []
        for file_path in file_paths:
            try:
                df = pd.read_csv(file_path)
                dfs.append(df)
                self.logger.info(f"Loaded {len(df)} items from {file_path}")
            except Exception as e:
                self.logger.error(f"Error loading file {file_path}: {str(e)}")
                self.stats["errors"] += 1
        
        if not dfs:
            return None
        
        # Combine all DataFrames
        combined_df = pd.concat(dfs, ignore_index=True)
        
        # Remove duplicates
        original_count = len(combined_df)
        combined_df = combined_df.drop_duplicates(subset=['itemId'])
        duplicate_count = original_count - len(combined_df)
        if duplicate_count > 0:
            self.logger.info(f"Removed {duplicate_count} duplicate items")
        
        return combined_df
    
    def _preprocess_items(self, df: pd.DataFrame, is_sold: bool = False) -> pd.DataFrame:
        """
        Preprocess items data
        
        Args:
            df (pd.DataFrame): DataFrame with items data
            is_sold (bool): Whether the items are sold items
            
        Returns:
            pd.DataFrame: Processed DataFrame
        """
        self.logger.info(f"Preprocessing {len(df)} {'sold' if is_sold else 'active'} items")
        
        # Make a copy to avoid modifying the original
        processed_df = df.copy()
        
        # Filter out items with missing critical data
        original_count = len(processed_df)
        processed_df = processed_df.dropna(subset=['itemId', 'price'])
        filtered_count = original_count - len(processed_df)
        self.stats["items_filtered_out"] += filtered_count
        
        if len(processed_df) == 0:
            self.logger.warning("No items left after filtering out missing critical data")
            return processed_df
        
        # Standardize date fields
        date_columns = ['listing_date', 'end_date', 'collection_date']
        if is_sold:
            date_columns.append('sold_date')
        
        for col in date_columns:
            if col in processed_df.columns:
                processed_df[col] = pd.to_datetime(processed_df[col], errors='coerce')
        
        # Calculate listing duration
        if 'listing_date' in processed_df.columns and 'end_date' in processed_df.columns:
            # Calculate listing duration in days
            processed_df['listing_duration_days'] = (processed_df['end_date'] - processed_df['listing_date']).dt.total_seconds() / (24 * 3600)
            
            # Replace negative or extreme values with NaN
            processed_df.loc[processed_df['listing_duration_days'] < 0, 'listing_duration_days'] = np.nan
            processed_df.loc[processed_df['listing_duration_days'] > 365, 'listing_duration_days'] = np.nan
        
        # Normalize condition
        if 'condition' in processed_df.columns:
            processed_df['condition_normalized'] = processed_df['condition'].map(
                lambda x: self.condition_mapping.get(str(x), 0.0) if pd.notna(x) else 0.0
            )
        
        # Extract item specifics into separate columns
        if 'item_specifics' in processed_df.columns:
            # Convert string representation of dict to actual dict
            processed_df['item_specifics'] = processed_df['item_specifics'].apply(
                lambda x: {} if pd.isna(x) else (json.loads(x) if isinstance(x, str) else x)
            )
            
            # Extract common specifics
            common_specifics = ['Brand', 'Model', 'Storage Capacity', 'Screen Size', 'Color', 'RAM']
            for specific in common_specifics:
                processed_df[f'specific_{specific.lower().replace(" ", "_")}'] = processed_df['item_specifics'].apply(
                    lambda x: x.get(specific, np.nan) if isinstance(x, dict) else np.nan
                )
        
        # Process location data
        if 'item_location' in processed_df.columns:
            # If item_location is a string that looks like a dictionary, convert it
            processed_df['item_location'] = processed_df['item_location'].apply(
                lambda x: json.loads(x) if isinstance(x, str) and x.startswith('{') else x
            )
            
            # Extract country information
            processed_df['country'] = processed_df['item_location'].apply(
                lambda x: x.get('country', np.nan) if isinstance(x, dict) else 
                           (x.split(',')[-1].strip() if isinstance(x, str) else np.nan)
            )
        
        # Convert string representation of lists to actual lists
        list_columns = ['shipping_options', 'image_urls']
        for col in list_columns:
            if col in processed_df.columns:
                processed_df[col] = processed_df[col].apply(
                    lambda x: [] if pd.isna(x) else (json.loads(x) if isinstance(x, str) else x)
                )
        
        # Extract shipping cost
        if 'shipping_options' in processed_df.columns:
            processed_df['shipping_cost'] = processed_df['shipping_options'].apply(
                lambda x: self._get_min_shipping_cost(x) if isinstance(x, list) else np.nan
            )
        
        # Calculate total price (item price + shipping)
        if 'price' in processed_df.columns and 'shipping_cost' in processed_df.columns:
            processed_df['total_price'] = processed_df['price'] + processed_df['shipping_cost'].fillna(0)
        
        # Add a flag for sold items
        processed_df['is_sold'] = is_sold
        
        # Handle image count
        if 'image_urls' in processed_df.columns:
            processed_df['image_count'] = processed_df['image_urls'].apply(
                lambda x: len(x) if isinstance(x, list) else 0
            )
        
        # Add has_image flag
        if 'image_count' in processed_df.columns:
            processed_df['has_image'] = processed_df['image_count'] > 0
        
        # Extract title length as a feature
        if 'title' in processed_df.columns:
            processed_df['title_length'] = processed_df['title'].apply(
                lambda x: len(str(x)) if pd.notna(x) else 0
            )
        
        # Update stats
        self.stats["items_processed"] += len(processed_df)
        
        return processed_df
    
    def _get_min_shipping_cost(self, shipping_options: List[Dict]) -> float:
        """
        Get minimum shipping cost from shipping options
        
        Args:
            shipping_options (List[Dict]): Shipping options
            
        Returns:
            float: Minimum shipping cost
        """
        if not shipping_options:
            return 0.0
        
        costs = []
        for option in shipping_options:
            if isinstance(option, dict) and 'shipping_cost' in option:
                cost = option['shipping_cost']
                if isinstance(cost, (int, float)):
                    costs.append(cost)
        
        if costs:
            return min(costs)
        return 0.0
    
    def _calculate_category_stats(self, active_df: Optional[pd.DataFrame], sold_df: Optional[pd.DataFrame]) -> Dict:
        """
        Calculate statistics for a category
        
        Args:
            active_df (Optional[pd.DataFrame]): DataFrame with active items
            sold_df (Optional[pd.DataFrame]): DataFrame with sold items
            
        Returns:
            Dict: Category statistics
        """
        stats = {}
        
        # Combine active and sold dataframes for overall stats
        dfs = []
        if active_df is not None and len(active_df) > 0:
            dfs.append(active_df)
        if sold_df is not None and len(sold_df) > 0:
            dfs.append(sold_df)
        
        if not dfs:
            return stats
        
        combined_df = pd.concat(dfs, ignore_index=True)
        
        # Basic price statistics
        if 'price' in combined_df.columns:
            price_stats = combined_df['price'].describe().to_dict()
            stats['price_mean'] = price_stats.get('mean', 0)
            stats['price_std'] = price_stats.get('std', 0)
            stats['price_min'] = price_stats.get('min', 0)
            stats['price_max'] = price_stats.get('max', 0)
            stats['price_25%'] = price_stats.get('25%', 0)
            stats['price_50%'] = price_stats.get('50%', 0)
            stats['price_75%'] = price_stats.get('75%', 0)
        
        # Total price statistics
        if 'total_price' in combined_df.columns:
            total_price_stats = combined_df['total_price'].describe().to_dict()
            stats['total_price_mean'] = total_price_stats.get('mean', 0)
            stats['total_price_std'] = total_price_stats.get('std', 0)
        
        # Condition distribution
        if 'condition_normalized' in combined_df.columns:
            condition_stats = combined_df['condition_normalized'].describe().to_dict()
            stats['condition_mean'] = condition_stats.get('mean', 0)
            stats['condition_std'] = condition_stats.get('std', 0)
        
        # Listing duration statistics
        if 'listing_duration_days' in combined_df.columns:
            duration_stats = combined_df['listing_duration_days'].describe().to_dict()
            stats['duration_mean'] = duration_stats.get('mean', 0)
            stats['duration_std'] = duration_stats.get('std', 0)
        
        # Item count statistics
        stats['total_items'] = len(combined_df)
        stats['active_items'] = len(active_df) if active_df is not None else 0
        stats['sold_items'] = len(sold_df) if sold_df is not None else 0
        
        # Sold percentage
        if stats['total_items'] > 0:
            stats['sold_percentage'] = (stats['sold_items'] / stats['total_items']) * 100
        else:
            stats['sold_percentage'] = 0
        
        # Price difference between sold and active
        if active_df is not None and sold_df is not None and 'price' in active_df.columns and 'price' in sold_df.columns:
            active_mean = active_df['price'].mean()
            sold_mean = sold_df['price'].mean()
            if not np.isnan(active_mean) and not np.isnan(sold_mean):
                stats['price_diff_sold_active'] = sold_mean - active_mean
                stats['price_ratio_sold_active'] = sold_mean / active_mean if active_mean > 0 else 1.0
        
        # Top brands
        if 'specific_brand' in combined_df.columns:
            brand_counts = combined_df['specific_brand'].value_counts().head(10).to_dict()
            stats['top_brands'] = brand_counts
        
        # Date range
        if 'collection_date' in combined_df.columns:
            stats['earliest_collection'] = combined_df['collection_date'].min().isoformat() if not pd.isna(combined_df['collection_date'].min()) else None
            stats['latest_collection'] = combined_df['collection_date'].max().isoformat() if not pd.isna(combined_df['collection_date'].max()) else None
        
        # Image statistics
        if 'image_count' in combined_df.columns:
            image_stats = combined_df['image_count'].describe().to_dict()
            stats['image_count_mean'] = image_stats.get('mean', 0)
            stats['has_image_percentage'] = (combined_df['has_image'].sum() / len(combined_df)) * 100 if 'has_image' in combined_df.columns else 0
        
        return stats

# Example usage code (commented out)
"""
def main():
    preprocessor = DataPreprocessor()
    stats = preprocessor.process_all_data()
    print(f"Data preprocessing completed: {stats['items_processed']} items processed")

if __name__ == "__main__":
    main()
"""

'\ndef main():\n    preprocessor = DataPreprocessor()\n    stats = preprocessor.process_all_data()\n    print(f"Data preprocessing completed: {stats[\'items_processed\']} items processed")\n\nif __name__ == "__main__":\n    main()\n'

In [10]:
import os
import json
import pandas as pd
import numpy as np
from datetime import datetime
from typing import Dict, List, Optional, Union, Any, Tuple
import glob
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
import pickle

from src.utils.logger import Logger
from src.utils.data_utils import save_json, load_json

class FeatureExtractor:
    """
    Feature extraction module for preparing data for the DRL model
    """
    def __init__(self, config_path: str = "/data/chats/p6wyr/workspace/config/config.json"):
        """
        Initialize the feature extractor
        
        Args:
            config_path (str): Path to configuration file
        """
        self.logger = Logger().get_logger()
        self.logger.info("Initializing FeatureExtractor module")
        
        # Load configuration
        self.config_path = config_path
        self._load_config()
        
        # Set up data directories
        self.processed_data_dir = "/data/chats/p6wyr/workspace/data/processed"
        self.features_dir = "/data/chats/p6wyr/workspace/data/features"
        os.makedirs(self.features_dir, exist_ok=True)
        
        # Track extraction statistics
        self.stats = {
            "features_extracted": 0,
            "categories_processed": 0,
            "errors": 0,
            "start_time": None,
            "end_time": None
        }
        
        # Initialize scalers
        self.scalers = {}
        self.pca_models = {}
        
    def _load_config(self) -> None:
        """
        Load configuration from file
        """
        try:
            with open(self.config_path, 'r') as f:
                self.config = json.load(f)
            
            # Extract relevant configuration
            self.categories = self.config.get("data", {}).get("categories", [])
            self.state_features = self.config.get("model", {}).get("state_features", [
                "item_condition", 
                "brand_popularity", 
                "season_demand", 
                "market_saturation",
                "avg_price", 
                "price_std", 
                "time_on_market",
                "conversion_rate"
            ])
            
            self.logger.info(f"Configuration loaded: {len(self.categories)} categories, "
                            f"{len(self.state_features)} state features")
        except Exception as e:
            self.logger.error(f"Error loading configuration: {str(e)}")
            # Use default values
            self.categories = [
                {"id": "9355", "name": "Laptops & Netbooks"},
                {"id": "15032", "name": "Cell Phones & Smartphones"}
            ]
            self.state_features = [
                "item_condition", 
                "brand_popularity", 
                "season_demand", 
                "market_saturation",
                "avg_price", 
                "price_std", 
                "time_on_market",
                "conversion_rate"
            ]
    
    def extract_all_features(self) -> Dict:
        """
        Extract features for all categories
        
        Returns:
            Dict: Extraction statistics
        """
        self.stats["start_time"] = datetime.now()
        self.logger.info("Starting feature extraction for all categories")
        
        for category in self.categories:
            try:
                self.extract_category_features(category)
                self.stats["categories_processed"] += 1
            except Exception as e:
                self.logger.error(f"Error extracting features for category {category['name']}: {str(e)}")
                self.stats["errors"] += 1
        
        self.stats["end_time"] = datetime.now()
        extraction_time = (self.stats["end_time"] - self.stats["start_time"]).total_seconds()
        self.logger.info(f"Feature extraction completed: {self.stats['features_extracted']} features extracted "
                         f"from {self.stats['categories_processed']} categories in {extraction_time:.2f} seconds")
        
        # Save extraction stats
        stats_file = os.path.join(self.features_dir, f"extraction_stats_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json")
        save_json(self.stats, stats_file)
        
        return self.stats
    
    def extract_category_features(self, category: Dict) -> None:
        """
        Extract features for a specific category
        
        Args:
            category (Dict): Category information with id and name
        """
        category_id = category["id"]
        category_name = category["name"]
        self.logger.info(f"Extracting features for category: {category_name} (ID: {category_id})")
        
        # Create category folder in features directory
        features_dir = os.path.join(self.features_dir, f"category_{category_id}")
        os.makedirs(features_dir, exist_ok=True)
        
        # Get processed data directory for this category
        processed_dir = os.path.join(self.processed_data_dir, f"category_{category_id}")
        
        # Find the latest active and sold item files
        active_files = sorted(glob.glob(os.path.join(processed_dir, "processed_active_items_*.csv")))
        sold_files = sorted(glob.glob(os.path.join(processed_dir, "processed_sold_items_*.csv")))
        
        if not active_files and not sold_files:
            self.logger.warning(f"No processed data found for category {category_name}")
            return
        
        # Load the latest files
        active_items_df = pd.read_csv(active_files[-1]) if active_files else None
        sold_items_df = pd.read_csv(sold_files[-1]) if sold_files else None
        
        # Combine active and sold items for feature extraction
        combined_df = self._combine_dataframes(active_items_df, sold_items_df)
        
        if combined_df is None or len(combined_df) == 0:
            self.logger.warning(f"No valid data for feature extraction in category {category_name}")
            return
        
        # Load category statistics
        stats_files = sorted(glob.glob(os.path.join(processed_dir, "category_stats_*.json")))
        if not stats_files:
            self.logger.warning(f"No category statistics found for {category_name}")
            category_stats = {}
        else:
            category_stats = load_json(stats_files[-1]) or {}
        
        # Extract features
        features_df = self._extract_features(combined_df, category_stats, category_id)
        
        # Save features
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        features_file = os.path.join(features_dir, f"features_{timestamp}.csv")
        features_df.to_csv(features_file, index=False)
        
        # Save scalers and models
        self._save_scalers_and_models(features_dir, category_id)
        
        # Create metadata
        metadata = {
            "category_id": category_id,
            "category_name": category_name,
            "timestamp": datetime.now().isoformat(),
            "features_count": len(features_df),
            "features_file": features_file,
            "feature_names": features_df.columns.tolist(),
            "source_files": {
                "active_items": active_files[-1] if active_files else None,
                "sold_items": sold_files[-1] if sold_files else None,
                "stats_file": stats_files[-1] if stats_files else None
            }
        }
        
        metadata_file = os.path.join(features_dir, f"metadata_{timestamp}.json")
        save_json(metadata, metadata_file)
        
        self.logger.info(f"Extracted {len(features_df)} feature records for category {category_name}")
        self.stats["features_extracted"] += len(features_df)
    
    def _combine_dataframes(self, active_df: Optional[pd.DataFrame], sold_df: Optional[pd.DataFrame]) -> Optional[pd.DataFrame]:
        """
        Combine active and sold item DataFrames
        
        Args:
            active_df (Optional[pd.DataFrame]): DataFrame with active items
            sold_df (Optional[pd.DataFrame]): DataFrame with sold items
            
        Returns:
            Optional[pd.DataFrame]: Combined DataFrame or None
        """
        dfs = []
        if active_df is not None and len(active_df) > 0:
            dfs.append(active_df)
        if sold_df is not None and len(sold_df) > 0:
            dfs.append(sold_df)
        
        if not dfs:
            return None
        
        combined_df = pd.concat(dfs, ignore_index=True)
        self.logger.info(f"Combined {len(combined_df)} items for feature extraction")
        return combined_df
    
    def _extract_features(self, df: pd.DataFrame, category_stats: Dict, category_id: str) -> pd.DataFrame:
        """
        Extract features from processed data
        
        Args:
            df (pd.DataFrame): DataFrame with processed items
            category_stats (Dict): Category statistics
            category_id (str): Category ID
            
        Returns:
            pd.DataFrame: Features DataFrame
        """
        self.logger.info(f"Extracting features from {len(df)} items")
        
        # Create a copy to avoid modifying the original
        features_df = df.copy()
        
        # Basic features
        basic_features = self._extract_basic_features(features_df)
        
        # Calculate temporal features
        temporal_features = self._extract_temporal_features(features_df)
        
        # Calculate market features
        market_features = self._extract_market_features(features_df, category_stats)
        
        # Calculate text features
        text_features = self._extract_text_features(features_df)
        
        # Combine all features
        all_features = pd.concat([
            features_df[['itemId', 'price', 'is_sold', 'category_id']],  # Keep these columns
            basic_features,
            temporal_features,
            market_features,
            text_features
        ], axis=1)
        
        # Normalize numerical features
        normalized_features = self._normalize_features(all_features, category_id)
        
        # Add target variable (for supervised learning)
        if 'is_sold' in normalized_features.columns and 'sold_date' in df.columns:
            # For sold items, calculate days to sell
            sold_items = df[df['is_sold'] == True]
            if 'listing_date' in sold_items.columns and 'sold_date' in sold_items.columns:
                sold_items['days_to_sell'] = (pd.to_datetime(sold_items['sold_date']) - 
                                            pd.to_datetime(sold_items['listing_date'])).dt.total_seconds() / (24 * 3600)
                
                # Add days_to_sell to the features DataFrame
                normalized_features = normalized_features.merge(
                    sold_items[['itemId', 'days_to_sell']], 
                    on='itemId', 
                    how='left'
                )
        
        # Final feature selection based on config
        selected_features = self._select_features(normalized_features)
        
        return selected_features
    
    def _extract_basic_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Extract basic features from item data
        
        Args:
            df (pd.DataFrame): DataFrame with processed items
            
        Returns:
            pd.DataFrame: Basic features DataFrame
        """
        features = pd.DataFrame(index=df.index)
        
        # Item condition (already normalized)
        if 'condition_normalized' in df.columns:
            features['item_condition'] = df['condition_normalized']
        
        # Shipping cost ratio
        if 'shipping_cost' in df.columns and 'price' in df.columns:
            features['shipping_cost_ratio'] = df['shipping_cost'] / df['price'].clip(lower=0.01)
            features['shipping_cost_ratio'] = features['shipping_cost_ratio'].clip(upper=1.0)
        
        # Has images feature
        if 'image_count' in df.columns:
            features['has_images'] = (df['image_count'] > 0).astype(float)
            features['image_count_normalized'] = df['image_count'].clip(upper=10) / 10  # Normalize to [0,1]
        
        # Brand popularity (placeholder - will be replaced in market features)
        features['brand_popularity'] = 0.5
        
        # One-hot encode country if available
        if 'country' in df.columns:
            # Get top 5 countries
            top_countries = df['country'].value_counts().nlargest(5).index
            for country in top_countries:
                features[f'country_{country}'] = (df['country'] == country).astype(float)
            features['country_other'] = (~df['country'].isin(top_countries)).astype(float)
        
        return features
    
    def _extract_temporal_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Extract temporal features from item data
        
        Args:
            df (pd.DataFrame): DataFrame with processed items
            
        Returns:
            pd.DataFrame: Temporal features DataFrame
        """
        features = pd.DataFrame(index=df.index)
        
        # Convert date columns to datetime if they aren't already
        date_columns = ['listing_date', 'end_date', 'collection_date']
        for col in date_columns:
            if col in df.columns:
                df[col] = pd.to_datetime(df[col], errors='coerce')
        
        # Time on market (for active items)
        if 'listing_date' in df.columns and 'collection_date' in df.columns:
            active_mask = df['is_sold'] == False
            if active_mask.any():
                df.loc[active_mask, 'time_on_market_days'] = (
                    df.loc[active_mask, 'collection_date'] - 
                    df.loc[active_mask, 'listing_date']
                ).dt.total_seconds() / (24 * 3600)
                
                # Normalize to [0,1] with 90 days as max
                features['time_on_market'] = df['time_on_market_days'].clip(lower=0, upper=90) / 90
        
        # Remaining time (for active items)
        if 'end_date' in df.columns and 'collection_date' in df.columns:
            active_mask = df['is_sold'] == False
            if active_mask.any():
                df.loc[active_mask, 'remaining_days'] = (
                    df.loc[active_mask, 'end_date'] - 
                    df.loc[active_mask, 'collection_date']
                ).dt.total_seconds() / (24 * 3600)
                
                # Normalize to [0,1] with 30 days as max
                features['remaining_time'] = df['remaining_days'].clip(lower=0, upper=30) / 30
        
        # Listing duration
        if 'listing_duration_days' in df.columns:
            features['listing_duration'] = df['listing_duration_days'].clip(lower=0, upper=90) / 90
        
        # Season demand (based on month)
        if 'listing_date' in df.columns:
            # Extract month
            df['listing_month'] = df['listing_date'].dt.month
            
            # Create seasonal feature (simplified)
            # Peak shopping seasons: holiday season (Nov-Dec), back-to-school (Aug-Sep)
            peak_seasons = [8, 9, 11, 12]  # August, September, November, December
            medium_seasons = [4, 5, 6, 7, 10]  # April, May, June, July, October
            low_seasons = [1, 2, 3]  # January, February, March
            
            features['season_demand'] = 0.5  # Default
            features.loc[df['listing_month'].isin(peak_seasons), 'season_demand'] = 1.0
            features.loc[df['listing_month'].isin(medium_seasons), 'season_demand'] = 0.7
            features.loc[df['listing_month'].isin(low_seasons), 'season_demand'] = 0.3
        
        return features
    
    def _extract_market_features(self, df: pd.DataFrame, category_stats: Dict) -> pd.DataFrame:
        """
        Extract market-related features
        
        Args:
            df (pd.DataFrame): DataFrame with processed items
            category_stats (Dict): Category statistics
            
        Returns:
            pd.DataFrame: Market features DataFrame
        """
        features = pd.DataFrame(index=df.index)
        
        # Price statistics
        price_mean = category_stats.get('price_mean', df['price'].mean())
        price_std = category_stats.get('price_std', df['price'].std())
        
        # Normalize price relative to category average
        features['price_relative'] = (df['price'] - price_mean) / price_std if price_std > 0 else 0
        features['price_relative'] = features['price_relative'].clip(lower=-3, upper=3)
        
        # Price competitiveness (lower price = higher competitiveness)
        features['price_competitiveness'] = 1 - (df['price'] / (price_mean * 1.5)).clip(lower=0, upper=1)
        
        # Market saturation (estimate based on category statistics)
        sold_percentage = category_stats.get('sold_percentage', 50)
        features['market_saturation'] = sold_percentage / 100  # Normalize to [0,1]
        
        # Calculate brand popularity if brand information is available
        if 'specific_brand' in df.columns:
            # Get brand counts
            brand_counts = df['specific_brand'].value_counts()
            total_items = len(df)
            
            # Map brands to their popularity (count / total)
            brand_popularity = brand_counts / total_items
            
            # Apply to features
            features['brand_popularity'] = df['specific_brand'].map(
                lambda x: brand_popularity.get(x, 0) if pd.notna(x) else 0
            )
        
        # Conversion rate (from category stats or estimated)
        if 'price_ratio_sold_active' in category_stats:
            features['conversion_rate'] = 1 / category_stats['price_ratio_sold_active']
            features['conversion_rate'] = features['conversion_rate'].clip(lower=0, upper=1)
        else:
            # Default based on price competitiveness
            features['conversion_rate'] = features['price_competitiveness'] * 0.8
        
        # Price standard deviation
        features['price_std'] = price_std / price_mean if price_mean > 0 else 0
        features['price_std'] = features['price_std'].clip(upper=1)
        
        # Average price (normalized)
        max_price = category_stats.get('price_max', df['price'].max())
        if max_price > 0:
            features['avg_price'] = price_mean / max_price
        else:
            features['avg_price'] = 0.5
        
        return features
    
    def _extract_text_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Extract features from text fields
        
        Args:
            df (pd.DataFrame): DataFrame with processed items
            
        Returns:
            pd.DataFrame: Text features DataFrame
        """
        features = pd.DataFrame(index=df.index)
        
        # Title length
        if 'title_length' in df.columns:
            # Normalize title length
            features['title_length_normalized'] = df['title_length'].clip(lower=0, upper=80) / 80
        
        # Simple keyword detection (could be expanded with NLP)
        if 'title' in df.columns:
            # Check for quality indicators
            quality_keywords = ['excellent', 'perfect', 'like new', 'mint', 'great']
            features['has_quality_indicator'] = df['title'].str.lower().apply(
                lambda x: any(kw in str(x).lower() for kw in quality_keywords) if pd.notna(x) else False
            ).astype(float)
            
            # Check for issue indicators
            issue_keywords = ['broken', 'damaged', 'cracked', 'issue', 'problem', 'scratched']
            features['has_issue_indicator'] = df['title'].str.lower().apply(
                lambda x: any(kw in str(x).lower() for kw in issue_keywords) if pd.notna(x) else False
            ).astype(float)
            
            # Check for urgency indicators
            urgency_keywords = ['quick sale', 'urgent', 'must sell', 'fast', 'immediate']
            features['has_urgency_indicator'] = df['title'].str.lower().apply(
                lambda x: any(kw in str(x).lower() for kw in urgency_keywords) if pd.notna(x) else False
            ).astype(float)
        
        return features
    
    def _normalize_features(self, df: pd.DataFrame, category_id: str) -> pd.DataFrame:
        """
        Normalize numerical features
        
        Args:
            df (pd.DataFrame): DataFrame with features
            category_id (str): Category ID for separate scalers
            
        Returns:
            pd.DataFrame: Normalized features DataFrame
        """
        # Make a copy of the DataFrame
        normalized_df = df.copy()
        
        # Identify numerical columns (excluding certain columns)
        exclude_cols = ['itemId', 'is_sold', 'category_id', 'days_to_sell']
        numerical_cols = [col for col in df.columns if col not in exclude_cols and pd.api.types.is_numeric_dtype(df[col])]
        
        if not numerical_cols:
            return normalized_df
        
        # Create or use existing scaler for this category
        scaler_key = f"scaler_{category_id}"
        if scaler_key not in self.scalers:
            self.scalers[scaler_key] = MinMaxScaler()
            normalized_df[numerical_cols] = self.scalers[scaler_key].fit_transform(df[numerical_cols].fillna(0))
        else:
            normalized_df[numerical_cols] = self.scalers[scaler_key].transform(df[numerical_cols].fillna(0))
        
        # Optional: Dimensionality reduction for feature sets with many columns
        if len(numerical_cols) > 15:
            pca_key = f"pca_{category_id}"
            pca_cols = [f"pca_{i}" for i in range(10)]  # 10 PCA components
            
            if pca_key not in self.pca_models:
                self.pca_models[pca_key] = PCA(n_components=10)
                pca_result = self.pca_models[pca_key].fit_transform(normalized_df[numerical_cols].fillna(0))
                
                # Add PCA components to DataFrame
                for i, col in enumerate(pca_cols):
                    normalized_df[col] = pca_result[:, i]
            else:
                pca_result = self.pca_models[pca_key].transform(normalized_df[numerical_cols].fillna(0))
                
                # Add PCA components to DataFrame
                for i, col in enumerate(pca_cols):
                    normalized_df[col] = pca_result[:, i]
        
        return normalized_df
    
    def _select_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Select features based on configuration
        
        Args:
            df (pd.DataFrame): DataFrame with all features
            
        Returns:
            pd.DataFrame: Selected features DataFrame
        """
        # Always keep these columns
        essential_cols = ['itemId', 'price', 'is_sold', 'category_id']
        
        # Map state features from config to actual DataFrame columns
        feature_mapping = {
            "item_condition": "item_condition",
            "brand_popularity": "brand_popularity",
            "season_demand": "season_demand",
            "market_saturation": "market_saturation",
            "avg_price": "avg_price",
            "price_std": "price_std",
            "time_on_market": "time_on_market",
            "conversion_rate": "conversion_rate"
        }
        
        # Get all available columns that match our state features
        available_cols = []
        for feature_name in self.state_features:
            col_name = feature_mapping.get(feature_name)
            if col_name in df.columns:
                available_cols.append(col_name)
        
        # Also include any PCA columns if they exist
        pca_cols = [col for col in df.columns if col.startswith("pca_")]
        
        # Create the final list of columns to keep
        selected_cols = essential_cols + available_cols + pca_cols
        
        # Add days_to_sell if available (for training)
        if 'days_to_sell' in df.columns:
            selected_cols.append('days_to_sell')
        
        # Return the selected columns
        return df[selected_cols].copy()
    
    def _save_scalers_and_models(self, features_dir: str, category_id: str) -> None:
        """
        Save scalers and models for future use
        
        Args:
            features_dir (str): Directory to save models
            category_id (str): Category ID
        """
        # Save scaler
        scaler_key = f"scaler_{category_id}"
        if scaler_key in self.scalers:
            scaler_path = os.path.join(features_dir, f"{scaler_key}.pkl")
            with open(scaler_path, 'wb') as f:
                pickle.dump(self.scalers[scaler_key], f)
            self.logger.info(f"Saved scaler to {scaler_path}")
        
        # Save PCA model
        pca_key = f"pca_{category_id}"
        if pca_key in self.pca_models:
            pca_path = os.path.join(features_dir, f"{pca_key}.pkl")
            with open(pca_path, 'wb') as f:
                pickle.dump(self.pca_models[pca_key], f)
            self.logger.info(f"Saved PCA model to {pca_path}")

# Example usage code
"""
def main():
    feature_extractor = FeatureExtractor()
    stats = feature_extractor.extract_all_features()
    print(f"Feature extraction completed: {stats['features_extracted']} features extracted")

if __name__ == "__main__":
    main()
"""

'\ndef main():\n    feature_extractor = FeatureExtractor()\n    stats = feature_extractor.extract_all_features()\n    print(f"Feature extraction completed: {stats[\'features_extracted\']} features extracted")\n\nif __name__ == "__main__":\n    main()\n'

In [11]:
import os
import asyncio
import argparse
import logging
import json
from datetime import datetime
import pandas as pd
from typing import Dict, List, Optional, Any

from src.api.ebay_api_client import EbayAPIClient
from src.data.data_collector import DataCollector
from src.data.data_preprocessor import DataPreprocessor
from src.data.feature_extractor import FeatureExtractor
from src.utils.logger import Logger

class EbayPricePredictionPipeline:
    """
    Main driver class that integrates all components of the eBay second-hand item price prediction system
    """
    def __init__(self, config_path: str = "/data/chats/p6wyr/workspace/config/config.json"):
        """
        Initialize the pipeline with configuration
        
        Args:
            config_path (str): Path to configuration file
        """
        self.logger = Logger().get_logger()
        self.logger.info("Initializing eBay Price Prediction Pipeline")
        
        # Load configuration
        self.config_path = config_path
        self._load_config()
        
        # Initialize components
        self.ebay_client = None  # Initialize later to avoid unnecessary token fetching
        self.data_collector = None
        self.data_preprocessor = DataPreprocessor(config_path=config_path)
        self.feature_extractor = FeatureExtractor(config_path=config_path)
    
    def _load_config(self) -> None:
        """
        Load configuration from file
        """
        try:
            with open(self.config_path, 'r') as f:
                self.config = json.load(f)
            self.logger.info(f"Configuration loaded successfully from {self.config_path}")
        except Exception as e:
            self.logger.error(f"Error loading configuration: {str(e)}")
            raise
    
    async def initialize_api_client(self) -> None:
        """
        Initialize the eBay API client
        """
        self.logger.info("Initializing eBay API client")
        self.ebay_client = EbayAPIClient(config_path=self.config_path)
        success = await self.ebay_client.authenticate()
        
        if success:
            self.logger.info("eBay API client successfully authenticated")
        else:
            self.logger.error("Failed to authenticate eBay API client")
            raise RuntimeError("API authentication failed")
    
    async def run_data_collection(self, categories: Optional[List[Dict]] = None) -> Dict:
        """
        Run the data collection process
        
        Args:
            categories (Optional[List[Dict]]): List of specific categories to collect, or None for all
            
        Returns:
            Dict: Collection statistics
        """
        self.logger.info("Starting data collection process")
        
        # Initialize data collector
        self.data_collector = DataCollector(config_path=self.config_path)
        
        # If specific categories provided, override the defaults
        if categories:
            self.data_collector.categories = categories
            self.logger.info(f"Using {len(categories)} provided categories for collection")
        
        # Run collection process
        stats = await self.data_collector.collect_data()
        self.logger.info(f"Data collection completed: {stats['items_collected']} items from {stats['categories_processed']} categories")
        
        return stats
    
    def run_data_preprocessing(self) -> Dict:
        """
        Run the data preprocessing step
        
        Returns:
            Dict: Preprocessing statistics
        """
        self.logger.info("Starting data preprocessing")
        stats = self.data_preprocessor.process_all_data()
        self.logger.info(f"Data preprocessing completed: {stats['items_processed']} items processed")
        
        return stats
    
    def run_feature_extraction(self) -> Dict:
        """
        Run the feature extraction process
        
        Returns:
            Dict: Feature extraction statistics
        """
        self.logger.info("Starting feature extraction")
        stats = self.feature_extractor.extract_all_features()
        self.logger.info(f"Feature extraction completed: {stats['features_extracted']} features extracted")
        
        return stats
    
    async def run_complete_pipeline(self, categories: Optional[List[Dict]] = None) -> Dict:
        """
        Run the complete data pipeline: collection, preprocessing, and feature extraction
        
        Args:
            categories (Optional[List[Dict]]): List of specific categories to process or None for all
            
        Returns:
            Dict: Complete pipeline statistics
        """
        pipeline_stats = {
            "start_time": datetime.now().isoformat(),
            "collection": None,
            "preprocessing": None,
            "feature_extraction": None,
            "end_time": None
        }
        
        try:
            # Initialize API client
            await self.initialize_api_client()
            
            # Run data collection
            collection_stats = await self.run_data_collection(categories)
            pipeline_stats["collection"] = collection_stats
            
            # Run data preprocessing
            preprocessing_stats = self.run_data_preprocessing()
            pipeline_stats["preprocessing"] = preprocessing_stats
            
            # Run feature extraction
            extraction_stats = self.run_feature_extraction()
            pipeline_stats["feature_extraction"] = extraction_stats
            
            # Complete pipeline
            pipeline_stats["end_time"] = datetime.now().isoformat()
            self.logger.info("Complete data pipeline execution finished successfully")
            
            # Save pipeline stats
            stats_file = os.path.join("/data/chats/p6wyr/workspace/data", f"pipeline_stats_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json")
            with open(stats_file, 'w') as f:
                json.dump(pipeline_stats, f, indent=2)
            
            return pipeline_stats
            
        except Exception as e:
            self.logger.error(f"Error in pipeline execution: {str(e)}")
            pipeline_stats["error"] = str(e)
            pipeline_stats["end_time"] = datetime.now().isoformat()
            return pipeline_stats

async def main():
    """
    Main entry point for the application
    """
    parser = argparse.ArgumentParser(description='eBay Second-hand Item Price Prediction System')
    parser.add_argument('--config', type=str, default="/data/chats/p6wyr/workspace/config/config.json", 
                        help='Path to configuration file')
    parser.add_argument('--mode', type=str, choices=['collect', 'preprocess', 'extract', 'full'], 
                        default='full', help='Operation mode')
    parser.add_argument('--category', type=str, help='Specific category ID to process')
    
    # For Jupyter notebook environment, use default arguments
    # args = parser.parse_args()
    args = parser.parse_args([])
    
    # Initialize logger
    logger = Logger().get_logger()
    logger.info(f"Starting application in {args.mode} mode")
    
    # Initialize pipeline
    pipeline = EbayPricePredictionPipeline(config_path=args.config)
    
    # Prepare categories if specific one provided
    categories = None
    if args.category:
        # Look up category name from config
        category_name = "Unknown"
        for cat in pipeline.config.get("data", {}).get("categories", []):
            if cat["id"] == args.category:
                category_name = cat["name"]
                break
        
        categories = [{"id": args.category, "name": category_name}]
        logger.info(f"Processing only category: {category_name} (ID: {args.category})")
    
    # Execute requested operation
    if args.mode == 'collect':
        await pipeline.initialize_api_client()
        stats = await pipeline.run_data_collection(categories)
        logger.info(f"Collection stats: {stats}")
    
    elif args.mode == 'preprocess':
        stats = pipeline.run_data_preprocessing()
        logger.info(f"Preprocessing stats: {stats}")
    
    elif args.mode == 'extract':
        stats = pipeline.run_feature_extraction()
        logger.info(f"Feature extraction stats: {stats}")
    
    elif args.mode == 'full':
        stats = await pipeline.run_complete_pipeline(categories)
        logger.info("Full pipeline execution completed")
    
    logger.info("Application execution completed")
    return stats

# For Jupyter notebook environment
if __name__ == "__main__":
    # Example of running in Jupyter notebook
    print("Starting eBay Price Prediction Pipeline")
    # Note: In Jupyter, we can't use asyncio.run() directly
    # Instead, we need to use the following approach:
    pipeline_stats = await main()  # Use await directly in Jupyter
    print(f"Pipeline completed with {pipeline_stats.get('collection', {}).get('items_collected', 0)} items collected")
    print("Pipeline execution completed")

ImportError: cannot import name 'DataCollector' from 'src.data.data_collector' (/data/chats/p6wyr/workspace/src/data/data_collector.py)

In [12]:
[reflection on previous impl]
The error message indicates that we're getting an `ImportError` when trying to import `DataCollector` from `src.data.data_collector`. This suggests that while we have the `data_collector.py` file in the expected location, the `DataCollector` class is not properly defined or exported in that file.

Looking at the code we had previously implemented, we created the `DataCollector` class but it appears that we didn't actually write the class to a file at the proper location. Instead, we wrote the code in a Jupyter notebook cell but didn't save it to the actual `data_collector.py` file in the project structure.

To fix this issue, we need to:

1. First create the `data_collector.py` file with our DataCollector class implementation
2. Then create the main driver script that imports and uses this class

We need to ensure that the `DataCollector` class is properly saved to the file system before we try to import it in our main driver script.

[improved impl]:
```python
from metagpt.tools.libs.terminal import Terminal
terminal = Terminal()

# Step 1: First, we need to create the data_collector.py file with our DataCollector class
data_collector_code = """
import os
import json
import pandas as pd
import asyncio
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Union, Any, Tuple
import time
import random
from tqdm import tqdm

from src.api.ebay_api_client import EbayAPIClient
from src.utils.logger import Logger
from src.utils.data_utils import generate_item_id, save_json, load_json, get_category_folder, save_items_to_csv, save_item_history

class DataCollector:
    \"\"\"
    Data collection module for retrieving second-hand item data from eBay
    \"\"\"
    def __init__(self, config_path: str = "/data/chats/p6wyr/workspace/config/config.json"):
        \"\"\"
        Initialize the data collector
        
        Args:
            config_path (str): Path to configuration file
        \"\"\"
        self.logger = Logger().get_logger()
        self.logger.info("Initializing DataCollector module")
        
        # Load configuration
        self.config_path = config_path
        self._load_config()
        
        # Initialize eBay API client
        self.ebay_client = EbayAPIClient(config_path=config_path)
        
        # Set up data directories
        self.raw_data_dir = "/data/chats/p6wyr/workspace/data/raw"
        os.makedirs(self.raw_data_dir, exist_ok=True)
        
        # Track collection statistics
        self.stats = {
            "items_collected": 0,
            "categories_processed": 0,
            "errors": 0,
            "start_time": None,
            "end_time": None
        }
    
    def _load_config(self) -> None:
        \"\"\"
        Load configuration from file
        \"\"\"
        try:
            with open(self.config_path, 'r') as f:
                self.config = json.load(f)
            
            # Extract relevant configuration
            self.categories = self.config.get("data", {}).get("categories", [])
            self.sample_size_per_category = self.config.get("data", {}).get("sample_size_per_category", 100)
            self.history_window = self.config.get("data", {}).get("history_window", 30)
            self.update_frequency = self.config.get("data", {}).get("update_frequency", 24)
            
            self.logger.info(f"Configuration loaded: {len(self.categories)} categories, "
                             f"{self.sample_size_per_category} items per category")
        except Exception as e:
            self.logger.error(f"Error loading configuration: {str(e)}")
            # Use default values
            self.categories = [
                {"id": "9355", "name": "Laptops & Netbooks"},
                {"id": "15032", "name": "Cell Phones & Smartphones"}
            ]
            self.sample_size_per_category = 100
            self.history_window = 30
            self.update_frequency = 24
    
    async def collect_data(self) -> Dict:
        \"\"\"
        Collect data for all configured categories
        
        Returns:
            Dict: Collection statistics
        \"\"\"
        self.stats["start_time"] = datetime.now()
        self.logger.info(f"Starting data collection for {len(self.categories)} categories")
        
        for category in self.categories:
            try:
                await self.collect_category_data(category)
                self.stats["categories_processed"] += 1
            except Exception as e:
                self.logger.error(f"Error collecting data for category {category['name']}: {str(e)}")
                self.stats["errors"] += 1
        
        self.stats["end_time"] = datetime.now()
        collection_time = (self.stats["end_time"] - self.stats["start_time"]).total_seconds()
        self.logger.info(f"Data collection completed: {self.stats['items_collected']} items collected "
                         f"from {self.stats['categories_processed']} categories in {collection_time:.2f} seconds")
        
        # Save collection stats
        stats_file = os.path.join(self.raw_data_dir, f"collection_stats_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json")
        save_json(self.stats, stats_file)
        
        return self.stats
    
    async def collect_category_data(self, category: Dict) -> None:
        \"\"\"
        Collect data for a specific category
        
        Args:
            category (Dict): Category information with id and name
        \"\"\"
        category_id = category["id"]
        category_name = category["name"]
        self.logger.info(f"Collecting data for category: {category_name} (ID: {category_id})")
        
        # Create category folder
        category_folder = get_category_folder(category_id)
        os.makedirs(category_folder, exist_ok=True)
        
        # Define search terms for this category
        search_terms = self._get_search_terms_for_category(category)
        
        # Collect active listings
        active_items = await self._collect_active_items(category, search_terms)
        
        # Collect sold items for price history
        sold_items = await self._collect_sold_items(category, search_terms)
        
        # Save category data
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        active_file = os.path.join(category_folder, f"active_items_{timestamp}.csv")
        sold_file = os.path.join(category_folder, f"sold_items_{timestamp}.csv")
        
        save_items_to_csv(active_items, active_file)
        save_items_to_csv(sold_items, sold_file)
        
        # Create a metadata file to track the collection
        metadata = {
            "category_id": category_id,
            "category_name": category_name,
            "timestamp": datetime.now().isoformat(),
            "active_items_count": len(active_items),
            "sold_items_count": len(sold_items),
            "active_items_file": active_file,
            "sold_items_file": sold_file
        }
        
        metadata_file = os.path.join(category_folder, f"metadata_{timestamp}.json")
        save_json(metadata, metadata_file)
        
        self.logger.info(f"Collected {len(active_items)} active items and {len(sold_items)} sold items "
                         f"for category {category_name}")
        
        # Update stats
        self.stats["items_collected"] += len(active_items) + len(sold_items)
    
    def _get_search_terms_for_category(self, category: Dict) -> List[str]:
        \"\"\"
        Get appropriate search terms for a category
        
        Args:
            category (Dict): Category information
            
        Returns:
            List[str]: List of search terms
        \"\"\"
        category_name = category["name"]
        # This could be expanded with more specific search terms per category
        base_terms = [category_name, f"used {category_name}", f"second hand {category_name}"]
        
        # Add category-specific terms
        if category["id"] == "9355":  # Laptops
            return base_terms + ["refurbished laptop", "used notebook", "macbook", "thinkpad", "gaming laptop"]
        elif category["id"] == "15032":  # Cell Phones
            return base_terms + ["used iphone", "used samsung galaxy", "refurbished phone", "unlocked phone"]
        elif category["id"] == "11450":  # Watches
            return base_terms + ["used rolex", "used omega", "vintage watch", "pre-owned watch"]
        elif category["id"] == "261007":  # Cameras
            return base_terms + ["used dslr", "second hand mirrorless", "vintage camera", "used sony camera"]
        else:
            return base_terms
    
    async def _collect_active_items(self, category: Dict, search_terms: List[str]) -> List[Dict]:
        \"\"\"
        Collect active listings for a category using multiple search terms
        
        Args:
            category (Dict): Category information
            search_terms (List[str]): List of search terms
            
        Returns:
            List[Dict]: Collected items data
        \"\"\"
        category_id = category["id"]
        collected_items = []
        
        for search_term in tqdm(search_terms, desc=f"Collecting active items for {category['name']}"):
            try:
                # Make the API request with reasonable limits
                response = await self.ebay_client.search_items(
                    keywords=search_term,
                    category_id=category_id,
                    limit=min(50, self.sample_size_per_category // len(search_terms))
                )
                
                if not response.success:
                    self.logger.warning(f"Error searching for '{search_term}' in category {category_id}: {response.error}")
                    continue
                
                # Extract items from response
                items = response.data.get("itemSummaries", [])
                self.logger.info(f"Found {len(items)} items for search term '{search_term}'")
                
                # Process each item
                for item in items:
                    try:
                        # Generate a consistent item ID
                        item_id = item.get("itemId") or generate_item_id(item)
                        
                        # Get more detailed information about the item
                        details_response = await self.ebay_client.get_item_details(item_id, use_cache=True)
                        
                        if details_response.success:
                            item_details = details_response.data
                            
                            # Merge summary and details
                            processed_item = self._process_item_data(item, item_details)
                            collected_items.append(processed_item)
                            
                            # Add a small delay to avoid overwhelming the API
                            await asyncio.sleep(random.uniform(0.1, 0.5))
                    except Exception as e:
                        self.logger.error(f"Error processing item {item.get('itemId', 'unknown')}: {str(e)}")
                
                # Add a delay between search terms
                await asyncio.sleep(random.uniform(1.0, 2.0))
                
            except Exception as e:
                self.logger.error(f"Error collecting items for search term '{search_term}': {str(e)}")
                self.stats["errors"] += 1
        
        # Deduplicate items based on itemId
        unique_items = []
        seen_ids = set()
        for item in collected_items:
            if item["itemId"] not in seen_ids:
                unique_items.append(item)
                seen_ids.add(item["itemId"])
        
        self.logger.info(f"Collected {len(unique_items)} unique active items for category {category['name']}")
        return unique_items
    
    async def _collect_sold_items(self, category: Dict, search_terms: List[str]) -> List[Dict]:
        \"\"\"
        Collect sold items data for price history
        
        Args:
            category (Dict): Category information
            search_terms (List[str]): List of search terms
            
        Returns:
            List[Dict]: Collected sold items data
        \"\"\"
        category_id = category["id"]
        sold_items = []
        
        for search_term in tqdm(search_terms[:2], desc=f"Collecting sold items for {category['name']}"):
            try:
                # Make the API request for sold items
                response = await self.ebay_client.get_sold_items(
                    keywords=search_term,
                    category_id=category_id,
                    days_back=self.history_window,
                    limit=min(50, self.sample_size_per_category // len(search_terms))
                )
                
                if not response.success:
                    self.logger.warning(f"Error searching for sold items with '{search_term}' in category {category_id}: {response.error}")
                    continue
                
                # The structure of the sold items response is different
                items = []
                try:
                    # The response structure might vary depending on the API version
                    search_result = response.data.get("findCompletedItemsResponse", [{}])[0]
                    result_items = search_result.get("searchResult", [{}])[0].get("item", [])
                    items.extend(result_items)
                except (KeyError, IndexError, TypeError) as e:
                    self.logger.warning(f"Error parsing sold items response: {str(e)}")
                
                self.logger.info(f"Found {len(items)} sold items for search term '{search_term}'")
                
                # Process sold items
                for item in items:
                    try:
                        processed_item = self._process_sold_item_data(item)
                        sold_items.append(processed_item)
                    except Exception as e:
                        self.logger.error(f"Error processing sold item: {str(e)}")
                
                # Add a delay between search terms
                await asyncio.sleep(random.uniform(1.0, 2.0))
                
            except Exception as e:
                self.logger.error(f"Error collecting sold items for search term '{search_term}': {str(e)}")
                self.stats["errors"] += 1
        
        # Deduplicate sold items
        unique_sold_items = []
        seen_ids = set()
        for item in sold_items:
            if item["itemId"] not in seen_ids:
                unique_sold_items.append(item)
                seen_ids.add(item["itemId"])
        
        self.logger.info(f"Collected {len(unique_sold_items)} unique sold items for category {category['name']}")
        return unique_sold_items
    
    def _process_item_data(self, item_summary: Dict, item_details: Dict) -> Dict:
        \"\"\"
        Process and normalize item data from API responses
        
        Args:
            item_summary (Dict): Item summary from search results
            item_details (Dict): Detailed item information
            
        Returns:
            Dict: Processed item data
        \"\"\"
        # Extract the base information from summary
        processed_item = {
            "itemId": item_summary.get("itemId", ""),
            "title": item_summary.get("title", ""),
            "condition": item_summary.get("condition", ""),
            "conditionId": item_summary.get("conditionId", ""),
            "price": self._extract_price(item_summary.get("price", {})),
            "currency": self._extract_currency(item_summary.get("price", {})),
            "category_id": item_details.get("categoryId", ""),
            "category_path": item_details.get("categoryPath", ""),
            "listing_date": item_summary.get("itemCreationDate", ""),
            "end_date": item_summary.get("itemEndDate", ""),
            "collection_date": datetime.now().isoformat(),
            "url": item_summary.get("itemWebUrl", ""),
            "item_location": self._extract_location(item_details),
            "shipping_options": self._extract_shipping(item_details),
            "item_specifics": self._extract_item_specifics(item_details),
            "image_urls": self._extract_image_urls(item_summary, item_details)
        }
        
        return processed_item
    
    def _process_sold_item_data(self, item: Dict) -> Dict:
        \"\"\"
        Process sold item data which has a different structure
        
        Args:
            item (Dict): Sold item data from API
            
        Returns:
            Dict: Processed sold item data
        \"\"\"
        # Extract listing info
        listing_info = item.get("listingInfo", {})
        selling_status = item.get("sellingStatus", [{}])[0]
        
        # Extract location
        location = item.get("location", "")
        country = item.get("country", "")
        
        # Build the processed item
        processed_item = {
            "itemId": item.get("itemId", ""),
            "title": item.get("title", [""])[0] if isinstance(item.get("title"), list) else item.get("title", ""),
            "condition": item.get("condition", {}).get("conditionDisplayName", ""),
            "conditionId": item.get("condition", {}).get("conditionId", ""),
            "price": float(selling_status.get("currentPrice", [{}])[0].get("__value__", 0)),
            "currency": selling_status.get("currentPrice", [{}])[0].get("@currencyId", "USD"),
            "category_id": item.get("primaryCategory", [{}])[0].get("categoryId", ""),
            "category_path": item.get("primaryCategory", [{}])[0].get("categoryName", ""),
            "listing_date": listing_info.get("startTime", ""),
            "end_date": listing_info.get("endTime", ""),
            "collection_date": datetime.now().isoformat(),
            "url": item.get("viewItemURL", [""])[0] if isinstance(item.get("viewItemURL"), list) else item.get("viewItemURL", ""),
            "item_location": f"{location}, {country}",
            "is_sold": True,
            "sold_date": listing_info.get("endTime", "")
        }
        
        return processed_item
    
    def _extract_price(self, price_data: Dict) -> float:
        \"\"\"
        Extract price value from price object
        
        Args:
            price_data (Dict): Price object from API
            
        Returns:
            float: Price value
        \"\"\"
        # The price structure might be different based on API version
        if isinstance(price_data, dict):
            value = price_data.get("value")
            if value is not None:
                try:
                    return float(value)
                except (ValueError, TypeError):
                    pass
        return 0.0
    
    def _extract_currency(self, price_data: Dict) -> str:
        \"\"\"
        Extract currency from price object
        
        Args:
            price_data (Dict): Price object from API
            
        Returns:
            str: Currency code
        \"\"\"
        if isinstance(price_data, dict):
            return price_data.get("currency", "USD")
        return "USD"
    
    def _extract_location(self, item_details: Dict) -> Dict:
        \"\"\"
        Extract location information from item details
        
        Args:
            item_details (Dict): Item details
            
        Returns:
            Dict: Location information
        \"\"\"
        location = {}
        
        # Try to extract from itemLocation
        item_location = item_details.get("itemLocation", {})
        if item_location:
            location["country"] = item_location.get("country", "")
            location["postal_code"] = item_location.get("postalCode", "")
            location["city"] = item_location.get("city", "")
            location["state_or_province"] = item_location.get("stateOrProvince", "")
        
        return location
    
    def _extract_shipping(self, item_details: Dict) -> List[Dict]:
        \"\"\"
        Extract shipping options from item details
        
        Args:
            item_details (Dict): Item details
            
        Returns:
            List[Dict]: Shipping options
        \"\"\"
        shipping_options = []
        
        # Extract from shippingOptions
        options = item_details.get("shippingOptions", [])
        for option in options:
            shipping_option = {
                "shipping_cost": self._extract_price(option.get("shippingCost", {})),
                "shipping_carrier": option.get("shippingServiceCode", ""),
                "shipping_type": option.get("type", "")
            }
            shipping_options.append(shipping_option)
        
        return shipping_options
    
    def _extract_item_specifics(self, item_details: Dict) -> Dict:
        \"\"\"
        Extract item specifics (attributes) from item details
        
        Args:
            item_details (Dict): Item details
            
        Returns:
            Dict: Item specifics
        \"\"\"
        specifics = {}
        
        # Extract from localizedAspects
        aspects = item_details.get("localizedAspects", [])
        for aspect in aspects:
            name = aspect.get("name", "")
            value = aspect.get("value", "")
            if name and value:
                specifics[name] = value
        
        return specifics
    
    def _extract_image_urls(self, item_summary: Dict, item_details: Dict) -> List[str]:
        \"\"\"
        Extract image URLs from item data
        
        Args:
            item_summary (Dict): Item summary
            item_details (Dict): Item details
            
        Returns:
            List[str]: List of image URLs
        \"\"\"
        image_urls = []
        
        # Try from item_summary first
        if "thumbnailImages" in item_summary:
            for image in item_summary.get("thumbnailImages", []):
                image_url = image.get("imageUrl", "")
                if image_url:
                    image_urls.append(image_url)
        
        # Try from item_details if no images found
        if not image_urls and "images" in item_details:
            for image in item_details.get("images", []):
                image_url = image.get("imageUrl", "")
                if image_url:
                    image_urls.append(image_url)
        
        return image_urls
    
    # Simple scheduling function instead of using ScheduledTaskManager
    async def schedule_data_collection_simple(self, interval_hours: int = None) -> None:
        \"\"\"
        Simple scheduling function for data collection
        
        Args:
            interval_hours (int): Interval between collections in hours
        \"\"\"
        if interval_hours is None:
            interval_hours = self.update_frequency
            
        self.logger.info(f"Starting simple scheduler to collect data every {interval_hours} hours")
        
        while True:
            try:
                self.logger.info("Running scheduled data collection")
                await self.collect_data()
                self.logger.info(f"Data collection complete. Next collection in {interval_hours} hours")
            except Exception as e:
                self.logger.error(f"Error in scheduled data collection: {str(e)}")
            
            # Sleep for the specified interval
            await asyncio.sleep(interval_hours * 3600)  # Convert to seconds
"""

# Step 2: Create the data_utils.py file if it doesn't exist
data_utils_code = """
import os
import json
import hashlib

def generate_item_id(item_data):
    \"\"\"
    Generate a unique ID for an item based on its properties
    \"\"\"
    # Create a string with important item attributes
    item_str = f"{item_data.get('title', '')}-{item_data.get('condition', '')}-{item_data.get('brand', '')}"
    # Generate a hash
    return hashlib.md5(item_str.encode('utf-8')).hexdigest()

def get_category_folder(category_id):
    \"\"\"
    Get the folder path for a specific category
    \"\"\"
    return os.path.join('/data/chats/p6wyr/workspace/data/raw', f'category_{category_id}')

def save_json(data, file_path):
    \"\"\"
    Save data as JSON
    \"\"\"
    directory = os.path.dirname(file_path)
    os.makedirs(directory, exist_ok=True)
    
    with open(file_path, 'w') as f:
        json.dump(data, f, indent=4)

def load_json(file_path):
    \"\"\"
    Load data from JSON
    \"\"\"
    if not os.path.exists(file_path):
        return None
    
    with open(file_path, 'r') as f:
        return json.load(f)

def save_items_to_csv(items, file_path):
    \"\"\"
    Save a list of items to CSV
    \"\"\"
    import pandas as pd
    df = pd.DataFrame(items)
    directory = os.path.dirname(file_path)
    os.makedirs(directory, exist_ok=True)
    df.to_csv(file_path, index=False)

def save_item_history(item_id, history_data, category_id=None):
    \"\"\"
    Save item price history
    \"\"\"
    folder = '/data/chats/p6wyr/workspace/data/raw/price_history'
    if category_id:
        folder = os.path.join(folder, f'category_{category_id}')
    
    os.makedirs(folder, exist_ok=True)
    file_path = os.path.join(folder, f'{item_id}_history.json')
    
    save_json(history_data, file_path)
"""

# Create the data_collector.py file
data_collector_path = "/data/chats/p6wyr/workspace/src/data/data_collector.py"
await terminal.run(f"touch {data_collector_path}")
with open(data_collector_path, "w") as f:
    f.write(data_collector_code)

# Create the data_utils.py file if it doesn't exist
data_utils_path = "/data/chats/p6wyr/workspace/src/utils/data_utils.py"
await terminal.run(f"touch {data_utils_path}")
with open(data_utils_path, "w") as f:
    f.write(data_utils_code)

# Step 3: Now create the main driver script
main_driver_code = """
import os
import asyncio
import argparse
import json
from datetime import datetime
import pandas as pd
from typing import Dict, List, Optional, Any

from src.api.ebay_api_client import EbayAPIClient
from src.data.data_collector import DataCollector
from src.data.data_preprocessor import DataPreprocessor
from src.data.feature_extractor import FeatureExtractor
from src.utils.logger import Logger

class EbayPricePredictionPipeline:
    \"\"\"
    Main driver class that integrates all components of the eBay second-hand item price prediction system
    \"\"\"
    def __init__(self, config_path: str = "/data/chats/p6wyr/workspace/config/config.json"):
        \"\"\"
        Initialize the pipeline with configuration
        
        Args:
            config_path (str): Path to configuration file
        \"\"\"
        self.logger = Logger().get_logger()
        self.logger.info("Initializing eBay Price Prediction Pipeline")
        
        # Load configuration
        self.config_path = config_path
        self._load_config()
        
        # Initialize components
        self.ebay_client = None  # Initialize later to avoid unnecessary token fetching
        self.data_collector = None
        self.data_preprocessor = DataPreprocessor(config_path=config_path)
        self.feature_extractor = FeatureExtractor(config_path=config_path)
    
    def _load_config(self) -> None:
        \"\"\"
        Load configuration from file
        \"\"\"
        try:
            with open(self.config_path, 'r') as f:
                self.config = json.load(f)
            self.logger.info(f"Configuration loaded successfully from {self.config_path}")
        except Exception as e:
            self.logger.error(f"Error loading configuration: {str(e)}")
            raise
    
    async def initialize_api_client(self) -> None:
        \"\"\"
        Initialize the eBay API client
        \"\"\"
        self.logger.info("Initializing eBay API client")
        self.ebay_client = EbayAPIClient(config_path=self.config_path)
        success = await self.ebay_client.authenticate()
        
        if success:
            self.logger.info("eBay API client successfully authenticated")
        else:
            self.logger.error("Failed to authenticate eBay API client")
            raise RuntimeError("API authentication failed")
    
    async def run_data_collection(self, categories: Optional[List[Dict]] = None) -> Dict:
        \"\"\"
        Run the data collection process
        
        Args:
            categories (Optional[List[Dict]]): List of specific categories to collect, or None for all
            
        Returns:
            Dict: Collection statistics
        \"\"\"
        self.logger.info("Starting data collection process")
        
        # Initialize data collector
        self.data_collector = DataCollector(config_path=self.config_path)
        
        # If specific categories provided, override the defaults
        if categories:
            self.data_collector.categories = categories
            self.logger.info(f"Using {len(categories)} provided categories for collection")
        
        # Run collection process
        stats = await self.data_collector.collect_data()
        self.logger.info(f"Data collection completed: {stats['items_collected']} items from {stats['categories_processed']} categories")
        
        return stats
    
    def run_data_preprocessing(self) -> Dict:
        \"\"\"
        Run the data preprocessing step
        
        Returns:
            Dict: Preprocessing statistics
        \"\"\"
        self.logger.info("Starting data preprocessing")
        stats = self.data_preprocessor.process_all_data()
        self.logger.info(f"Data preprocessing completed: {stats['items_processed']} items processed")
        
        return stats
    
    def run_feature_extraction(self) -> Dict:
        \"\"\"
        Run the feature extraction process
        
        Returns:
            Dict: Feature extraction statistics
        \"\"\"
        self.logger.info("Starting feature extraction")
        stats = self.feature_extractor.extract_all_features()
        self.logger.info(f"Feature extraction completed: {stats['features_extracted']} features extracted")
        
        return stats
    
    async def run_complete_pipeline(self, categories: Optional[List[Dict]] = None) -> Dict:
        \"\"\"
        Run the complete data pipeline: collection, preprocessing, and feature extraction
        
        Args:
            categories (Optional[List[Dict]]): List of specific categories to process or None for all
            
        Returns:
            Dict: Complete pipeline statistics
        \"\"\"
        pipeline_stats = {
            "start_time": datetime.now().isoformat(),
            "collection": None,
            "preprocessing": None,
            "feature_extraction": None,
            "end_time": None
        }
        
        try:
            # Initialize API client
            await self.initialize_api_client()
            
            # Run data collection
            collection_stats = await self.run_data_collection(categories)
            pipeline_stats["collection"] = collection_stats
            
            # Run data preprocessing
            preprocessing_

SyntaxError: unterminated string literal (detected at line 2) (2283540264.py, line 2)

In [13]:
from metagpt.tools.libs.terminal import Terminal
terminal = Terminal()

# Create the data_collector.py file with our DataCollector class implementation
data_collector_code = '''import os
import json
import pandas as pd
import asyncio
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Union, Any, Tuple
import time
import random
from tqdm import tqdm

from src.api.ebay_api_client import EbayAPIClient
from src.utils.logger import Logger

# Helper functions (simplified versions) to avoid extra imports
def generate_item_id(item_data):
    """Generate a unique ID for an item based on its properties"""
    import hashlib
    item_str = f"{item_data.get('title', '')}-{item_data.get('condition', '')}"
    return hashlib.md5(item_str.encode('utf-8')).hexdigest()

def get_category_folder(category_id):
    """Get the folder path for a specific category"""
    return os.path.join('/data/chats/p6wyr/workspace/data/raw', f'category_{category_id}')

def save_json(data, file_path):
    """Save data as JSON"""
    directory = os.path.dirname(file_path)
    os.makedirs(directory, exist_ok=True)
    with open(file_path, 'w') as f:
        json.dump(data, f, indent=4)

def save_items_to_csv(items, file_path):
    """Save a list of items to CSV"""
    df = pd.DataFrame(items)
    directory = os.path.dirname(file_path)
    os.makedirs(directory, exist_ok=True)
    df.to_csv(file_path, index=False)

class DataCollector:
    """
    Data collection module for retrieving second-hand item data from eBay
    """
    def __init__(self, config_path: str = "/data/chats/p6wyr/workspace/config/config.json"):
        """
        Initialize the data collector
        
        Args:
            config_path (str): Path to configuration file
        """
        self.logger = Logger().get_logger()
        self.logger.info("Initializing DataCollector module")
        
        # Load configuration
        self.config_path = config_path
        self._load_config()
        
        # Initialize eBay API client
        self.ebay_client = EbayAPIClient(config_path=config_path)
        
        # Set up data directories
        self.raw_data_dir = "/data/chats/p6wyr/workspace/data/raw"
        os.makedirs(self.raw_data_dir, exist_ok=True)
        
        # Track collection statistics
        self.stats = {
            "items_collected": 0,
            "categories_processed": 0,
            "errors": 0,
            "start_time": None,
            "end_time": None
        }
    
    def _load_config(self) -> None:
        """
        Load configuration from file
        """
        try:
            with open(self.config_path, 'r') as f:
                self.config = json.load(f)
            
            # Extract relevant configuration
            self.categories = self.config.get("data", {}).get("categories", [])
            self.sample_size_per_category = self.config.get("data", {}).get("sample_size_per_category", 100)
            self.history_window = self.config.get("data", {}).get("history_window", 30)
            self.update_frequency = self.config.get("data", {}).get("update_frequency", 24)
            
            self.logger.info(f"Configuration loaded: {len(self.categories)} categories")
        except Exception as e:
            self.logger.error(f"Error loading configuration: {str(e)}")
            # Use default values
            self.categories = [
                {"id": "9355", "name": "Laptops & Netbooks"},
                {"id": "15032", "name": "Cell Phones & Smartphones"}
            ]
            self.sample_size_per_category = 100
            self.history_window = 30
            self.update_frequency = 24
    
    async def collect_data(self) -> Dict:
        """
        Collect data for all configured categories
        
        Returns:
            Dict: Collection statistics
        """
        self.stats["start_time"] = datetime.now()
        self.logger.info(f"Starting data collection for {len(self.categories)} categories")
        
        for category in self.categories:
            try:
                await self.collect_category_data(category)
                self.stats["categories_processed"] += 1
            except Exception as e:
                self.logger.error(f"Error collecting data for category {category['name']}: {str(e)}")
                self.stats["errors"] += 1
        
        self.stats["end_time"] = datetime.now()
        collection_time = (self.stats["end_time"] - self.stats["start_time"]).total_seconds()
        self.logger.info(f"Data collection completed: {self.stats['items_collected']} items collected "
                         f"from {self.stats['categories_processed']} categories in {collection_time:.2f} seconds")
        
        # Save collection stats
        stats_file = os.path.join(self.raw_data_dir, f"collection_stats_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json")
        save_json(self.stats, stats_file)
        
        return self.stats
    
    async def collect_category_data(self, category: Dict) -> None:
        """
        Collect data for a specific category
        
        Args:
            category (Dict): Category information with id and name
        """
        category_id = category["id"]
        category_name = category["name"]
        self.logger.info(f"Collecting data for category: {category_name} (ID: {category_id})")
        
        # Create category folder
        category_folder = get_category_folder(category_id)
        os.makedirs(category_folder, exist_ok=True)
        
        # Define search terms for this category
        search_terms = self._get_search_terms_for_category(category)
        
        # Collect active listings
        active_items = await self._collect_active_items(category, search_terms)
        
        # Collect sold items for price history
        sold_items = await self._collect_sold_items(category, search_terms)
        
        # Save category data
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        active_file = os.path.join(category_folder, f"active_items_{timestamp}.csv")
        sold_file = os.path.join(category_folder, f"sold_items_{timestamp}.csv")
        
        save_items_to_csv(active_items, active_file)
        save_items_to_csv(sold_items, sold_file)
        
        # Create a metadata file to track the collection
        metadata = {
            "category_id": category_id,
            "category_name": category_name,
            "timestamp": datetime.now().isoformat(),
            "active_items_count": len(active_items),
            "sold_items_count": len(sold_items),
            "active_items_file": active_file,
            "sold_items_file": sold_file
        }
        
        metadata_file = os.path.join(category_folder, f"metadata_{timestamp}.json")
        save_json(metadata, metadata_file)
        
        # Update stats
        self.stats["items_collected"] += len(active_items) + len(sold_items)
    
    def _get_search_terms_for_category(self, category: Dict) -> List[str]:
        """Get appropriate search terms for a category"""
        category_name = category["name"]
        base_terms = [category_name, f"used {category_name}"]
        
        # Add category-specific terms
        if category["id"] == "9355":  # Laptops
            return base_terms + ["refurbished laptop", "macbook", "thinkpad"]
        elif category["id"] == "15032":  # Cell Phones
            return base_terms + ["used iphone", "used samsung galaxy"]
        else:
            return base_terms
    
    async def _collect_active_items(self, category: Dict, search_terms: List[str]) -> List[Dict]:
        """Collect active listings for a category using multiple search terms"""
        category_id = category["id"]
        collected_items = []
        
        for search_term in search_terms:
            try:
                # Make the API request
                response = await self.ebay_client.search_items(
                    keywords=search_term,
                    category_id=category_id,
                    limit=min(50, self.sample_size_per_category // len(search_terms))
                )
                
                if not response.success:
                    continue
                
                # Extract items from response
                items = response.data.get("itemSummaries", [])
                
                # Process each item
                for item in items:
                    try:
                        # Generate a consistent item ID
                        item_id = item.get("itemId") or generate_item_id(item)
                        
                        # Get more detailed information about the item
                        details_response = await self.ebay_client.get_item_details(item_id, use_cache=True)
                        
                        if details_response.success:
                            item_details = details_response.data
                            
                            # Process and add the item
                            processed_item = self._process_item_data(item, item_details)
                            collected_items.append(processed_item)
                            
                            # Small delay to avoid overwhelming the API
                            await asyncio.sleep(0.1)
                    except Exception as e:
                        self.logger.error(f"Error processing item: {str(e)}")
                
                # Add a delay between search terms
                await asyncio.sleep(1.0)
                
            except Exception as e:
                self.logger.error(f"Error collecting items: {str(e)}")
        
        # Deduplicate items
        unique_items = []
        seen_ids = set()
        for item in collected_items:
            if item["itemId"] not in seen_ids:
                unique_items.append(item)
                seen_ids.add(item["itemId"])
        
        return unique_items
    
    async def _collect_sold_items(self, category: Dict, search_terms: List[str]) -> List[Dict]:
        """Collect sold items data for price history"""
        category_id = category["id"]
        sold_items = []
        
        for search_term in search_terms[:2]:
            try:
                # Make the API request for sold items
                response = await self.ebay_client.get_sold_items(
                    keywords=search_term,
                    category_id=category_id,
                    days_back=self.history_window,
                    limit=min(50, self.sample_size_per_category // len(search_terms))
                )
                
                if not response.success:
                    continue
                
                # The structure of the sold items response is different
                items = []
                try:
                    search_result = response.data.get("findCompletedItemsResponse", [{}])[0]
                    result_items = search_result.get("searchResult", [{}])[0].get("item", [])
                    items.extend(result_items)
                except (KeyError, IndexError) as e:
                    self.logger.warning(f"Error parsing sold items response: {str(e)}")
                
                # Process sold items
                for item in items:
                    try:
                        processed_item = self._process_sold_item_data(item)
                        sold_items.append(processed_item)
                    except Exception as e:
                        self.logger.error(f"Error processing sold item: {str(e)}")
                
                # Add a delay between search terms
                await asyncio.sleep(1.0)
                
            except Exception as e:
                self.logger.error(f"Error collecting sold items: {str(e)}")
        
        # Deduplicate sold items
        unique_sold_items = []
        seen_ids = set()
        for item in sold_items:
            if item["itemId"] not in seen_ids:
                unique_sold_items.append(item)
                seen_ids.add(item["itemId"])
        
        return unique_sold_items
    
    def _process_item_data(self, item_summary: Dict, item_details: Dict) -> Dict:
        """Process and normalize item data from API responses"""
        # Basic processing - simplified for brevity
        return {
            "itemId": item_summary.get("itemId", ""),
            "title": item_summary.get("title", ""),
            "condition": item_summary.get("condition", ""),
            "price": self._extract_price(item_summary.get("price", {})),
            "currency": self._extract_currency(item_summary.get("price", {})),
            "category_id": item_details.get("categoryId", ""),
            "listing_date": item_summary.get("itemCreationDate", ""),
            "end_date": item_summary.get("itemEndDate", ""),
            "collection_date": datetime.now().isoformat(),
            "url": item_summary.get("itemWebUrl", "")
        }
    
    def _process_sold_item_data(self, item: Dict) -> Dict:
        """Process sold item data which has a different structure"""
        listing_info = item.get("listingInfo", {})
        selling_status = item.get("sellingStatus", [{}])[0]
        
        return {
            "itemId": item.get("itemId", ""),
            "title": item.get("title", [""])[0] if isinstance(item.get("title"), list) else item.get("title", ""),
            "condition": item.get("condition", {}).get("conditionDisplayName", ""),
            "price": float(selling_status.get("currentPrice", [{}])[0].get("__value__", 0)),
            "currency": selling_status.get("currentPrice", [{}])[0].get("@currencyId", "USD"),
            "category_id": item.get("primaryCategory", [{}])[0].get("categoryId", ""),
            "listing_date": listing_info.get("startTime", ""),
            "end_date": listing_info.get("endTime", ""),
            "collection_date": datetime.now().isoformat(),
            "is_sold": True,
            "sold_date": listing_info.get("endTime", "")
        }
    
    def _extract_price(self, price_data: Dict) -> float:
        """Extract price value from price object"""
        if isinstance(price_data, dict):
            value = price_data.get("value")
            if value is not None:
                try:
                    return float(value)
                except (ValueError, TypeError):
                    pass
        return 0.0
    
    def _extract_currency(self, price_data: Dict) -> str:
        """Extract currency from price object"""
        if isinstance(price_data, dict):
            return price_data.get("currency", "USD")
        return "USD"
'''

# Create the data_collector.py file
data_collector_path = "/data/chats/p6wyr/workspace/src/data/data_collector.py"
await terminal.run(f"rm -f {data_collector_path}")  # Delete if exists to avoid appending
await terminal.run(f"touch {data_collector_path}")
with open(data_collector_path, "w") as f:
    f.write(data_collector_code)

# Now update our main driver script
main_script = '''import os
import asyncio
import argparse
import json
from datetime import datetime
import pandas as pd
from typing import Dict, List, Optional, Any

from src.api.ebay_api_client import EbayAPIClient
from src.data.data_collector import DataCollector
from src.data.data_preprocessor import DataPreprocessor
from src.data.feature_extractor import FeatureExtractor
from src.utils.logger import Logger

class EbayPricePredictionPipeline:
    """
    Main driver class that integrates all components of the eBay second-hand item price prediction system
    """
    def __init__(self, config_path: str = "/data/chats/p6wyr/workspace/config/config.json"):
        """Initialize the pipeline with configuration"""
        self.logger = Logger().get_logger()
        self.logger.info("Initializing eBay Price Prediction Pipeline")
        
        # Load configuration
        self.config_path = config_path
        self._load_config()
        
        # Initialize components
        self.ebay_client = None  # Initialize later to avoid unnecessary token fetching
        self.data_collector = None
        self.data_preprocessor = DataPreprocessor(config_path=config_path)
        self.feature_extractor = FeatureExtractor(config_path=config_path)
    
    def _load_config(self) -> None:
        """Load configuration from file"""
        try:
            with open(self.config_path, 'r') as f:
                self.config = json.load(f)
            self.logger.info(f"Configuration loaded successfully from {self.config_path}")
        except Exception as e:
            self.logger.error(f"Error loading configuration: {str(e)}")
            raise
    
    async def initialize_api_client(self) -> None:
        """Initialize the eBay API client"""
        self.logger.info("Initializing eBay API client")
        self.ebay_client = EbayAPIClient(config_path=self.config_path)
        success = await self.ebay_client.authenticate()
        
        if success:
            self.logger.info("eBay API client successfully authenticated")
        else:
            self.logger.error("Failed to authenticate eBay API client")
            raise RuntimeError("API authentication failed")
    
    async def run_data_collection(self, categories: Optional[List[Dict]] = None) -> Dict:
        """Run the data collection process"""
        self.logger.info("Starting data collection process")
        
        # Initialize data collector
        self.data_collector = DataCollector(config_path=self.config_path)
        
        # If specific categories provided, override the defaults
        if categories:
            self.data_collector.categories = categories
            self.logger.info(f"Using {len(categories)} provided categories for collection")
        
        # Run collection process
        stats = await self.data_collector.collect_data()
        return stats
    
    def run_data_preprocessing(self) -> Dict:
        """Run the data preprocessing step"""
        self.logger.info("Starting data preprocessing")
        stats = self.data_preprocessor.process_all_data()
        return stats
    
    def run_feature_extraction(self) -> Dict:
        """Run the feature extraction process"""
        self.logger.info("Starting feature extraction")
        stats = self.feature_extractor.extract_all_features()
        return stats
    
    async def run_complete_pipeline(self, categories: Optional[List[Dict]] = None) -> Dict:
        """Run the complete data pipeline: collection, preprocessing, and feature extraction"""
        pipeline_stats = {
            "start_time": datetime.now().isoformat(),
            "collection": None,
            "preprocessing": None,
            "feature_extraction": None,
            "end_time": None
        }
        
        try:
            # Initialize API client
            await self.initialize_api_client()
            
            # Run data collection
            collection_stats = await self.run_data_collection(categories)
            pipeline_stats["collection"] = collection_stats
            
            # Run data preprocessing
            preprocessing_stats = self.run_data_preprocessing()
            pipeline_stats["preprocessing"] = preprocessing_stats
            
            # Run feature extraction
            extraction_stats = self.run_feature_extraction()
            pipeline_stats["feature_extraction"] = extraction_stats
            
            # Complete pipeline
            pipeline_stats["end_time"] = datetime.now().isoformat()
            
            return pipeline_stats
            
        except Exception as e:
            self.logger.error(f"Error in pipeline execution: {str(e)}")
            pipeline_stats["error"] = str(e)
            pipeline_stats["end_time"] = datetime.now().isoformat()
            return pipeline_stats

async def main():
    """Main entry point for the application"""
    parser = argparse.ArgumentParser(description='eBay Second-hand Item Price Prediction System')
    parser.add_argument('--config', type=str, default="/data/chats/p6wyr/workspace/config/config.json", 
                        help='Path to configuration file')
    parser.add_argument('--mode', type=str, choices=['collect', 'preprocess', 'extract', 'full'], 
                        default='full', help='Operation mode')
    parser.add_argument('--category', type=str, help='Specific category ID to process')
    
    # For Jupyter notebook environment, use default arguments
    args = parser.parse_args([])
    
    # Initialize logger
    logger = Logger().get_logger()
    logger.info(f"Starting application in {args.mode} mode")
    
    # Initialize pipeline
    pipeline = EbayPricePredictionPipeline(config_path=args.config)
    
    # Prepare categories if specific one provided
    categories = None
    if args.category:
        # Look up category name from config
        category_name = "Unknown"
        for cat in pipeline.config.get("data", {}).get("categories", []):
            if cat["id"] == args.category:
                category_name = cat["name"]
                break
        
        categories = [{"id": args.category, "name": category_name}]
    
    # Execute requested operation
    if args.mode == 'collect':
        await pipeline.initialize_api_client()
        stats = await pipeline.run_data_collection(categories)
    
    elif args.mode == 'preprocess':
        stats = pipeline.run_data_preprocessing()
    
    elif args.mode == 'extract':
        stats = pipeline.run_feature_extraction()
    
    elif args.mode == 'full':
        stats = await pipeline.run_complete_pipeline(categories)
    
    return stats

# For Jupyter notebook environment
print("Starting eBay Price Prediction Pipeline")
pipeline_stats = await main()  # Use await directly in Jupyter
print(f"Pipeline completed with {pipeline_stats.get('collection', {}).get('items_collected', 0)} items collected")
'''

# Save main script to a file that can be executed in Jupyter
main_file_path = '/data/chats/p6wyr/workspace/main_driver.py'
await terminal.run(f"touch {main_file_path}")
with open(main_file_path, 'w') as f:
    f.write(main_script)

print("Created data_collector.py and main_driver.py successfully.")
print("To use in Jupyter, import the newly created modules and run the main script.")

Created data_collector.py and main_driver.py successfully.
To use in Jupyter, import the newly created modules and run the main script.


In [14]:
# Let's check the directory structure to understand what files and data we have
from metagpt.tools.libs.terminal import Terminal
terminal = Terminal()
print("Directory structure:")
output = await terminal.run("ls -la /data/chats/p6wyr/workspace/")
print(output)

# Check if we have any data in the data directory
print("\nData directory contents:")
output = await terminal.run("ls -la /data/chats/p6wyr/workspace/data/")
print(output)

# Check if we have any raw data
print("\nRaw data directory contents:")
output = await terminal.run("ls -la /data/chats/p6wyr/workspace/data/raw/")
print(output)

# Check if we have any processed data
print("\nProcessed data directory contents:")
output = await terminal.run("ls -la /data/chats/p6wyr/workspace/data/processed/")
print(output)

# Check if we have any features data
print("\nFeatures directory contents:")
output = await terminal.run("ls -la /data/chats/p6wyr/workspace/data/features/")
print(output)

# Check config file
print("\nConfig file contents:")
import json
try:
    with open('/data/chats/p6wyr/workspace/config/config.json', 'r') as f:
        config = json.load(f)
    print(json.dumps(config, indent=2))
except:
    print("Config file not found or invalid")

Directory structure:
total 10732
drwxr-xr-x 114 root root 1859584 Apr 22 23:29 .
drwxr-xr-x 280 root root 7307264 Apr 22 21:04 ..
-rw-r--r--   1 root root  828536 Apr 22 21:25 .MGXEnv.json
drwxr-xr-x  50 root root  208896 Apr 22 20:55 .git
-rw-r--r--   1 root root    1472 Apr 22 20:55 .gitattributes
-rw-r--r--   1 root root     476 Apr 22 20:55 .gitignore
-rw-r--r--   1 root root  121574 Apr 22 23:29 .timeline.json
-rw-r--r--   1 root root    1769 Apr 22 21:22 README.md
-rw-r--r--   1 root root  380696 Apr 22 23:28 code.ipynb
drwxr-xr-x   3 root root    8192 Apr 22 21:22 config
drwxr-xr-x  13 root root   49152 Apr 22 21:17 data
drwxr-xr-x   7 root root   45056 Apr 22 21:14 docs
drwxr-xr-x   2 root root    4096 Apr 22 21:22 logs
-rw-r--r--   1 root root    6906 Apr 22 23:28 main_driver.py
-rw-r--r--   1 root root     301 Apr 22 21:22 requirements.txt
drwxr-xr-x  34 root root  163840 Apr 22 21:17 src


Data directory contents:

total 1908
drwxr-xr-x  13 root root   49152 Apr 22 21:17 .
d

In [15]:
I'll now implement the Deep Reinforcement Learning model for price prediction based on our system design. This implementation will integrate with our existing data pipeline.

```python
from metagpt.tools.libs.terminal import Terminal
terminal = Terminal()

# Install required packages for the DRL model
await terminal.run('pip install torch gym numpy stable-baselines3 tensorboard')

import os
import json
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from typing import Dict, List, Tuple, Optional, Any
from collections import deque, namedtuple
import random
import glob
from datetime import datetime
import pickle
import copy
from tqdm import tqdm
import matplotlib.pyplot as plt

from src.utils.logger import Logger

# Define constants for the DRL model
Experience = namedtuple('Experience', ['state', 'action', 'reward', 'next_state', 'done'])

class ReplayBuffer:
    """
    Replay buffer for storing and sampling experiences for DRL training
    """
    def __init__(self, capacity: int):
        """Initialize replay buffer with given capacity"""
        self.buffer = deque(maxlen=capacity)
    
    def push(self, state, action, reward, next_state, done):
        """Add experience to the buffer"""
        self.buffer.append(Experience(state, action, reward, next_state, done))
    
    def sample(self, batch_size: int):
        """Sample random batch of experiences"""
        experiences = random.sample(self.buffer, batch_size)
        states = torch.from_numpy(np.vstack([e.state for e in experiences])).float()
        actions = torch.from_numpy(np.vstack([e.action for e in experiences])).long()
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences])).float()
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences])).float()
        dones = torch.from_numpy(np.vstack([e.done for e in experiences]).astype(np.uint8)).float()
        return states, actions, rewards, next_states, dones
    
    def __len__(self):
        """Current size of replay buffer"""
        return len(self.buffer)

class DQNetwork(nn.Module):
    """
    Deep Q-Network for price prediction
    """
    def __init__(self, state_size: int, action_size: int, config: Dict):
        """
        Initialize DQN with state and action dimensions
        
        Args:
            state_size (int): Dimension of state space
            action_size (int): Dimension of action space
            config (Dict): Model configuration
        """
        super(DQNetwork, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        
        # Get configuration parameters
        hidden_layer_size = config.get("hidden_layer_size", 128)
        num_hidden_layers = config.get("num_hidden_layers", 3)
        
        # Create the network layers
        layers = [nn.Linear(state_size, hidden_layer_size), nn.ReLU()]
        
        # Add hidden layers
        for _ in range(num_hidden_layers - 1):
            layers.append(nn.Linear(hidden_layer_size, hidden_layer_size))
            layers.append(nn.ReLU())
        
        # Add output layer
        layers.append(nn.Linear(hidden_layer_size, action_size))
        
        # Create sequential model
        self.network = nn.Sequential(*layers)
    
    def forward(self, state):
        """Forward pass through network"""
        return self.network(state)

class PricePredictionDRLAgent:
    """
    Deep Reinforcement Learning agent for price prediction
    """
    def __init__(self, state_size: int, action_size: int, category_id: str, config_path: str = "/data/chats/p6wyr/workspace/config/config.json"):
        """
        Initialize the DRL agent
        
        Args:
            state_size (int): Dimension of state space
            action_size (int): Dimension of action space
            category_id (str): Category ID
            config_path (str): Path to configuration file
        """
        self.logger = Logger().get_logger()
        self.logger.info("Initializing DRL agent for price prediction")
        
        # Load configuration
        self.config_path = config_path
        self._load_config()
        
        # Get DRL model configuration
        drl_config = self.config.get("model", {}).get("drl", {})
        
        # Set up model parameters
        self.state_size = state_size
        self.action_size = action_size
        self.category_id = category_id
        self.learning_rate = drl_config.get("learning_rate", 0.001)
        self.discount_factor = drl_config.get("discount_factor", 0.95)
        self.exploration_rate = drl_config.get("exploration_rate", 0.1)
        self.batch_size = drl_config.get("batch_size", 64)
        self.memory_size = drl_config.get("memory_size", 10000)
        self.target_update_freq = drl_config.get("target_update_frequency", 1000)
        
        # Set device (GPU if available)
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.logger.info(f"Using device: {self.device}")
        
        # Initialize networks
        self.qnetwork_local = DQNetwork(state_size, action_size, drl_config).to(self.device)
        self.qnetwork_target = DQNetwork(state_size, action_size, drl_config).to(self.device)
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=self.learning_rate)
        
        # Initialize replay memory
        self.memory = ReplayBuffer(self.memory_size)
        
        # Initialize step counter for target network updates
        self.t_step = 0
        
        # Set up model directories
        self.models_dir = f"/data/chats/p6wyr/workspace/data/models/category_{self.category_id}"
        os.makedirs(self.models_dir, exist_ok=True)
        
        # Set up tensorboard logging
        self.tb_dir = f"/data/chats/p6wyr/workspace/logs/tensorboard/category_{self.category_id}"
        os.makedirs(self.tb_dir, exist_ok=True)
        self.writer = SummaryWriter(log_dir=self.tb_dir)
        
        # Reward function weights
        reward_config = self.config.get("model", {}).get("reward_function", {})
        self.profit_weight = reward_config.get("profit_weight", 0.7)
        self.time_weight = reward_config.get("time_weight", 0.3)
        
        # Track statistics
        self.stats = {
            "episodes": 0,
            "avg_rewards": [],
            "avg_losses": [],
            "exploration_rate": []
        }
    
    def _load_config(self) -> None:
        """
        Load configuration from file
        """
        try:
            with open(self.config_path, 'r') as f:
                self.config = json.load(f)
            self.logger.info("Configuration loaded successfully")
        except Exception as e:
            self.logger.error(f"Error loading configuration: {str(e)}")
            # Use default configuration
            self.config = {
                "model": {
                    "drl": {
                        "model_type": "dqn",
                        "learning_rate": 0.001,
                        "discount_factor": 0.95,
                        "exploration_rate": 0.1,
                        "target_update_frequency": 1000,
                        "batch_size": 64,
                        "memory_size": 10000,
                        "num_hidden_layers": 3,
                        "hidden_layer_size": 128,
                        "activation": "relu"
                    },
                    "reward_function": {
                        "profit_weight": 0.7,
                        "time_weight": 0.3
                    }
                }
            }
    
    def step(self, state, action, reward, next_state, done):
        """
        Update the agent's knowledge based on experience
        
        Args:
            state: Current state
            action: Selected action
            reward: Received reward
            next_state: Next state
            done: Whether episode is done
        """
        # Save experience in replay memory
        self.memory.push(state, action, reward, next_state, done)
        
        # Learn every self.batch_size steps, if enough samples in memory
        self.t_step = (self.t_step + 1) % self.target_update_freq
        if len(self.memory) > self.batch_size:
            experiences = self.memory.sample(self.batch_size)
            loss = self._learn(experiences)
            return loss
        
        # Update target network
        if self.t_step == 0:
            self._update_target_network()
            return 0
        
        return 0
    
    def act(self, state, evaluate=False):
        """
        Choose action based on current state using epsilon-greedy policy
        
        Args:
            state: Current state
            evaluate (bool): Whether to evaluate (no exploration)
            
        Returns:
            int: Selected action index
        """
        # Convert state to tensor
        state = torch.from_numpy(state).float().unsqueeze(0).to(self.device)
        
        # Set network to evaluation mode
        self.qnetwork_local.eval()
        
        with torch.no_grad():
            action_values = self.qnetwork_local(state)
        
        # Set network back to training mode
        self.qnetwork_local.train()
        
        # Epsilon-greedy action selection
        if not evaluate and random.random() < self.exploration_rate:
            return random.choice(np.arange(self.action_size))
        else:
            return np.argmax(action_values.cpu().data.numpy())
    
    def _learn(self, experiences):
        """
        Update value parameters using batch of experiences
        
        Args:
            experiences: Tuple of (states, actions, rewards, next_states, dones)
            
        Returns:
            float: Loss value
        """
        states, actions, rewards, next_states, dones = experiences
        
        # Move to device
        states = states.to(self.device)
        actions = actions.to(self.device)
        rewards = rewards.to(self.device)
        next_states = next_states.to(self.device)
        dones = dones.to(self.device)
        
        # Get max predicted Q values for next states from target model
        Q_targets_next = self.qnetwork_target(next_states).detach().max(1)[0].unsqueeze(1)
        
        # Compute Q targets for current states
        Q_targets = rewards + (self.discount_factor * Q_targets_next * (1 - dones))
        
        # Get expected Q values from local model
        Q_expected = self.qnetwork_local(states).gather(1, actions)
        
        # Compute loss
        loss = F.mse_loss(Q_expected, Q_targets)
        
        # Minimize the loss
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        
        return loss.item()
    
    def _update_target_network(self):
        """Update target network with parameters from local network"""
        self.qnetwork_target.load_state_dict(self.qnetwork_local.state_dict())
    
    def save_model(self, episode: int = None):
        """
        Save the agent's model
        
        Args:
            episode (int): Current episode number (used for filename)
        """
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        episode_str = f"_episode_{episode}" if episode is not None else ""
        
        model_path = os.path.join(self.models_dir, f"drl_model_{timestamp}{episode_str}.pt")
        
        torch.save({
            'qnetwork_local_state_dict': self.qnetwork_local.state_dict(),
            'qnetwork_target_state_dict': self.qnetwork_target.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'exploration_rate': self.exploration_rate,
            'stats': self.stats
        }, model_path)
        
        self.logger.info(f"Model saved to {model_path}")
        return model_path
    
    def load_model(self, model_path: str):
        """
        Load a saved model
        
        Args:
            model_path (str): Path to saved model
        """
        if not os.path.exists(model_path):
            self.logger.error(f"Model file not found: {model_path}")
            return False
        
        try:
            checkpoint = torch.load(model_path, map_location=self.device)
            
            self.qnetwork_local.load_state_dict(checkpoint['qnetwork_local_state_dict'])
            self.qnetwork_target.load_state_dict(checkpoint['qnetwork_target_state_dict'])
            self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            self.exploration_rate = checkpoint.get('exploration_rate', 0.1)
            self.stats = checkpoint.get('stats', self.stats)
            
            self.logger.info(f"Model loaded from {model_path}")
            return True
        except Exception as e:
            self.logger.error(f"Error loading model: {str(e)}")
            return False
    
    def compute_reward(self, state, action, next_state, price, sold_price=None, time_to_sell=None):
        """
        Compute the reward for a state-action pair
        
        Args:
            state: Current state
            action: Action taken
            next_state: Resulting state
            price: Original item price
            sold_price: Final selling price (if sold)
            time_to_sell: Time to sell in days (if sold)
            
        Returns:
            float: Computed reward
        """
        # If item sold, use actual profit and time
        if sold_price is not None and time_to_sell is not None:
            profit_margin = (sold_price - price) / price if price > 0 else 0
            
            # Normalize time to sell (consider 30 days as max)
            norm_time = max(0, 1 - (time_to_sell / 30)) if time_to_sell > 0 else 0
            
            # Combine profit and time components
            reward = (self.profit_weight * profit_margin) + (self.time_weight * norm_time)
            
            return reward * 10  # Scale reward
        
        # If not sold yet, use market-based reward estimation
        price_competitiveness = next_state[4] if len(next_state) > 4 else 0.5  # Use price competitiveness feature
        conversion_rate = next_state[7] if len(next_state) > 7 else 0.5  # Use conversion rate feature
        
        # Estimate reward based on price position and market conditions
        estimated_reward = (price_competitiveness * 0.6) + (conversion_rate * 0.4)
        
        return estimated_reward * 5  # Scale reward
    
    def decay_exploration_rate(self, decay_factor=0.995, min_rate=0.01):
        """
        Decay the exploration rate
        
        Args:
            decay_factor (float): Factor to decay by
            min_rate (float): Minimum exploration rate
        """
        self.exploration_rate = max(min_rate, self.exploration_rate * decay_factor)
        return self.exploration_rate


class PriceEnvironment:
    """
    Environment for price prediction, simulating market dynamics
    """
    def __init__(self, features_df: pd.DataFrame, category_id: str, num_price_levels: int = 10):
        """
        Initialize the environment
        
        Args:
            features_df (pd.DataFrame): DataFrame with item features
            category_id (str): Category ID
            num_price_levels (int): Number of discrete price levels
        """
        self.logger = Logger().get_logger()
        self.features_df = features_df
        self.category_id = category_id
        
        # Get sold items for statistics
        self.sold_items = features_df[features_df['is_sold'] == 1]
        self.unsold_items = features_df[features_df['is_sold'] == 0]
        
        # Extract price range
        self.min_price = features_df['price'].min()
        self.max_price = features_df['price'].max()
        self.avg_price = features_df['price'].mean()
        
        # Calculate price levels (actions)
        self.num_price_levels = num_price_levels
        self.price_levels = np.linspace(0.5, 1.5, num_price_levels)  # 0.5x to 1.5x of item's base price
        
        # Current state info
        self.current_item_idx = 0
        self.current_state = None
        self.base_price = 0
        
        self.logger.info(f"Initialized price environment with {len(features_df)} items, {num_price_levels} price levels")
    
    def reset(self, item_idx=None):
        """
        Reset the environment and get new item
        
        Args:
            item_idx (int): Index of item to use (None for random)
            
        Returns:
            np.array: Initial state
        """
        # Select item (random if not specified)
        if item_idx is None:
            self.current_item_idx = np.random.randint(0, len(self.features_df))
        else:
            self.current_item_idx = item_idx % len(self.features_df)
        
        # Get item data
        item = self.features_df.iloc[self.current_item_idx]
        
        # Extract state features (excluding ID, price, is_sold, category_id, days_to_sell)
        state_columns = [col for col in self.features_df.columns 
                         if col not in ['itemId', 'price', 'is_sold', 'category_id', 'days_to_sell']]
        
        # Set current state
        self.current_state = item[state_columns].values.astype(np.float32)
        
        # Set base price
        self.base_price = item['price']
        
        return self.current_state
    
    def step(self, action):
        """
        Take a step in the environment
        
        Args:
            action (int): Action index (price level)
            
        Returns:
            tuple: (next_state, reward, done, info)
        """
        # Get selected price level
        price_factor = self.price_levels[action]
        suggested_price = self.base_price * price_factor
        
        # Get current item
        item = self.features_df.iloc[self.current_item_idx]
        
        # Check if item sold in historical data
        is_sold = item['is_sold'] == 1
        days_to_sell = item.get('days_to_sell', 30)  # Default to 30 days if unknown
        
        # Get actual sold price if available
        sold_price = item['price'] if is_sold else None
        
        # Calculate reward
        reward = self._calculate_reward(suggested_price, is_sold, sold_price, days_to_sell)
        
        # Return the same state (as this is not a sequential environment)
        next_state = self.current_state
        
        # Episode is done after one step (predict price only once per item)
        done = True
        
        # Additional info
        info = {
            'item_id': item.get('itemId', self.current_item_idx),
            'base_price': self.base_price,
            'suggested_price': suggested_price,
            'price_factor': price_factor,
            'is_sold': is_sold,
            'days_to_sell': days_to_sell if is_sold else None,
            'sold_price': sold_price
        }
        
        return next_state, reward, done, info
    
    def _calculate_reward(self, suggested_price, is_sold, sold_price, days_to_sell):
        """
        Calculate reward for the action
        
        Args:
            suggested_price (float): Price suggested by agent
            is_sold (bool): Whether item was sold
            sold_price (float): Actual sold price
            days_to_sell (float): Days taken to sell
            
        Returns:
            float: Reward value
        """
        # For sold items, compare the suggested price to actual sold price
        if is_sold and sold_price is not None:
            # Penalize overpricing that could prevent sale
            if suggested_price > 1.2 * sold_price:
                return -1.0
            
            # Reward accurate pricing (within 10% of actual)
            if 0.9 * sold_price <= suggested_price <= 1.1 * sold_price:
                return 1.0
            
            # Smaller reward for reasonable pricing (within 20% of actual)
            if 0.8 * sold_price <= suggested_price <= 1.2 * sold_price:
                return 0.5
            
            # Penalize significantly underpricing
            if suggested_price < 0.8 * sold_price:
                return -0.5
        
        # For unsold items, evaluate based on market dynamics
        else:
            # Get similar items
            similar_sold = self._find_similar_items(is_sold=True)
            
            if len(similar_sold) > 0:
                avg_similar_price = similar_sold['price'].mean()
                
                # Reward pricing close to successful similar items
                if 0.9 * avg_similar_price <= suggested_price <= 1.1 * avg_similar_price:
                    return 0.8
                
                # Smaller reward for reasonable pricing
                if 0.8 * avg_similar_price <= suggested_price <= 1.2 * avg_similar_price:
                    return 0.4
                
                # Penalize significant deviation from market
                return -0.3
            
        # Default small reward if we can't determine better
        return 0.1
    
    def _find_similar_items(self, is_sold=True):
        """
        Find items similar to current item
        
        Args:
            is_sold (bool): Whether to find sold or unsold items
            
        Returns:
            pd.DataFrame: Similar items
        """
        # Current item
        item = self.features_df.iloc[self.current_item_idx]
        
        # Filter by sold status
        items_subset = self.sold_items if is_sold else self.unsold_items
        
        if len(items_subset) == 0:
            return pd.DataFrame()
        
        # Extract state features (excluding ID, price, is_sold, category_id, days_to_sell)
        feature_columns = [col for col in self.features_df.columns 
                          if col not in ['itemId', 'price', 'is_sold', 'category_id', 'days_to_sell']]
        
        if len(feature_columns) == 0:
            return items_subset.head(5)
        
        # Simple similarity: use feature values to find similar items
        # For more complex scenarios, consider using cosine similarity or KNN
        
        # Extract feature values for current item
        item_features = item[feature_columns].values
        
        # Calculate simple Euclidean distance for each item
        def calculate_distance(row):
            return np.sum((row[feature_columns].values - item_features) ** 2)
        
        # Calculate distances and sort by similarity
        items_subset = items_subset.copy()
        items_subset['distance'] = items_subset.apply(calculate_distance, axis=1)
        items_subset = items_subset.sort_values('distance')
        
        # Return top 5 most similar items
        return items_subset.head(5)


class PricePredictionTrainer:
    """
    Trainer class for price prediction model
    """
    def __init__(self, category_id: str, config_path: str = "/data/chats/p6wyr/workspace/config/config.json"):
        """
        Initialize the trainer
        
        Args:
            category_id (str): Category ID to train model for
            config_path (str): Path to configuration file
        """
        self.logger = Logger().get_logger()
        self.logger.info(f"Initializing price prediction trainer for category {category_id}")
        
        self.category_id = category_id
        self.config_path = config_path
        
        # Load configuration
        self._load_config()
        
        # Set paths
        self.features_dir = f"/data/chats/p6wyr/workspace/data/features/category_{category_id}"
        self.models_dir = f"/data/chats/p6wyr/workspace/data/models/category_{category_id}"
        
        os.makedirs(self.models_dir, exist_ok=True)
        
        # Load data
        self.features_df = self._load_features()
        
        # Set up environment and agent
        if self.features_df is not None:
            self._setup_env_and_agent()
        else:
            self.logger.error("Failed to load features data. Cannot set up environment and agent.")
            self.environment = None
            self.agent = None
    
    def _load_config(self) -> None:
        """
        Load configuration from file
        """
        try:
            with open(self.config_path, 'r') as f:
                self.config = json.load(f)
            self.logger.info("Configuration loaded successfully")
        except Exception as e:
            self.logger.error(f"Error loading configuration: {str(e)}")
            # Use default configuration
            self.config = {
                "model": {
                    "drl": {
                        "model_type": "dqn",
                        "learning_rate": 0.001,
                        "discount_factor": 0.95,
                        "exploration_rate": 0.2,
                        "target_update_frequency": 1000,
                        "batch_size": 64,
                        "memory_size": 10000,
                        "num_hidden_layers": 3,
                        "hidden_layer_size": 128,
                        "activation": "relu"
                    }
                },
                "data": {
                    "min_items_for_training": 500
                }
            }
    
    def _load_features(self) -> Optional[pd.DataFrame]:
        """
        Load features data for the category
        
        Returns:
            Optional[pd.DataFrame]: Features DataFrame or None if not found
        """
        # Find the latest features file
        feature_files = sorted(glob.glob(os.path.join(self.features_dir, "features_*.csv")))
        
        if not feature_files:
            self.logger.error(f"No feature files found in {self.features_dir}")
            return None
        
        # Load latest features file
        latest_features_file = feature_files[-1]
        self.logger.info(f"Loading features from {latest_features_file}")
        
        try:
            df = pd.read_csv(latest_features_file)
            self.logger.info(f"Loaded {len(df)} feature records")
            return df
        except Exception as e:
            self.logger.error(f"Error loading features: {str(e)}")
            return None
    
    def _setup_env_and_agent(self) -> None:
        """
        Set up environment and DRL agent
        """
        # Define number of price levels (actions)
        self.num_price_levels = 10
        
        # Create environment
        self.environment = PriceEnvironment(
            features_df=self.features_df,
            category_id=self.category_id,
            num_price_levels=self.num_price_levels
        )
        
        # Get state size (number of features)
        state_columns = [col for col in self.features_df.columns 
                         if col not in ['itemId', 'price', 'is_sold', 'category_id', 'days_to_sell']]
        state_size = len(state_columns)
        
        # Create agent
        self.agent = PricePredictionDRLAgent(
            state_size=state_size,
            action_size=self.num_price_levels,
            category_id=self.category_id,
            config_path=self.config_path
        )
        
        self.logger.info(f"Set up environment and agent with state size {state_size} and {self.num_price_levels} actions")
    
    def train(self, num_episodes: int = 2000, batch_size: int = 64, checkpoint_interval: int = 200):
        """
        Train the DRL agent
        
        Args:
            num_episodes (int): Number of training episodes
            batch_size (int): Batch size for training
            checkpoint_interval (int): Interval for saving model checkpoints
            
        Returns:
            Dict: Training statistics
        """
        if self.environment is None or self.agent is None:
            self.logger.error("Environment or agent not initialized. Cannot train.")
            return None
        
        self.logger.info(f"Starting training for {num_episodes} episodes")
        
        # Statistics tracking
        rewards = []
        losses = []
        exploration_rates = []
        
        # Sample item indices
        n_items = len(self.features_df)
        item_indices = np.random.choice(n_items, min(n_items, num_episodes), replace=(n_items < num_episodes))
        
        # Training loop
        for episode in tqdm(range(num_episodes)):
            # Reset environment with a specific item
            item_idx = item_indices[episode % len(item_indices)]
            state = self.environment.reset(item_idx)
            
            # Choose action
            action = self.agent.act(state)
            
            # Take action
            next_state, reward, done, info = self.environment.step(action)
            
            # Learn from experience
            loss = self.agent.step(state, action, reward, next_state, done)
            
            # Track statistics
            rewards.append(reward)
            if loss != 0:  # Only track non-zero losses
                losses.append(loss)
            exploration_rates.append(self.agent.exploration_rate)
            
            # Decay exploration rate
            if episode % 100 == 0:
                self.agent.decay_exploration_rate()
            
            # Log progress
            if episode % 100 == 0:
                avg_reward = np.mean(rewards[-100:]) if rewards else 0
                avg_loss = np.mean(losses[-100:]) if losses else 0
                
                self.logger.info(f"Episode {episode}/{num_episodes} | "
                                f"Avg Reward: {avg_reward:.4f} | "
                                f"Avg Loss: {avg_loss:.4f} | "
                                f"Exploration Rate: {self.agent.exploration_rate:.4f}")
                
                # Log to tensorboard
                self.agent.writer.add_scalar('Reward/Average', avg_reward, episode)
                self.agent.writer.add_scalar('Loss/Average', avg_loss, episode)
                self.agent.writer.add_scalar('Exploration/Rate', self.agent.exploration_rate, episode)
                
                # Save to agent stats
                self.agent.stats["avg_rewards"].append((episode, avg_reward))
                self.agent.stats["avg_losses

SyntaxError: unterminated string literal (detected at line 1) (3333821276.py, line 1)

In [16]:
[reflection on previous impl]
The error in my previous implementation is due to a syntax error in the code. The specific error is an "unterminated string literal" detected at line 1 of the input. Looking at the code, I can see that there is likely an incomplete string in the PricePredictionTrainer class implementation, specifically in the `train` method where the code is adding to the agent's statistics. The line with `self.agent.stats["avg_losses'` is incomplete and should be `self.agent.stats["avg_losses"].append((episode, avg_loss))`.

Additionally, the code also needs to properly finish the implementation of the `train` method and add a main execution block to demonstrate the DRL model in action. I need to ensure all strings are properly terminated and the entire implementation is syntactically correct.

[improved impl]:
```python
from metagpt.tools.libs.terminal import Terminal
terminal = Terminal()

# Install required packages for the DRL model
await terminal.run('pip install torch gym numpy stable-baselines3 tensorboard')

import os
import json
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from typing import Dict, List, Tuple, Optional, Any
from collections import deque, namedtuple
import random
import glob
from datetime import datetime
import pickle
import copy
from tqdm import tqdm
import matplotlib.pyplot as plt

from src.utils.logger import Logger

# Define constants for the DRL model
Experience = namedtuple('Experience', ['state', 'action', 'reward', 'next_state', 'done'])

class ReplayBuffer:
    """
    Replay buffer for storing and sampling experiences for DRL training
    """
    def __init__(self, capacity: int):
        """Initialize replay buffer with given capacity"""
        self.buffer = deque(maxlen=capacity)
    
    def push(self, state, action, reward, next_state, done):
        """Add experience to the buffer"""
        self.buffer.append(Experience(state, action, reward, next_state, done))
    
    def sample(self, batch_size: int):
        """Sample random batch of experiences"""
        experiences = random.sample(self.buffer, batch_size)
        states = torch.from_numpy(np.vstack([e.state for e in experiences])).float()
        actions = torch.from_numpy(np.vstack([e.action for e in experiences])).long()
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences])).float()
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences])).float()
        dones = torch.from_numpy(np.vstack([e.done for e in experiences]).astype(np.uint8)).float()
        return states, actions, rewards, next_states, dones
    
    def __len__(self):
        """Current size of replay buffer"""
        return len(self.buffer)

class DQNetwork(nn.Module):
    """
    Deep Q-Network for price prediction
    """
    def __init__(self, state_size: int, action_size: int, config: Dict):
        """
        Initialize DQN with state and action dimensions
        
        Args:
            state_size (int): Dimension of state space
            action_size (int): Dimension of action space
            config (Dict): Model configuration
        """
        super(DQNetwork, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        
        # Get configuration parameters
        hidden_layer_size = config.get("hidden_layer_size", 128)
        num_hidden_layers = config.get("num_hidden_layers", 3)
        
        # Create the network layers
        layers = [nn.Linear(state_size, hidden_layer_size), nn.ReLU()]
        
        # Add hidden layers
        for _ in range(num_hidden_layers - 1):
            layers.append(nn.Linear(hidden_layer_size, hidden_layer_size))
            layers.append(nn.ReLU())
        
        # Add output layer
        layers.append(nn.Linear(hidden_layer_size, action_size))
        
        # Create sequential model
        self.network = nn.Sequential(*layers)
    
    def forward(self, state):
        """Forward pass through network"""
        return self.network(state)

class PricePredictionDRLAgent:
    """
    Deep Reinforcement Learning agent for price prediction
    """
    def __init__(self, state_size: int, action_size: int, category_id: str, config_path: str = "/data/chats/p6wyr/workspace/config/config.json"):
        """
        Initialize the DRL agent
        
        Args:
            state_size (int): Dimension of state space
            action_size (int): Dimension of action space
            category_id (str): Category ID
            config_path (str): Path to configuration file
        """
        self.logger = Logger().get_logger()
        self.logger.info("Initializing DRL agent for price prediction")
        
        # Load configuration
        self.config_path = config_path
        self._load_config()
        
        # Get DRL model configuration
        drl_config = self.config.get("model", {}).get("drl", {})
        
        # Set up model parameters
        self.state_size = state_size
        self.action_size = action_size
        self.category_id = category_id
        self.learning_rate = drl_config.get("learning_rate", 0.001)
        self.discount_factor = drl_config.get("discount_factor", 0.95)
        self.exploration_rate = drl_config.get("exploration_rate", 0.1)
        self.batch_size = drl_config.get("batch_size", 64)
        self.memory_size = drl_config.get("memory_size", 10000)
        self.target_update_freq = drl_config.get("target_update_frequency", 1000)
        
        # Set device (GPU if available)
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.logger.info(f"Using device: {self.device}")
        
        # Initialize networks
        self.qnetwork_local = DQNetwork(state_size, action_size, drl_config).to(self.device)
        self.qnetwork_target = DQNetwork(state_size, action_size, drl_config).to(self.device)
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=self.learning_rate)
        
        # Initialize replay memory
        self.memory = ReplayBuffer(self.memory_size)
        
        # Initialize step counter for target network updates
        self.t_step = 0
        
        # Set up model directories
        self.models_dir = f"/data/chats/p6wyr/workspace/data/models/category_{self.category_id}"
        os.makedirs(self.models_dir, exist_ok=True)
        
        # Set up tensorboard logging
        self.tb_dir = f"/data/chats/p6wyr/workspace/logs/tensorboard/category_{self.category_id}"
        os.makedirs(self.tb_dir, exist_ok=True)
        self.writer = SummaryWriter(log_dir=self.tb_dir)
        
        # Reward function weights
        reward_config = self.config.get("model", {}).get("reward_function", {})
        self.profit_weight = reward_config.get("profit_weight", 0.7)
        self.time_weight = reward_config.get("time_weight", 0.3)
        
        # Track statistics
        self.stats = {
            "episodes": 0,
            "avg_rewards": [],
            "avg_losses": [],
            "exploration_rate": []
        }
    
    def _load_config(self) -> None:
        """
        Load configuration from file
        """
        try:
            with open(self.config_path, 'r') as f:
                self.config = json.load(f)
            self.logger.info("Configuration loaded successfully")
        except Exception as e:
            self.logger.error(f"Error loading configuration: {str(e)}")
            # Use default configuration
            self.config = {
                "model": {
                    "drl": {
                        "model_type": "dqn",
                        "learning_rate": 0.001,
                        "discount_factor": 0.95,
                        "exploration_rate": 0.1,
                        "target_update_frequency": 1000,
                        "batch_size": 64,
                        "memory_size": 10000,
                        "num_hidden_layers": 3,
                        "hidden_layer_size": 128,
                        "activation": "relu"
                    },
                    "reward_function": {
                        "profit_weight": 0.7,
                        "time_weight": 0.3
                    }
                }
            }
    
    def step(self, state, action, reward, next_state, done):
        """
        Update the agent's knowledge based on experience
        
        Args:
            state: Current state
            action: Selected action
            reward: Received reward
            next_state: Next state
            done: Whether episode is done
        """
        # Save experience in replay memory
        self.memory.push(state, action, reward, next_state, done)
        
        # Learn every self.batch_size steps, if enough samples in memory
        self.t_step = (self.t_step + 1) % self.target_update_freq
        if len(self.memory) > self.batch_size:
            experiences = self.memory.sample(self.batch_size)
            loss = self._learn(experiences)
            return loss
        
        # Update target network
        if self.t_step == 0:
            self._update_target_network()
            return 0
        
        return 0
    
    def act(self, state, evaluate=False):
        """
        Choose action based on current state using epsilon-greedy policy
        
        Args:
            state: Current state
            evaluate (bool): Whether to evaluate (no exploration)
            
        Returns:
            int: Selected action index
        """
        # Convert state to tensor
        state = torch.from_numpy(state).float().unsqueeze(0).to(self.device)
        
        # Set network to evaluation mode
        self.qnetwork_local.eval()
        
        with torch.no_grad():
            action_values = self.qnetwork_local(state)
        
        # Set network back to training mode
        self.qnetwork_local.train()
        
        # Epsilon-greedy action selection
        if not evaluate and random.random() < self.exploration_rate:
            return random.choice(np.arange(self.action_size))
        else:
            return np.argmax(action_values.cpu().data.numpy())
    
    def _learn(self, experiences):
        """
        Update value parameters using batch of experiences
        
        Args:
            experiences: Tuple of (states, actions, rewards, next_states, dones)
            
        Returns:
            float: Loss value
        """
        states, actions, rewards, next_states, dones = experiences
        
        # Move to device
        states = states.to(self.device)
        actions = actions.to(self.device)
        rewards = rewards.to(self.device)
        next_states = next_states.to(self.device)
        dones = dones.to(self.device)
        
        # Get max predicted Q values for next states from target model
        Q_targets_next = self.qnetwork_target(next_states).detach().max(1)[0].unsqueeze(1)
        
        # Compute Q targets for current states
        Q_targets = rewards + (self.discount_factor * Q_targets_next * (1 - dones))
        
        # Get expected Q values from local model
        Q_expected = self.qnetwork_local(states).gather(1, actions)
        
        # Compute loss
        loss = F.mse_loss(Q_expected, Q_targets)
        
        # Minimize the loss
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        
        return loss.item()
    
    def _update_target_network(self):
        """Update target network with parameters from local network"""
        self.qnetwork_target.load_state_dict(self.qnetwork_local.state_dict())
    
    def save_model(self, episode: int = None):
        """
        Save the agent's model
        
        Args:
            episode (int): Current episode number (used for filename)
        """
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        episode_str = f"_episode_{episode}" if episode is not None else ""
        
        model_path = os.path.join(self.models_dir, f"drl_model_{timestamp}{episode_str}.pt")
        
        torch.save({
            'qnetwork_local_state_dict': self.qnetwork_local.state_dict(),
            'qnetwork_target_state_dict': self.qnetwork_target.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'exploration_rate': self.exploration_rate,
            'stats': self.stats
        }, model_path)
        
        self.logger.info(f"Model saved to {model_path}")
        return model_path
    
    def load_model(self, model_path: str):
        """
        Load a saved model
        
        Args:
            model_path (str): Path to saved model
        """
        if not os.path.exists(model_path):
            self.logger.error(f"Model file not found: {model_path}")
            return False
        
        try:
            checkpoint = torch.load(model_path, map_location=self.device)
            
            self.qnetwork_local.load_state_dict(checkpoint['qnetwork_local_state_dict'])
            self.qnetwork_target.load_state_dict(checkpoint['qnetwork_target_state_dict'])
            self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            self.exploration_rate = checkpoint.get('exploration_rate', 0.1)
            self.stats = checkpoint.get('stats', self.stats)
            
            self.logger.info(f"Model loaded from {model_path}")
            return True
        except Exception as e:
            self.logger.error(f"Error loading model: {str(e)}")
            return False
    
    def compute_reward(self, state, action, next_state, price, sold_price=None, time_to_sell=None):
        """
        Compute the reward for a state-action pair
        
        Args:
            state: Current state
            action: Action taken
            next_state: Resulting state
            price: Original item price
            sold_price: Final selling price (if sold)
            time_to_sell: Time to sell in days (if sold)
            
        Returns:
            float: Computed reward
        """
        # If item sold, use actual profit and time
        if sold_price is not None and time_to_sell is not None:
            profit_margin = (sold_price - price) / price if price > 0 else 0
            
            # Normalize time to sell (consider 30 days as max)
            norm_time = max(0, 1 - (time_to_sell / 30)) if time_to_sell > 0 else 0
            
            # Combine profit and time components
            reward = (self.profit_weight * profit_margin) + (self.time_weight * norm_time)
            
            return reward * 10  # Scale reward
        
        # If not sold yet, use market-based reward estimation
        price_competitiveness = next_state[4] if len(next_state) > 4 else 0.5  # Use price competitiveness feature
        conversion_rate = next_state[7] if len(next_state) > 7 else 0.5  # Use conversion rate feature
        
        # Estimate reward based on price position and market conditions
        estimated_reward = (price_competitiveness * 0.6) + (conversion_rate * 0.4)
        
        return estimated_reward * 5  # Scale reward
    
    def decay_exploration_rate(self, decay_factor=0.995, min_rate=0.01):
        """
        Decay the exploration rate
        
        Args:
            decay_factor (float): Factor to decay by
            min_rate (float): Minimum exploration rate
        """
        self.exploration_rate = max(min_rate, self.exploration_rate * decay_factor)
        return self.exploration_rate


class PriceEnvironment:
    """
    Environment for price prediction, simulating market dynamics
    """
    def __init__(self, features_df: pd.DataFrame, category_id: str, num_price_levels: int = 10):
        """
        Initialize the environment
        
        Args:
            features_df (pd.DataFrame): DataFrame with item features
            category_id (str): Category ID
            num_price_levels (int): Number of discrete price levels
        """
        self.logger = Logger().get_logger()
        self.features_df = features_df
        self.category_id = category_id
        
        # Get sold items for statistics
        self.sold_items = features_df[features_df['is_sold'] == 1]
        self.unsold_items = features_df[features_df['is_sold'] == 0]
        
        # Extract price range
        self.min_price = features_df['price'].min()
        self.max_price = features_df['price'].max()
        self.avg_price = features_df['price'].mean()
        
        # Calculate price levels (actions)
        self.num_price_levels = num_price_levels
        self.price_levels = np.linspace(0.5, 1.5, num_price_levels)  # 0.5x to 1.5x of item's base price
        
        # Current state info
        self.current_item_idx = 0
        self.current_state = None
        self.base_price = 0
        
        self.logger.info(f"Initialized price environment with {len(features_df)} items, {num_price_levels} price levels")
    
    def reset(self, item_idx=None):
        """
        Reset the environment and get new item
        
        Args:
            item_idx (int): Index of item to use (None for random)
            
        Returns:
            np.array: Initial state
        """
        # Select item (random if not specified)
        if item_idx is None:
            self.current_item_idx = np.random.randint(0, len(self.features_df))
        else:
            self.current_item_idx = item_idx % len(self.features_df)
        
        # Get item data
        item = self.features_df.iloc[self.current_item_idx]
        
        # Extract state features (excluding ID, price, is_sold, category_id, days_to_sell)
        state_columns = [col for col in self.features_df.columns 
                         if col not in ['itemId', 'price', 'is_sold', 'category_id', 'days_to_sell']]
        
        # Set current state
        self.current_state = item[state_columns].values.astype(np.float32)
        
        # Set base price
        self.base_price = item['price']
        
        return self.current_state
    
    def step(self, action):
        """
        Take a step in the environment
        
        Args:
            action (int): Action index (price level)
            
        Returns:
            tuple: (next_state, reward, done, info)
        """
        # Get selected price level
        price_factor = self.price_levels[action]
        suggested_price = self.base_price * price_factor
        
        # Get current item
        item = self.features_df.iloc[self.current_item_idx]
        
        # Check if item sold in historical data
        is_sold = item['is_sold'] == 1
        days_to_sell = item.get('days_to_sell', 30)  # Default to 30 days if unknown
        
        # Get actual sold price if available
        sold_price = item['price'] if is_sold else None
        
        # Calculate reward
        reward = self._calculate_reward(suggested_price, is_sold, sold_price, days_to_sell)
        
        # Return the same state (as this is not a sequential environment)
        next_state = self.current_state
        
        # Episode is done after one step (predict price only once per item)
        done = True
        
        # Additional info
        info = {
            'item_id': item.get('itemId', self.current_item_idx),
            'base_price': self.base_price,
            'suggested_price': suggested_price,
            'price_factor': price_factor,
            'is_sold': is_sold,
            'days_to_sell': days_to_sell if is_sold else None,
            'sold_price': sold_price
        }
        
        return next_state, reward, done, info
    
    def _calculate_reward(self, suggested_price, is_sold, sold_price, days_to_sell):
        """
        Calculate reward for the action
        
        Args:
            suggested_price (float): Price suggested by agent
            is_sold (bool): Whether item was sold
            sold_price (float): Actual sold price
            days_to_sell (float): Days taken to sell
            
        Returns:
            float: Reward value
        """
        # For sold items, compare the suggested price to actual sold price
        if is_sold and sold_price is not None:
            # Penalize overpricing that could prevent sale
            if suggested_price > 1.2 * sold_price:
                return -1.0
            
            # Reward accurate pricing (within 10% of actual)
            if 0.9 * sold_price <= suggested_price <= 1.1 * sold_price:
                return 1.0
            
            # Smaller reward for reasonable pricing (within 20% of actual)
            if 0.8 * sold_price <= suggested_price <= 1.2 * sold_price:
                return 0.5
            
            # Penalize significantly underpricing
            if suggested_price < 0.8 * sold_price:
                return -0.5
        
        # For unsold items, evaluate based on market dynamics
        else:
            # Get similar items
            similar_sold = self._find_similar_items(is_sold=True)
            
            if len(similar_sold) > 0:
                avg_similar_price = similar_sold['price'].mean()
                
                # Reward pricing close to successful similar items
                if 0.9 * avg_similar_price <= suggested_price <= 1.1 * avg_similar_price:
                    return 0.8
                
                # Smaller reward for reasonable pricing
                if 0.8 * avg_similar_price <= suggested_price <= 1.2 * avg_similar_price:
                    return 0.4
                
                # Penalize significant deviation from market
                return -0.3
            
        # Default small reward if we can't determine better
        return 0.1
    
    def _find_similar_items(self, is_sold=True):
        """
        Find items similar to current item
        
        Args:
            is_sold (bool): Whether to find sold or unsold items
            
        Returns:
            pd.DataFrame: Similar items
        """
        # Current item
        item = self.features_df.iloc[self.current_item_idx]
        
        # Filter by sold status
        items_subset = self.sold_items if is_sold else self.unsold_items
        
        if len(items_subset) == 0:
            return pd.DataFrame()
        
        # Extract state features (excluding ID, price, is_sold, category_id, days_to_sell)
        feature_columns = [col for col in self.features_df.columns 
                          if col not in ['itemId', 'price', 'is_sold', 'category_id', 'days_to_sell']]
        
        if len(feature_columns) == 0:
            return items_subset.head(5)
        
        # Simple similarity: use feature values to find similar items
        # For more complex scenarios, consider using cosine similarity or KNN
        
        # Extract feature values for current item
        item_features = item[feature_columns].values
        
        # Calculate simple Euclidean distance for each item
        def calculate_distance(row):
            return np.sum((row[feature_columns].values - item_features) ** 2)
        
        # Calculate distances and sort by similarity
        items_subset = items_subset.copy()
        items_subset['distance'] = items_subset.apply(calculate_distance, axis=1)
        items_subset = items_subset.sort_values('distance')
        
        # Return top 5 most similar items
        return items_subset.head(5)


class PricePredictionTrainer:
    """
    Trainer class for price prediction model
    """
    def __init__(self, category_id: str, config_path: str = "/data/chats/p6wyr/workspace/config/config.json"):
        """
        Initialize the trainer
        
        Args:
            category_id (str): Category ID to train model for
            config_path (str): Path to configuration file
        """
        self.logger = Logger().get_logger()
        self.logger.info(f"Initializing price prediction trainer for category {category_id}")
        
        self.category_id = category_id
        self.config_path = config_path
        
        # Load configuration
        self._load_config()
        
        # Set paths
        self.features_dir = f"/data/chats/p6wyr/workspace/data/features/category_{category_id}"
        self.models_dir = f"/data/chats/p6wyr/workspace/data/models/category_{category_id}"
        
        os.makedirs(self.models_dir, exist_ok=True)
        
        # Load data
        self.features_df = self._load_features()
        
        # Set up environment and agent
        if self.features_df is not None:
            self._setup_env_and_agent()
        else:
            self.logger.error("Failed to load features data. Cannot set up environment and agent.")
            self.environment = None
            self.agent = None
    
    def _load_config(self) -> None:
        """
        Load configuration from file
        """
        try:
            with open(self.config_path, 'r') as f:
                self.config = json.load(f)
            self.logger.info("Configuration loaded successfully")
        except Exception as e:
            self.logger.error(f"Error loading configuration: {str(e)}")
            # Use default configuration
            self.config = {
                "model": {
                    "drl": {
                        "model_type": "dqn",
                        "learning_rate": 0.001,
                        "discount_factor": 0.95,
                        "exploration_rate": 0.2,
                        "target_update_frequency": 1000,
                        "batch_size": 64,
                        "memory_size": 10000,
                        "num_hidden_layers": 3,
                        "hidden_layer_size": 128,
                        "activation": "relu"
                    }
                },
                "data": {
                    "min_items_for_training": 500
                }
            }
    
    def _load_features(self) -> Optional[pd.DataFrame]:
        """
        Load features data for the category
        
        Returns:
            Optional[pd.DataFrame]: Features DataFrame or None if not found
        """
        # Find the latest features file
        feature_files = sorted(glob.glob(os.path.join(self.features_dir, "features_*.csv")))
        
        if not feature_files:
            self.logger.error(f"No feature files found in {self.features_dir}")
            # Create synthetic data for demonstration if no real data exists
            return self._create_synthetic_data()
        
        # Load latest features file
        latest_features_file = feature_files[-1]
        self.logger.info(f"Loading features from {latest_features_file}")
        
        try:
            df = pd.read_csv(latest_features_file)
            self.logger.info(f"Loaded {len(df)} feature records")
            return df
        except Exception as e:
            self.logger.error(f"Error loading features: {str(e)}")
            # Create synthetic data as a fallback
            return self._create_synthetic_data()

    def _create_synthetic_data(self) -> pd.DataFrame:
        """
        Create synthetic data for demonstration purposes
        
        Returns:
            pd.DataFrame: Synthetic features data
        """
        self.logger.warning("Creating synthetic data for demonstration")
        
        # Number of synthetic items
        n_items = 500
        
        # Create random feature data
        np.random.seed(42)  # For reproducibility
        
        # Get state features from config
        state_features = self.config.get("model", {}).get("state_features", [
            "item_condition", 
            "brand_popularity", 
            "season_demand", 
            "market_saturation",
            "avg_price", 
            "price_std", 
            "time_on_market",
            "conversion_rate"
        ])
        
        # Create DataFrame
        data = {
            'itemId': [f"synth_{i}" for i in range(n_items)],
            'price': np.random.uniform(50, 1000, n_items),
            'is_sold': np.random.choice([0, 1], n_items, p=[0.6, 0.4]),
            'category_id': self.category_id
        }
        
        # Add state features
        for feature in state_features:
            data[feature] = np.random.uniform(0, 1, n_items)
        
        # Add days_to_sell for sold items
        df = pd.DataFrame(data)
        sold_mask = df['is_sold'] == 1
        df.loc[sold_mask, 'days_to_sell'] = np.random.uniform(1, 30, sold_mask.sum())
        
        self.logger.info(f"Created synthetic dataset with {n_items} items")
        return df
    
    def _setup_env_and_agent(self) -> None:
        """
        Set up environment and DRL agent
        """
        # Define number of price levels (actions)
        self.num_price_levels = 10
        
        # Create environment
        self.environment = PriceEnvironment(
            features_df=self.features_df,
            category_id=self.category_id,
            num_price_levels=self.num_price_levels
        )
        
        # Get state size (number of features)
        state_columns = [col for col in self.features_df.columns 
                         if col not in ['itemId', 'price', 'is_sold', 'category_id', 'days_to_sell']]
        state_size = len(state_columns)
        
        # Create agent
        self.agent = PricePredictionDRLAgent(
            state_size=state_size,
            action_size=self.num_price_levels,
            category_id=self.category_id,
            config_path=self.config_path
        )
        
        self.logger.info(f"Set up environment and agent with state size {state_size} and {self.num_price_levels} actions")
    
    def train(self, num_episodes: int = 2000, batch_size: int = 64, checkpoint_interval: int = 200):
        """
        Train the DRL agent
        
        Args:
            num_episodes (int): Number of training episodes
            batch_size (int): Batch size for training
            checkpoint_interval (int): Interval for saving model checkpoints

SyntaxError: unterminated triple-quoted string literal (detected at line 795) (2999313802.py, line 789)

In [17]:
[reflection on previous impl]
I see the issue in my previous implementation. There are multiple syntax errors in the code:

1. In the PricePredictionTrainer.train() method, I have an incomplete docstring ("triple-quoted string") that needs to be properly closed.
2. I also didn't complete the train() method implementation and didn't add the evaluation and main execution code.
3. The `self.agent.stats["avg_losses"]` line is incomplete and needs to be correctly appended with data.
4. I need to make sure all the methods are fully implemented and all strings are properly terminated.

I need to fix these syntax errors and complete the implementation with proper evaluation functionality and a main execution block to demonstrate the DRL model in action.

[improved impl]:
```python
from metagpt.tools.libs.terminal import Terminal
terminal = Terminal()

# Install required packages for the DRL model
await terminal.run('pip install torch numpy matplotlib tqdm tensorboard')

import os
import json
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from typing import Dict, List, Tuple, Optional, Any
from collections import deque, namedtuple
import random
import glob
from datetime import datetime
import pickle
import copy
from tqdm import tqdm
import matplotlib.pyplot as plt

from src.utils.logger import Logger

# Define constants for the DRL model
Experience = namedtuple('Experience', ['state', 'action', 'reward', 'next_state', 'done'])

class ReplayBuffer:
    """
    Replay buffer for storing and sampling experiences for DRL training
    """
    def __init__(self, capacity: int):
        """Initialize replay buffer with given capacity"""
        self.buffer = deque(maxlen=capacity)
    
    def push(self, state, action, reward, next_state, done):
        """Add experience to the buffer"""
        self.buffer.append(Experience(state, action, reward, next_state, done))
    
    def sample(self, batch_size: int):
        """Sample random batch of experiences"""
        experiences = random.sample(self.buffer, batch_size)
        states = torch.from_numpy(np.vstack([e.state for e in experiences])).float()
        actions = torch.from_numpy(np.vstack([e.action for e in experiences])).long()
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences])).float()
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences])).float()
        dones = torch.from_numpy(np.vstack([e.done for e in experiences]).astype(np.uint8)).float()
        return states, actions, rewards, next_states, dones
    
    def __len__(self):
        """Current size of replay buffer"""
        return len(self.buffer)

class DQNetwork(nn.Module):
    """
    Deep Q-Network for price prediction
    """
    def __init__(self, state_size: int, action_size: int, config: Dict):
        """
        Initialize DQN with state and action dimensions
        
        Args:
            state_size (int): Dimension of state space
            action_size (int): Dimension of action space
            config (Dict): Model configuration
        """
        super(DQNetwork, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        
        # Get configuration parameters
        hidden_layer_size = config.get("hidden_layer_size", 128)
        num_hidden_layers = config.get("num_hidden_layers", 3)
        
        # Create the network layers
        layers = [nn.Linear(state_size, hidden_layer_size), nn.ReLU()]
        
        # Add hidden layers
        for _ in range(num_hidden_layers - 1):
            layers.append(nn.Linear(hidden_layer_size, hidden_layer_size))
            layers.append(nn.ReLU())
        
        # Add output layer
        layers.append(nn.Linear(hidden_layer_size, action_size))
        
        # Create sequential model
        self.network = nn.Sequential(*layers)
    
    def forward(self, state):
        """Forward pass through network"""
        return self.network(state)

class PricePredictionDRLAgent:
    """
    Deep Reinforcement Learning agent for price prediction
    """
    def __init__(self, state_size: int, action_size: int, category_id: str, config_path: str = "/data/chats/p6wyr/workspace/config/config.json"):
        """
        Initialize the DRL agent
        
        Args:
            state_size (int): Dimension of state space
            action_size (int): Dimension of action space
            category_id (str): Category ID
            config_path (str): Path to configuration file
        """
        self.logger = Logger().get_logger()
        self.logger.info("Initializing DRL agent for price prediction")
        
        # Load configuration
        self.config_path = config_path
        self._load_config()
        
        # Get DRL model configuration
        drl_config = self.config.get("model", {}).get("drl", {})
        
        # Set up model parameters
        self.state_size = state_size
        self.action_size = action_size
        self.category_id = category_id
        self.learning_rate = drl_config.get("learning_rate", 0.001)
        self.discount_factor = drl_config.get("discount_factor", 0.95)
        self.exploration_rate = drl_config.get("exploration_rate", 0.1)
        self.batch_size = drl_config.get("batch_size", 64)
        self.memory_size = drl_config.get("memory_size", 10000)
        self.target_update_freq = drl_config.get("target_update_frequency", 1000)
        
        # Set device (GPU if available)
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.logger.info(f"Using device: {self.device}")
        
        # Initialize networks
        self.qnetwork_local = DQNetwork(state_size, action_size, drl_config).to(self.device)
        self.qnetwork_target = DQNetwork(state_size, action_size, drl_config).to(self.device)
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=self.learning_rate)
        
        # Initialize replay memory
        self.memory = ReplayBuffer(self.memory_size)
        
        # Initialize step counter for target network updates
        self.t_step = 0
        
        # Set up model directories
        self.models_dir = f"/data/chats/p6wyr/workspace/data/models/category_{self.category_id}"
        os.makedirs(self.models_dir, exist_ok=True)
        
        # Set up tensorboard logging
        self.tb_dir = f"/data/chats/p6wyr/workspace/logs/tensorboard/category_{self.category_id}"
        os.makedirs(self.tb_dir, exist_ok=True)
        self.writer = SummaryWriter(log_dir=self.tb_dir)
        
        # Reward function weights
        reward_config = self.config.get("model", {}).get("reward_function", {})
        self.profit_weight = reward_config.get("profit_weight", 0.7)
        self.time_weight = reward_config.get("time_weight", 0.3)
        
        # Track statistics
        self.stats = {
            "episodes": 0,
            "avg_rewards": [],
            "avg_losses": [],
            "exploration_rate": []
        }
    
    def _load_config(self) -> None:
        """
        Load configuration from file
        """
        try:
            with open(self.config_path, 'r') as f:
                self.config = json.load(f)
            self.logger.info("Configuration loaded successfully")
        except Exception as e:
            self.logger.error(f"Error loading configuration: {str(e)}")
            # Use default configuration
            self.config = {
                "model": {
                    "drl": {
                        "model_type": "dqn",
                        "learning_rate": 0.001,
                        "discount_factor": 0.95,
                        "exploration_rate": 0.1,
                        "target_update_frequency": 1000,
                        "batch_size": 64,
                        "memory_size": 10000,
                        "num_hidden_layers": 3,
                        "hidden_layer_size": 128,
                        "activation": "relu"
                    },
                    "reward_function": {
                        "profit_weight": 0.7,
                        "time_weight": 0.3
                    }
                }
            }
    
    def step(self, state, action, reward, next_state, done):
        """
        Update the agent's knowledge based on experience
        
        Args:
            state: Current state
            action: Selected action
            reward: Received reward
            next_state: Next state
            done: Whether episode is done
            
        Returns:
            float: Loss value (0 if not learning)
        """
        # Save experience in replay memory
        self.memory.push(state, action, reward, next_state, done)
        
        # Learn every self.batch_size steps, if enough samples in memory
        self.t_step = (self.t_step + 1) % self.target_update_freq
        if len(self.memory) > self.batch_size:
            experiences = self.memory.sample(self.batch_size)
            loss = self._learn(experiences)
            return loss
        
        # Update target network
        if self.t_step == 0:
            self._update_target_network()
            return 0
        
        return 0
    
    def act(self, state, evaluate=False):
        """
        Choose action based on current state using epsilon-greedy policy
        
        Args:
            state: Current state
            evaluate (bool): Whether to evaluate (no exploration)
            
        Returns:
            int: Selected action index
        """
        # Convert state to tensor
        state = torch.from_numpy(state).float().unsqueeze(0).to(self.device)
        
        # Set network to evaluation mode
        self.qnetwork_local.eval()
        
        with torch.no_grad():
            action_values = self.qnetwork_local(state)
        
        # Set network back to training mode
        self.qnetwork_local.train()
        
        # Epsilon-greedy action selection
        if not evaluate and random.random() < self.exploration_rate:
            return random.choice(np.arange(self.action_size))
        else:
            return np.argmax(action_values.cpu().data.numpy())
    
    def _learn(self, experiences):
        """
        Update value parameters using batch of experiences
        
        Args:
            experiences: Tuple of (states, actions, rewards, next_states, dones)
            
        Returns:
            float: Loss value
        """
        states, actions, rewards, next_states, dones = experiences
        
        # Move to device
        states = states.to(self.device)
        actions = actions.to(self.device)
        rewards = rewards.to(self.device)
        next_states = next_states.to(self.device)
        dones = dones.to(self.device)
        
        # Get max predicted Q values for next states from target model
        Q_targets_next = self.qnetwork_target(next_states).detach().max(1)[0].unsqueeze(1)
        
        # Compute Q targets for current states
        Q_targets = rewards + (self.discount_factor * Q_targets_next * (1 - dones))
        
        # Get expected Q values from local model
        Q_expected = self.qnetwork_local(states).gather(1, actions)
        
        # Compute loss
        loss = F.mse_loss(Q_expected, Q_targets)
        
        # Minimize the loss
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        
        return loss.item()
    
    def _update_target_network(self):
        """Update target network with parameters from local network"""
        self.qnetwork_target.load_state_dict(self.qnetwork_local.state_dict())
    
    def save_model(self, episode: int = None):
        """
        Save the agent's model
        
        Args:
            episode (int): Current episode number (used for filename)
            
        Returns:
            str: Path to saved model
        """
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        episode_str = f"_episode_{episode}" if episode is not None else ""
        
        model_path = os.path.join(self.models_dir, f"drl_model_{timestamp}{episode_str}.pt")
        
        torch.save({
            'qnetwork_local_state_dict': self.qnetwork_local.state_dict(),
            'qnetwork_target_state_dict': self.qnetwork_target.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'exploration_rate': self.exploration_rate,
            'stats': self.stats
        }, model_path)
        
        self.logger.info(f"Model saved to {model_path}")
        return model_path
    
    def load_model(self, model_path: str):
        """
        Load a saved model
        
        Args:
            model_path (str): Path to saved model
            
        Returns:
            bool: True if loaded successfully, False otherwise
        """
        if not os.path.exists(model_path):
            self.logger.error(f"Model file not found: {model_path}")
            return False
        
        try:
            checkpoint = torch.load(model_path, map_location=self.device)
            
            self.qnetwork_local.load_state_dict(checkpoint['qnetwork_local_state_dict'])
            self.qnetwork_target.load_state_dict(checkpoint['qnetwork_target_state_dict'])
            self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            self.exploration_rate = checkpoint.get('exploration_rate', 0.1)
            self.stats = checkpoint.get('stats', self.stats)
            
            self.logger.info(f"Model loaded from {model_path}")
            return True
        except Exception as e:
            self.logger.error(f"Error loading model: {str(e)}")
            return False
    
    def compute_reward(self, state, action, next_state, price, sold_price=None, time_to_sell=None):
        """
        Compute the reward for a state-action pair
        
        Args:
            state: Current state
            action: Action taken
            next_state: Resulting state
            price: Original item price
            sold_price: Final selling price (if sold)
            time_to_sell: Time to sell in days (if sold)
            
        Returns:
            float: Computed reward
        """
        # If item sold, use actual profit and time
        if sold_price is not None and time_to_sell is not None:
            profit_margin = (sold_price - price) / price if price > 0 else 0
            
            # Normalize time to sell (consider 30 days as max)
            norm_time = max(0, 1 - (time_to_sell / 30)) if time_to_sell > 0 else 0
            
            # Combine profit and time components
            reward = (self.profit_weight * profit_margin) + (self.time_weight * norm_time)
            
            return reward * 10  # Scale reward
        
        # If not sold yet, use market-based reward estimation
        price_competitiveness = next_state[4] if len(next_state) > 4 else 0.5  # Use price competitiveness feature
        conversion_rate = next_state[7] if len(next_state) > 7 else 0.5  # Use conversion rate feature
        
        # Estimate reward based on price position and market conditions
        estimated_reward = (price_competitiveness * 0.6) + (conversion_rate * 0.4)
        
        return estimated_reward * 5  # Scale reward
    
    def decay_exploration_rate(self, decay_factor=0.995, min_rate=0.01):
        """
        Decay the exploration rate
        
        Args:
            decay_factor (float): Factor to decay by
            min_rate (float): Minimum exploration rate
            
        Returns:
            float: New exploration rate
        """
        self.exploration_rate = max(min_rate, self.exploration_rate * decay_factor)
        return self.exploration_rate


class PriceEnvironment:
    """
    Environment for price prediction, simulating market dynamics
    """
    def __init__(self, features_df: pd.DataFrame, category_id: str, num_price_levels: int = 10):
        """
        Initialize the environment
        
        Args:
            features_df (pd.DataFrame): DataFrame with item features
            category_id (str): Category ID
            num_price_levels (int): Number of discrete price levels
        """
        self.logger = Logger().get_logger()
        self.features_df = features_df
        self.category_id = category_id
        
        # Get sold items for statistics
        self.sold_items = features_df[features_df['is_sold'] == 1]
        self.unsold_items = features_df[features_df['is_sold'] == 0]
        
        # Extract price range
        self.min_price = features_df['price'].min()
        self.max_price = features_df['price'].max()
        self.avg_price = features_df['price'].mean()
        
        # Calculate price levels (actions)
        self.num_price_levels = num_price_levels
        self.price_levels = np.linspace(0.5, 1.5, num_price_levels)  # 0.5x to 1.5x of item's base price
        
        # Current state info
        self.current_item_idx = 0
        self.current_state = None
        self.base_price = 0
        
        self.logger.info(f"Initialized price environment with {len(features_df)} items, {num_price_levels} price levels")
    
    def reset(self, item_idx=None):
        """
        Reset the environment and get new item
        
        Args:
            item_idx (int): Index of item to use (None for random)
            
        Returns:
            np.array: Initial state
        """
        # Select item (random if not specified)
        if item_idx is None:
            self.current_item_idx = np.random.randint(0, len(self.features_df))
        else:
            self.current_item_idx = item_idx % len(self.features_df)
        
        # Get item data
        item = self.features_df.iloc[self.current_item_idx]
        
        # Extract state features (excluding ID, price, is_sold, category_id, days_to_sell)
        state_columns = [col for col in self.features_df.columns 
                         if col not in ['itemId', 'price', 'is_sold', 'category_id', 'days_to_sell']]
        
        # Set current state
        self.current_state = item[state_columns].values.astype(np.float32)
        
        # Set base price
        self.base_price = item['price']
        
        return self.current_state
    
    def step(self, action):
        """
        Take a step in the environment
        
        Args:
            action (int): Action index (price level)
            
        Returns:
            tuple: (next_state, reward, done, info)
        """
        # Get selected price level
        price_factor = self.price_levels[action]
        suggested_price = self.base_price * price_factor
        
        # Get current item
        item = self.features_df.iloc[self.current_item_idx]
        
        # Check if item sold in historical data
        is_sold = item['is_sold'] == 1
        days_to_sell = item.get('days_to_sell', 30)  # Default to 30 days if unknown
        
        # Get actual sold price if available
        sold_price = item['price'] if is_sold else None
        
        # Calculate reward
        reward = self._calculate_reward(suggested_price, is_sold, sold_price, days_to_sell)
        
        # Return the same state (as this is not a sequential environment)
        next_state = self.current_state
        
        # Episode is done after one step (predict price only once per item)
        done = True
        
        # Additional info
        info = {
            'item_id': item.get('itemId', self.current_item_idx),
            'base_price': self.base_price,
            'suggested_price': suggested_price,
            'price_factor': price_factor,
            'is_sold': is_sold,
            'days_to_sell': days_to_sell if is_sold else None,
            'sold_price': sold_price
        }
        
        return next_state, reward, done, info
    
    def _calculate_reward(self, suggested_price, is_sold, sold_price, days_to_sell):
        """
        Calculate reward for the action
        
        Args:
            suggested_price (float): Price suggested by agent
            is_sold (bool): Whether item was sold
            sold_price (float): Actual sold price
            days_to_sell (float): Days taken to sell
            
        Returns:
            float: Reward value
        """
        # For sold items, compare the suggested price to actual sold price
        if is_sold and sold_price is not None:
            # Penalize overpricing that could prevent sale
            if suggested_price > 1.2 * sold_price:
                return -1.0
            
            # Reward accurate pricing (within 10% of actual)
            if 0.9 * sold_price <= suggested_price <= 1.1 * sold_price:
                return 1.0
            
            # Smaller reward for reasonable pricing (within 20% of actual)
            if 0.8 * sold_price <= suggested_price <= 1.2 * sold_price:
                return 0.5
            
            # Penalize significantly underpricing
            if suggested_price < 0.8 * sold_price:
                return -0.5
        
        # For unsold items, evaluate based on market dynamics
        else:
            # Get similar items
            similar_sold = self._find_similar_items(is_sold=True)
            
            if len(similar_sold) > 0:
                avg_similar_price = similar_sold['price'].mean()
                
                # Reward pricing close to successful similar items
                if 0.9 * avg_similar_price <= suggested_price <= 1.1 * avg_similar_price:
                    return 0.8
                
                # Smaller reward for reasonable pricing
                if 0.8 * avg_similar_price <= suggested_price <= 1.2 * avg_similar_price:
                    return 0.4
                
                # Penalize significant deviation from market
                return -0.3
            
        # Default small reward if we can't determine better
        return 0.1
    
    def _find_similar_items(self, is_sold=True):
        """
        Find items similar to current item
        
        Args:
            is_sold (bool): Whether to find sold or unsold items
            
        Returns:
            pd.DataFrame: Similar items
        """
        # Current item
        item = self.features_df.iloc[self.current_item_idx]
        
        # Filter by sold status
        items_subset = self.sold_items if is_sold else self.unsold_items
        
        if len(items_subset) == 0:
            return pd.DataFrame()
        
        # Extract state features (excluding ID, price, is_sold, category_id, days_to_sell)
        feature_columns = [col for col in self.features_df.columns 
                          if col not in ['itemId', 'price', 'is_sold', 'category_id', 'days_to_sell']]
        
        if len(feature_columns) == 0:
            return items_subset.head(5)
        
        # Simple similarity: use feature values to find similar items
        # For more complex scenarios, consider using cosine similarity or KNN
        
        # Extract feature values for current item
        item_features = item[feature_columns].values
        
        # Calculate simple Euclidean distance for each item
        def calculate_distance(row):
            return np.sum((row[feature_columns].values - item_features) ** 2)
        
        # Calculate distances and sort by similarity
        items_subset = items_subset.copy()
        items_subset['distance'] = items_subset.apply(calculate_distance, axis=1)
        items_subset = items_subset.sort_values('distance')
        
        # Return top 5 most similar items
        return items_subset.head(5)


class PricePredictionTrainer:
    """
    Trainer class for price prediction model
    """
    def __init__(self, category_id: str, config_path: str = "/data/chats/p6wyr/workspace/config/config.json"):
        """
        Initialize the trainer
        
        Args:
            category_id (str): Category ID to train model for
            config_path (str): Path to configuration file
        """
        self.logger = Logger().get_logger()
        self.logger.info(f"Initializing price prediction trainer for category {category_id}")
        
        self.category_id = category_id
        self.config_path = config_path
        
        # Load configuration
        self._load_config()
        
        # Set paths
        self.features_dir = f"/data/chats/p6wyr/workspace/data/features/category_{category_id}"
        self.models_dir = f"/data/chats/p6wyr/workspace/data/models/category_{category_id}"
        
        os.makedirs(self.models_dir, exist_ok=True)
        
        # Load data
        self.features_df = self._load_features()
        
        # Set up environment and agent
        if self.features_df is not None:
            self._setup_env_and_agent()
        else:
            self.logger.error("Failed to load features data. Cannot set up environment and agent.")
            self.environment = None
            self.agent = None
    
    def _load_config(self) -> None:
        """
        Load configuration from file
        """
        try:
            with open(self.config_path, 'r') as f:
                self.config = json.load(f)
            self.logger.info("Configuration loaded successfully")
        except Exception as e:
            self.logger.error(f"Error loading configuration: {str(e)}")
            # Use default configuration
            self.config = {
                "model": {
                    "drl": {
                        "model_type": "dqn",
                        "learning_rate": 0.001,
                        "discount_factor": 0.95,
                        "exploration_rate": 0.2,
                        "target_update_frequency": 1000,
                        "batch_size": 64,
                        "memory_size": 10000,
                        "num_hidden_layers": 3,
                        "hidden_layer_size": 128,
                        "activation": "relu"
                    }
                },
                "data": {
                    "min_items_for_training": 500
                }
            }
    
    def _load_features(self) -> Optional[pd.DataFrame]:
        """
        Load features data for the category
        
        Returns:
            Optional[pd.DataFrame]: Features DataFrame or None if not found
        """
        # Find the latest features file
        feature_files = sorted(glob.glob(os.path.join(self.features_dir, "features_*.csv")))
        
        if not feature_files:
            self.logger.error(f"No feature files found in {self.features_dir}")
            # Create synthetic data for demonstration if no real data exists
            return self._create_synthetic_data()
        
        # Load latest features file
        latest_features_file = feature_files[-1]
        self.logger.info(f"Loading features from {latest_features_file}")
        
        try:
            df = pd.read_csv(latest_features_file)
            self.logger.info(f"Loaded {len(df)} feature records")
            return df
        except Exception as e:
            self.logger.error(f"Error loading features: {str(e)}")
            # Create synthetic data as a fallback
            return self._create_synthetic_data()

    def _create_synthetic_data(self) -> pd.DataFrame:
        """
        Create synthetic data for demonstration purposes
        
        Returns:
            pd.DataFrame: Synthetic features data
        """
        self.logger.warning("Creating synthetic data for demonstration")
        
        # Number of synthetic items
        n_items = 500
        
        # Create random feature data
        np.random.seed(42)  # For reproducibility
        
        # Get state features from config
        state_features = self.config.get("model", {}).get("state_features", [
            "item_condition", 
            "brand_popularity", 
            "season_demand", 
            "market_saturation",
            "avg_price", 
            "price_std", 
            "time_on_market",
            "conversion_rate"
        ])
        
        # Create DataFrame
        data = {
            'itemId': [f"synth_{i}" for i in range(n_items)],
            'price': np.random.uniform(50, 1000, n_items),
            'is_sold': np.random.choice([0, 1], n_items, p=[0.6, 0.4]),
            'category_id': self.category_id
        }
        
        # Add state features
        for feature in state_features:
            data[feature] = np.random.uniform(0, 1, n_items)
        
        # Add days_to_sell for sold items
        df = pd.DataFrame(data)
        sold_mask = df['is_sold'] == 1
        df.loc[sold_mask, 'days_to_sell'] = np.random.uniform(1, 30, sold_mask.sum())
        
        self.logger.info(f"Created synthetic dataset with {n_items} items")
        return df
    
    def _setup_env_and_agent(self) -> None:
        """
        Set up environment and DRL agent
        """
        # Define number of price levels (actions)
        self.num_price_levels = 10
        
        # Create environment
        self.environment = PriceEnvironment(
            features_df=self.features_df,
            category_id=self.category_id,
            num_price_levels=self.num_price_levels
        )
        
        # Get state size (number of features)
        state_columns = [col for col in self.features_df.columns 
                         if col not in ['itemId', 'price', 'is_sold', 'category_id', 'days_to_sell']]
        state_size = len(state_columns)
        
        # Create agent
        self.agent = PricePredictionDRLAgent(
            state_size=state_size,
            action_size=self.num_price_levels,
            category_id=self.category_id,
            config_path=self.config_path
        )
        
        self.logger.info(f"Set up environment and agent with state size {state_size} and {self.num_price_levels} actions")
    
    def train(self, num_episodes: int = 2000, batch_size: int = 64, checkpoint_interval: int = 200):
        """
        Train the DRL agent
        
        Args

SyntaxError: unterminated triple-quoted string literal (detected at line 809) (820449835.py, line 806)

In [18]:
import os
import json
from datetime import datetime
import pandas as pd
import glob
from src.utils.logger import Logger

logger = Logger().get_logger()

# Check if any data has been collected
raw_data_dir = "/data/chats/p6wyr/workspace/data/raw"
processed_data_dir = "/data/chats/p6wyr/workspace/data/processed"
features_dir = "/data/chats/p6wyr/workspace/data/features"

# Check directories existence
print(f"Raw data directory exists: {os.path.exists(raw_data_dir)}")
print(f"Processed data directory exists: {os.path.exists(processed_data_dir)}")
print(f"Features directory exists: {os.path.exists(features_dir)}")

# Check for data files
categories = []
# Load config to get categories
config_path = "/data/chats/p6wyr/workspace/config/config.json"
if os.path.exists(config_path):
    try:
        with open(config_path, 'r') as f:
            config = json.load(f)
        categories = config.get("data", {}).get("categories", [])
        print(f"Found {len(categories)} categories in config:")
        for cat in categories:
            print(f"- {cat['name']} (ID: {cat['id']})")
    except Exception as e:
        print(f"Error loading config: {str(e)}")

# Check for collected data
for cat in categories:
    cat_id = cat["id"]
    cat_folder = os.path.join(raw_data_dir, f"category_{cat_id}")
    if os.path.exists(cat_folder):
        active_files = glob.glob(os.path.join(cat_folder, "active_items_*.csv"))
        sold_files = glob.glob(os.path.join(cat_folder, "sold_items_*.csv"))
        print(f"\nCategory {cat['name']} (ID: {cat_id}):")
        print(f"- Active item files: {len(active_files)}")
        print(f"- Sold item files: {len(sold_files)}")
        
        # If data exists, load the latest file and print column info
        if active_files:
            latest_file = sorted(active_files)[-1]
            try:
                df = pd.read_csv(latest_file)
                print(f"\nSample of active items data from {os.path.basename(latest_file)}:")
                print(f"- Shape: {df.shape}")
                print(f"- Columns: {df.columns.tolist()}")
                
                from metagpt.tools.libs.data_preprocess import get_column_info
                column_info = get_column_info(df)
                print("\nColumn information:")
                print(column_info)
                
                # Print a few sample records
                print("\nSample records:")
                print(df.head(2).to_string())
                break  # Just show one category's data for brevity
            except Exception as e:
                print(f"Error loading data: {str(e)}")

Raw data directory exists: True
Processed data directory exists: True
Features directory exists: True
Found 4 categories in config:
- Laptops & Netbooks (ID: 9355)
- Cell Phones & Smartphones (ID: 15032)
- Wristwatch (ID: 11450)
- Digital Cameras (ID: 261007)


In [19]:
from metagpt.tools.libs.terminal import Terminal
terminal = Terminal()

# Install required packages for the DRL model
await terminal.run('pip install torch numpy matplotlib tqdm tensorboard')

import os
import json
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from typing import Dict, List, Tuple, Optional, Any
from collections import deque, namedtuple
import random
import glob
from datetime import datetime
import pickle
import copy
from tqdm import tqdm
import matplotlib.pyplot as plt

from src.utils.logger import Logger

# Define constants for the DRL model
Experience = namedtuple('Experience', ['state', 'action', 'reward', 'next_state', 'done'])

class ReplayBuffer:
    """
    Replay buffer for storing and sampling experiences for DRL training
    """
    def __init__(self, capacity: int):
        """Initialize replay buffer with given capacity"""
        self.buffer = deque(maxlen=capacity)
    
    def push(self, state, action, reward, next_state, done):
        """Add experience to the buffer"""
        self.buffer.append(Experience(state, action, reward, next_state, done))
    
    def sample(self, batch_size: int):
        """Sample random batch of experiences"""
        experiences = random.sample(self.buffer, batch_size)
        states = torch.from_numpy(np.vstack([e.state for e in experiences])).float()
        actions = torch.from_numpy(np.vstack([e.action for e in experiences])).long()
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences])).float()
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences])).float()
        dones = torch.from_numpy(np.vstack([e.done for e in experiences]).astype(np.uint8)).float()
        return states, actions, rewards, next_states, dones
    
    def __len__(self):
        """Current size of replay buffer"""
        return len(self.buffer)

class DQNetwork(nn.Module):
    """
    Deep Q-Network for price prediction
    """
    def __init__(self, state_size: int, action_size: int, config: Dict):
        """
        Initialize DQN with state and action dimensions
        
        Args:
            state_size (int): Dimension of state space
            action_size (int): Dimension of action space
            config (Dict): Model configuration
        """
        super(DQNetwork, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        
        # Get configuration parameters
        hidden_layer_size = config.get("hidden_layer_size", 128)
        num_hidden_layers = config.get("num_hidden_layers", 3)
        dropout_rate = config.get("dropout_rate", 0.1)
        
        # Create the network layers with batch normalization and dropout for better performance
        layers = [nn.Linear(state_size, hidden_layer_size), 
                 nn.BatchNorm1d(hidden_layer_size),
                 nn.ReLU(),
                 nn.Dropout(dropout_rate)]
        
        # Add hidden layers
        for _ in range(num_hidden_layers - 1):
            layers.append(nn.Linear(hidden_layer_size, hidden_layer_size))
            layers.append(nn.BatchNorm1d(hidden_layer_size))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout_rate))
        
        # Add output layer
        layers.append(nn.Linear(hidden_layer_size, action_size))
        
        # Create sequential model
        self.network = nn.Sequential(*layers)
        
        # Initialize weights using Xavier/Glorot initialization for better convergence
        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        """Initialize network weights"""
        if isinstance(module, nn.Linear):
            nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                module.bias.data.fill_(0.01)
    
    def forward(self, state):
        """Forward pass through network"""
        # Handle both single and batch inputs
        if state.dim() == 1:
            state = state.unsqueeze(0)
            
        return self.network(state)

class DuelingDQNetwork(nn.Module):
    """
    Dueling Deep Q-Network for improved performance
    Separates state value and action advantages for better estimation
    """
    def __init__(self, state_size: int, action_size: int, config: Dict):
        """
        Initialize Dueling DQN with state and action dimensions
        
        Args:
            state_size (int): Dimension of state space
            action_size (int): Dimension of action space
            config (Dict): Model configuration
        """
        super(DuelingDQNetwork, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        
        # Get configuration parameters
        hidden_layer_size = config.get("hidden_layer_size", 128)
        num_hidden_layers = config.get("num_hidden_layers", 3)
        dropout_rate = config.get("dropout_rate", 0.1)
        
        # Feature extraction layers
        self.feature_layers = nn.Sequential(
            nn.Linear(state_size, hidden_layer_size),
            nn.BatchNorm1d(hidden_layer_size),
            nn.ReLU(),
            nn.Dropout(dropout_rate)
        )
        
        # Add hidden layers for feature extraction
        for _ in range(num_hidden_layers - 2):  # -2 because we already added one and will add one more for each stream
            self.feature_layers.extend([
                nn.Linear(hidden_layer_size, hidden_layer_size),
                nn.BatchNorm1d(hidden_layer_size),
                nn.ReLU(),
                nn.Dropout(dropout_rate)
            ])
        
        # Value stream - estimates state value V(s)
        self.value_stream = nn.Sequential(
            nn.Linear(hidden_layer_size, hidden_layer_size),
            nn.BatchNorm1d(hidden_layer_size),
            nn.ReLU(),
            nn.Linear(hidden_layer_size, 1)  # Single value output
        )
        
        # Advantage stream - estimates advantages for each action A(s,a)
        self.advantage_stream = nn.Sequential(
            nn.Linear(hidden_layer_size, hidden_layer_size),
            nn.BatchNorm1d(hidden_layer_size),
            nn.ReLU(),
            nn.Linear(hidden_layer_size, action_size)  # One output per action
        )
        
        # Initialize weights
        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        """Initialize network weights"""
        if isinstance(module, nn.Linear):
            nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                module.bias.data.fill_(0.01)
    
    def forward(self, state):
        """Forward pass through dueling network architecture"""
        # Handle both single and batch inputs
        if state.dim() == 1:
            state = state.unsqueeze(0)
            
        features = self.feature_layers(state)
        
        # Calculate state value and advantages
        value = self.value_stream(features)
        advantages = self.advantage_stream(features)
        
        # Combine value and advantages to get Q-values
        # Q(s,a) = V(s) + (A(s,a) - mean(A(s,a')))
        # This formula ensures identifiability
        qvalues = value + (advantages - advantages.mean(dim=1, keepdim=True))
        
        return qvalues

class PricePredictionDRLAgent:
    """
    Deep Reinforcement Learning agent for price prediction
    """
    def __init__(self, state_size: int, action_size: int, category_id: str, 
                 config_path: str = "/data/chats/p6wyr/workspace/config/config.json"):
        """
        Initialize the DRL agent
        
        Args:
            state_size (int): Dimension of state space
            action_size (int): Dimension of action space
            category_id (str): Category ID
            config_path (str): Path to configuration file
        """
        self.logger = Logger().get_logger()
        self.logger.info("Initializing DRL agent for price prediction")
        
        # Load configuration
        self.config_path = config_path
        self._load_config()
        
        # Get DRL model configuration
        self.drl_config = self.config.get("model", {}).get("drl", {})
        
        # Set up model parameters
        self.state_size = state_size
        self.action_size = action_size
        self.category_id = category_id
        self.learning_rate = self.drl_config.get("learning_rate", 0.001)
        self.discount_factor = self.drl_config.get("discount_factor", 0.95)
        self.exploration_rate = self.drl_config.get("exploration_rate", 0.1)
        self.batch_size = self.drl_config.get("batch_size", 64)
        self.memory_size = self.drl_config.get("memory_size", 10000)
        self.target_update_freq = self.drl_config.get("target_update_frequency", 1000)
        self.double_dqn = self.drl_config.get("double_dqn", True)
        self.dueling_network = self.drl_config.get("dueling_network", True)
        
        # Set device (GPU if available)
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.logger.info(f"Using device: {self.device}")
        
        # Initialize networks
        self._initialize_networks()
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=self.learning_rate)
        
        # Initialize replay memory
        self.memory = ReplayBuffer(self.memory_size)
        
        # Initialize step counter for target network updates
        self.t_step = 0
        self.train_step = 0
        
        # Set up model directories
        self.models_dir = f"/data/chats/p6wyr/workspace/data/models/category_{self.category_id}"
        os.makedirs(self.models_dir, exist_ok=True)
        
        # Set up tensorboard logging
        self.tb_dir = f"/data/chats/p6wyr/workspace/logs/tensorboard/category_{self.category_id}"
        os.makedirs(self.tb_dir, exist_ok=True)
        self.writer = SummaryWriter(log_dir=self.tb_dir)
        
        # Reward function weights
        reward_config = self.config.get("model", {}).get("reward_function", {})
        self.profit_weight = reward_config.get("profit_weight", 0.7)
        self.time_weight = reward_config.get("time_weight", 0.3)
        
        # Track statistics
        self.stats = {
            "episodes": 0,
            "avg_rewards": [],
            "avg_losses": [],
            "exploration_rate": []
        }
    
    def _load_config(self) -> None:
        """
        Load configuration from file
        """
        try:
            with open(self.config_path, 'r') as f:
                self.config = json.load(f)
            self.logger.info("Configuration loaded successfully")
        except Exception as e:
            self.logger.error(f"Error loading configuration: {str(e)}")
            # Use default configuration
            self.config = {
                "model": {
                    "drl": {
                        "model_type": "dqn",
                        "learning_rate": 0.001,
                        "discount_factor": 0.95,
                        "exploration_rate": 0.1,
                        "target_update_frequency": 1000,
                        "batch_size": 64,
                        "memory_size": 10000,
                        "num_hidden_layers": 3,
                        "hidden_layer_size": 128,
                        "activation": "relu",
                        "double_dqn": True,
                        "dueling_network": True,
                        "dropout_rate": 0.1
                    },
                    "reward_function": {
                        "profit_weight": 0.7,
                        "time_weight": 0.3
                    }
                }
            }
    
    def _initialize_networks(self):
        """Initialize Q-networks based on configuration"""
        # Choose network architecture based on config
        if self.dueling_network:
            self.qnetwork_local = DuelingDQNetwork(
                self.state_size, self.action_size, self.drl_config).to(self.device)
            self.qnetwork_target = DuelingDQNetwork(
                self.state_size, self.action_size, self.drl_config).to(self.device)
            self.logger.info("Using Dueling DQN architecture")
        else:
            self.qnetwork_local = DQNetwork(
                self.state_size, self.action_size, self.drl_config).to(self.device)
            self.qnetwork_target = DQNetwork(
                self.state_size, self.action_size, self.drl_config).to(self.device)
            self.logger.info("Using standard DQN architecture")
        
        # Copy weights from local to target network
        self.qnetwork_target.load_state_dict(self.qnetwork_local.state_dict())
    
    def step(self, state, action, reward, next_state, done):
        """
        Update the agent's knowledge based on experience
        
        Args:
            state: Current state
            action: Selected action
            reward: Received reward
            next_state: Next state
            done: Whether episode is done
            
        Returns:
            float: Loss value (0 if not learning)
        """
        # Save experience in replay memory
        self.memory.push(state, action, reward, next_state, done)
        
        # Update step counter
        self.t_step = (self.t_step + 1) % self.target_update_freq
        
        # Learn every self.batch_size steps, if enough samples in memory
        if len(self.memory) > self.batch_size:
            experiences = self.memory.sample(self.batch_size)
            loss = self._learn(experiences)
            
            # Log loss to tensorboard
            self.train_step += 1
            self.writer.add_scalar('Loss/train', loss, self.train_step)
            
            # Update target network periodically
            if self.t_step == 0:
                self._update_target_network()
            
            return loss
        
        return 0
    
    def act(self, state, evaluate=False):
        """
        Choose action based on current state using epsilon-greedy policy
        
        Args:
            state: Current state
            evaluate (bool): Whether to evaluate (no exploration)
            
        Returns:
            int: Selected action index
        """
        # Convert state to tensor
        state = torch.from_numpy(state).float().unsqueeze(0).to(self.device)
        
        # Set network to evaluation mode
        self.qnetwork_local.eval()
        
        with torch.no_grad():
            action_values = self.qnetwork_local(state)
        
        # Set network back to training mode
        self.qnetwork_local.train()
        
        # Epsilon-greedy action selection
        if not evaluate and random.random() < self.exploration_rate:
            return random.choice(np.arange(self.action_size))
        else:
            return np.argmax(action_values.cpu().data.numpy())
    
    def _learn(self, experiences):
        """
        Update value parameters using batch of experiences
        
        Args:
            experiences: Tuple of (states, actions, rewards, next_states, dones)
            
        Returns:
            float: Loss value
        """
        states, actions, rewards, next_states, dones = experiences
        
        # Move to device
        states = states.to(self.device)
        actions = actions.to(self.device)
        rewards = rewards.to(self.device)
        next_states = next_states.to(self.device)
        dones = dones.to(self.device)
        
        if self.double_dqn:
            # Double DQN: Select actions using local network, evaluate using target network
            # Get actions that would be selected by local network
            self.qnetwork_local.eval()
            with torch.no_grad():
                local_best_actions = self.qnetwork_local(next_states).argmax(dim=1, keepdim=True)
            self.qnetwork_local.train()
            
            # Evaluate those actions using the target network
            Q_targets_next = self.qnetwork_target(next_states).gather(1, local_best_actions)
        else:
            # Standard DQN: Get max predicted Q values for next states from target model
            Q_targets_next = self.qnetwork_target(next_states).detach().max(1)[0].unsqueeze(1)
        
        # Compute Q targets for current states
        Q_targets = rewards + (self.discount_factor * Q_targets_next * (1 - dones))
        
        # Get expected Q values from local model
        Q_expected = self.qnetwork_local(states).gather(1, actions)
        
        # Compute loss - Huber loss for stability
        loss = F.smooth_l1_loss(Q_expected, Q_targets)
        
        # Minimize the loss
        self.optimizer.zero_grad()
        loss.backward()
        
        # Gradient clipping to prevent exploding gradients
        torch.nn.utils.clip_grad_norm_(self.qnetwork_local.parameters(), 1.0)
        
        self.optimizer.step()
        
        return loss.item()
    
    def _update_target_network(self):
        """Update target network with parameters from local network"""
        # Soft update of the target network's weights (θ_target = τ*θ_local + (1 - τ)*θ_target)
        tau = 0.001  # Small update rate for stability
        for target_param, local_param in zip(self.qnetwork_target.parameters(), self.qnetwork_local.parameters()):
            target_param.data.copy_(tau * local_param.data + (1.0 - tau) * target_param.data)
    
    def save_model(self, episode: int = None):
        """
        Save the agent's model
        
        Args:
            episode (int): Current episode number (used for filename)
            
        Returns:
            str: Path to saved model
        """
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        episode_str = f"_episode_{episode}" if episode is not None else ""
        
        model_path = os.path.join(self.models_dir, f"drl_model_{timestamp}{episode_str}.pt")
        
        torch.save({
            'qnetwork_local_state_dict': self.qnetwork_local.state_dict(),
            'qnetwork_target_state_dict': self.qnetwork_target.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'exploration_rate': self.exploration_rate,
            'stats': self.stats,
            'config': {
                'state_size': self.state_size,
                'action_size': self.action_size,
                'double_dqn': self.double_dqn,
                'dueling_network': self.dueling_network
            }
        }, model_path)
        
        self.logger.info(f"Model saved to {model_path}")
        return model_path
    
    def load_model(self, model_path: str):
        """
        Load a saved model
        
        Args:
            model_path (str): Path to saved model
            
        Returns:
            bool: True if loaded successfully, False otherwise
        """
        if not os.path.exists(model_path):
            self.logger.error(f"Model file not found: {model_path}")
            return False
        
        try:
            checkpoint = torch.load(model_path, map_location=self.device)
            
            # Check if model architecture matches
            config = checkpoint.get('config', {})
            if config.get('state_size') != self.state_size or config.get('action_size') != self.action_size:
                self.logger.warning(f"Model architecture mismatch. Reinitializing networks.")
                
                # Handle architecture mismatch by reinitializing networks
                self.dueling_network = config.get('dueling_network', self.dueling_network)
                self.double_dqn = config.get('double_dqn', self.double_dqn)
                self._initialize_networks()
            
            self.qnetwork_local.load_state_dict(checkpoint['qnetwork_local_state_dict'])
            self.qnetwork_target.load_state_dict(checkpoint['qnetwork_target_state_dict'])
            self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            self.exploration_rate = checkpoint.get('exploration_rate', 0.1)
            self.stats = checkpoint.get('stats', self.stats)
            
            self.logger.info(f"Model loaded from {model_path}")
            return True
        except Exception as e:
            self.logger.error(f"Error loading model: {str(e)}")
            return False
    
    def compute_reward(self, state, action, next_state, price, sold_price=None, time_to_sell=None):
        """
        Compute the reward for a state-action pair
        
        Args:
            state: Current state
            action: Action taken
            next_state: Resulting state
            price: Original item price
            sold_price: Final selling price (if sold)
            time_to_sell: Time to sell in days (if sold)
            
        Returns:
            float: Computed reward
        """
        # If item sold, use actual profit and time
        if sold_price is not None and time_to_sell is not None:
            profit_margin = (sold_price - price) / price if price > 0 else 0
            
            # Normalize time to sell (consider 30 days as max)
            norm_time = max(0, 1 - (time_to_sell / 30)) if time_to_sell > 0 else 0
            
            # Combine profit and time components
            reward = (self.profit_weight * profit_margin) + (self.time_weight * norm_time)
            
            return reward * 10  # Scale reward
        
        # If not sold yet, use market-based reward estimation
        price_competitiveness = next_state[4] if len(next_state) > 4 else 0.5  # Use price competitiveness feature
        conversion_rate = next_state[7] if len(next_state) > 7 else 0.5  # Use conversion rate feature
        
        # Penalize extreme pricing - too high or too low
        # Action represents price level index - convert to relative price
        price_level = action / (self.action_size - 1)  # normalize to [0,1]
        price_penalty = -2.0 * abs(price_level - 0.5)  # penalize deviation from middle price
        
        # Combine factors for reward estimation
        estimated_reward = (price_competitiveness * 0.6) + (conversion_rate * 0.4) + price_penalty
        
        return estimated_reward * 5  # Scale reward
    
    def decay_exploration_rate(self, decay_factor=0.995, min_rate=0.01):
        """
        Decay the exploration rate
        
        Args:
            decay_factor (float): Factor to decay by
            min_rate (float): Minimum exploration rate
            
        Returns:
            float: New exploration rate
        """
        self.exploration_rate = max(min_rate, self.exploration_rate * decay_factor)
        self.stats["exploration_rate"].append(self.exploration_rate)
        return self.exploration_rate

2025-04-22 23:42:35.215208: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745390555.304829     422 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745390555.325474     422 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-22 23:42:35.444858: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [20]:
import os
import glob
import pandas as pd

# Check if there's any data collected
raw_data_dir = "/data/chats/p6wyr/workspace/data/raw"
processed_data_dir = "/data/chats/p6wyr/workspace/data/processed"
features_dir = "/data/chats/p6wyr/workspace/data/features"

# Check data directories
print(f"Data directories exist:")
print(f"Raw data: {os.path.exists(raw_data_dir)}")
print(f"Processed data: {os.path.exists(processed_data_dir)}")
print(f"Features data: {os.path.exists(features_dir)}")

# Look for category folders in raw data
category_folders = glob.glob(os.path.join(raw_data_dir, "category_*"))
print(f"\nFound {len(category_folders)} category folders")
if category_folders:
    for folder in category_folders:
        print(f"- {os.path.basename(folder)}")
        
        # Check for collected data files
        active_files = glob.glob(os.path.join(folder, "active_items_*.csv"))
        sold_files = glob.glob(os.path.join(folder, "sold_items_*.csv"))
        
        if active_files:
            latest_active = sorted(active_files)[-1]
            print(f"  Latest active items file: {os.path.basename(latest_active)}")
            if os.path.exists(latest_active):
                df_active = pd.read_csv(latest_active)
                print(f"  Active items: {len(df_active)} rows")
                
                # Get column info for active items
                from metagpt.tools.libs.data_preprocess import get_column_info
                column_info = get_column_info(df_active)
                print("\nActive items column_info:")
                print(column_info)
                
        if sold_files:
            latest_sold = sorted(sold_files)[-1]
            print(f"  Latest sold items file: {os.path.basename(latest_sold)}")
            if os.path.exists(latest_sold):
                df_sold = pd.read_csv(latest_sold)
                print(f"  Sold items: {len(df_sold)} rows")
                
                # Get column info for sold items
                from metagpt.tools.libs.data_preprocess import get_column_info
                column_info = get_column_info(df_sold)
                print("\nSold items column_info:")
                print(column_info)
else:
    print("No data has been collected yet.")

# Check config file
config_path = "/data/chats/p6wyr/workspace/config/config.json"
if os.path.exists(config_path):
    import json
    with open(config_path, 'r') as f:
        config = json.load(f)
    print("\nConfiguration:")
    if "data" in config and "categories" in config["data"]:
        print(f"Categories configured: {len(config['data']['categories'])}")
        for cat in config["data"]["categories"]:
            print(f"- {cat['name']} (ID: {cat['id']})")
    else:
        print("No categories configured in config file")
else:
    print("\nConfig file not found")

Data directories exist:
Raw data: True
Processed data: True
Features data: True

Found 0 category folders
No data has been collected yet.

Configuration:
Categories configured: 4
- Laptops & Netbooks (ID: 9355)
- Cell Phones & Smartphones (ID: 15032)
- Wristwatch (ID: 11450)
- Digital Cameras (ID: 261007)


In [21]:
I'll implement the price environment and trainer class for the deep reinforcement learning model. These components will allow the model to interact with the environment and learn optimal pricing strategies.

```python
import os
import json
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
from tqdm import tqdm
from typing import Dict, List, Tuple, Optional, Any
from datetime import datetime
import random
import glob

from src.utils.logger import Logger

class PriceEnvironment:
    """
    Environment for price prediction, simulating market dynamics
    """
    def __init__(self, features_df: pd.DataFrame, category_id: str, num_price_levels: int = 10):
        """
        Initialize the environment
        
        Args:
            features_df (pd.DataFrame): DataFrame with item features
            category_id (str): Category ID
            num_price_levels (int): Number of discrete price levels
        """
        self.logger = Logger().get_logger()
        self.features_df = features_df
        self.category_id = category_id
        
        # Get sold items for statistics
        self.sold_items = features_df[features_df['is_sold'] == 1]
        self.unsold_items = features_df[features_df['is_sold'] == 0]
        
        # Extract price range
        self.min_price = features_df['price'].min()
        self.max_price = features_df['price'].max()
        self.avg_price = features_df['price'].mean()
        
        # Calculate price levels (actions)
        self.num_price_levels = num_price_levels
        # Dynamic price range based on data: from 70% to 130% of item's base price
        self.price_levels = np.linspace(0.7, 1.3, num_price_levels)
        
        # Current state info
        self.current_item_idx = 0
        self.current_state = None
        self.base_price = 0
        
        self.logger.info(f"Initialized price environment with {len(features_df)} items, {num_price_levels} price levels")
    
    def reset(self, item_idx=None):
        """
        Reset the environment and get new item
        
        Args:
            item_idx (int): Index of item to use (None for random)
            
        Returns:
            np.array: Initial state
        """
        # Select item (random if not specified)
        if item_idx is None:
            self.current_item_idx = np.random.randint(0, len(self.features_df))
        else:
            self.current_item_idx = item_idx % len(self.features_df)
        
        # Get item data
        item = self.features_df.iloc[self.current_item_idx]
        
        # Extract state features (excluding ID, price, is_sold, category_id, days_to_sell)
        state_columns = [col for col in self.features_df.columns 
                         if col not in ['itemId', 'price', 'is_sold', 'category_id', 'days_to_sell']]
        
        # Set current state
        self.current_state = item[state_columns].values.astype(np.float32)
        
        # Set base price
        self.base_price = item['price']
        
        return self.current_state
    
    def step(self, action):
        """
        Take a step in the environment
        
        Args:
            action (int): Action index (price level)
            
        Returns:
            tuple: (next_state, reward, done, info)
        """
        # Get selected price level
        price_factor = self.price_levels[action]
        suggested_price = self.base_price * price_factor
        
        # Get current item
        item = self.features_df.iloc[self.current_item_idx]
        
        # Check if item sold in historical data
        is_sold = item['is_sold'] == 1
        days_to_sell = item.get('days_to_sell', 30) if is_sold else None
        
        # Get actual sold price if available
        sold_price = item['price'] if is_sold else None
        
        # Calculate reward
        reward = self._calculate_reward(suggested_price, is_sold, sold_price, days_to_sell)
        
        # Return the same state (as this is not a sequential environment)
        next_state = self.current_state
        
        # Episode is done after one step (predict price only once per item)
        done = True
        
        # Additional info
        info = {
            'item_id': item.get('itemId', str(self.current_item_idx)),
            'base_price': self.base_price,
            'suggested_price': suggested_price,
            'price_factor': price_factor,
            'is_sold': is_sold,
            'days_to_sell': days_to_sell,
            'sold_price': sold_price
        }
        
        return next_state, reward, done, info
    
    def _calculate_reward(self, suggested_price, is_sold, sold_price, days_to_sell):
        """
        Calculate reward for the action
        
        Args:
            suggested_price (float): Price suggested by agent
            is_sold (bool): Whether item was sold
            sold_price (float): Actual sold price
            days_to_sell (float): Days taken to sell
            
        Returns:
            float: Reward value
        """
        # For sold items, compare the suggested price to actual sold price
        if is_sold and sold_price is not None:
            # Calculate profit margin
            profit_margin = (suggested_price / sold_price) - 1
            
            # Optimize for selling close to maximum price possible
            # Overpricing penalty (gets more severe as price increases)
            if suggested_price > 1.2 * sold_price:
                return -2.0 * (suggested_price / sold_price - 1.2)
            
            # Reward for accurate pricing (within 10% of actual)
            if 0.95 * sold_price <= suggested_price <= 1.05 * sold_price:
                # Additional reward for faster selling items
                time_factor = 1.0
                if days_to_sell is not None:
                    time_factor = max(0.5, 1.0 - (days_to_sell / 30))
                
                return 2.0 * time_factor
            
            # Smaller reward for reasonable pricing (within 20% of actual)
            if 0.9 * sold_price <= suggested_price <= 1.1 * sold_price:
                return 1.0
            
            # Penalize significantly underpricing (lost potential profit)
            if suggested_price < 0.8 * sold_price:
                return -1.0 * (0.8 - suggested_price / sold_price)
            
            # Slight reward for other cases based on how close to optimal
            return 0.5 - abs(profit_margin)
        
        # For unsold items, evaluate based on market dynamics
        else:
            # Get similar items
            similar_sold = self._find_similar_items(is_sold=True)
            
            if len(similar_sold) > 0:
                avg_similar_price = similar_sold['price'].mean()
                avg_days_to_sell = similar_sold['days_to_sell'].mean() if 'days_to_sell' in similar_sold else 30
                
                # Price ratio compared to similar items
                price_ratio = suggested_price / avg_similar_price
                
                # Reward pricing close to successful similar items
                if 0.9 * avg_similar_price <= suggested_price <= 1.1 * avg_similar_price:
                    return 1.0
                
                # Smaller reward for reasonable pricing
                if 0.8 * avg_similar_price <= suggested_price <= 1.2 * avg_similar_price:
                    return 0.5
                
                # Penalize significant deviation from market
                if price_ratio > 1.3 or price_ratio < 0.7:
                    return -1.0 * (abs(price_ratio - 1) - 0.3)
                
                # Slight reward/penalty based on proximity to market price
                return 0.2 - abs(price_ratio - 1)
            
        # Default small negative reward if we can't determine better
        # This encourages the agent to prefer states where it can make better predictions
        return -0.1
    
    def _find_similar_items(self, is_sold=True, top_n=5):
        """
        Find items similar to current item
        
        Args:
            is_sold (bool): Whether to find sold or unsold items
            top_n (int): Number of similar items to return
            
        Returns:
            pd.DataFrame: Similar items
        """
        # Current item
        item = self.features_df.iloc[self.current_item_idx]
        
        # Filter by sold status
        items_subset = self.sold_items if is_sold else self.unsold_items
        
        if len(items_subset) == 0:
            return pd.DataFrame()
        
        # Extract state features (excluding ID, price, is_sold, category_id, days_to_sell)
        feature_columns = [col for col in self.features_df.columns 
                          if col not in ['itemId', 'price', 'is_sold', 'category_id', 'days_to_sell']]
        
        if len(feature_columns) == 0:
            # If no features available, return random subset
            return items_subset.sample(min(top_n, len(items_subset)))
        
        # For more sophisticated similarity, use feature values
        current_features = item[feature_columns].values
        
        # Calculate distance for each item
        distances = []
        for idx, row in items_subset.iterrows():
            # Calculate Euclidean distance in feature space
            distance = np.sqrt(np.sum((row[feature_columns].values - current_features) ** 2))
            distances.append((idx, distance))
        
        # Sort by distance and get top_n closest items
        distances.sort(key=lambda x: x[1])
        similar_indices = [idx for idx, _ in distances[:top_n]]
        
        return items_subset.loc[similar_indices]


class PricePredictionTrainer:
    """
    Trainer class for price prediction model
    """
    def __init__(self, category_id: str, config_path: str = "/data/chats/p6wyr/workspace/config/config.json"):
        """
        Initialize the trainer
        
        Args:
            category_id (str): Category ID to train model for
            config_path (str): Path to configuration file
        """
        self.logger = Logger().get_logger()
        self.logger.info(f"Initializing price prediction trainer for category {category_id}")
        
        self.category_id = category_id
        self.config_path = config_path
        
        # Load configuration
        self._load_config()
        
        # Set paths
        self.features_dir = f"/data/chats/p6wyr/workspace/data/features/category_{category_id}"
        self.models_dir = f"/data/chats/p6wyr/workspace/data/models/category_{category_id}"
        self.results_dir = f"/data/chats/p6wyr/workspace/data/results/category_{category_id}"
        
        os.makedirs(self.models_dir, exist_ok=True)
        os.makedirs(self.results_dir, exist_ok=True)
        
        # Load data
        self.features_df = self._load_features()
        
        # Set up environment and agent
        if self.features_df is not None:
            self._setup_env_and_agent()
        else:
            self.logger.error("Failed to load features data. Cannot set up environment and agent.")
            self.environment = None
            self.agent = None
    
    def _load_config(self) -> None:
        """
        Load configuration from file
        """
        try:
            with open(self.config_path, 'r') as f:
                self.config = json.load(f)
            self.logger.info("Configuration loaded successfully")
        except Exception as e:
            self.logger.error(f"Error loading configuration: {str(e)}")
            # Use default configuration
            self.config = {
                "model": {
                    "drl": {
                        "model_type": "dqn",
                        "learning_rate": 0.001,
                        "discount_factor": 0.95,
                        "exploration_rate": 0.2,
                        "target_update_frequency": 1000,
                        "batch_size": 64,
                        "memory_size": 10000,
                        "num_hidden_layers": 3,
                        "hidden_layer_size": 128,
                        "activation": "relu",
                        "double_dqn": True,
                        "dueling_network": True
                    }
                },
                "data": {
                    "min_items_for_training": 500
                }
            }
    
    def _load_features(self) -> Optional[pd.DataFrame]:
        """
        Load features data for the category
        
        Returns:
            Optional[pd.DataFrame]: Features DataFrame or None if not found
        """
        # Find the latest features file
        feature_files = sorted(glob.glob(os.path.join(self.features_dir, "features_*.csv")))
        
        if not feature_files:
            self.logger.error(f"No feature files found in {self.features_dir}")
            # Create synthetic data for demonstration if no real data exists
            return self._create_synthetic_data()
        
        # Load latest features file
        latest_features_file = feature_files[-1]
        self.logger.info(f"Loading features from {latest_features_file}")
        
        try:
            df = pd.read_csv(latest_features_file)
            self.logger.info(f"Loaded {len(df)} feature records")
            return df
        except Exception as e:
            self.logger.error(f"Error loading features: {str(e)}")
            # Create synthetic data as a fallback
            return self._create_synthetic_data()

    def _create_synthetic_data(self) -> pd.DataFrame:
        """
        Create synthetic data for demonstration purposes
        
        Returns:
            pd.DataFrame: Synthetic features data
        """
        self.logger.warning("Creating synthetic data for demonstration")
        
        # Number of synthetic items
        n_items = 1000  # Increased for better training
        
        # Create random feature data with controlled distributions
        np.random.seed(42)  # For reproducibility
        
        # Get state features from config
        state_features = self.config.get("model", {}).get("state_features", [
            "item_condition", 
            "brand_popularity", 
            "season_demand", 
            "market_saturation",
            "avg_price", 
            "price_std", 
            "time_on_market",
            "conversion_rate"
        ])
        
        # Create item IDs and basic properties
        data = {
            'itemId': [f"synth_{i}" for i in range(n_items)],
            'category_id': self.category_id
        }
        
        # Create prices with log-normal distribution (more realistic)
        mean_price = 500
        data['price'] = np.random.lognormal(mean=np.log(mean_price), sigma=0.5, size=n_items)
        
        # Create is_sold with price-dependent probability
        # Lower-priced items are more likely to be sold
        price_normalized = (data['price'] - data['price'].min()) / (data['price'].max() - data['price'].min())
        sold_prob = 0.7 - 0.4 * price_normalized  # Higher price → lower probability of being sold
        data['is_sold'] = np.random.binomial(1, sold_prob)
        
        # Add state features with some correlations to make it more realistic
        for feature in state_features:
            if feature == "item_condition":
                # Better condition items tend to be more expensive
                condition_base = np.random.uniform(0, 1, n_items)
                price_effect = 0.3 * price_normalized
                data[feature] = np.clip(condition_base + price_effect, 0, 1)
            
            elif feature == "brand_popularity":
                # Create a few popular brands, many unpopular ones
                brands = np.random.power(0.3, n_items)  # Power distribution
                data[feature] = brands
            
            elif feature == "season_demand":
                # Random seasonal demand
                data[feature] = np.random.uniform(0, 1, n_items)
            
            elif feature == "market_saturation":
                # Market saturation correlates negatively with price
                saturation_base = np.random.uniform(0.3, 0.7, n_items)
                price_effect = -0.2 * price_normalized  # Higher price → lower saturation
                data[feature] = np.clip(saturation_base + price_effect, 0, 1)
            
            elif feature == "avg_price":
                # Normalize price to [0,1]
                data[feature] = price_normalized
            
            elif feature == "price_std":
                # Random price standard deviation
                data[feature] = np.random.uniform(0.05, 0.2, n_items)
            
            elif feature == "time_on_market":
                # More expensive items tend to stay longer on market
                time_base = np.random.uniform(0, 0.5, n_items)
                price_effect = 0.5 * price_normalized  # Higher price → longer time
                data[feature] = np.clip(time_base + price_effect, 0, 1)
            
            elif feature == "conversion_rate":
                # Conversion rate correlates negatively with price
                conversion_base = np.random.uniform(0.3, 0.8, n_items)
                price_effect = -0.4 * price_normalized  # Higher price → lower conversion
                data[feature] = np.clip(conversion_base + price_effect, 0, 1)
            
            else:
                # Random values for other features
                data[feature] = np.random.uniform(0, 1, n_items)
        
        # Add days_to_sell for sold items with price dependency
        df = pd.DataFrame(data)
        sold_mask = df['is_sold'] == 1
        
        # Higher-priced items take longer to sell
        days_base = np.random.uniform(1, 15, sold_mask.sum())
        price_effect = 15 * price_normalized[sold_mask]
        df.loc[sold_mask, 'days_to_sell'] = days_base + price_effect
        
        self.logger.info(f"Created synthetic dataset with {n_items} items")
        return df
    
    def _setup_env_and_agent(self) -> None:
        """
        Set up environment and DRL agent
        """
        # Define number of price levels (actions)
        self.num_price_levels = 15  # Increased for finer price granularity
        
        # Create environment
        self.environment = PriceEnvironment(
            features_df=self.features_df,
            category_id=self.category_id,
            num_price_levels=self.num_price_levels
        )
        
        # Get state size (number of features)
        state_columns = [col for col in self.features_df.columns 
                         if col not in ['itemId', 'price', 'is_sold', 'category_id', 'days_to_sell']]
        state_size = len(state_columns)
        
        # Create agent with improved configuration
        from src.models.drl_agent import PricePredictionDRLAgent  # Import the agent class we defined earlier
        
        self.agent = PricePredictionDRLAgent(
            state_size=state_size,
            action_size=self.num_price_levels,
            category_id=self.category_id,
            config_path=self.config_path
        )
        
        self.logger.info(f"Set up environment and agent with state size {state_size} and {self.num_price_levels} actions")
    
    def train(self, num_episodes: int = 2000, batch_size: int = 64, checkpoint_interval: int = 200):
        """
        Train the DRL agent
        
        Args:
            num_episodes (int): Number of training episodes
            batch_size (int): Batch size for learning
            checkpoint_interval (int): Interval for saving model checkpoints
            
        Returns:
            dict: Training statistics
        """
        if self.agent is None or self.environment is None:
            self.logger.error("Agent or environment not initialized. Cannot train.")
            return {"error": "Agent or environment not initialized"}
        
        self.logger.info(f"Starting training for {num_episodes} episodes")
        
        # Record start time
        start_time = datetime.now()
        
        # Initialize progress tracking
        rewards = []
        losses = []
        epsilons = []
        
        # Training loop with tqdm progress bar
        for episode in tqdm(range(num_episodes), desc="Training"):
            # Reset environment
            state = self.environment.reset()
            
            # Select action using epsilon-greedy policy
            action = self.agent.act(state)
            
            # Take step in environment
            next_state, reward, done, info = self.environment.step(action)
            
            # Update agent
            loss = self.agent.step(state, action, reward, next_state, done)
            
            # Track statistics
            rewards.append(reward)
            if loss > 0:
                losses.append(loss)
            epsilons.append(self.agent.exploration_rate)
            
            # Log to tensorboard periodically
            if episode % 10 == 0:
                self.agent.writer.add_scalar('Reward', reward, episode)
                self.agent.writer.add_scalar('Exploration Rate', self.agent.exploration_rate, episode)
                if loss > 0:
                    self.agent.writer.add_scalar('Loss', loss, episode)
            
            # Save checkpoint periodically
            if episode > 0 and episode % checkpoint_interval == 0:
                self.agent.save_model(episode=episode)
                
                # Calculate running averages
                avg_reward = np.mean(rewards[-checkpoint_interval:])
                avg_loss = np.mean(losses[-min(checkpoint_interval, len(losses)):]) if losses else 0
                
                self.agent.stats["episodes"] = episode
                self.agent.stats["avg_rewards"].append((episode, avg_reward))
                self.agent.stats["avg_losses"].append((episode, avg_loss))
                
                self.logger.info(f"Episode {episode}/{num_episodes} - Avg Reward: {avg_reward:.4f}, Avg Loss: {avg_loss:.4f}, Epsilon: {self.agent.exploration_rate:.4f}")
            
            # Decay exploration rate
            self.agent.decay_exploration_rate()
        
        # Final model save
        self.agent.save_model(episode=num_episodes)
        
        # Calculate training time
        training_time = datetime.now() - start_time
        
        # Generate training statistics
        train_stats = {
            "episodes": num_episodes,
            "final_exploration_rate": self.agent.exploration_rate,
            "avg_reward_last_100": np.mean(rewards[-100:]),
            "avg_loss_last_100": np.mean(losses[-100:]) if len(losses) >= 100 else np.mean(losses),
            "training_time_seconds": training_time.total_seconds()
        }
        
        # Save training stats
        stats_path = os.path.join(self.results_dir, f"training_stats_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json")
        with open(stats_path, 'w') as f:
            json.dump(train_stats, f, indent=4)
        
        self.logger.info(f"Training completed in {training_time}. Stats saved to {stats_path}")
        
        # Plot learning curves
        self._plot_learning_curves(rewards, losses, epsilons, num_episodes)
        
        return train_stats
    
    def _plot_learning_curves(self, rewards, losses, epsilons, num_episodes):
        """
        Plot learning curves from training data
        
        Args:
            rewards (list): Episode rewards
            losses (list): Episode losses
            epsilons (list): Exploration rates
            num_episodes (int): Total number of episodes
        """
        # Create plots directory
        plots_dir = os.path.join(self.results_dir, "plots")
        os.makedirs(plots_dir, exist_ok=True)
        
        # Calculate moving averages
        window = min(100, num_episodes // 20)
        rewards_smoothed = self._moving_average(rewards, window)
        losses_smoothed = self._moving_average(losses, window) if losses else []
        
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        # Plot rewards
        plt.figure(figsize=(10, 5))
        plt.plot(rewards, alpha=0.3, color='blue')
        plt.plot(rewards_smoothed, color='blue', linewidth=2)
        plt.title('Episode Rewards')
        plt.xlabel('Episode')
        plt.ylabel('Reward')
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(os.path.join(plots_dir, f"rewards_{timestamp}.png"))
        
        # Plot losses if available
        if losses:
            plt.figure(figsize=(10, 5))
            plt.plot(losses, alpha=0.3, color='red')
            plt.plot(losses_smoothed, color='red', linewidth=2)
            plt.title('Episode Losses')
            plt.xlabel('Episode')
            plt.ylabel('Loss')
            plt.grid(True)
            plt.tight_layout()
            plt.savefig(os.path.join(plots_dir, f"losses_{timestamp}.png"))
        
        # Plot exploration rate
        plt.figure(figsize=(10, 5))
        plt.plot(epsilons, color='green', linewidth=2)
        plt.title('Exploration Rate')
        plt.xlabel('Episode')
        plt.ylabel('Epsilon')
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(os.path.join(plots_dir, f"epsilon_{timestamp}.png"))
        
        plt.close('all')
    
    def _moving_average(self, values, window):
        """
        Calculate the moving average of a list
        
        Args:
            values (list): List of values
            window (int): Window size
            
        Returns:
            list: Moving average values
        """
        weights = np.repeat(1.0, window) / window
        return np.convolve(values, weights, 'valid')
    
    def evaluate(self, num_episodes: int = 100, visualize: bool = True):
        """
        Evaluate the trained agent
        
        Args:
            num_episodes (int): Number of evaluation episodes
            visualize (bool): Whether to visualize results
            
        Returns:
            dict: Evaluation statistics
        """
        if self.agent is None or self.environment is None:
            self.logger.error("Agent or environment not initialized. Cannot evaluate.")
            return {"error": "Agent or environment not initialized"}
        
        self.logger.info(f"Evaluating agent for {num_episodes} episodes")
        
        # Initialize metrics
        rewards = []
        price_diffs = []
        price_ratios = []
        correct_direction = 0  # Count when agent correctly identifies if price should go up or down
        
        # Set agent to evaluation mode
        self.agent.exploration_rate = 0.0
        
        # Evaluation loop
        results = []
        for episode in tqdm(range(num_episodes), desc="Evaluating"):
            # Reset environment and get random item
            state = self.environment.reset()
            
            # Get base price of item
            base_price = self.environment.base_price
            
            # Select action without exploration
            action = self.agent.act(state, evaluate=True)
            
            # Take step in environment
            next_state, reward, done, info = self.environment.step(action)
            
            # Track metrics
            rewards.append(reward)
            
            # Calculate price difference and ratio
            suggested_price = info['suggested_price']
            sold_price = info['sold_price']
            
            if sold_price is not None:
                price_diff = suggested_price - sold_price
                price_ratio = suggested_price / sold_price
                price_diffs.append(price_diff)
                price_ratios.append(price_ratio)
                
                # Check if agent correctly identified direction
                if (suggested_price > base_price and sold_price > base_price) or \
                   (suggested_price < base_price and sold_price < base_price):
                    correct_direction += 1
            
            # Save detailed results
            results.append(info)
        
        # Calculate evaluation metrics
        accuracy_metrics = {
            "avg_reward": np.mean(rewards),
            "median_reward": np.median(rewards),
            "avg_price_diff": np.mean(price_diffs) if price_diffs else None,
            "avg_price_ratio": np.mean(price_ratios) if price_ratios else None,
            "direction_accuracy": correct_direction / len(price_diffs) if price_diffs else None,
            "within_5pct": sum(0.95 <= ratio <= 1.05 for ratio in price_ratios) / len(price_ratios) if price_ratios else None,
            "within_10pct": sum(0.9 <= ratio <= 1.1 for ratio in price_ratios) / len(price_ratios) if price_ratios else None,
        }
        
        # Save evaluation results
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        eval_path = os.path.join(self.results_dir, f"evaluation_{timestamp}.json")
        
        eval_results = {
            "metrics": accuracy_metrics,
            "detailed_results": results
        }
        
        with open(eval_path, 'w') as f:
            # Convert numpy values to Python native types for JSON serialization
            json_results = json.dumps(self._convert_to_serializable(eval_results), indent=4)
            f.write(json_results)
        
        self.logger.info(f"Evaluation completed. Results saved to {eval_path}")
        
        # Create visualizations
        if visualize:
            self._visualize_evaluation_results(price_ratios, rewards, results)
        
        return accuracy_metrics
    
    def _convert_to_serializable(self, obj):
        """
        Convert numpy types to Python native types for JSON serialization
        
        Args:
            obj: Object to convert
            
        Returns:
            object: Serializable object
        """
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, dict):
            return {k: self._convert_to_serializable(v) for k, v in obj.items()}
        elif isinstance(obj, list):
            return [self._convert_to_serializable(item) for item in obj]
        else:
            return obj
    
    def _visualize_evaluation_results(self, price_ratios, rewards, results):
        """
        Create visualizations of evaluation results
        
        Args:
            price_ratios (list): List of price ratios
            rewards (list): List of rewards
            results (list): List of detailed results
        """
        plots_dir = os.path.join(self.results_dir, "plots")
        os.makedirs(plots_dir, exist_ok=True)
        
        timestamp = datetime.now().str

SyntaxError: unterminated string literal (detected at line 1) (2496708699.py, line 1)

In [22]:
timestamp = datetime.now().str

AttributeError: 'datetime.datetime' object has no attribute 'str'

In [23]:
timestamp = datetime.now().str

AttributeError: 'datetime.datetime' object has no attribute 'str'

In [24]:
# Check directory structure 
import os
from pathlib import Path
import glob

# Check important directories
workspace_root = "/data/chats/p6wyr/workspace"
data_dirs = [
    "data/raw",
    "data/processed",
    "data/features",
    "data/cache/api_responses"
]

print("Directory structure:")
for dir_path in data_dirs:
    full_path = os.path.join(workspace_root, dir_path)
    exists = os.path.exists(full_path)
    contents = os.listdir(full_path) if exists else []
    print(f"{full_path}: {'EXISTS' if exists else 'MISSING'}, {len(contents)} items")

# Check for category data
raw_categories = glob.glob(f"{workspace_root}/data/raw/category_*")
processed_categories = glob.glob(f"{workspace_root}/data/processed/category_*")
feature_categories = glob.glob(f"{workspace_root}/data/features/category_*")

print("\nCategories found:")
print(f"Raw data categories: {len(raw_categories)}")
print(f"Processed data categories: {len(processed_categories)}")
print(f"Feature categories: {len(feature_categories)}")

# Check for config file
config_path = f"{workspace_root}/config/config.json"
config_exists = os.path.exists(config_path)
print(f"\nConfig file exists: {config_exists}")

if config_exists:
    import json
    with open(config_path, 'r') as f:
        config = json.load(f)
    print(f"Config categories: {len(config.get('data', {}).get('categories', []))}")
    categories = config.get('data', {}).get('categories', [])
    for cat in categories:
        print(f"  - {cat.get('id')}: {cat.get('name')}")
else:
    print("Config file not found. Using default categories.")

# Check for feature data
feature_files = glob.glob(f"{workspace_root}/data/features/category_*/features_*.csv")
print(f"\nFeature files found: {len(feature_files)}")

if feature_files:
    import pandas as pd
    # Load the most recent feature file to examine
    latest_feature_file = sorted(feature_files)[-1]
    print(f"Loading most recent feature file: {latest_feature_file}")
    df = pd.read_csv(latest_feature_file)
    
    from metagpt.tools.libs.data_preprocess import get_column_info
    column_info = get_column_info(df)
    print("\nFeature data columns:")
    print(column_info)
else:
    print("No feature files found.")

Directory structure:
/data/chats/p6wyr/workspace/data/raw: EXISTS, 1 items
/data/chats/p6wyr/workspace/data/processed: EXISTS, 1 items
/data/chats/p6wyr/workspace/data/features: EXISTS, 1 items
/data/chats/p6wyr/workspace/data/cache/api_responses: EXISTS, 0 items

Categories found:
Raw data categories: 0
Processed data categories: 0
Feature categories: 0

Config file exists: True
Config categories: 4
  - 9355: Laptops & Netbooks
  - 15032: Cell Phones & Smartphones
  - 11450: Wristwatch
  - 261007: Digital Cameras

Feature files found: 0
No feature files found.


In [25]:
I'll implement the PriceEnvironment and PricePredictionTrainer classes for the deep reinforcement learning model. These components will allow us to simulate the second-hand item pricing environment and train our DRL agent.

```python
import os
import json
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from datetime import datetime
from typing import Dict, List, Tuple, Optional, Any
import random
import glob

from src.utils.logger import Logger


class PriceEnvironment:
    """
    Environment for price prediction, simulating market dynamics
    """
    def __init__(self, features_df: pd.DataFrame, category_id: str, num_price_levels: int = 10):
        """
        Initialize the environment
        
        Args:
            features_df (pd.DataFrame): DataFrame with item features
            category_id (str): Category ID
            num_price_levels (int): Number of discrete price levels
        """
        self.logger = Logger().get_logger()
        self.features_df = features_df.copy()
        self.category_id = category_id
        
        # Get sold items for statistics
        self.sold_items = self.features_df[self.features_df['is_sold'] == 1]
        self.unsold_items = self.features_df[self.features_df['is_sold'] == 0]
        
        # Extract price range
        self.min_price = self.features_df['price'].min()
        self.max_price = self.features_df['price'].max()
        self.avg_price = self.features_df['price'].mean()
        
        # Calculate price levels (actions)
        self.num_price_levels = num_price_levels
        self.price_levels = np.linspace(0.5, 1.5, num_price_levels)  # 0.5x to 1.5x of item's base price
        
        # Current state info
        self.current_item_idx = 0
        self.current_state = None
        self.base_price = 0
        
        self.logger.info(f"Initialized price environment with {len(features_df)} items, {num_price_levels} price levels")
    
    def reset(self, item_idx=None):
        """
        Reset the environment and get new item
        
        Args:
            item_idx (int): Index of item to use (None for random)
            
        Returns:
            np.array: Initial state
        """
        # Select item (random if not specified)
        if item_idx is None:
            self.current_item_idx = np.random.randint(0, len(self.features_df))
        else:
            self.current_item_idx = item_idx % len(self.features_df)
        
        # Get item data
        item = self.features_df.iloc[self.current_item_idx]
        
        # Extract state features (excluding ID, price, is_sold, category_id, days_to_sell)
        state_columns = [col for col in self.features_df.columns 
                         if col not in ['itemId', 'price', 'is_sold', 'category_id', 'days_to_sell']]
        
        # Set current state
        self.current_state = item[state_columns].values.astype(np.float32)
        
        # Set base price
        self.base_price = item['price']
        
        return self.current_state
    
    def step(self, action):
        """
        Take a step in the environment
        
        Args:
            action (int): Action index (price level)
            
        Returns:
            tuple: (next_state, reward, done, info)
        """
        # Get selected price level
        price_factor = self.price_levels[action]
        suggested_price = self.base_price * price_factor
        
        # Get current item
        item = self.features_df.iloc[self.current_item_idx]
        
        # Check if item sold in historical data
        is_sold = item['is_sold'] == 1
        days_to_sell = item.get('days_to_sell', 30)  # Default to 30 days if unknown
        
        # Get actual sold price if available
        sold_price = item['price'] if is_sold else None
        
        # Calculate reward
        reward = self._calculate_reward(suggested_price, is_sold, sold_price, days_to_sell)
        
        # Return the same state (as this is not a sequential environment)
        next_state = self.current_state
        
        # Episode is done after one step (predict price only once per item)
        done = True
        
        # Additional info
        info = {
            'item_id': item.get('itemId', self.current_item_idx),
            'base_price': self.base_price,
            'suggested_price': suggested_price,
            'price_factor': price_factor,
            'is_sold': is_sold,
            'days_to_sell': days_to_sell if is_sold else None,
            'sold_price': sold_price
        }
        
        return next_state, reward, done, info
    
    def _calculate_reward(self, suggested_price, is_sold, sold_price, days_to_sell):
        """
        Calculate reward for the action
        
        Args:
            suggested_price (float): Price suggested by agent
            is_sold (bool): Whether item was sold
            sold_price (float): Actual sold price
            days_to_sell (float): Days taken to sell
            
        Returns:
            float: Reward value
        """
        # For sold items, compare the suggested price to actual sold price
        if is_sold and sold_price is not None:
            # Penalize overpricing that could prevent sale
            if suggested_price > 1.2 * sold_price:
                return -2.0
            
            # Reward accurate pricing (within 5% of actual)
            if 0.95 * sold_price <= suggested_price <= 1.05 * sold_price:
                return 2.0
            
            # Good pricing (within 10% of actual)
            if 0.9 * sold_price <= suggested_price <= 1.1 * sold_price:
                return 1.5
            
            # Reasonable pricing (within 20% of actual)
            if 0.8 * sold_price <= suggested_price <= 1.2 * sold_price:
                return 0.8
            
            # Penalize significantly underpricing
            if suggested_price < 0.8 * sold_price:
                return -1.0
            
            # Default small reward for other cases
            return 0.2
        
        # For unsold items, evaluate based on market dynamics
        else:
            # Get similar items
            similar_sold = self._find_similar_items(is_sold=True)
            
            if len(similar_sold) > 0:
                avg_similar_price = similar_sold['price'].mean()
                
                # Reward pricing close to successful similar items
                if 0.9 * avg_similar_price <= suggested_price <= 1.1 * avg_similar_price:
                    return 1.2
                
                # Smaller reward for reasonable pricing
                if 0.8 * avg_similar_price <= suggested_price <= 1.2 * avg_similar_price:
                    return 0.6
                
                # Penalize significant deviation from market
                if suggested_price > 1.3 * avg_similar_price:
                    return -1.5
                
                if suggested_price < 0.7 * avg_similar_price:
                    return -0.8
                
                # Small negative reward for other cases
                return -0.3
        
        # Default small neutral reward if we can't determine better
        return 0.0
    
    def _find_similar_items(self, is_sold=True, top_n=5):
        """
        Find items similar to current item
        
        Args:
            is_sold (bool): Whether to find sold or unsold items
            top_n (int): Number of similar items to return
            
        Returns:
            pd.DataFrame: Similar items
        """
        # Current item
        item = self.features_df.iloc[self.current_item_idx]
        
        # Filter by sold status
        items_subset = self.sold_items if is_sold else self.unsold_items
        
        if len(items_subset) == 0:
            return pd.DataFrame()
        
        # Extract state features (excluding ID, price, is_sold, category_id, days_to_sell)
        feature_columns = [col for col in self.features_df.columns 
                          if col not in ['itemId', 'price', 'is_sold', 'category_id', 'days_to_sell']]
        
        if len(feature_columns) == 0 or len(items_subset) <= top_n:
            return items_subset.head(top_n)
        
        # Calculate similarity using Euclidean distance
        item_features = item[feature_columns].values
        
        def euclidean_distance(row):
            return np.sqrt(np.sum((row[feature_columns].values - item_features) ** 2))
        
        # Calculate distances and sort by similarity
        items_subset = items_subset.copy()
        items_subset['distance'] = items_subset.apply(euclidean_distance, axis=1)
        items_subset = items_subset.sort_values('distance')
        
        # Return top N most similar items
        return items_subset.head(top_n)


class PricePredictionTrainer:
    """
    Trainer class for price prediction model
    """
    def __init__(self, category_id: str, config_path: str = "/data/chats/p6wyr/workspace/config/config.json"):
        """
        Initialize the trainer
        
        Args:
            category_id (str): Category ID to train model for
            config_path (str): Path to configuration file
        """
        self.logger = Logger().get_logger()
        self.logger.info(f"Initializing price prediction trainer for category {category_id}")
        
        self.category_id = category_id
        self.config_path = config_path
        
        # Load configuration
        self._load_config()
        
        # Set paths
        self.features_dir = f"/data/chats/p6wyr/workspace/data/features/category_{category_id}"
        self.models_dir = f"/data/chats/p6wyr/workspace/data/models/category_{category_id}"
        
        os.makedirs(self.models_dir, exist_ok=True)
        
        # Load data
        self.features_df = self._load_features()
        
        # Set up environment and agent
        if self.features_df is not None:
            self._setup_env_and_agent()
        else:
            self.logger.error("Failed to load features data. Cannot set up environment and agent.")
            self.environment = None
            self.agent = None
    
    def _load_config(self) -> None:
        """
        Load configuration from file
        """
        try:
            with open(self.config_path, 'r') as f:
                self.config = json.load(f)
            self.logger.info("Configuration loaded successfully")
        except Exception as e:
            self.logger.error(f"Error loading configuration: {str(e)}")
            # Use default configuration
            self.config = {
                "model": {
                    "drl": {
                        "model_type": "dqn",
                        "learning_rate": 0.001,
                        "discount_factor": 0.95,
                        "exploration_rate": 0.2,
                        "exploration_decay_rate": 0.995,
                        "min_exploration_rate": 0.01,
                        "target_update_frequency": 100,
                        "batch_size": 64,
                        "memory_size": 10000,
                        "num_hidden_layers": 3,
                        "hidden_layer_size": 128,
                        "activation": "relu",
                        "double_dqn": True,
                        "dueling_network": True
                    },
                    "reward_function": {
                        "profit_weight": 0.7,
                        "time_weight": 0.3,
                    },
                    "training": {
                        "num_episodes": 3000,
                        "checkpoint_interval": 500,
                        "evaluation_interval": 100
                    }
                },
                "data": {
                    "min_items_for_training": 100
                }
            }
    
    def _load_features(self) -> Optional[pd.DataFrame]:
        """
        Load features data for the category
        
        Returns:
            Optional[pd.DataFrame]: Features DataFrame or None if not found
        """
        # Find the latest features file
        feature_files = sorted(glob.glob(os.path.join(self.features_dir, "features_*.csv")))
        
        if not feature_files:
            self.logger.error(f"No feature files found in {self.features_dir}")
            # Create synthetic data for demonstration if no real data exists
            return self._create_synthetic_data()
        
        # Load latest features file
        latest_features_file = feature_files[-1]
        self.logger.info(f"Loading features from {latest_features_file}")
        
        try:
            df = pd.read_csv(latest_features_file)
            self.logger.info(f"Loaded {len(df)} feature records")
            return df
        except Exception as e:
            self.logger.error(f"Error loading features: {str(e)}")
            # Create synthetic data as a fallback
            return self._create_synthetic_data()

    def _create_synthetic_data(self) -> pd.DataFrame:
        """
        Create synthetic data for demonstration purposes
        
        Returns:
            pd.DataFrame: Synthetic features data
        """
        self.logger.warning("Creating synthetic data for demonstration")
        
        # Number of synthetic items
        n_items = 1000
        
        # Create random feature data
        np.random.seed(42)  # For reproducibility
        
        # Get state features from config
        state_features = self.config.get("model", {}).get("state_features", [
            "item_condition", 
            "brand_popularity", 
            "season_demand", 
            "market_saturation",
            "avg_price", 
            "price_std", 
            "time_on_market",
            "conversion_rate"
        ])
        
        # Create DataFrame
        data = {
            'itemId': [f"synth_{i}" for i in range(n_items)],
            'price': np.random.uniform(50, 1000, n_items),
            'is_sold': np.random.choice([0, 1], n_items, p=[0.6, 0.4]),
            'category_id': self.category_id
        }
        
        # Add state features
        for feature in state_features:
            data[feature] = np.random.uniform(0, 1, n_items)
        
        # Add days_to_sell for sold items
        df = pd.DataFrame(data)
        sold_mask = df['is_sold'] == 1
        df.loc[sold_mask, 'days_to_sell'] = np.random.exponential(scale=10, size=sold_mask.sum()).clip(1, 60)
        
        # Log the creation of synthetic data
        self.logger.info(f"Created synthetic dataset with {n_items} items")
        
        return df
    
    def _setup_env_and_agent(self) -> None:
        """
        Set up environment and DRL agent
        """
        # Define number of price levels (actions)
        self.num_price_levels = 15  # More granular price options
        
        # Create environment
        self.environment = PriceEnvironment(
            features_df=self.features_df,
            category_id=self.category_id,
            num_price_levels=self.num_price_levels
        )
        
        # Get state size (number of features)
        state_columns = [col for col in self.features_df.columns 
                        if col not in ['itemId', 'price', 'is_sold', 'category_id', 'days_to_sell']]
        state_size = len(state_columns)
        
        # Create agent with configuration
        self.agent = PricePredictionDRLAgent(
            state_size=state_size,
            action_size=self.num_price_levels,
            category_id=self.category_id,
            config_path=self.config_path
        )
        
        self.logger.info(f"Set up environment and agent with state size {state_size} and {self.num_price_levels} actions")
    
    def train(self, num_episodes: Optional[int] = None, batch_size: Optional[int] = None, 
              checkpoint_interval: Optional[int] = None, evaluation_interval: Optional[int] = None) -> Dict:
        """
        Train the DRL agent
        
        Args:
            num_episodes (Optional[int]): Number of training episodes
            batch_size (Optional[int]): Batch size for learning
            checkpoint_interval (Optional[int]): Interval for saving checkpoints
            evaluation_interval (Optional[int]): Interval for evaluation
            
        Returns:
            Dict: Training statistics
        """
        if self.agent is None or self.environment is None:
            self.logger.error("Agent or environment not initialized. Cannot train.")
            return {"status": "failed", "reason": "initialization_failed"}
        
        # Get training configuration
        training_config = self.config.get("model", {}).get("training", {})
        drl_config = self.config.get("model", {}).get("drl", {})
        
        # Set parameters, using provided values or defaults from config
        num_episodes = num_episodes or training_config.get("num_episodes", 3000)
        batch_size = batch_size or drl_config.get("batch_size", 64)
        checkpoint_interval = checkpoint_interval or training_config.get("checkpoint_interval", 500)
        evaluation_interval = evaluation_interval or training_config.get("evaluation_interval", 100)
        
        # Get exploration decay parameters
        exploration_decay_rate = drl_config.get("exploration_decay_rate", 0.995)
        min_exploration_rate = drl_config.get("min_exploration_rate", 0.01)
        
        # Initialize statistics
        self.agent.stats["episodes"] = num_episodes
        self.agent.stats["avg_rewards"] = []
        self.agent.stats["avg_losses"] = []
        self.agent.stats["exploration_rate"] = [self.agent.exploration_rate]
        
        # Track best model performance
        best_eval_reward = float("-inf")
        best_model_path = None
        
        self.logger.info(f"Starting training for {num_episodes} episodes")
        
        # Main training loop
        for episode in tqdm(range(1, num_episodes + 1), desc="Training"):
            # Select a random item for training
            state = self.environment.reset()
            
            # Select action using epsilon-greedy policy
            action = self.agent.act(state)
            
            # Take action in the environment
            next_state, reward, done, info = self.environment.step(action)
            
            # Learn from experience
            loss = self.agent.step(state, action, reward, next_state, done)
            
            # Decay exploration rate
            if episode % 10 == 0:  # Decay every 10 episodes
                self.agent.decay_exploration_rate(
                    decay_factor=exploration_decay_rate,
                    min_rate=min_exploration_rate
                )
            
            # Log training progress to tensorboard
            if episode % 10 == 0:  # Log every 10 episodes
                self.agent.writer.add_scalar('Training/Reward', reward, episode)
                if loss > 0:
                    self.agent.writer.add_scalar('Training/Loss', loss, episode)
                self.agent.writer.add_scalar('Training/Exploration_Rate', 
                                            self.agent.exploration_rate, episode)
            
            # Store statistics
            if episode % 50 == 0:  # Update stats periodically
                self.agent.stats["avg_rewards"].append(self._evaluate_agent(10))
                self.agent.stats["avg_losses"].append(loss if loss > 0 else 0)
                self.agent.stats["exploration_rate"].append(self.agent.exploration_rate)
                
                # Print progress
                self.logger.info(
                    f"Episode {episode}/{num_episodes} | "
                    f"Avg Reward: {self.agent.stats['avg_rewards'][-1]:.3f} | "
                    f"Loss: {self.agent.stats['avg_losses'][-1]:.3f} | "
                    f"Exploration Rate: {self.agent.exploration_rate:.3f}"
                )
            
            # Save checkpoints
            if episode % checkpoint_interval == 0:
                checkpoint_path = self.agent.save_model(episode=episode)
                self.logger.info(f"Saved checkpoint at episode {episode} to {checkpoint_path}")
            
            # Evaluate agent and save best model
            if episode % evaluation_interval == 0:
                eval_reward = self._evaluate_agent(50)  # More thorough evaluation
                self.logger.info(f"Evaluation at episode {episode}: Average reward = {eval_reward:.3f}")
                
                # Log to tensorboard
                self.agent.writer.add_scalar('Evaluation/Average_Reward', eval_reward, episode)
                
                # Save best model
                if eval_reward > best_eval_reward:
                    best_eval_reward = eval_reward
                    best_model_path = self.agent.save_model(episode=episode)
                    self.logger.info(f"New best model at episode {episode} with reward {best_eval_reward:.3f}")
        
        # Final evaluation
        final_eval_reward = self._evaluate_agent(100)
        self.logger.info(f"Final evaluation: Average reward = {final_eval_reward:.3f}")
        
        # Save final model
        final_model_path = self.agent.save_model(episode=num_episodes)
        
        # Close tensorboard writer
        self.agent.writer.close()
        
        # Return training statistics
        return {
            "status": "success",
            "episodes_trained": num_episodes,
            "final_exploration_rate": self.agent.exploration_rate,
            "final_evaluation_reward": final_eval_reward,
            "best_evaluation_reward": best_eval_reward,
            "best_model_path": best_model_path,
            "final_model_path": final_model_path
        }
    
    def _evaluate_agent(self, num_episodes: int = 10) -> float:
        """
        Evaluate the agent's performance
        
        Args:
            num_episodes (int): Number of episodes to evaluate
            
        Returns:
            float: Average reward
        """
        rewards = []
        
        for _ in range(num_episodes):
            state = self.environment.reset()
            action = self.agent.act(state, evaluate=True)  # No exploration during evaluation
            _, reward, _, _ = self.environment.step(action)
            rewards.append(reward)
        
        return sum(rewards) / num_episodes if rewards else 0
    
    def evaluate_model(self, model_path: Optional[str] = None, num_episodes: int = 100) -> Dict:
        """
        Evaluate a trained model
        
        Args:
            model_path (Optional[str]): Path to model to evaluate (uses the latest if None)
            num_episodes (int): Number of episodes to evaluate
            
        Returns:
            Dict: Evaluation results
        """
        if self.agent is None or self.environment is None:
            self.logger.error("Agent or environment not initialized. Cannot evaluate.")
            return {"status": "failed", "reason": "initialization_failed"}
        
        # If no model path provided, find the latest model
        if model_path is None:
            model_files = sorted(glob.glob(os.path.join(self.models_dir, "drl_model_*.pt")))
            if not model_files:
                self.logger.error("No trained models found")
                return {"status": "failed", "reason": "no_models_found"}
            model_path = model_files[-1]
        
        # Load the model
        if not self.agent.load_model(model_path):
            return {"status": "failed", "reason": "model_loading_failed"}
        
        self.logger.info(f"Evaluating model from {model_path}")
        
        # Run evaluation
        rewards = []
        price_accuracies = []  # Track how close predicted prices are to actual prices
        actions_taken = []
        
        for episode in tqdm(range(num_episodes), desc="Evaluating"):
            state = self.environment.reset()
            action = self.agent.act(state, evaluate=True)
            _, reward, _, info = self.environment.step(action)
            
            rewards.append(reward)
            actions_taken.append(action)
            
            # Calculate price accuracy for sold items
            if info.get('is_sold') and info.get('sold_price'):
                accuracy = abs(info['suggested_price'] - info['sold_price']) / info['sold_price']
                price_accuracies.append(1 - accuracy)  # Convert to accuracy: 1 = perfect, 0 = completely off
        
        # Calculate statistics
        avg_reward = sum(rewards) / len(rewards) if rewards else 0
        avg_price_accuracy = sum(price_accuracies) / len(price_accuracies) if price_accuracies else 0
        action_distribution = {action: actions_taken.count(action) / len(actions_taken) 
                              for action in set(actions_taken)}
        
        # Create evaluation results
        results = {
            "status": "success",
            "model_path": model_path,
            "episodes_evaluated": num_episodes,
            "average_reward": avg_reward,
            "price_accuracy": avg_price_accuracy,
            "action_distribution": action_distribution
        }
        
        self.logger.info(f"Evaluation results: Avg reward = {avg_reward:.3f}, Price accuracy = {avg_price_accuracy:.3f}")
        
        return results
    
    def predict_price(self, features: Dict) -> Dict:
        """
        Predict the optimal price for an item
        
        Args:
            features (Dict): Item features
            
        Returns:
            Dict: Prediction results
        """
        if self.agent is None or self.environment is None:
            self.logger.error("Agent or environment not initialized. Cannot predict.")
            return {"status": "failed", "reason": "initialization_failed"}
        
        # Find the latest model if no model is loaded
        if not hasattr(self.agent, 'qnetwork_local') or self.agent.qnetwork_local is None:
            model_files = sorted(glob.glob(os.path.join(self.models_dir, "drl_model_*.pt")))
            if not model_files:
                self.logger.error("No trained models found")
                return {"status": "failed", "reason": "no_models_found"}
            
            # Load the latest model
            if not self.agent.load_model(model_files[-1]):
                return {"status": "failed", "reason": "model_loading_failed"}
        
        # Format features as a state vector
        try:
            # Get relevant features
            state_columns = [col for col in self.features_df.columns 
                           if col not in ['itemId', 'price', 'is_sold', 'category_id', 'days_to_sell']]
            
            # Create state array
            state = np.zeros(len(state_columns), dtype=np.float32)
            for i, col in enumerate(state_columns):
                state[i] = features.get(col, 0.0)
            
            # Get base price
            base_price = features.get('price', self.environment.avg_price)
            
            # Predict action
            action = self.agent.act(state, evaluate=True)
            price_factor = self.environment.price_levels[action]
            suggested_price = base_price * price_factor
            
            # Calculate price range
            confidence = 0.9  # 90% confidence
            lower_bound = suggested_price * (1 - 0.05)
            upper_bound = suggested_price * (1 + 0.05)
            
            return {
                "status": "success",
                "base_price": base_price,
                "suggested_price": suggested_price,
                "price_factor": price_factor,
                "price_range": {
                    "lower": lower_bound,
                    "upper": upper_bound,
                    "confidence": confidence
                },
                "action": int(action)
            }
            
        except Exception as e:
            self.logger.error(f"Error predicting price: {str(e)}")
            return {"status": "failed", "reason": str(e)}
    
    def plot_training_progress(self, save_path: Optional[str] = None):
        """
        Plot training progress metrics
        
        Args:
            save_path (Optional[str]): Path to save the plot
        """
        if not self.agent or not self.agent.stats or not self.agent.stats["avg_rewards"]:
            self.logger.error("No training statistics available")
            return
        
        # Create figure with 3 subplots
        fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10, 15), sharex=True)
        
        # Plot average rewards
        episodes = list(range(0, len(self.agent.stats["avg_rewards"]) * 50, 50))
        ax1.plot(episodes, self.agent.stats["avg_rewards"])
        ax1.set_title("Average Reward")
        ax1.set_ylabel("Reward")
        ax1.grid(True)
        
        # Plot average losses
        if self.agent.stats.get("avg_losses"):
            ax2.plot(episodes, self.agent.stats["avg_losses"])
            ax2.set_title("Average Loss")
            ax2.set_ylabel("Loss")
            ax2.grid(True)
        
        # Plot exploration rate
        exploration_rate_episodes = list(range(0, len(self.agent.stats["exploration_rate"]) * 50, 50))
        ax3.plot(exploration_rate_episodes, self.agent.stats["exploration_rate"])
        ax3.set_title("Exploration Rate")
        ax3.set_xlabel("Episode")
        ax3.set_ylabel("Rate")
        ax3.grid(True)
        
        plt.tight_layout()
        
        # Save or show plot
        if save_path:
            plt.savefig(save_path)
            self.logger.info(f"Training progress plot saved to {save_path}")
        else:
            plt.show()


# Example usage function (for demonstration)
async def train_model_for_category(category_id="9355", num_episodes=1000):
    """
    Train a price prediction model for a specified category
    
    Args:
        category_id (str): Category ID to train for
        num_episodes (int): Number of training episodes
    
    Returns:
        Dict: Training results
    """
    logger = Logger().get_logger()
    logger.info(f"Starting model training for category {category_id}")
    
    # Create trainer
    trainer = PricePredictionTrainer(category_id=category_id)
    
    # Train the model
    training_results = trainer.train(num_episodes=num_episodes)
    
    # Evaluate the model
    if training_results.get("best_model_path"):
        evaluation_results = trainer.evaluate_model(model_path=training_results["best_model_path"])
    else:
        evaluation_results = trainer.evaluate_model()
    
    # Plot training progress
    plot_path = f"/data/chats/p6

SyntaxError: unterminated string literal (detected at line 1) (2015793525.py, line 1)

In [26]:
[reflection on previous impl]
The previous implementation for PriceEnvironment and PricePredictionTrainer is mostly correct in functionality, but it contains a syntax error. The error message indicates an unterminated string literal at line 1. This is likely due to a truncation or formatting issue with the code when it was sent to the execution environment. The error specifically occurs in the example usage function where the path string is cut off: `plot_path = f"/data/chats/p6"` is incomplete.

Additionally, the code imports the PricePredictionDRLAgent class but doesn't check if it's already defined or imported. Since the PricePredictionDRLAgent class was implemented in a previous step, we should avoid redefining it.

I need to fix these issues by:
1. Properly terminating all string literals
2. Making sure all paths are complete
3. Ensuring proper imports for the PricePredictionDRLAgent class
4. Checking that all brackets and quotes are properly paired
5. Keeping the training function but ensuring it doesn't have syntax errors

[improved impl]:
```python
import os
import json
import numpy as np
import pandas as pd
import torch
from datetime import datetime
from typing import Dict, List, Tuple, Optional, Any
import random
import glob
import matplotlib.pyplot as plt
from tqdm import tqdm

from src.utils.logger import Logger

# Importing previously defined PricePredictionDRLAgent class
# We assume it's already defined in a previous code execution

class PriceEnvironment:
    """
    Environment for price prediction, simulating market dynamics
    """
    def __init__(self, features_df: pd.DataFrame, category_id: str, num_price_levels: int = 10):
        """
        Initialize the environment
        
        Args:
            features_df (pd.DataFrame): DataFrame with item features
            category_id (str): Category ID
            num_price_levels (int): Number of discrete price levels
        """
        self.logger = Logger().get_logger()
        self.features_df = features_df.copy()
        self.category_id = category_id
        
        # Get sold items for statistics
        self.sold_items = self.features_df[self.features_df['is_sold'] == 1]
        self.unsold_items = self.features_df[self.features_df['is_sold'] == 0]
        
        # Extract price range
        self.min_price = self.features_df['price'].min()
        self.max_price = self.features_df['price'].max()
        self.avg_price = self.features_df['price'].mean()
        
        # Calculate price levels (actions)
        self.num_price_levels = num_price_levels
        self.price_levels = np.linspace(0.5, 1.5, num_price_levels)  # 0.5x to 1.5x of item's base price
        
        # Current state info
        self.current_item_idx = 0
        self.current_state = None
        self.base_price = 0
        
        self.logger.info(f"Initialized price environment with {len(features_df)} items, {num_price_levels} price levels")
    
    def reset(self, item_idx=None):
        """
        Reset the environment and get new item
        
        Args:
            item_idx (int): Index of item to use (None for random)
            
        Returns:
            np.array: Initial state
        """
        # Select item (random if not specified)
        if item_idx is None:
            self.current_item_idx = np.random.randint(0, len(self.features_df))
        else:
            self.current_item_idx = item_idx % len(self.features_df)
        
        # Get item data
        item = self.features_df.iloc[self.current_item_idx]
        
        # Extract state features (excluding ID, price, is_sold, category_id, days_to_sell)
        state_columns = [col for col in self.features_df.columns 
                         if col not in ['itemId', 'price', 'is_sold', 'category_id', 'days_to_sell']]
        
        # Set current state
        self.current_state = item[state_columns].values.astype(np.float32)
        
        # Set base price
        self.base_price = item['price']
        
        return self.current_state
    
    def step(self, action):
        """
        Take a step in the environment
        
        Args:
            action (int): Action index (price level)
            
        Returns:
            tuple: (next_state, reward, done, info)
        """
        # Get selected price level
        price_factor = self.price_levels[action]
        suggested_price = self.base_price * price_factor
        
        # Get current item
        item = self.features_df.iloc[self.current_item_idx]
        
        # Check if item sold in historical data
        is_sold = item['is_sold'] == 1
        days_to_sell = item.get('days_to_sell', 30)  # Default to 30 days if unknown
        
        # Get actual sold price if available
        sold_price = item['price'] if is_sold else None
        
        # Calculate reward
        reward = self._calculate_reward(suggested_price, is_sold, sold_price, days_to_sell)
        
        # Return the same state (as this is not a sequential environment)
        next_state = self.current_state
        
        # Episode is done after one step (predict price only once per item)
        done = True
        
        # Additional info
        info = {
            'item_id': item.get('itemId', self.current_item_idx),
            'base_price': self.base_price,
            'suggested_price': suggested_price,
            'price_factor': price_factor,
            'is_sold': is_sold,
            'days_to_sell': days_to_sell if is_sold else None,
            'sold_price': sold_price
        }
        
        return next_state, reward, done, info
    
    def _calculate_reward(self, suggested_price, is_sold, sold_price, days_to_sell):
        """
        Calculate reward for the action
        
        Args:
            suggested_price (float): Price suggested by agent
            is_sold (bool): Whether item was sold
            sold_price (float): Actual sold price
            days_to_sell (float): Days taken to sell
            
        Returns:
            float: Reward value
        """
        # For sold items, compare the suggested price to actual sold price
        if is_sold and sold_price is not None:
            # Penalize overpricing that could prevent sale
            if suggested_price > 1.2 * sold_price:
                return -2.0
            
            # Reward accurate pricing (within 5% of actual)
            if 0.95 * sold_price <= suggested_price <= 1.05 * sold_price:
                return 2.0
            
            # Good pricing (within 10% of actual)
            if 0.9 * sold_price <= suggested_price <= 1.1 * sold_price:
                return 1.5
            
            # Reasonable pricing (within 20% of actual)
            if 0.8 * sold_price <= suggested_price <= 1.2 * sold_price:
                return 0.8
            
            # Penalize significantly underpricing
            if suggested_price < 0.8 * sold_price:
                return -1.0
            
            # Default small reward for other cases
            return 0.2
        
        # For unsold items, evaluate based on market dynamics
        else:
            # Get similar items
            similar_sold = self._find_similar_items(is_sold=True)
            
            if len(similar_sold) > 0:
                avg_similar_price = similar_sold['price'].mean()
                
                # Reward pricing close to successful similar items
                if 0.9 * avg_similar_price <= suggested_price <= 1.1 * avg_similar_price:
                    return 1.2
                
                # Smaller reward for reasonable pricing
                if 0.8 * avg_similar_price <= suggested_price <= 1.2 * avg_similar_price:
                    return 0.6
                
                # Penalize significant deviation from market
                if suggested_price > 1.3 * avg_similar_price:
                    return -1.5
                
                if suggested_price < 0.7 * avg_similar_price:
                    return -0.8
                
                # Small negative reward for other cases
                return -0.3
        
        # Default small neutral reward if we can't determine better
        return 0.0
    
    def _find_similar_items(self, is_sold=True, top_n=5):
        """
        Find items similar to current item
        
        Args:
            is_sold (bool): Whether to find sold or unsold items
            top_n (int): Number of similar items to return
            
        Returns:
            pd.DataFrame: Similar items
        """
        # Current item
        item = self.features_df.iloc[self.current_item_idx]
        
        # Filter by sold status
        items_subset = self.sold_items if is_sold else self.unsold_items
        
        if len(items_subset) == 0:
            return pd.DataFrame()
        
        # Extract state features (excluding ID, price, is_sold, category_id, days_to_sell)
        feature_columns = [col for col in self.features_df.columns 
                          if col not in ['itemId', 'price', 'is_sold', 'category_id', 'days_to_sell']]
        
        if len(feature_columns) == 0 or len(items_subset) <= top_n:
            return items_subset.head(top_n)
        
        # Calculate similarity using Euclidean distance
        item_features = item[feature_columns].values
        
        def euclidean_distance(row):
            return np.sqrt(np.sum((row[feature_columns].values - item_features) ** 2))
        
        # Calculate distances and sort by similarity
        items_subset = items_subset.copy()
        items_subset['distance'] = items_subset.apply(euclidean_distance, axis=1)
        items_subset = items_subset.sort_values('distance')
        
        # Return top N most similar items
        return items_subset.head(top_n)


class PricePredictionTrainer:
    """
    Trainer class for price prediction model
    """
    def __init__(self, category_id: str, config_path: str = "/data/chats/p6wyr/workspace/config/config.json"):
        """
        Initialize the trainer
        
        Args:
            category_id (str): Category ID to train model for
            config_path (str): Path to configuration file
        """
        self.logger = Logger().get_logger()
        self.logger.info(f"Initializing price prediction trainer for category {category_id}")
        
        self.category_id = category_id
        self.config_path = config_path
        
        # Load configuration
        self._load_config()
        
        # Set paths
        self.features_dir = f"/data/chats/p6wyr/workspace/data/features/category_{category_id}"
        self.models_dir = f"/data/chats/p6wyr/workspace/data/models/category_{category_id}"
        
        os.makedirs(self.models_dir, exist_ok=True)
        
        # Load data
        self.features_df = self._load_features()
        
        # Set up environment and agent
        if self.features_df is not None:
            self._setup_env_and_agent()
        else:
            self.logger.error("Failed to load features data. Cannot set up environment and agent.")
            self.environment = None
            self.agent = None
    
    def _load_config(self) -> None:
        """
        Load configuration from file
        """
        try:
            with open(self.config_path, 'r') as f:
                self.config = json.load(f)
            self.logger.info("Configuration loaded successfully")
        except Exception as e:
            self.logger.error(f"Error loading configuration: {str(e)}")
            # Use default configuration
            self.config = {
                "model": {
                    "drl": {
                        "model_type": "dqn",
                        "learning_rate": 0.001,
                        "discount_factor": 0.95,
                        "exploration_rate": 0.2,
                        "exploration_decay_rate": 0.995,
                        "min_exploration_rate": 0.01,
                        "target_update_frequency": 100,
                        "batch_size": 64,
                        "memory_size": 10000,
                        "num_hidden_layers": 3,
                        "hidden_layer_size": 128,
                        "activation": "relu",
                        "double_dqn": True,
                        "dueling_network": True
                    },
                    "reward_function": {
                        "profit_weight": 0.7,
                        "time_weight": 0.3
                    },
                    "training": {
                        "num_episodes": 3000,
                        "checkpoint_interval": 500,
                        "evaluation_interval": 100
                    }
                },
                "data": {
                    "min_items_for_training": 100
                }
            }
    
    def _load_features(self) -> Optional[pd.DataFrame]:
        """
        Load features data for the category
        
        Returns:
            Optional[pd.DataFrame]: Features DataFrame or None if not found
        """
        # Find the latest features file
        feature_files = sorted(glob.glob(os.path.join(self.features_dir, "features_*.csv")))
        
        if not feature_files:
            self.logger.error(f"No feature files found in {self.features_dir}")
            # Create synthetic data for demonstration if no real data exists
            return self._create_synthetic_data()
        
        # Load latest features file
        latest_features_file = feature_files[-1]
        self.logger.info(f"Loading features from {latest_features_file}")
        
        try:
            df = pd.read_csv(latest_features_file)
            self.logger.info(f"Loaded {len(df)} feature records")
            return df
        except Exception as e:
            self.logger.error(f"Error loading features: {str(e)}")
            # Create synthetic data as a fallback
            return self._create_synthetic_data()

    def _create_synthetic_data(self) -> pd.DataFrame:
        """
        Create synthetic data for demonstration purposes
        
        Returns:
            pd.DataFrame: Synthetic features data
        """
        self.logger.warning("Creating synthetic data for demonstration")
        
        # Number of synthetic items
        n_items = 1000
        
        # Create random feature data
        np.random.seed(42)  # For reproducibility
        
        # Get state features from config
        state_features = self.config.get("model", {}).get("state_features", [
            "item_condition", 
            "brand_popularity", 
            "season_demand", 
            "market_saturation",
            "avg_price", 
            "price_std", 
            "time_on_market",
            "conversion_rate"
        ])
        
        # Create DataFrame
        data = {
            'itemId': [f"synth_{i}" for i in range(n_items)],
            'price': np.random.uniform(50, 1000, n_items),
            'is_sold': np.random.choice([0, 1], n_items, p=[0.6, 0.4]),
            'category_id': self.category_id
        }
        
        # Add state features
        for feature in state_features:
            data[feature] = np.random.uniform(0, 1, n_items)
        
        # Add days_to_sell for sold items
        df = pd.DataFrame(data)
        sold_mask = df['is_sold'] == 1
        df.loc[sold_mask, 'days_to_sell'] = np.random.exponential(scale=10, size=sold_mask.sum()).clip(1, 60)
        
        # Log the creation of synthetic data
        self.logger.info(f"Created synthetic dataset with {n_items} items")
        
        return df
    
    def _setup_env_and_agent(self) -> None:
        """
        Set up environment and DRL agent
        """
        # Define number of price levels (actions)
        self.num_price_levels = 15  # More granular price options
        
        # Create environment
        self.environment = PriceEnvironment(
            features_df=self.features_df,
            category_id=self.category_id,
            num_price_levels=self.num_price_levels
        )
        
        # Get state size (number of features)
        state_columns = [col for col in self.features_df.columns 
                        if col not in ['itemId', 'price', 'is_sold', 'category_id', 'days_to_sell']]
        state_size = len(state_columns)
        
        # Create agent with configuration
        # Assuming PricePredictionDRLAgent was defined in a previous step
        self.agent = PricePredictionDRLAgent(
            state_size=state_size,
            action_size=self.num_price_levels,
            category_id=self.category_id,
            config_path=self.config_path
        )
        
        self.logger.info(f"Set up environment and agent with state size {state_size} and {self.num_price_levels} actions")
    
    def train(self, num_episodes: Optional[int] = None, batch_size: Optional[int] = None, 
              checkpoint_interval: Optional[int] = None, evaluation_interval: Optional[int] = None) -> Dict:
        """
        Train the DRL agent
        
        Args:
            num_episodes (Optional[int]): Number of training episodes
            batch_size (Optional[int]): Batch size for learning
            checkpoint_interval (Optional[int]): Interval for saving checkpoints
            evaluation_interval (Optional[int]): Interval for evaluation
            
        Returns:
            Dict: Training statistics
        """
        if self.agent is None or self.environment is None:
            self.logger.error("Agent or environment not initialized. Cannot train.")
            return {"status": "failed", "reason": "initialization_failed"}
        
        # Get training configuration
        training_config = self.config.get("model", {}).get("training", {})
        drl_config = self.config.get("model", {}).get("drl", {})
        
        # Set parameters, using provided values or defaults from config
        num_episodes = num_episodes or training_config.get("num_episodes", 3000)
        batch_size = batch_size or drl_config.get("batch_size", 64)
        checkpoint_interval = checkpoint_interval or training_config.get("checkpoint_interval", 500)
        evaluation_interval = evaluation_interval or training_config.get("evaluation_interval", 100)
        
        # Get exploration decay parameters
        exploration_decay_rate = drl_config.get("exploration_decay_rate", 0.995)
        min_exploration_rate = drl_config.get("min_exploration_rate", 0.01)
        
        # Initialize statistics
        self.agent.stats["episodes"] = num_episodes
        self.agent.stats["avg_rewards"] = []
        self.agent.stats["avg_losses"] = []
        self.agent.stats["exploration_rate"] = [self.agent.exploration_rate]
        
        # Track best model performance
        best_eval_reward = float("-inf")
        best_model_path = None
        
        self.logger.info(f"Starting training for {num_episodes} episodes")
        
        # Main training loop
        for episode in tqdm(range(1, num_episodes + 1), desc="Training"):
            # Select a random item for training
            state = self.environment.reset()
            
            # Select action using epsilon-greedy policy
            action = self.agent.act(state)
            
            # Take action in the environment
            next_state, reward, done, info = self.environment.step(action)
            
            # Learn from experience
            loss = self.agent.step(state, action, reward, next_state, done)
            
            # Decay exploration rate
            if episode % 10 == 0:  # Decay every 10 episodes
                self.agent.decay_exploration_rate(
                    decay_factor=exploration_decay_rate,
                    min_rate=min_exploration_rate
                )
            
            # Log training progress to tensorboard
            if episode % 10 == 0:  # Log every 10 episodes
                self.agent.writer.add_scalar('Training/Reward', reward, episode)
                if loss > 0:
                    self.agent.writer.add_scalar('Training/Loss', loss, episode)
                self.agent.writer.add_scalar('Training/Exploration_Rate', 
                                            self.agent.exploration_rate, episode)
            
            # Store statistics
            if episode % 50 == 0:  # Update stats periodically
                self.agent.stats["avg_rewards"].append(self._evaluate_agent(10))
                self.agent.stats["avg_losses"].append(loss if loss > 0 else 0)
                self.agent.stats["exploration_rate"].append(self.agent.exploration_rate)
                
                # Print progress
                self.logger.info(
                    f"Episode {episode}/{num_episodes} | "
                    f"Avg Reward: {self.agent.stats['avg_rewards'][-1]:.3f} | "
                    f"Loss: {self.agent.stats['avg_losses'][-1]:.3f} | "
                    f"Exploration Rate: {self.agent.exploration_rate:.3f}"
                )
            
            # Save checkpoints
            if episode % checkpoint_interval == 0:
                checkpoint_path = self.agent.save_model(episode=episode)
                self.logger.info(f"Saved checkpoint at episode {episode} to {checkpoint_path}")
            
            # Evaluate agent and save best model
            if episode % evaluation_interval == 0:
                eval_reward = self._evaluate_agent(50)  # More thorough evaluation
                self.logger.info(f"Evaluation at episode {episode}: Average reward = {eval_reward:.3f}")
                
                # Log to tensorboard
                self.agent.writer.add_scalar('Evaluation/Average_Reward', eval_reward, episode)
                
                # Save best model
                if eval_reward > best_eval_reward:
                    best_eval_reward = eval_reward
                    best_model_path = self.agent.save_model(episode=episode)
                    self.logger.info(f"New best model at episode {episode} with reward {best_eval_reward:.3f}")
        
        # Final evaluation
        final_eval_reward = self._evaluate_agent(100)
        self.logger.info(f"Final evaluation: Average reward = {final_eval_reward:.3f}")
        
        # Save final model
        final_model_path = self.agent.save_model(episode=num_episodes)
        
        # Close tensorboard writer
        self.agent.writer.close()
        
        # Return training statistics
        return {
            "status": "success",
            "episodes_trained": num_episodes,
            "final_exploration_rate": self.agent.exploration_rate,
            "final_evaluation_reward": final_eval_reward,
            "best_evaluation_reward": best_eval_reward,
            "best_model_path": best_model_path,
            "final_model_path": final_model_path
        }
    
    def _evaluate_agent(self, num_episodes: int = 10) -> float:
        """
        Evaluate the agent's performance
        
        Args:
            num_episodes (int): Number of episodes to evaluate
            
        Returns:
            float: Average reward
        """
        rewards = []
        
        for _ in range(num_episodes):
            state = self.environment.reset()
            action = self.agent.act(state, evaluate=True)  # No exploration during evaluation
            _, reward, _, _ = self.environment.step(action)
            rewards.append(reward)
        
        return sum(rewards) / num_episodes if rewards else 0
    
    def evaluate_model(self, model_path: Optional[str] = None, num_episodes: int = 100) -> Dict:
        """
        Evaluate a trained model
        
        Args:
            model_path (Optional[str]): Path to model to evaluate (uses the latest if None)
            num_episodes (int): Number of episodes to evaluate
            
        Returns:
            Dict: Evaluation results
        """
        if self.agent is None or self.environment is None:
            self.logger.error("Agent or environment not initialized. Cannot evaluate.")
            return {"status": "failed", "reason": "initialization_failed"}
        
        # If no model path provided, find the latest model
        if model_path is None:
            model_files = sorted(glob.glob(os.path.join(self.models_dir, "drl_model_*.pt")))
            if not model_files:
                self.logger.error("No trained models found")
                return {"status": "failed", "reason": "no_models_found"}
            model_path = model_files[-1]
        
        # Load the model
        if not self.agent.load_model(model_path):
            return {"status": "failed", "reason": "model_loading_failed"}
        
        self.logger.info(f"Evaluating model from {model_path}")
        
        # Run evaluation
        rewards = []
        price_accuracies = []  # Track how close predicted prices are to actual prices
        actions_taken = []
        
        for episode in tqdm(range(num_episodes), desc="Evaluating"):
            state = self.environment.reset()
            action = self.agent.act(state, evaluate=True)
            _, reward, _, info = self.environment.step(action)
            
            rewards.append(reward)
            actions_taken.append(action)
            
            # Calculate price accuracy for sold items
            if info.get('is_sold') and info.get('sold_price'):
                accuracy = abs(info['suggested_price'] - info['sold_price']) / info['sold_price']
                price_accuracies.append(1 - accuracy)  # Convert to accuracy: 1 = perfect, 0 = completely off
        
        # Calculate statistics
        avg_reward = sum(rewards) / len(rewards) if rewards else 0
        avg_price_accuracy = sum(price_accuracies) / len(price_accuracies) if price_accuracies else 0
        action_distribution = {action: actions_taken.count(action) / len(actions_taken) 
                              for action in set(actions_taken)}
        
        # Create evaluation results
        results = {
            "status": "success",
            "model_path": model_path,
            "episodes_evaluated": num_episodes,
            "average_reward": avg_reward,
            "price_accuracy": avg_price_accuracy,
            "action_distribution": action_distribution
        }
        
        self.logger.info(f"Evaluation results: Avg reward = {avg_reward:.3f}, Price accuracy = {avg_price_accuracy:.3f}")
        
        return results
    
    def predict_price(self, features: Dict) -> Dict:
        """
        Predict the optimal price for an item
        
        Args:
            features (Dict): Item features
            
        Returns:
            Dict: Prediction results
        """
        if self.agent is None or self.environment is None:
            self.logger.error("Agent or environment not initialized. Cannot predict.")
            return {"status": "failed", "reason": "initialization_failed"}
        
        # Find the latest model if no model is loaded
        if not hasattr(self.agent, 'qnetwork_local') or self.agent.qnetwork_local is None:
            model_files = sorted(glob.glob(os.path.join(self.models_dir, "drl_model_*.pt")))
            if not model_files:
                self.logger.error("No trained models found")
                return {"status": "failed", "reason": "no_models_found"}
            
            # Load the latest model
            if not self.agent.load_model(model_files[-1]):
                return {"status": "failed", "reason": "model_loading_failed"}
        
        # Format features as a state vector
        try:
            # Get relevant features
            state_columns = [col for col in self.features_df.columns 
                           if col not in ['itemId', 'price', 'is_sold', 'category_id', 'days_to_sell']]
            
            # Create state array
            state = np.zeros(len(state_columns), dtype=np.float32)
            for i, col in enumerate(state_columns):
                state[i] = features.get(col, 0.0)
            
            # Get base price
            base_price = features.get('price', self.environment.avg_price)
            
            # Predict action
            action = self.agent.act(state, evaluate=True)
            price_factor = self.environment.price_levels[action]
            suggested_price = base_price * price_factor
            
            # Calculate price range
            confidence = 0.9  # 90% confidence
            lower_bound = suggested_price * (1 - 0.05)
            upper_bound = suggested_price * (1 + 0.05)
            
            return {
                "status": "success",
                "base_price": base_price,
                "suggested_price": suggested_price,
                "price_factor": price_factor,
                "price_range": {
                    "lower": lower_bound,
                    "upper": upper_bound,
                    "confidence": confidence
                },
                "action": int(action)
            }
            
        except Exception as e:
            self.logger.error(f"Error predicting price: {str(e)}")
            return {"status": "failed", "reason": str(e)}
    
    def plot_training_progress(self, save_path: Optional[str] = None):
        """
        Plot training progress metrics
        
        Args:
            save_path (Optional[str]): Path to save the plot
        """
        if not self.agent or not self.agent.stats or not self.agent.stats["avg_rewards"]:
            self.logger.error("No training statistics available")
            return
        
        # Create figure with 3 subplots
        fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10, 15), sharex=True)
        
        # Plot average rewards
        episodes = list(range(0, len(self.agent.stats["avg_rewards"]) * 50, 50))
        ax1.plot(episodes, self.agent.stats["avg_rewards"])
        ax1.set_title("Average Reward")
        ax1.set_ylabel("Reward")
        ax1.grid(True)
        
        # Plot average losses
        if self.agent.stats.get("avg_losses"):
            ax2.plot(episodes, self.agent.stats["avg_losses"])
            ax2.set_title("Average Loss")
            ax2.set_ylabel("Loss")
            ax2.grid(True)
        
        # Plot exploration rate
        exploration_rate_episodes = list(range(0, len(self.agent.stats["exploration_rate"]) * 50, 50))
        ax3.plot(exploration_rate_episodes, self.agent.stats["exploration_rate"])
        ax3.set_title("Exploration Rate")
        ax3.set_xlabel("Episode")
        ax3.set_ylabel("Rate")
        ax3.grid(True)
        
        plt.tight_layout()
        
        # Save or show plot
        if save_path:
            plt.savefig(save_path)
            self.logger.info(f"Training progress plot saved to {save_path}")
        else:
            plt.show()

# Example usage function (fixed the path issue)
async def train_model_for_category(category_id="9355", num_episodes=

SyntaxError: unterminated string literal (detected at line 11) (633384038.py, line 11)

In [27]:
import os
import json
import numpy as np
import pandas as pd
import torch
from datetime import datetime
from typing import Dict, List, Tuple, Optional, Any
import random
import glob
import matplotlib.pyplot as plt
from tqdm import tqdm

from src.utils.logger import Logger

class PriceEnvironment:
    """
    Environment for price prediction, simulating market dynamics
    """
    def __init__(self, features_df: pd.DataFrame, category_id: str, num_price_levels: int = 10):
        """
        Initialize the environment
        
        Args:
            features_df (pd.DataFrame): DataFrame with item features
            category_id (str): Category ID
            num_price_levels (int): Number of discrete price levels
        """
        self.logger = Logger().get_logger()
        self.features_df = features_df.copy()
        self.category_id = category_id
        
        # Get sold items for statistics
        self.sold_items = self.features_df[self.features_df['is_sold'] == 1]
        self.unsold_items = self.features_df[self.features_df['is_sold'] == 0]
        
        # Extract price range
        self.min_price = self.features_df['price'].min()
        self.max_price = self.features_df['price'].max()
        self.avg_price = self.features_df['price'].mean()
        
        # Calculate price levels (actions)
        self.num_price_levels = num_price_levels
        self.price_levels = np.linspace(0.5, 1.5, num_price_levels)  # 0.5x to 1.5x of item's base price
        
        # Current state info
        self.current_item_idx = 0
        self.current_state = None
        self.base_price = 0
        
        self.logger.info(f"Initialized price environment with {len(features_df)} items, {num_price_levels} price levels")
    
    def reset(self, item_idx=None):
        """
        Reset the environment and get new item
        
        Args:
            item_idx (int): Index of item to use (None for random)
            
        Returns:
            np.array: Initial state
        """
        # Select item (random if not specified)
        if item_idx is None:
            self.current_item_idx = np.random.randint(0, len(self.features_df))
        else:
            self.current_item_idx = item_idx % len(self.features_df)
        
        # Get item data
        item = self.features_df.iloc[self.current_item_idx]
        
        # Extract state features (excluding ID, price, is_sold, category_id, days_to_sell)
        state_columns = [col for col in self.features_df.columns 
                         if col not in ['itemId', 'price', 'is_sold', 'category_id', 'days_to_sell']]
        
        # Set current state
        self.current_state = item[state_columns].values.astype(np.float32)
        
        # Set base price
        self.base_price = item['price']
        
        return self.current_state
    
    def step(self, action):
        """
        Take a step in the environment
        
        Args:
            action (int): Action index (price level)
            
        Returns:
            tuple: (next_state, reward, done, info)
        """
        # Get selected price level
        price_factor = self.price_levels[action]
        suggested_price = self.base_price * price_factor
        
        # Get current item
        item = self.features_df.iloc[self.current_item_idx]
        
        # Check if item sold in historical data
        is_sold = item['is_sold'] == 1
        days_to_sell = item.get('days_to_sell', 30)  # Default to 30 days if unknown
        
        # Get actual sold price if available
        sold_price = item['price'] if is_sold else None
        
        # Calculate reward
        reward = self._calculate_reward(suggested_price, is_sold, sold_price, days_to_sell)
        
        # Return the same state (as this is not a sequential environment)
        next_state = self.current_state
        
        # Episode is done after one step (predict price only once per item)
        done = True
        
        # Additional info
        info = {
            'item_id': item.get('itemId', self.current_item_idx),
            'base_price': self.base_price,
            'suggested_price': suggested_price,
            'price_factor': price_factor,
            'is_sold': is_sold,
            'days_to_sell': days_to_sell if is_sold else None,
            'sold_price': sold_price
        }
        
        return next_state, reward, done, info
    
    def _calculate_reward(self, suggested_price, is_sold, sold_price, days_to_sell):
        """
        Calculate reward for the action
        
        Args:
            suggested_price (float): Price suggested by agent
            is_sold (bool): Whether item was sold
            sold_price (float): Actual sold price
            days_to_sell (float): Days taken to sell
            
        Returns:
            float: Reward value
        """
        # For sold items, compare the suggested price to actual sold price
        if is_sold and sold_price is not None:
            # Penalize overpricing that could prevent sale
            if suggested_price > 1.2 * sold_price:
                return -2.0
            
            # Reward accurate pricing (within 5% of actual)
            if 0.95 * sold_price <= suggested_price <= 1.05 * sold_price:
                return 2.0
            
            # Good pricing (within 10% of actual)
            if 0.9 * sold_price <= suggested_price <= 1.1 * sold_price:
                return 1.5
            
            # Reasonable pricing (within 20% of actual)
            if 0.8 * sold_price <= suggested_price <= 1.2 * sold_price:
                return 0.8
            
            # Penalize significantly underpricing
            if suggested_price < 0.8 * sold_price:
                return -1.0
            
            # Default small reward for other cases
            return 0.2
        
        # For unsold items, evaluate based on market dynamics
        else:
            # Get similar items
            similar_sold = self._find_similar_items(is_sold=True)
            
            if len(similar_sold) > 0:
                avg_similar_price = similar_sold['price'].mean()
                
                # Reward pricing close to successful similar items
                if 0.9 * avg_similar_price <= suggested_price <= 1.1 * avg_similar_price:
                    return 1.2
                
                # Smaller reward for reasonable pricing
                if 0.8 * avg_similar_price <= suggested_price <= 1.2 * avg_similar_price:
                    return 0.6
                
                # Penalize significant deviation from market
                if suggested_price > 1.3 * avg_similar_price:
                    return -1.5
                
                if suggested_price < 0.7 * avg_similar_price:
                    return -0.8
                
                # Small negative reward for other cases
                return -0.3
        
        # Default small neutral reward if we can't determine better
        return 0.0
    
    def _find_similar_items(self, is_sold=True, top_n=5):
        """
        Find items similar to current item
        
        Args:
            is_sold (bool): Whether to find sold or unsold items
            top_n (int): Number of similar items to return
            
        Returns:
            pd.DataFrame: Similar items
        """
        # Current item
        item = self.features_df.iloc[self.current_item_idx]
        
        # Filter by sold status
        items_subset = self.sold_items if is_sold else self.unsold_items
        
        if len(items_subset) == 0:
            return pd.DataFrame()
        
        # Extract state features (excluding ID, price, is_sold, category_id, days_to_sell)
        feature_columns = [col for col in self.features_df.columns 
                          if col not in ['itemId', 'price', 'is_sold', 'category_id', 'days_to_sell']]
        
        if len(feature_columns) == 0 or len(items_subset) <= top_n:
            return items_subset.head(top_n)
        
        # Calculate similarity using Euclidean distance
        item_features = item[feature_columns].values
        
        def euclidean_distance(row):
            return np.sqrt(np.sum((row[feature_columns].values - item_features) ** 2))
        
        # Calculate distances and sort by similarity
        items_subset = items_subset.copy()
        items_subset['distance'] = items_subset.apply(euclidean_distance, axis=1)
        items_subset = items_subset.sort_values('distance')
        
        # Return top N most similar items
        return items_subset.head(top_n)


class PricePredictionTrainer:
    """
    Trainer class for price prediction model
    """
    def __init__(self, category_id: str, config_path: str = "/data/chats/p6wyr/workspace/config/config.json"):
        """
        Initialize the trainer
        
        Args:
            category_id (str): Category ID to train model for
            config_path (str): Path to configuration file
        """
        self.logger = Logger().get_logger()
        self.logger.info(f"Initializing price prediction trainer for category {category_id}")
        
        self.category_id = category_id
        self.config_path = config_path
        
        # Load configuration
        self._load_config()
        
        # Set paths
        self.features_dir = f"/data/chats/p6wyr/workspace/data/features/category_{category_id}"
        self.models_dir = f"/data/chats/p6wyr/workspace/data/models/category_{category_id}"
        
        os.makedirs(self.models_dir, exist_ok=True)
        
        # Load data
        self.features_df = self._load_features()
        
        # Set up environment and agent
        if self.features_df is not None:
            self._setup_env_and_agent()
        else:
            self.logger.error("Failed to load features data. Cannot set up environment and agent.")
            self.environment = None
            self.agent = None
    
    def _load_config(self) -> None:
        """
        Load configuration from file
        """
        try:
            with open(self.config_path, 'r') as f:
                self.config = json.load(f)
            self.logger.info("Configuration loaded successfully")
        except Exception as e:
            self.logger.error(f"Error loading configuration: {str(e)}")
            # Use default configuration
            self.config = {
                "model": {
                    "drl": {
                        "model_type": "dqn",
                        "learning_rate": 0.001,
                        "discount_factor": 0.95,
                        "exploration_rate": 0.2,
                        "exploration_decay_rate": 0.995,
                        "min_exploration_rate": 0.01,
                        "target_update_frequency": 100,
                        "batch_size": 64,
                        "memory_size": 10000,
                        "num_hidden_layers": 3,
                        "hidden_layer_size": 128,
                        "activation": "relu",
                        "double_dqn": True,
                        "dueling_network": True
                    },
                    "reward_function": {
                        "profit_weight": 0.7,
                        "time_weight": 0.3
                    },
                    "training": {
                        "num_episodes": 3000,
                        "checkpoint_interval": 500,
                        "evaluation_interval": 100
                    }
                },
                "data": {
                    "min_items_for_training": 100
                }
            }
    
    def _load_features(self) -> Optional[pd.DataFrame]:
        """
        Load features data for the category
        
        Returns:
            Optional[pd.DataFrame]: Features DataFrame or None if not found
        """
        # Find the latest features file
        feature_files = sorted(glob.glob(os.path.join(self.features_dir, "features_*.csv")))
        
        if not feature_files:
            self.logger.error(f"No feature files found in {self.features_dir}")
            # Create synthetic data for demonstration if no real data exists
            return self._create_synthetic_data()
        
        # Load latest features file
        latest_features_file = feature_files[-1]
        self.logger.info(f"Loading features from {latest_features_file}")
        
        try:
            df = pd.read_csv(latest_features_file)
            self.logger.info(f"Loaded {len(df)} feature records")
            return df
        except Exception as e:
            self.logger.error(f"Error loading features: {str(e)}")
            # Create synthetic data as a fallback
            return self._create_synthetic_data()

    def _create_synthetic_data(self) -> pd.DataFrame:
        """
        Create synthetic data for demonstration purposes
        
        Returns:
            pd.DataFrame: Synthetic features data
        """
        self.logger.warning("Creating synthetic data for demonstration")
        
        # Number of synthetic items
        n_items = 1000
        
        # Create random feature data
        np.random.seed(42)  # For reproducibility
        
        # Get state features from config
        state_features = self.config.get("model", {}).get("state_features", [
            "item_condition", 
            "brand_popularity", 
            "season_demand", 
            "market_saturation",
            "avg_price", 
            "price_std", 
            "time_on_market",
            "conversion_rate"
        ])
        
        # Create DataFrame
        data = {
            'itemId': [f"synth_{i}" for i in range(n_items)],
            'price': np.random.uniform(50, 1000, n_items),
            'is_sold': np.random.choice([0, 1], n_items, p=[0.6, 0.4]),
            'category_id': self.category_id
        }
        
        # Add state features
        for feature in state_features:
            data[feature] = np.random.uniform(0, 1, n_items)
        
        # Add days_to_sell for sold items
        df = pd.DataFrame(data)
        sold_mask = df['is_sold'] == 1
        df.loc[sold_mask, 'days_to_sell'] = np.random.exponential(scale=10, size=sold_mask.sum()).clip(1, 60)
        
        # Log the creation of synthetic data
        self.logger.info(f"Created synthetic dataset with {n_items} items")
        
        return df
    
    def _setup_env_and_agent(self) -> None:
        """
        Set up environment and DRL agent
        """
        # Define number of price levels (actions)
        self.num_price_levels = 15  # More granular price options
        
        # Create environment
        self.environment = PriceEnvironment(
            features_df=self.features_df,
            category_id=self.category_id,
            num_price_levels=self.num_price_levels
        )
        
        # Get state size (number of features)
        state_columns = [col for col in self.features_df.columns 
                        if col not in ['itemId', 'price', 'is_sold', 'category_id', 'days_to_sell']]
        state_size = len(state_columns)
        
        # Create agent - we're assuming PricePredictionDRLAgent is already imported from previous code
        from src.models.drl_agent import PricePredictionDRLAgent
        
        self.agent = PricePredictionDRLAgent(
            state_size=state_size,
            action_size=self.num_price_levels,
            category_id=self.category_id,
            config_path=self.config_path
        )
        
        self.logger.info(f"Set up environment and agent with state size {state_size} and {self.num_price_levels} actions")
    
    def train(self, num_episodes: Optional[int] = None, batch_size: Optional[int] = None, 
              checkpoint_interval: Optional[int] = None, evaluation_interval: Optional[int] = None) -> Dict:
        """
        Train the DRL agent
        
        Args:
            num_episodes (Optional[int]): Number of training episodes
            batch_size (Optional[int]): Batch size for learning
            checkpoint_interval (Optional[int]): Interval for saving checkpoints
            evaluation_interval (Optional[int]): Interval for evaluation
            
        Returns:
            Dict: Training statistics
        """
        if self.agent is None or self.environment is None:
            self.logger.error("Agent or environment not initialized. Cannot train.")
            return {"status": "failed", "reason": "initialization_failed"}
        
        # Get training configuration
        training_config = self.config.get("model", {}).get("training", {})
        drl_config = self.config.get("model", {}).get("drl", {})
        
        # Set parameters, using provided values or defaults from config
        num_episodes = num_episodes or training_config.get("num_episodes", 3000)
        batch_size = batch_size or drl_config.get("batch_size", 64)
        checkpoint_interval = checkpoint_interval or training_config.get("checkpoint_interval", 500)
        evaluation_interval = evaluation_interval or training_config.get("evaluation_interval", 100)
        
        # Get exploration decay parameters
        exploration_decay_rate = drl_config.get("exploration_decay_rate", 0.995)
        min_exploration_rate = drl_config.get("min_exploration_rate", 0.01)
        
        # Initialize statistics
        self.agent.stats["episodes"] = num_episodes
        self.agent.stats["avg_rewards"] = []
        self.agent.stats["avg_losses"] = []
        self.agent.stats["exploration_rate"] = [self.agent.exploration_rate]
        
        # Track best model performance
        best_eval_reward = float("-inf")
        best_model_path = None
        
        self.logger.info(f"Starting training for {num_episodes} episodes")
        
        # Main training loop
        for episode in tqdm(range(1, num_episodes + 1), desc="Training"):
            # Select a random item for training
            state = self.environment.reset()
            
            # Select action using epsilon-greedy policy
            action = self.agent.act(state)
            
            # Take action in the environment
            next_state, reward, done, info = self.environment.step(action)
            
            # Learn from experience
            loss = self.agent.step(state, action, reward, next_state, done)
            
            # Decay exploration rate
            if episode % 10 == 0:  # Decay every 10 episodes
                self.agent.decay_exploration_rate(
                    decay_factor=exploration_decay_rate,
                    min_rate=min_exploration_rate
                )
            
            # Log training progress to tensorboard
            if episode % 10 == 0:  # Log every 10 episodes
                self.agent.writer.add_scalar('Training/Reward', reward, episode)
                if loss > 0:
                    self.agent.writer.add_scalar('Training/Loss', loss, episode)
                self.agent.writer.add_scalar('Training/Exploration_Rate', 
                                            self.agent.exploration_rate, episode)
            
            # Store statistics
            if episode % 50 == 0:  # Update stats periodically
                self.agent.stats["avg_rewards"].append(self._evaluate_agent(10))
                self.agent.stats["avg_losses"].append(loss if loss > 0 else 0)
                self.agent.stats["exploration_rate"].append(self.agent.exploration_rate)
                
                # Print progress
                self.logger.info(
                    f"Episode {episode}/{num_episodes} | "
                    f"Avg Reward: {self.agent.stats['avg_rewards'][-1]:.3f} | "
                    f"Loss: {self.agent.stats['avg_losses'][-1]:.3f} | "
                    f"Exploration Rate: {self.agent.exploration_rate:.3f}"
                )
            
            # Save checkpoints
            if episode % checkpoint_interval == 0:
                checkpoint_path = self.agent.save_model(episode=episode)
                self.logger.info(f"Saved checkpoint at episode {episode} to {checkpoint_path}")
            
            # Evaluate agent and save best model
            if episode % evaluation_interval == 0:
                eval_reward = self._evaluate_agent(50)  # More thorough evaluation
                self.logger.info(f"Evaluation at episode {episode}: Average reward = {eval_reward:.3f}")
                
                # Log to tensorboard
                self.agent.writer.add_scalar('Evaluation/Average_Reward', eval_reward, episode)
                
                # Save best model
                if eval_reward > best_eval_reward:
                    best_eval_reward = eval_reward
                    best_model_path = self.agent.save_model(episode=episode)
                    self.logger.info(f"New best model at episode {episode} with reward {best_eval_reward:.3f}")
        
        # Final evaluation
        final_eval_reward = self._evaluate_agent(100)
        self.logger.info(f"Final evaluation: Average reward = {final_eval_reward:.3f}")
        
        # Save final model
        final_model_path = self.agent.save_model(episode=num_episodes)
        
        # Close tensorboard writer
        self.agent.writer.close()
        
        # Return training statistics
        return {
            "status": "success",
            "episodes_trained": num_episodes,
            "final_exploration_rate": self.agent.exploration_rate,
            "final_evaluation_reward": final_eval_reward,
            "best_evaluation_reward": best_eval_reward,
            "best_model_path": best_model_path,
            "final_model_path": final_model_path
        }
    
    def _evaluate_agent(self, num_episodes: int = 10) -> float:
        """
        Evaluate the agent's performance
        
        Args:
            num_episodes (int): Number of episodes to evaluate
            
        Returns:
            float: Average reward
        """
        rewards = []
        
        for _ in range(num_episodes):
            state = self.environment.reset()
            action = self.agent.act(state, evaluate=True)  # No exploration during evaluation
            _, reward, _, _ = self.environment.step(action)
            rewards.append(reward)
        
        return sum(rewards) / num_episodes if rewards else 0
    
    def evaluate_model(self, model_path: Optional[str] = None, num_episodes: int = 100) -> Dict:
        """
        Evaluate a trained model
        
        Args:
            model_path (Optional[str]): Path to model to evaluate (uses the latest if None)
            num_episodes (int): Number of episodes to evaluate
            
        Returns:
            Dict: Evaluation results
        """
        if self.agent is None or self.environment is None:
            self.logger.error("Agent or environment not initialized. Cannot evaluate.")
            return {"status": "failed", "reason": "initialization_failed"}
        
        # If no model path provided, find the latest model
        if model_path is None:
            model_files = sorted(glob.glob(os.path.join(self.models_dir, "drl_model_*.pt")))
            if not model_files:
                self.logger.error("No trained models found")
                return {"status": "failed", "reason": "no_models_found"}
            model_path = model_files[-1]
        
        # Load the model
        if not self.agent.load_model(model_path):
            return {"status": "failed", "reason": "model_loading_failed"}
        
        self.logger.info(f"Evaluating model from {model_path}")
        
        # Run evaluation
        rewards = []
        price_accuracies = []  # Track how close predicted prices are to actual prices
        actions_taken = []
        
        for episode in tqdm(range(num_episodes), desc="Evaluating"):
            state = self.environment.reset()
            action = self.agent.act(state, evaluate=True)
            _, reward, _, info = self.environment.step(action)
            
            rewards.append(reward)
            actions_taken.append(action)
            
            # Calculate price accuracy for sold items
            if info.get('is_sold') and info.get('sold_price'):
                accuracy = abs(info['suggested_price'] - info['sold_price']) / info['sold_price']
                price_accuracies.append(1 - accuracy)  # Convert to accuracy: 1 = perfect, 0 = completely off
        
        # Calculate statistics
        avg_reward = sum(rewards) / len(rewards) if rewards else 0
        avg_price_accuracy = sum(price_accuracies) / len(price_accuracies) if price_accuracies else 0
        action_distribution = {action: actions_taken.count(action) / len(actions_taken) 
                              for action in set(actions_taken)}
        
        # Create evaluation results
        results = {
            "status": "success",
            "model_path": model_path,
            "episodes_evaluated": num_episodes,
            "average_reward": avg_reward,
            "price_accuracy": avg_price_accuracy,
            "action_distribution": action_distribution
        }
        
        self.logger.info(f"Evaluation results: Avg reward = {avg_reward:.3f}, Price accuracy = {avg_price_accuracy:.3f}")
        
        return results
    
    def predict_price(self, features: Dict) -> Dict:
        """
        Predict the optimal price for an item
        
        Args:
            features (Dict): Item features
            
        Returns:
            Dict: Prediction results
        """
        if self.agent is None or self.environment is None:
            self.logger.error("Agent or environment not initialized. Cannot predict.")
            return {"status": "failed", "reason": "initialization_failed"}
        
        # Find the latest model if no model is loaded
        if not hasattr(self.agent, 'qnetwork_local') or self.agent.qnetwork_local is None:
            model_files = sorted(glob.glob(os.path.join(self.models_dir, "drl_model_*.pt")))
            if not model_files:
                self.logger.error("No trained models found")
                return {"status": "failed", "reason": "no_models_found"}
            
            # Load the latest model
            if not self.agent.load_model(model_files[-1]):
                return {"status": "failed", "reason": "model_loading_failed"}
        
        # Format features as a state vector
        try:
            # Get relevant features
            state_columns = [col for col in self.features_df.columns 
                           if col not in ['itemId', 'price', 'is_sold', 'category_id', 'days_to_sell']]
            
            # Create state array
            state = np.zeros(len(state_columns), dtype=np.float32)
            for i, col in enumerate(state_columns):
                state[i] = features.get(col, 0.0)
            
            # Get base price
            base_price = features.get('price', self.environment.avg_price)
            
            # Predict action
            action = self.agent.act(state, evaluate=True)
            price_factor = self.environment.price_levels[action]
            suggested_price = base_price * price_factor
            
            # Calculate price range
            confidence = 0.9  # 90% confidence
            lower_bound = suggested_price * (1 - 0.05)
            upper_bound = suggested_price * (1 + 0.05)
            
            return {
                "status": "success",
                "base_price": base_price,
                "suggested_price": suggested_price,
                "price_factor": price_factor,
                "price_range": {
                    "lower": lower_bound,
                    "upper": upper_bound,
                    "confidence": confidence
                },
                "action": int(action)
            }
            
        except Exception as e:
            self.logger.error(f"Error predicting price: {str(e)}")
            return {"status": "failed", "reason": str(e)}
    
    def plot_training_progress(self, save_path: Optional[str] = None):
        """
        Plot training progress metrics
        
        Args:
            save_path (Optional[str]): Path to save the plot
        """
        if not self.agent or not self.agent.stats or not self.agent.stats["avg_rewards"]:
            self.logger.error("No training statistics available")
            return
        
        # Create figure with 3 subplots
        fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10, 15), sharex=True)
        
        # Plot average rewards
        episodes = list(range(0, len(self.agent.stats["avg_rewards"]) * 50, 50))
        ax1.plot(episodes, self.agent.stats["avg_rewards"])
        ax1.set_title("Average Reward")
        ax1.set_ylabel("Reward")
        ax1.grid(True)
        
        # Plot average losses
        if self.agent.stats.get("avg_losses"):
            ax2.plot(episodes, self.agent.stats["avg_losses"])
            ax2.set_title("Average Loss")
            ax2.set_ylabel("Loss")
            ax2.grid(True)
        
        # Plot exploration rate
        exploration_rate_episodes = list(range(0, len(self.agent.stats["exploration_rate"]) * 50, 50))
        ax3.plot(exploration_rate_episodes, self.agent.stats["exploration_rate"])
        ax3.set_title("Exploration Rate")
        ax3.set_xlabel("Episode")
        ax3.set_ylabel("Rate")
        ax3.grid(True)
        
        plt.tight_layout()
        
        # Save or show plot
        if save_path:
            plt.savefig(save_path)
            self.logger.info(f"Training progress plot saved to {save_path}")
        else:
            plt.show()

In [28]:
import os
import json
import pandas as pd
from pathlib import Path
from datetime import datetime

# Check what directories have been created
print("Directories created:")
data_dir = Path("/data/chats/p6wyr/workspace/data")
for subdir in ["raw", "processed", "features", "cache"]:
    full_path = data_dir / subdir
    exists = full_path.exists()
    print(f"- {subdir}: {'exists' if exists else 'does not exist'}")

# Check if there's any config file
config_path = "/data/chats/p6wyr/workspace/config/config.json"
if os.path.exists(config_path):
    with open(config_path, 'r') as f:
        config = json.load(f)
    print("\nConfig file found. Categories configured:")
    for category in config.get("data", {}).get("categories", []):
        print(f"- {category['name']} (ID: {category['id']})")
else:
    print("\nNo config file found at", config_path)

# Check if there's any data collected in processed directory
processed_dir = data_dir / "processed"
if processed_dir.exists():
    category_dirs = [d for d in processed_dir.iterdir() if d.is_dir() and d.name.startswith("category_")]
    print(f"\nFound {len(category_dirs)} category directories in processed data")
    
    # Try to load a sample DataFrame if any CSV exists
    csv_files = []
    for cat_dir in category_dirs:
        csv_files.extend(list(cat_dir.glob("*.csv")))
    
    if csv_files:
        print(f"Found {len(csv_files)} CSV files. Loading most recent one for inspection:")
        latest_csv = max(csv_files, key=lambda x: x.stat().st_mtime)
        print(f"Loading: {latest_csv}")
        df = pd.read_csv(latest_csv)
        
        # Using the specified function to inspect columns
        from metagpt.tools.libs.data_preprocess import get_column_info
        column_info = get_column_info(df)
        print("\nColumn information:")
        print(column_info)
    else:
        print("No CSV files found in the processed directories.")
else:
    print("\nNo processed data directory found.")

Directories created:
- raw: exists
- processed: exists
- features: exists
- cache: exists

Config file found. Categories configured:
- Laptops & Netbooks (ID: 9355)
- Cell Phones & Smartphones (ID: 15032)
- Wristwatch (ID: 11450)
- Digital Cameras (ID: 261007)

Found 0 category directories in processed data
No CSV files found in the processed directories.


In [29]:
I'll create a main script to demonstrate the DRL model functionality that integrates the previously implemented components.

```python
from metagpt.tools.libs.terminal import Terminal
terminal = Terminal()

# Ensure the required directories exist
await terminal.run('mkdir -p /data/chats/p6wyr/workspace/src/models')

# Create the drl_agent.py file with our PricePredictionDRLAgent implementation
drl_agent_path = "/data/chats/p6wyr/workspace/src/models/drl_agent.py"

drl_agent_code = """
import os
import json
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from typing import Dict, List, Tuple, Optional, Any
from collections import deque, namedtuple
import random
from datetime import datetime

from src.utils.logger import Logger

# Define constants for the DRL model
Experience = namedtuple('Experience', ['state', 'action', 'reward', 'next_state', 'done'])

class ReplayBuffer:
    \"\"\"
    Replay buffer for storing and sampling experiences for DRL training
    \"\"\"
    def __init__(self, capacity: int):
        \"\"\"Initialize replay buffer with given capacity\"\"\"
        self.buffer = deque(maxlen=capacity)
    
    def push(self, state, action, reward, next_state, done):
        \"\"\"Add experience to the buffer\"\"\"
        self.buffer.append(Experience(state, action, reward, next_state, done))
    
    def sample(self, batch_size: int):
        \"\"\"Sample random batch of experiences\"\"\"
        experiences = random.sample(self.buffer, batch_size)
        states = torch.from_numpy(np.vstack([e.state for e in experiences])).float()
        actions = torch.from_numpy(np.vstack([e.action for e in experiences])).long()
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences])).float()
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences])).float()
        dones = torch.from_numpy(np.vstack([e.done for e in experiences]).astype(np.uint8)).float()
        return states, actions, rewards, next_states, dones
    
    def __len__(self):
        \"\"\"Current size of replay buffer\"\"\"
        return len(self.buffer)

class DQNetwork(nn.Module):
    \"\"\"
    Deep Q-Network for price prediction
    \"\"\"
    def __init__(self, state_size: int, action_size: int, config: Dict):
        \"\"\"Initialize DQN with state and action dimensions\"\"\"
        super(DQNetwork, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        
        # Get configuration parameters
        hidden_layer_size = config.get("hidden_layer_size", 128)
        num_hidden_layers = config.get("num_hidden_layers", 3)
        dropout_rate = config.get("dropout_rate", 0.1)
        
        # Create the network layers with batch normalization and dropout
        layers = [nn.Linear(state_size, hidden_layer_size), 
                 nn.BatchNorm1d(hidden_layer_size),
                 nn.ReLU(),
                 nn.Dropout(dropout_rate)]
        
        # Add hidden layers
        for _ in range(num_hidden_layers - 1):
            layers.append(nn.Linear(hidden_layer_size, hidden_layer_size))
            layers.append(nn.BatchNorm1d(hidden_layer_size))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout_rate))
        
        # Add output layer
        layers.append(nn.Linear(hidden_layer_size, action_size))
        
        # Create sequential model
        self.network = nn.Sequential(*layers)
        
        # Initialize weights using Xavier/Glorot initialization
        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        \"\"\"Initialize network weights\"\"\"
        if isinstance(module, nn.Linear):
            nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                module.bias.data.fill_(0.01)
    
    def forward(self, state):
        \"\"\"Forward pass through network\"\"\"
        # Handle both single and batch inputs
        if state.dim() == 1:
            state = state.unsqueeze(0)
            
        return self.network(state)

class DuelingDQNetwork(nn.Module):
    \"\"\"
    Dueling Deep Q-Network for improved performance
    Separates state value and action advantages for better estimation
    \"\"\"
    def __init__(self, state_size: int, action_size: int, config: Dict):
        \"\"\"Initialize Dueling DQN with state and action dimensions\"\"\"
        super(DuelingDQNetwork, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        
        # Get configuration parameters
        hidden_layer_size = config.get("hidden_layer_size", 128)
        num_hidden_layers = config.get("num_hidden_layers", 3)
        dropout_rate = config.get("dropout_rate", 0.1)
        
        # Feature extraction layers
        self.feature_layers = nn.Sequential(
            nn.Linear(state_size, hidden_layer_size),
            nn.BatchNorm1d(hidden_layer_size),
            nn.ReLU(),
            nn.Dropout(dropout_rate)
        )
        
        # Add hidden layers for feature extraction
        for _ in range(num_hidden_layers - 2):
            self.feature_layers.extend([
                nn.Linear(hidden_layer_size, hidden_layer_size),
                nn.BatchNorm1d(hidden_layer_size),
                nn.ReLU(),
                nn.Dropout(dropout_rate)
            ])
        
        # Value stream - estimates state value V(s)
        self.value_stream = nn.Sequential(
            nn.Linear(hidden_layer_size, hidden_layer_size),
            nn.BatchNorm1d(hidden_layer_size),
            nn.ReLU(),
            nn.Linear(hidden_layer_size, 1)
        )
        
        # Advantage stream - estimates advantages for each action A(s,a)
        self.advantage_stream = nn.Sequential(
            nn.Linear(hidden_layer_size, hidden_layer_size),
            nn.BatchNorm1d(hidden_layer_size),
            nn.ReLU(),
            nn.Linear(hidden_layer_size, action_size)
        )
        
        # Initialize weights
        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        \"\"\"Initialize network weights\"\"\"
        if isinstance(module, nn.Linear):
            nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                module.bias.data.fill_(0.01)
    
    def forward(self, state):
        \"\"\"Forward pass through dueling network architecture\"\"\"
        # Handle both single and batch inputs
        if state.dim() == 1:
            state = state.unsqueeze(0)
            
        features = self.feature_layers(state)
        
        # Calculate state value and advantages
        value = self.value_stream(features)
        advantages = self.advantage_stream(features)
        
        # Q(s,a) = V(s) + (A(s,a) - mean(A(s,a')))
        qvalues = value + (advantages - advantages.mean(dim=1, keepdim=True))
        
        return qvalues

class PricePredictionDRLAgent:
    \"\"\"
    Deep Reinforcement Learning agent for price prediction
    \"\"\"
    def __init__(self, state_size: int, action_size: int, category_id: str, 
                 config_path: str = "/data/chats/p6wyr/workspace/config/config.json"):
        \"\"\"Initialize the DRL agent\"\"\"
        self.logger = Logger().get_logger()
        self.logger.info("Initializing DRL agent for price prediction")
        
        # Load configuration
        self.config_path = config_path
        self._load_config()
        
        # Get DRL model configuration
        self.drl_config = self.config.get("model", {}).get("drl", {})
        
        # Set up model parameters
        self.state_size = state_size
        self.action_size = action_size
        self.category_id = category_id
        self.learning_rate = self.drl_config.get("learning_rate", 0.001)
        self.discount_factor = self.drl_config.get("discount_factor", 0.95)
        self.exploration_rate = self.drl_config.get("exploration_rate", 0.1)
        self.batch_size = self.drl_config.get("batch_size", 64)
        self.memory_size = self.drl_config.get("memory_size", 10000)
        self.target_update_freq = self.drl_config.get("target_update_frequency", 1000)
        self.double_dqn = self.drl_config.get("double_dqn", True)
        self.dueling_network = self.drl_config.get("dueling_network", True)
        
        # Set device (GPU if available)
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.logger.info(f"Using device: {self.device}")
        
        # Initialize networks
        self._initialize_networks()
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=self.learning_rate)
        
        # Initialize replay memory
        self.memory = ReplayBuffer(self.memory_size)
        
        # Initialize step counter for target network updates
        self.t_step = 0
        self.train_step = 0
        
        # Set up model directories
        self.models_dir = f"/data/chats/p6wyr/workspace/data/models/category_{self.category_id}"
        os.makedirs(self.models_dir, exist_ok=True)
        
        # Set up tensorboard logging
        self.tb_dir = f"/data/chats/p6wyr/workspace/logs/tensorboard/category_{self.category_id}"
        os.makedirs(self.tb_dir, exist_ok=True)
        self.writer = SummaryWriter(log_dir=self.tb_dir)
        
        # Reward function weights
        reward_config = self.config.get("model", {}).get("reward_function", {})
        self.profit_weight = reward_config.get("profit_weight", 0.7)
        self.time_weight = reward_config.get("time_weight", 0.3)
        
        # Track statistics
        self.stats = {
            "episodes": 0,
            "avg_rewards": [],
            "avg_losses": [],
            "exploration_rate": []
        }
    
    def _load_config(self) -> None:
        \"\"\"Load configuration from file\"\"\"
        try:
            with open(self.config_path, 'r') as f:
                self.config = json.load(f)
            self.logger.info("Configuration loaded successfully")
        except Exception as e:
            self.logger.error(f"Error loading configuration: {str(e)}")
            # Use default configuration
            self.config = {
                "model": {
                    "drl": {
                        "model_type": "dqn",
                        "learning_rate": 0.001,
                        "discount_factor": 0.95,
                        "exploration_rate": 0.1,
                        "target_update_frequency": 1000,
                        "batch_size": 64,
                        "memory_size": 10000,
                        "num_hidden_layers": 3,
                        "hidden_layer_size": 128,
                        "activation": "relu",
                        "double_dqn": True,
                        "dueling_network": True,
                        "dropout_rate": 0.1
                    },
                    "reward_function": {
                        "profit_weight": 0.7,
                        "time_weight": 0.3
                    }
                }
            }
    
    def _initialize_networks(self):
        \"\"\"Initialize Q-networks based on configuration\"\"\"
        # Choose network architecture based on config
        if self.dueling_network:
            self.qnetwork_local = DuelingDQNetwork(
                self.state_size, self.action_size, self.drl_config).to(self.device)
            self.qnetwork_target = DuelingDQNetwork(
                self.state_size, self.action_size, self.drl_config).to(self.device)
            self.logger.info("Using Dueling DQN architecture")
        else:
            self.qnetwork_local = DQNetwork(
                self.state_size, self.action_size, self.drl_config).to(self.device)
            self.qnetwork_target = DQNetwork(
                self.state_size, self.action_size, self.drl_config).to(self.device)
            self.logger.info("Using standard DQN architecture")
        
        # Copy weights from local to target network
        self.qnetwork_target.load_state_dict(self.qnetwork_local.state_dict())
    
    def step(self, state, action, reward, next_state, done):
        \"\"\"Update the agent's knowledge based on experience\"\"\"
        # Save experience in replay memory
        self.memory.push(state, action, reward, next_state, done)
        
        # Update step counter
        self.t_step = (self.t_step + 1) % self.target_update_freq
        
        # Learn every self.batch_size steps, if enough samples in memory
        if len(self.memory) > self.batch_size:
            experiences = self.memory.sample(self.batch_size)
            loss = self._learn(experiences)
            
            # Log loss to tensorboard
            self.train_step += 1
            self.writer.add_scalar('Loss/train', loss, self.train_step)
            
            # Update target network periodically
            if self.t_step == 0:
                self._update_target_network()
            
            return loss
        
        return 0
    
    def act(self, state, evaluate=False):
        \"\"\"Choose action based on current state using epsilon-greedy policy\"\"\"
        # Convert state to tensor
        state = torch.from_numpy(state).float().unsqueeze(0).to(self.device)
        
        # Set network to evaluation mode
        self.qnetwork_local.eval()
        
        with torch.no_grad():
            action_values = self.qnetwork_local(state)
        
        # Set network back to training mode
        self.qnetwork_local.train()
        
        # Epsilon-greedy action selection
        if not evaluate and random.random() < self.exploration_rate:
            return random.choice(np.arange(self.action_size))
        else:
            return np.argmax(action_values.cpu().data.numpy())
    
    def _learn(self, experiences):
        \"\"\"Update value parameters using batch of experiences\"\"\"
        states, actions, rewards, next_states, dones = experiences
        
        # Move to device
        states = states.to(self.device)
        actions = actions.to(self.device)
        rewards = rewards.to(self.device)
        next_states = next_states.to(self.device)
        dones = dones.to(self.device)
        
        if self.double_dqn:
            # Double DQN: Select actions using local network, evaluate using target network
            self.qnetwork_local.eval()
            with torch.no_grad():
                local_best_actions = self.qnetwork_local(next_states).argmax(dim=1, keepdim=True)
            self.qnetwork_local.train()
            
            # Evaluate those actions using the target network
            Q_targets_next = self.qnetwork_target(next_states).gather(1, local_best_actions)
        else:
            # Standard DQN: Get max predicted Q values from target model
            Q_targets_next = self.qnetwork_target(next_states).detach().max(1)[0].unsqueeze(1)
        
        # Compute Q targets for current states
        Q_targets = rewards + (self.discount_factor * Q_targets_next * (1 - dones))
        
        # Get expected Q values from local model
        Q_expected = self.qnetwork_local(states).gather(1, actions)
        
        # Compute loss - Huber loss for stability
        loss = F.smooth_l1_loss(Q_expected, Q_targets)
        
        # Minimize the loss
        self.optimizer.zero_grad()
        loss.backward()
        
        # Gradient clipping to prevent exploding gradients
        torch.nn.utils.clip_grad_norm_(self.qnetwork_local.parameters(), 1.0)
        
        self.optimizer.step()
        
        return loss.item()
    
    def _update_target_network(self):
        \"\"\"Update target network with parameters from local network\"\"\"
        # Soft update of the target network's weights
        tau = 0.001  # Small update rate for stability
        for target_param, local_param in zip(self.qnetwork_target.parameters(), self.qnetwork_local.parameters()):
            target_param.data.copy_(tau * local_param.data + (1.0 - tau) * target_param.data)
    
    def save_model(self, episode: int = None):
        \"\"\"Save the agent's model\"\"\"
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        episode_str = f"_episode_{episode}" if episode is not None else ""
        
        model_path = os.path.join(self.models_dir, f"drl_model_{timestamp}{episode_str}.pt")
        
        torch.save({
            'qnetwork_local_state_dict': self.qnetwork_local.state_dict(),
            'qnetwork_target_state_dict': self.qnetwork_target.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'exploration_rate': self.exploration_rate,
            'stats': self.stats,
            'config': {
                'state_size': self.state_size,
                'action_size': self.action_size,
                'double_dqn': self.double_dqn,
                'dueling_network': self.dueling_network
            }
        }, model_path)
        
        self.logger.info(f"Model saved to {model_path}")
        return model_path
    
    def load_model(self, model_path: str):
        \"\"\"Load a saved model\"\"\"
        if not os.path.exists(model_path):
            self.logger.error(f"Model file not found: {model_path}")
            return False
        
        try:
            checkpoint = torch.load(model_path, map_location=self.device)
            
            # Check if model architecture matches
            config = checkpoint.get('config', {})
            if config.get('state_size') != self.state_size or config.get('action_size') != self.action_size:
                self.logger.warning(f"Model architecture mismatch. Reinitializing networks.")
                
                # Handle architecture mismatch by reinitializing networks
                self.dueling_network = config.get('dueling_network', self.dueling_network)
                self.double_dqn = config.get('double_dqn', self.double_dqn)
                self._initialize_networks()
            
            self.qnetwork_local.load_state_dict(checkpoint['qnetwork_local_state_dict'])
            self.qnetwork_target.load_state_dict(checkpoint['qnetwork_target_state_dict'])
            self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            self.exploration_rate = checkpoint.get('exploration_rate', 0.1)
            self.stats = checkpoint.get('stats', self.stats)
            
            self.logger.info(f"Model loaded from {model_path}")
            return True
        except Exception as e:
            self.logger.error(f"Error loading model: {str(e)}")
            return False
    
    def compute_reward(self, state, action, next_state, price, sold_price=None, time_to_sell=None):
        \"\"\"Compute the reward for a state-action pair\"\"\"
        # If item sold, use actual profit and time
        if sold_price is not None and time_to_sell is not None:
            profit_margin = (sold_price - price) / price if price > 0 else 0
            
            # Normalize time to sell (consider 30 days as max)
            norm_time = max(0, 1 - (time_to_sell / 30)) if time_to_sell > 0 else 0
            
            # Combine profit and time components
            reward = (self.profit_weight * profit_margin) + (self.time_weight * norm_time)
            
            return reward * 10  # Scale reward
        
        # If not sold yet, use market-based reward estimation
        price_competitiveness = next_state[4] if len(next_state) > 4 else 0.5
        conversion_rate = next_state[7] if len(next_state) > 7 else 0.5
        
        # Penalize extreme pricing
        price_level = action / (self.action_size - 1)
        price_penalty = -2.0 * abs(price_level - 0.5)
        
        # Combine factors for reward estimation
        estimated_reward = (price_competitiveness * 0.6) + (conversion_rate * 0.4) + price_penalty
        
        return estimated_reward * 5
    
    def decay_exploration_rate(self, decay_factor=0.995, min_rate=0.01):
        \"\"\"Decay the exploration rate\"\"\"
        self.exploration_rate = max(min_rate, self.exploration_rate * decay_factor)
        self.stats["exploration_rate"].append(self.exploration_rate)
        return self.exploration_rate
"""

# Create the environment.py file
environment_path = "/data/chats/p6wyr/workspace/src/models/environment.py"

environment_code = """
import numpy as np
import pandas as pd
from typing import Dict, List, Tuple, Optional, Any
import random

from src.utils.logger import Logger

class PriceEnvironment:
    \"\"\"
    Environment for price prediction, simulating market dynamics
    \"\"\"
    def __init__(self, features_df: pd.DataFrame, category_id: str, num_price_levels: int = 10):
        \"\"\"Initialize the environment\"\"\"
        self.logger = Logger().get_logger()
        self.features_df = features_df.copy()
        self.category_id = category_id
        
        # Get sold items for statistics
        self.sold_items = self.features_df[self.features_df['is_sold'] == 1]
        self.unsold_items = self.features_df[self.features_df['is_sold'] == 0]
        
        # Extract price range
        self.min_price = self.features_df['price'].min()
        self.max_price = self.features_df['price'].max()
        self.avg_price = self.features_df['price'].mean()
        
        # Calculate price levels (actions)
        self.num_price_levels = num_price_levels
        self.price_levels = np.linspace(0.5, 1.5, num_price_levels)
        
        # Current state info
        self.current_item_idx = 0
        self.current_state = None
        self.base_price = 0
        
        self.logger.info(f"Initialized price environment with {len(features_df)} items, {num_price_levels} price levels")
    
    def reset(self, item_idx=None):
        \"\"\"Reset the environment and get new item\"\"\"
        # Select item (random if not specified)
        if item_idx is None:
            self.current_item_idx = np.random.randint(0, len(self.features_df))
        else:
            self.current_item_idx = item_idx % len(self.features_df)
        
        # Get item data
        item = self.features_df.iloc[self.current_item_idx]
        
        # Extract state features (excluding ID, price, is_sold, category_id, days_to_sell)
        state_columns = [col for col in self.features_df.columns 
                         if col not in ['itemId', 'price', 'is_sold', 'category_id', 'days_to_sell']]
        
        # Set current state
        self.current_state = item[state_columns].values.astype(np.float32)
        
        # Set base price
        self.base_price = item['price']
        
        return self.current_state
    
    def step(self, action):
        \"\"\"Take a step in the environment\"\"\"
        # Get selected price level
        price_factor = self.price_levels[action]
        suggested_price = self.base_price * price_factor
        
        # Get current item
        item = self.features_df.iloc[self.current_item_idx]
        
        # Check if item sold in historical data
        is_sold = item['is_sold'] == 1
        days_to_sell = item.get('days_to_sell', 30)
        
        # Get actual sold price if available
        sold_price = item['price'] if is_sold else None
        
        # Calculate reward
        reward = self._calculate_reward(suggested_price, is_sold, sold_price, days_to_sell)
        
        # Return the same state (as this is not a sequential environment)
        next_state = self.current_state
        
        # Episode is done after one step
        done = True
        
        # Additional info
        info = {
            'item_id': item.get('itemId', self.current_item_idx),
            'base_price': self.base_price,
            'suggested_price': suggested_price,
            'price_factor': price_factor,
            'is_sold': is_sold,
            'days_to_sell': days_to_sell if is_sold else None,
            'sold_price': sold_price
        }
        
        return next_state, reward, done, info
    
    def _calculate_reward(self, suggested_price, is_sold, sold_price, days_to_sell):
        \"\"\"Calculate reward for the action\"\"\"
        # For sold items, compare the suggested price to actual sold price
        if is_sold and sold_price is not None:
            # Penalize overpricing that could prevent sale
            if suggested_price > 1.2 * sold_price:
                return -2.0
            
            # Reward accurate pricing (within 5% of actual)
            if 0.95 * sold_price <= suggested_price <= 1.05 * sold_price:
                return 2.0
            
            # Good pricing (within 10% of actual)
            if 0.9 * sold_price <= suggested_price <= 1.1 * sold_price:
                return 1.5
            
            # Reasonable pricing (within 20% of actual)
            if 0.8 * sold_price <= suggested_price <= 1.2 * sold_price:
                return 0.8
            
            # Penalize significantly underpricing
            if suggested_price < 0.8 * sold_price:
                return -1.0
            
            return 0.2
        
        # For unsold items, evaluate based on market dynamics
        else:
            # Get similar items
            similar_sold = self._find_similar_items(is_sold=True)
            
            if len(similar_sold) > 0:
                avg_similar_price = similar_sold['price'].mean()
                
                # Reward pricing close to successful similar items
                if 0.9 * avg_similar_price <= suggested_price <= 1.1 * avg_similar_price:
                    return 1.2
                
                # Smaller reward for reasonable pricing
                if 0.8 * avg_similar_price <= suggested_price <= 1.2 * avg_similar_price:
                    return 0.6
                
                # Penalize significant deviation from market
                if suggested_price > 1.3 * avg_similar_price:
                    return -1.5
                
                if suggested_price < 0.7 * avg_similar_price:
                    return -0.8
                
                return -0.3
        
        return 0.0
    
    def _find_similar_items(self, is_sold=True, top_n=5):
        \"\"\"Find items similar to current item\"\"\"
        # Current item
        item = self.features_df.iloc[self.current_item_idx]
        
        # Filter by sold status
        items_subset = self.sold_items if is_sold else self.unsold_items
        
        if len(items_subset) == 0:
            return pd.DataFrame()
        
        # Extract state features
        feature_columns = [col for col in self.features_df.columns 
                          if col not in ['itemId', 'price', 'is_sold', 'category_id', 'days_to_sell']]
        
        if len(feature_columns) == 0 or len(items_subset) <= top_n:
            return items_subset.head(top_n)
        
        # Calculate similarity using Euclidean distance
        item_features = item[feature_columns].values
        
        def euclidean_distance(row):
            return np.sqrt(np.sum((row[feature_columns].values - item_features) ** 2))
        
        # Calculate distances and sort by similarity
        items_subset = items_subset.copy()
        items_subset['distance'] = items_subset.apply(euclidean_distance, axis=1)
        items_subset = items_subset.sort_values('distance')
        
        # Return top N most similar items
        return items_subset.head(top_n)
"""

# Create the trainer.py file
trainer_path = "/data/chats/p6wyr/workspace/src/models/trainer.py"

trainer_code = """
import os
import json
import glob
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple, Optional, Any
from tqdm import tqdm
from datetime import datetime

from src.utils.logger import Logger
from src.models.environment import PriceEnvironment
from src.models.drl_agent import PricePredictionDRLAgent

class PricePredictionTrainer:
    \"\"\"
    Trainer class for price prediction model
    \"\"\"
    def __init__(self, category_id: str, config_path: str = "/data/chats/p6wyr/workspace/config/config.json"):
        \"\"\"Initialize the trainer\"\"\"
        self.logger = Logger().get_logger()
        self.logger.info(f"Initializing price prediction trainer for category {category_id}")
        
        self.category_id = category_id
        self.config_path = config_path
        
        # Load configuration
        self._load_config()
        
        # Set paths
        self.features_dir = f"/data/chats/p6wyr/workspace/data/features/category_{category_id}"
        self.models_dir = f"/data/chats/p6wyr/workspace/data/models

SyntaxError: unterminated string literal (detected at line 1) (3552987608.py, line 1)

In [30]:
I need to fix the error in the code. It looks like we have an unterminated string literal in the trainer code and didn't finish the main script implementation.

[reflection on previous impl]
In my previous implementation, I was writing three Python module files (drl_agent.py, environment.py, and trainer.py) and started creating the main.py script. However, I never completed the trainer.py file as it ended with an unterminated string literal at `self.models_dir = f"/data/chats/p6wyr/workspace/data/models`. Additionally, I never got to creating the actual main.py script that demonstrates the model functionality.

I need to:
1. Complete the trainer.py file by properly ending the string literal
2. Create a main.py script that integrates all components
3. Ensure the implementation allows for training and evaluating the model on synthetic data if real data isn't available

[improved impl]:
```python
from metagpt.tools.libs.terminal import Terminal
terminal = Terminal()

# Ensure required directories exist
await terminal.run('mkdir -p /data/chats/p6wyr/workspace/src/models')
await terminal.run('mkdir -p /data/chats/p6wyr/workspace/data/models')
await terminal.run('mkdir -p /data/chats/p6wyr/workspace/logs/tensorboard')

# Create the drl_agent.py file
drl_agent_path = "/data/chats/p6wyr/workspace/src/models/drl_agent.py"
await terminal.run(f"touch {drl_agent_path}")

drl_agent_code = '''
import os
import json
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from typing import Dict, List, Tuple, Optional, Any
from collections import deque, namedtuple
import random
from datetime import datetime

from src.utils.logger import Logger

# Define constants for the DRL model
Experience = namedtuple('Experience', ['state', 'action', 'reward', 'next_state', 'done'])

class ReplayBuffer:
    """
    Replay buffer for storing and sampling experiences for DRL training
    """
    def __init__(self, capacity: int):
        """Initialize replay buffer with given capacity"""
        self.buffer = deque(maxlen=capacity)
    
    def push(self, state, action, reward, next_state, done):
        """Add experience to the buffer"""
        self.buffer.append(Experience(state, action, reward, next_state, done))
    
    def sample(self, batch_size: int):
        """Sample random batch of experiences"""
        experiences = random.sample(self.buffer, batch_size)
        states = torch.from_numpy(np.vstack([e.state for e in experiences])).float()
        actions = torch.from_numpy(np.vstack([e.action for e in experiences])).long()
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences])).float()
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences])).float()
        dones = torch.from_numpy(np.vstack([e.done for e in experiences]).astype(np.uint8)).float()
        return states, actions, rewards, next_states, dones
    
    def __len__(self):
        """Current size of replay buffer"""
        return len(self.buffer)

class DQNetwork(nn.Module):
    """
    Deep Q-Network for price prediction
    """
    def __init__(self, state_size: int, action_size: int, config: Dict):
        """Initialize DQN with state and action dimensions"""
        super(DQNetwork, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        
        # Get configuration parameters
        hidden_layer_size = config.get("hidden_layer_size", 128)
        num_hidden_layers = config.get("num_hidden_layers", 3)
        dropout_rate = config.get("dropout_rate", 0.1)
        
        # Create the network layers with batch normalization and dropout
        layers = [nn.Linear(state_size, hidden_layer_size), 
                 nn.BatchNorm1d(hidden_layer_size),
                 nn.ReLU(),
                 nn.Dropout(dropout_rate)]
        
        # Add hidden layers
        for _ in range(num_hidden_layers - 1):
            layers.append(nn.Linear(hidden_layer_size, hidden_layer_size))
            layers.append(nn.BatchNorm1d(hidden_layer_size))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout_rate))
        
        # Add output layer
        layers.append(nn.Linear(hidden_layer_size, action_size))
        
        # Create sequential model
        self.network = nn.Sequential(*layers)
        
        # Initialize weights using Xavier/Glorot initialization
        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        """Initialize network weights"""
        if isinstance(module, nn.Linear):
            nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                module.bias.data.fill_(0.01)
    
    def forward(self, state):
        """Forward pass through network"""
        # Handle both single and batch inputs
        if state.dim() == 1:
            state = state.unsqueeze(0)
            
        return self.network(state)

class DuelingDQNetwork(nn.Module):
    """
    Dueling Deep Q-Network for improved performance
    Separates state value and action advantages for better estimation
    """
    def __init__(self, state_size: int, action_size: int, config: Dict):
        """Initialize Dueling DQN with state and action dimensions"""
        super(DuelingDQNetwork, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        
        # Get configuration parameters
        hidden_layer_size = config.get("hidden_layer_size", 128)
        num_hidden_layers = config.get("num_hidden_layers", 3)
        dropout_rate = config.get("dropout_rate", 0.1)
        
        # Feature extraction layers
        self.feature_layers = nn.Sequential(
            nn.Linear(state_size, hidden_layer_size),
            nn.BatchNorm1d(hidden_layer_size),
            nn.ReLU(),
            nn.Dropout(dropout_rate)
        )
        
        # Add hidden layers for feature extraction
        for _ in range(num_hidden_layers - 2):
            self.feature_layers.extend([
                nn.Linear(hidden_layer_size, hidden_layer_size),
                nn.BatchNorm1d(hidden_layer_size),
                nn.ReLU(),
                nn.Dropout(dropout_rate)
            ])
        
        # Value stream - estimates state value V(s)
        self.value_stream = nn.Sequential(
            nn.Linear(hidden_layer_size, hidden_layer_size),
            nn.BatchNorm1d(hidden_layer_size),
            nn.ReLU(),
            nn.Linear(hidden_layer_size, 1)
        )
        
        # Advantage stream - estimates advantages for each action A(s,a)
        self.advantage_stream = nn.Sequential(
            nn.Linear(hidden_layer_size, hidden_layer_size),
            nn.BatchNorm1d(hidden_layer_size),
            nn.ReLU(),
            nn.Linear(hidden_layer_size, action_size)
        )
        
        # Initialize weights
        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        """Initialize network weights"""
        if isinstance(module, nn.Linear):
            nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                module.bias.data.fill_(0.01)
    
    def forward(self, state):
        """Forward pass through dueling network architecture"""
        # Handle both single and batch inputs
        if state.dim() == 1:
            state = state.unsqueeze(0)
            
        features = self.feature_layers(state)
        
        # Calculate state value and advantages
        value = self.value_stream(features)
        advantages = self.advantage_stream(features)
        
        # Q(s,a) = V(s) + (A(s,a) - mean(A(s,a')))
        qvalues = value + (advantages - advantages.mean(dim=1, keepdim=True))
        
        return qvalues

class PricePredictionDRLAgent:
    """
    Deep Reinforcement Learning agent for price prediction
    """
    def __init__(self, state_size: int, action_size: int, category_id: str, 
                 config_path: str = "/data/chats/p6wyr/workspace/config/config.json"):
        """Initialize the DRL agent"""
        self.logger = Logger().get_logger()
        self.logger.info("Initializing DRL agent for price prediction")
        
        # Load configuration
        self.config_path = config_path
        self._load_config()
        
        # Get DRL model configuration
        self.drl_config = self.config.get("model", {}).get("drl", {})
        
        # Set up model parameters
        self.state_size = state_size
        self.action_size = action_size
        self.category_id = category_id
        self.learning_rate = self.drl_config.get("learning_rate", 0.001)
        self.discount_factor = self.drl_config.get("discount_factor", 0.95)
        self.exploration_rate = self.drl_config.get("exploration_rate", 0.1)
        self.batch_size = self.drl_config.get("batch_size", 64)
        self.memory_size = self.drl_config.get("memory_size", 10000)
        self.target_update_freq = self.drl_config.get("target_update_frequency", 1000)
        self.double_dqn = self.drl_config.get("double_dqn", True)
        self.dueling_network = self.drl_config.get("dueling_network", True)
        
        # Set device (GPU if available)
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.logger.info(f"Using device: {self.device}")
        
        # Initialize networks
        self._initialize_networks()
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=self.learning_rate)
        
        # Initialize replay memory
        self.memory = ReplayBuffer(self.memory_size)
        
        # Initialize step counter for target network updates
        self.t_step = 0
        self.train_step = 0
        
        # Set up model directories
        self.models_dir = f"/data/chats/p6wyr/workspace/data/models/category_{self.category_id}"
        os.makedirs(self.models_dir, exist_ok=True)
        
        # Set up tensorboard logging
        self.tb_dir = f"/data/chats/p6wyr/workspace/logs/tensorboard/category_{self.category_id}"
        os.makedirs(self.tb_dir, exist_ok=True)
        self.writer = SummaryWriter(log_dir=self.tb_dir)
        
        # Reward function weights
        reward_config = self.config.get("model", {}).get("reward_function", {})
        self.profit_weight = reward_config.get("profit_weight", 0.7)
        self.time_weight = reward_config.get("time_weight", 0.3)
        
        # Track statistics
        self.stats = {
            "episodes": 0,
            "avg_rewards": [],
            "avg_losses": [],
            "exploration_rate": []
        }
    
    def _load_config(self) -> None:
        """Load configuration from file"""
        try:
            with open(self.config_path, 'r') as f:
                self.config = json.load(f)
            self.logger.info("Configuration loaded successfully")
        except Exception as e:
            self.logger.error(f"Error loading configuration: {str(e)}")
            # Use default configuration
            self.config = {
                "model": {
                    "drl": {
                        "model_type": "dqn",
                        "learning_rate": 0.001,
                        "discount_factor": 0.95,
                        "exploration_rate": 0.1,
                        "target_update_frequency": 1000,
                        "batch_size": 64,
                        "memory_size": 10000,
                        "num_hidden_layers": 3,
                        "hidden_layer_size": 128,
                        "activation": "relu",
                        "double_dqn": True,
                        "dueling_network": True,
                        "dropout_rate": 0.1
                    },
                    "reward_function": {
                        "profit_weight": 0.7,
                        "time_weight": 0.3
                    }
                }
            }
    
    def _initialize_networks(self):
        """Initialize Q-networks based on configuration"""
        # Choose network architecture based on config
        if self.dueling_network:
            self.qnetwork_local = DuelingDQNetwork(
                self.state_size, self.action_size, self.drl_config).to(self.device)
            self.qnetwork_target = DuelingDQNetwork(
                self.state_size, self.action_size, self.drl_config).to(self.device)
            self.logger.info("Using Dueling DQN architecture")
        else:
            self.qnetwork_local = DQNetwork(
                self.state_size, self.action_size, self.drl_config).to(self.device)
            self.qnetwork_target = DQNetwork(
                self.state_size, self.action_size, self.drl_config).to(self.device)
            self.logger.info("Using standard DQN architecture")
        
        # Copy weights from local to target network
        self.qnetwork_target.load_state_dict(self.qnetwork_local.state_dict())
    
    def step(self, state, action, reward, next_state, done):
        """Update the agent's knowledge based on experience"""
        # Save experience in replay memory
        self.memory.push(state, action, reward, next_state, done)
        
        # Update step counter
        self.t_step = (self.t_step + 1) % self.target_update_freq
        
        # Learn every self.batch_size steps, if enough samples in memory
        if len(self.memory) > self.batch_size:
            experiences = self.memory.sample(self.batch_size)
            loss = self._learn(experiences)
            
            # Log loss to tensorboard
            self.train_step += 1
            self.writer.add_scalar('Loss/train', loss, self.train_step)
            
            # Update target network periodically
            if self.t_step == 0:
                self._update_target_network()
            
            return loss
        
        return 0
    
    def act(self, state, evaluate=False):
        """Choose action based on current state using epsilon-greedy policy"""
        # Convert state to tensor
        state = torch.from_numpy(state).float().unsqueeze(0).to(self.device)
        
        # Set network to evaluation mode
        self.qnetwork_local.eval()
        
        with torch.no_grad():
            action_values = self.qnetwork_local(state)
        
        # Set network back to training mode
        self.qnetwork_local.train()
        
        # Epsilon-greedy action selection
        if not evaluate and random.random() < self.exploration_rate:
            return random.choice(np.arange(self.action_size))
        else:
            return np.argmax(action_values.cpu().data.numpy())
    
    def _learn(self, experiences):
        """Update value parameters using batch of experiences"""
        states, actions, rewards, next_states, dones = experiences
        
        # Move to device
        states = states.to(self.device)
        actions = actions.to(self.device)
        rewards = rewards.to(self.device)
        next_states = next_states.to(self.device)
        dones = dones.to(self.device)
        
        if self.double_dqn:
            # Double DQN: Select actions using local network, evaluate using target network
            self.qnetwork_local.eval()
            with torch.no_grad():
                local_best_actions = self.qnetwork_local(next_states).argmax(dim=1, keepdim=True)
            self.qnetwork_local.train()
            
            # Evaluate those actions using the target network
            Q_targets_next = self.qnetwork_target(next_states).gather(1, local_best_actions)
        else:
            # Standard DQN: Get max predicted Q values from target model
            Q_targets_next = self.qnetwork_target(next_states).detach().max(1)[0].unsqueeze(1)
        
        # Compute Q targets for current states
        Q_targets = rewards + (self.discount_factor * Q_targets_next * (1 - dones))
        
        # Get expected Q values from local model
        Q_expected = self.qnetwork_local(states).gather(1, actions)
        
        # Compute loss - Huber loss for stability
        loss = F.smooth_l1_loss(Q_expected, Q_targets)
        
        # Minimize the loss
        self.optimizer.zero_grad()
        loss.backward()
        
        # Gradient clipping to prevent exploding gradients
        torch.nn.utils.clip_grad_norm_(self.qnetwork_local.parameters(), 1.0)
        
        self.optimizer.step()
        
        return loss.item()
    
    def _update_target_network(self):
        """Update target network with parameters from local network"""
        # Soft update of the target network's weights
        tau = 0.001  # Small update rate for stability
        for target_param, local_param in zip(self.qnetwork_target.parameters(), self.qnetwork_local.parameters()):
            target_param.data.copy_(tau * local_param.data + (1.0 - tau) * target_param.data)
    
    def save_model(self, episode: int = None):
        """Save the agent's model"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        episode_str = f"_episode_{episode}" if episode is not None else ""
        
        model_path = os.path.join(self.models_dir, f"drl_model_{timestamp}{episode_str}.pt")
        
        torch.save({
            'qnetwork_local_state_dict': self.qnetwork_local.state_dict(),
            'qnetwork_target_state_dict': self.qnetwork_target.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'exploration_rate': self.exploration_rate,
            'stats': self.stats,
            'config': {
                'state_size': self.state_size,
                'action_size': self.action_size,
                'double_dqn': self.double_dqn,
                'dueling_network': self.dueling_network
            }
        }, model_path)
        
        self.logger.info(f"Model saved to {model_path}")
        return model_path
    
    def load_model(self, model_path: str):
        """Load a saved model"""
        if not os.path.exists(model_path):
            self.logger.error(f"Model file not found: {model_path}")
            return False
        
        try:
            checkpoint = torch.load(model_path, map_location=self.device)
            
            # Check if model architecture matches
            config = checkpoint.get('config', {})
            if config.get('state_size') != self.state_size or config.get('action_size') != self.action_size:
                self.logger.warning(f"Model architecture mismatch. Reinitializing networks.")
                
                # Handle architecture mismatch by reinitializing networks
                self.dueling_network = config.get('dueling_network', self.dueling_network)
                self.double_dqn = config.get('double_dqn', self.double_dqn)
                self._initialize_networks()
            
            self.qnetwork_local.load_state_dict(checkpoint['qnetwork_local_state_dict'])
            self.qnetwork_target.load_state_dict(checkpoint['qnetwork_target_state_dict'])
            self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            self.exploration_rate = checkpoint.get('exploration_rate', 0.1)
            self.stats = checkpoint.get('stats', self.stats)
            
            self.logger.info(f"Model loaded from {model_path}")
            return True
        except Exception as e:
            self.logger.error(f"Error loading model: {str(e)}")
            return False
    
    def compute_reward(self, state, action, next_state, price, sold_price=None, time_to_sell=None):
        """Compute the reward for a state-action pair"""
        # If item sold, use actual profit and time
        if sold_price is not None and time_to_sell is not None:
            profit_margin = (sold_price - price) / price if price > 0 else 0
            
            # Normalize time to sell (consider 30 days as max)
            norm_time = max(0, 1 - (time_to_sell / 30)) if time_to_sell > 0 else 0
            
            # Combine profit and time components
            reward = (self.profit_weight * profit_margin) + (self.time_weight * norm_time)
            
            return reward * 10  # Scale reward
        
        # If not sold yet, use market-based reward estimation
        price_competitiveness = next_state[4] if len(next_state) > 4 else 0.5
        conversion_rate = next_state[7] if len(next_state) > 7 else 0.5
        
        # Penalize extreme pricing
        price_level = action / (self.action_size - 1)
        price_penalty = -2.0 * abs(price_level - 0.5)
        
        # Combine factors for reward estimation
        estimated_reward = (price_competitiveness * 0.6) + (conversion_rate * 0.4) + price_penalty
        
        return estimated_reward * 5
    
    def decay_exploration_rate(self, decay_factor=0.995, min_rate=0.01):
        """Decay the exploration rate"""
        self.exploration_rate = max(min_rate, self.exploration_rate * decay_factor)
        self.stats["exploration_rate"].append(self.exploration_rate)
        return self.exploration_rate
'''

with open(drl_agent_path, 'w') as f:
    f.write(drl_agent_code)

# Create the environment.py file
environment_path = "/data/chats/p6wyr/workspace/src/models/environment.py"
await terminal.run(f"touch {environment_path}")

environment_code = '''
import numpy as np
import pandas as pd
from typing import Dict, List, Tuple, Optional, Any
import random

from src.utils.logger import Logger

class PriceEnvironment:
    """
    Environment for price prediction, simulating market dynamics
    """
    def __init__(self, features_df: pd.DataFrame, category_id: str, num_price_levels: int = 10):
        """Initialize the environment"""
        self.logger = Logger().get_logger()
        self.features_df = features_df.copy()
        self.category_id = category_id
        
        # Get sold items for statistics
        self.sold_items = self.features_df[self.features_df['is_sold'] == 1]
        self.unsold_items = self.features_df[self.features_df['is_sold'] == 0]
        
        # Extract price range
        self.min_price = self.features_df['price'].min()
        self.max_price = self.features_df['price'].max()
        self.avg_price = self.features_df['price'].mean()
        
        # Calculate price levels (actions)
        self.num_price_levels = num_price_levels
        self.price_levels = np.linspace(0.5, 1.5, num_price_levels)
        
        # Current state info
        self.current_item_idx = 0
        self.current_state = None
        self.base_price = 0
        
        self.logger.info(f"Initialized price environment with {len(features_df)} items, {num_price_levels} price levels")
    
    def reset(self, item_idx=None):
        """Reset the environment and get new item"""
        # Select item (random if not specified)
        if item_idx is None:
            self.current_item_idx = np.random.randint(0, len(self.features_df))
        else:
            self.current_item_idx = item_idx % len(self.features_df)
        
        # Get item data
        item = self.features_df.iloc[self.current_item_idx]
        
        # Extract state features (excluding ID, price, is_sold, category_id, days_to_sell)
        state_columns = [col for col in self.features_df.columns 
                         if col not in ['itemId', 'price', 'is_sold', 'category_id', 'days_to_sell']]
        
        # Set current state
        self.current_state = item[state_columns].values.astype(np.float32)
        
        # Set base price
        self.base_price = item['price']
        
        return self.current_state
    
    def step(self, action):
        """Take a step in the environment"""
        # Get selected price level
        price_factor = self.price_levels[action]
        suggested_price = self.base_price * price_factor
        
        # Get current item
        item = self.features_df.iloc[self.current_item_idx]
        
        # Check if item sold in historical data
        is_sold = item['is_sold'] == 1
        days_to_sell = item.get('days_to_sell', 30)
        
        # Get actual sold price if available
        sold_price = item['price'] if is_sold else None
        
        # Calculate reward
        reward = self._calculate_reward(suggested_price, is_sold, sold_price, days_to_sell)
        
        # Return the same state (as this is not a sequential environment)
        next_state = self.current_state
        
        # Episode is done after one step
        done = True
        
        # Additional info
        info = {
            'item_id': item.get('itemId', self.current_item_idx),
            'base_price': self.base_price,
            'suggested_price': suggested_price,
            'price_factor': price_factor,
            'is_sold': is_sold,
            'days_to_sell': days_to_sell if is_sold else None,
            'sold_price': sold_price
        }
        
        return next_state, reward, done, info
    
    def _calculate_reward(self, suggested_price, is_sold, sold_price, days_to_sell):
        """Calculate reward for the action"""
        # For sold items, compare the suggested price to actual sold price
        if is_sold and sold_price is not None:
            # Penalize overpricing that could prevent sale
            if suggested_price > 1.2 * sold_price:
                return -2.0
            
            # Reward accurate pricing (within 5% of actual)
            if 0.95 * sold_price <= suggested_price <= 1.05 * sold_price:
                return 2.0
            
            # Good pricing (within 10% of actual)
            if 0.9 * sold_price <= suggested_price <= 1.1 * sold_price:
                return 1.5
            
            # Reasonable pricing (within 20% of actual)
            if 0.8 * sold_price <= suggested_price <= 1.2 * sold_price:
                return 0.8
            
            # Penalize significantly underpricing
            if suggested_price < 0.8 * sold_price:
                return -1.0
            
            return 0.2
        
        # For unsold items, evaluate based on market dynamics
        else:
            # Get similar items
            similar_sold = self._find_similar_items(is_sold=True)
            
            if len(similar_sold) > 0:
                avg_similar_price = similar_sold['price'].mean()
                
                # Reward pricing close to successful similar items
                if 0.9 * avg_similar_price <= suggested_price <= 1.1 * avg_similar_price:
                    return 1.2
                
                # Smaller reward for reasonable pricing
                if 0.8 * avg_similar_price <= suggested_price <= 1.2 * avg_similar_price:
                    return 0.6
                
                # Penalize significant deviation from market
                if suggested_price > 1.3 * avg_similar_price:
                    return -1.5
                
                if suggested_price < 0.7 * avg_similar_price:
                    return -0.8
                
                return -0.3
        
        return 0.0
    
    def _find_similar_items(self, is_sold=True, top_n=5):
        """Find items similar to current item"""
        # Current item
        item = self.features_df.iloc[self.current_item_idx]
        
        # Filter by sold status
        items_subset = self.sold_items if is_sold else self.unsold_items
        
        if len(items_subset) == 0:
            return pd.DataFrame()
        
        # Extract state features
        feature_columns = [col for col in self.features_df.columns 
                          if col not in ['itemId', 'price', 'is_sold', 'category_id', 'days_to_sell']]
        
        if len(feature_columns) == 0 or len(items_subset) <= top_n:
            return items_subset.head(top_n)
        
        # Calculate similarity using Euclidean distance
        item_features = item[feature_columns].values
        
        def euclidean_distance(row):
            return np.sqrt(np.sum((row[feature_columns].values - item_features) ** 2))
        
        # Calculate distances and sort by similarity
        items_subset = items_subset.copy()
        items_subset['distance'] = items_subset.apply(euclidean_distance, axis=1)
        items_subset = items_subset.sort_values('distance')
        
        # Return top N most similar items
        return items_subset.head(top_n)
'''

with open(environment_path, 'w') as f:
    f.write(environment_code)

# Create the trainer.py file - fixing the unterminated string issue
trainer_path = "/data/chats/p6wyr/workspace/src/models/trainer.py"
await terminal.run(f"touch {trainer_path}")

trainer_code = '''
import os
import json
import glob
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple, Optional, Any
from tqdm import tqdm
from datetime import datetime

from src.utils.logger import Logger
from src.models.environment import PriceEnvironment
from src.models.drl_agent import PricePredictionDRLAgent

class PriceP

SyntaxError: unterminated string literal (detected at line 1) (2121117381.py, line 1)

In [31]:
I need to implement a main script for the DRL model for price prediction. I'll analyze what went wrong in my previous implementations and provide a complete solution.

[reflection on previous impl]
In my previous implementations, I was trying to create multiple Python module files for the DRL model components but ran into syntax errors. I identified two main issues:

1. The trainer.py file had an unterminated string literal at `self.models_dir = f"/data/chats/p6wyr/workspace/data/models"` - the string wasn't properly closed.
2. I didn't implement the main.py script to demonstrate the DRL model functionality.

Additionally, the execution shows that there are no CSV files in the processed directories, which means we'll need to work with synthetic data as there's no real data available yet.

My approach will be to:
1. Create a comprehensive main.py script that integrates all previous components
2. Implement synthetic data generation since real data isn't available
3. Include functionality to train and evaluate the model
4. Add visualization for the results

[improved impl]:
```python
from metagpt.tools.libs.terminal import Terminal
terminal = Terminal()

# Install required packages first
await terminal.run('pip install torch numpy matplotlib tqdm tensorboard')

# Make sure all required directories exist
await terminal.run('mkdir -p /data/chats/p6wyr/workspace/src/models')
await terminal.run('mkdir -p /data/chats/p6wyr/workspace/data/models/category_9355')  # Laptops
await terminal.run('mkdir -p /data/chats/p6wyr/workspace/logs/tensorboard/category_9355')

# Create the main.py file to demonstrate the DRL model functionality
main_path = "/data/chats/p6wyr/workspace/src/models/main.py"
await terminal.run(f"touch {main_path}")

main_code = '''
import os
import sys
import json
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
from typing import Dict, List, Optional, Any
import argparse
import asyncio
import random
from datetime import datetime
import logging

# Set up basic logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger("ebay_price_predictor")

# Make sure the models directory is in the path
sys.path.append("/data/chats/p6wyr/workspace")

# We'll define these classes inline since they may not exist yet in the environment
# In a real application, you would import these from their respective modules
class PriceEnvironment:
    """
    Environment for price prediction, simulating market dynamics
    """
    def __init__(self, features_df: pd.DataFrame, category_id: str, num_price_levels: int = 10):
        """Initialize the environment"""
        self.logger = logger
        self.features_df = features_df.copy()
        self.category_id = category_id
        
        # Get sold items for statistics
        self.sold_items = self.features_df[self.features_df['is_sold'] == 1]
        self.unsold_items = self.features_df[self.features_df['is_sold'] == 0]
        
        # Extract price range
        self.min_price = features_df['price'].min()
        self.max_price = features_df['price'].max()
        self.avg_price = features_df['price'].mean()
        
        # Calculate price levels (actions)
        self.num_price_levels = num_price_levels
        self.price_levels = np.linspace(0.5, 1.5, num_price_levels)
        
        # Current state info
        self.current_item_idx = 0
        self.current_state = None
        self.base_price = 0
        
        self.logger.info(f"Initialized price environment with {len(features_df)} items, {num_price_levels} price levels")
    
    def reset(self, item_idx=None):
        """Reset the environment and get new item"""
        # Select item (random if not specified)
        if item_idx is None:
            self.current_item_idx = np.random.randint(0, len(self.features_df))
        else:
            self.current_item_idx = item_idx % len(self.features_df)
        
        # Get item data
        item = self.features_df.iloc[self.current_item_idx]
        
        # Extract state features (excluding ID, price, is_sold, category_id, days_to_sell)
        state_columns = [col for col in self.features_df.columns 
                         if col not in ['itemId', 'price', 'is_sold', 'category_id', 'days_to_sell']]
        
        # Set current state
        self.current_state = item[state_columns].values.astype(np.float32)
        
        # Set base price
        self.base_price = item['price']
        
        return self.current_state
    
    def step(self, action):
        """Take a step in the environment"""
        # Get selected price level
        price_factor = self.price_levels[action]
        suggested_price = self.base_price * price_factor
        
        # Get current item
        item = self.features_df.iloc[self.current_item_idx]
        
        # Check if item sold in historical data
        is_sold = item['is_sold'] == 1
        days_to_sell = item.get('days_to_sell', 30)
        
        # Get actual sold price if available
        sold_price = item['price'] if is_sold else None
        
        # Calculate reward
        reward = self._calculate_reward(suggested_price, is_sold, sold_price, days_to_sell)
        
        # Return the same state (as this is not a sequential environment)
        next_state = self.current_state
        
        # Episode is done after one step
        done = True
        
        # Additional info
        info = {
            'item_id': item.get('itemId', self.current_item_idx),
            'base_price': self.base_price,
            'suggested_price': suggested_price,
            'price_factor': price_factor,
            'is_sold': is_sold,
            'days_to_sell': days_to_sell if is_sold else None,
            'sold_price': sold_price
        }
        
        return next_state, reward, done, info
    
    def _calculate_reward(self, suggested_price, is_sold, sold_price, days_to_sell):
        """Calculate reward for the action"""
        # For sold items, compare the suggested price to actual sold price
        if is_sold and sold_price is not None:
            # Penalize overpricing that could prevent sale
            if suggested_price > 1.2 * sold_price:
                return -2.0
            
            # Reward accurate pricing (within 5% of actual)
            if 0.95 * sold_price <= suggested_price <= 1.05 * sold_price:
                return 2.0
            
            # Good pricing (within 10% of actual)
            if 0.9 * sold_price <= suggested_price <= 1.1 * sold_price:
                return 1.5
            
            # Reasonable pricing (within 20% of actual)
            if 0.8 * sold_price <= suggested_price <= 1.2 * sold_price:
                return 0.8
            
            # Penalize significantly underpricing
            if suggested_price < 0.8 * sold_price:
                return -1.0
            
            return 0.2
        
        # For unsold items, evaluate based on market dynamics
        else:
            # Get similar items
            similar_sold = self._find_similar_items(is_sold=True)
            
            if len(similar_sold) > 0:
                avg_similar_price = similar_sold['price'].mean()
                
                # Reward pricing close to successful similar items
                if 0.9 * avg_similar_price <= suggested_price <= 1.1 * avg_similar_price:
                    return 1.2
                
                # Smaller reward for reasonable pricing
                if 0.8 * avg_similar_price <= suggested_price <= 1.2 * avg_similar_price:
                    return 0.6
                
                # Penalize significant deviation from market
                if suggested_price > 1.3 * avg_similar_price:
                    return -1.5
                
                if suggested_price < 0.7 * avg_similar_price:
                    return -0.8
                
                return -0.3
        
        return 0.0
    
    def _find_similar_items(self, is_sold=True, top_n=5):
        """Find items similar to current item"""
        # Current item
        item = self.features_df.iloc[self.current_item_idx]
        
        # Filter by sold status
        items_subset = self.sold_items if is_sold else self.unsold_items
        
        if len(items_subset) == 0:
            return pd.DataFrame()
        
        # Extract state features
        feature_columns = [col for col in self.features_df.columns 
                          if col not in ['itemId', 'price', 'is_sold', 'category_id', 'days_to_sell']]
        
        if len(feature_columns) == 0 or len(items_subset) <= top_n:
            return items_subset.head(top_n)
        
        # Calculate similarity using Euclidean distance
        item_features = item[feature_columns].values
        
        def euclidean_distance(row):
            return np.sqrt(np.sum((row[feature_columns].values - item_features) ** 2))
        
        # Calculate distances and sort by similarity
        items_subset = items_subset.copy()
        items_subset['distance'] = items_subset.apply(euclidean_distance, axis=1)
        items_subset = items_subset.sort_values('distance')
        
        # Return top N most similar items
        return items_subset.head(top_n)


# Define replay buffer for DRL agent
import torch
from collections import deque, namedtuple
import random
import numpy as np

# Define Experience namedtuple for replay buffer
Experience = namedtuple('Experience', ['state', 'action', 'reward', 'next_state', 'done'])

class ReplayBuffer:
    """Replay buffer for storing and sampling experiences"""
    def __init__(self, capacity: int):
        self.buffer = deque(maxlen=capacity)
    
    def push(self, state, action, reward, next_state, done):
        self.buffer.append(Experience(state, action, reward, next_state, done))
    
    def sample(self, batch_size: int):
        experiences = random.sample(self.buffer, batch_size)
        states = torch.from_numpy(np.vstack([e.state for e in experiences])).float()
        actions = torch.from_numpy(np.vstack([e.action for e in experiences])).long()
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences])).float()
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences])).float()
        dones = torch.from_numpy(np.vstack([e.done for e in experiences]).astype(np.uint8)).float()
        return states, actions, rewards, next_states, dones
    
    def __len__(self):
        return len(self.buffer)


# Define neural network models
import torch.nn as nn
import torch.nn.functional as F

class DQNetwork(nn.Module):
    """Deep Q-Network for price prediction"""
    def __init__(self, state_size: int, action_size: int, hidden_size: int = 128):
        super(DQNetwork, self).__init__()
        self.fc1 = nn.Linear(state_size, hidden_size)
        self.bn1 = nn.BatchNorm1d(hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.bn2 = nn.BatchNorm1d(hidden_size)
        self.fc3 = nn.Linear(hidden_size, action_size)
        
    def forward(self, state):
        # Handle single state input
        if state.dim() == 1:
            state = state.unsqueeze(0)
        
        x = F.relu(self.bn1(self.fc1(state)))
        x = F.relu(self.bn2(self.fc2(x)))
        return self.fc3(x)


class DuelingDQNetwork(nn.Module):
    """Dueling Deep Q-Network for price prediction"""
    def __init__(self, state_size: int, action_size: int, hidden_size: int = 128):
        super(DuelingDQNetwork, self).__init__()
        # Feature extraction
        self.fc1 = nn.Linear(state_size, hidden_size)
        self.bn1 = nn.BatchNorm1d(hidden_size)
        
        # Value stream
        self.fc_value = nn.Linear(hidden_size, hidden_size)
        self.bn_value = nn.BatchNorm1d(hidden_size)
        self.value = nn.Linear(hidden_size, 1)
        
        # Advantage stream
        self.fc_adv = nn.Linear(hidden_size, hidden_size)
        self.bn_adv = nn.BatchNorm1d(hidden_size)
        self.advantage = nn.Linear(hidden_size, action_size)
    
    def forward(self, state):
        if state.dim() == 1:
            state = state.unsqueeze(0)
        
        features = F.relu(self.bn1(self.fc1(state)))
        
        value = F.relu(self.bn_value(self.fc_value(features)))
        value = self.value(value)
        
        adv = F.relu(self.bn_adv(self.fc_adv(features)))
        adv = self.advantage(adv)
        
        # Q = V + (A - mean(A))
        return value + adv - adv.mean(dim=1, keepdim=True)


# DRL Agent implementation
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

class DRLAgent:
    """Deep Reinforcement Learning agent for price prediction"""
    def __init__(self, state_size, action_size, dueling=True, double_dqn=True, 
                 learning_rate=0.001, discount_factor=0.99, category_id='9355'):
        self.state_size = state_size
        self.action_size = action_size
        self.dueling = dueling
        self.double_dqn = double_dqn
        
        # Hyperparameters
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.epsilon = 0.2  # Initial exploration rate
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01
        self.batch_size = 64
        self.memory_size = 10000
        self.target_update_freq = 10
        self.step_count = 0
        
        # Set device
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        logger.info(f"Using device: {self.device}")
        
        # Initialize networks
        if dueling:
            self.qnetwork_local = DuelingDQNetwork(state_size, action_size).to(self.device)
            self.qnetwork_target = DuelingDQNetwork(state_size, action_size).to(self.device)
        else:
            self.qnetwork_local = DQNetwork(state_size, action_size).to(self.device)
            self.qnetwork_target = DQNetwork(state_size, action_size).to(self.device)
        
        # Copy weights from local to target network
        self.qnetwork_target.load_state_dict(self.qnetwork_local.state_dict())
        
        # Initialize optimizer
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=learning_rate)
        
        # Initialize replay buffer
        self.memory = ReplayBuffer(self.memory_size)
        
        # Set up model directory and tensorboard
        self.models_dir = f"/data/chats/p6wyr/workspace/data/models/category_{category_id}"
        os.makedirs(self.models_dir, exist_ok=True)
        
        self.tb_dir = f"/data/chats/p6wyr/workspace/logs/tensorboard/category_{category_id}"
        os.makedirs(self.tb_dir, exist_ok=True)
        self.writer = SummaryWriter(log_dir=self.tb_dir)
        
        # Stats tracking
        self.stats = {
            "episodes": 0,
            "rewards": [],
            "losses": [],
            "epsilons": [],
        }
    
    def step(self, state, action, reward, next_state, done):
        """Add experience to memory and learn if enough samples"""
        # Add experience to replay buffer
        self.memory.push(state, action, reward, next_state, done)
        
        # Learn from experiences if enough samples
        if len(self.memory) > self.batch_size:
            experiences = self.memory.sample(self.batch_size)
            loss = self._learn(experiences)
            
            # Track step count for target network update
            self.step_count += 1
            
            # Update target network periodically
            if self.step_count % self.target_update_freq == 0:
                self._update_target_network()
            
            return loss
        
        return 0
    
    def act(self, state, eval_mode=False):
        """Choose an action using epsilon-greedy policy"""
        # Convert state to tensor
        state = torch.from_numpy(state).float().unsqueeze(0).to(self.device)
        
        # Set network to evaluation mode
        self.qnetwork_local.eval()
        
        with torch.no_grad():
            action_values = self.qnetwork_local(state)
        
        # Set network back to train mode
        self.qnetwork_local.train()
        
        # Epsilon-greedy action selection
        if not eval_mode and random.random() < self.epsilon:
            return random.choice(np.arange(self.action_size))
        else:
            return np.argmax(action_values.cpu().data.numpy())
    
    def _learn(self, experiences):
        """Update network parameters based on batch of experiences"""
        states, actions, rewards, next_states, dones = experiences
        
        # Move to device
        states = states.to(self.device)
        actions = actions.to(self.device)
        rewards = rewards.to(self.device)
        next_states = next_states.to(self.device)
        dones = dones.to(self.device)
        
        if self.double_dqn:
            # Double DQN: Get actions from local network but values from target network
            self.qnetwork_local.eval()
            with torch.no_grad():
                best_actions = self.qnetwork_local(next_states).argmax(dim=1, keepdim=True)
            self.qnetwork_local.train()
            
            next_q_values = self.qnetwork_target(next_states).gather(1, best_actions)
        else:
            # Standard DQN
            next_q_values = self.qnetwork_target(next_states).max(1)[0].unsqueeze(1)
        
        # Calculate Q targets
        q_targets = rewards + (self.discount_factor * next_q_values * (1 - dones))
        
        # Get current Q values from local model
        q_expected = self.qnetwork_local(states).gather(1, actions)
        
        # Compute loss
        loss = F.smooth_l1_loss(q_expected, q_targets)
        
        # Minimize loss
        self.optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.qnetwork_local.parameters(), 1.0)
        self.optimizer.step()
        
        return loss.item()
    
    def _update_target_network(self):
        """Update target network with local network weights"""
        self.qnetwork_target.load_state_dict(self.qnetwork_local.state_dict())
    
    def decay_epsilon(self):
        """Decay exploration rate"""
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)
        self.stats["epsilons"].append(self.epsilon)
        return self.epsilon
    
    def save(self, filename=None):
        """Save model weights and stats"""
        if filename is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"drl_model_{timestamp}.pt"
        
        save_path = os.path.join(self.models_dir, filename)
        
        torch.save({
            'qnetwork_local_state_dict': self.qnetwork_local.state_dict(),
            'qnetwork_target_state_dict': self.qnetwork_target.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'epsilon': self.epsilon,
            'stats': self.stats,
            'hyperparams': {
                'state_size': self.state_size,
                'action_size': self.action_size,
                'dueling': self.dueling,
                'double_dqn': self.double_dqn
            }
        }, save_path)
        
        logger.info(f"Model saved to {save_path}")
        return save_path
    
    def load(self, filepath):
        """Load model weights and stats"""
        if not os.path.exists(filepath):
            logger.error(f"Model file not found: {filepath}")
            return False
        
        try:
            checkpoint = torch.load(filepath, map_location=self.device)
            
            # Load network weights
            self.qnetwork_local.load_state_dict(checkpoint['qnetwork_local_state_dict'])
            self.qnetwork_target.load_state_dict(checkpoint['qnetwork_target_state_dict'])
            
            # Load optimizer state
            self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            
            # Load other parameters
            self.epsilon = checkpoint.get('epsilon', self.epsilon)
            self.stats = checkpoint.get('stats', self.stats)
            
            logger.info(f"Model loaded from {filepath}")
            return True
        except Exception as e:
            logger.error(f"Error loading model: {e}")
            return False


def create_synthetic_data(num_items=1000, category_id="9355"):
    """Create synthetic data for training and testing the model"""
    logger.info(f"Creating synthetic data with {num_items} items for category {category_id}")
    
    # Set random seed for reproducibility
    np.random.seed(42)
    
    # Generate random data
    data = {
        'itemId': [f"item_{i}" for i in range(num_items)],
        'price': np.random.uniform(50, 500, num_items),
        'is_sold': np.random.choice([0, 1], num_items, p=[0.6, 0.4]),
        'category_id': [category_id] * num_items,
        
        # State features
        'item_condition': np.random.uniform(0, 1, num_items),
        'brand_popularity': np.random.uniform(0, 1, num_items),
        'season_demand': np.random.uniform(0, 1, num_items),
        'market_saturation': np.random.uniform(0, 1, num_items),
        'avg_price': np.random.uniform(0, 1, num_items),
        'price_std': np.random.uniform(0, 0.5, num_items),
        'time_on_market': np.random.uniform(0, 1, num_items),
        'conversion_rate': np.random.uniform(0, 1, num_items)
    }
    
    # Create DataFrame
    df = pd.DataFrame(data)
    
    # Add days_to_sell for sold items
    sold_mask = df['is_sold'] == 1
    df.loc[sold_mask, 'days_to_sell'] = np.random.exponential(scale=10, size=sold_mask.sum()).clip(1, 60)
    
    return df


def train_agent(environment, agent, num_episodes=1000, max_episodes_length=1000, 
                print_every=50, save_every=500):
    """Train DRL agent using the given environment"""
    logger.info(f"Starting training for {num_episodes} episodes")
    
    rewards = []
    losses = []
    epsilons = []
    
    for episode in range(1, num_episodes+1):
        state = environment.reset()
        action = agent.act(state)
        next_state, reward, done, info = environment.step(action)
        loss = agent.step(state, action, reward, next_state, done)
        
        rewards.append(reward)
        if loss > 0:
            losses.append(loss)
        
        # Decay epsilon
        if episode % 10 == 0:
            epsilon = agent.decay_epsilon()
            epsilons.append(epsilon)
        
        # Print progress
        if episode % print_every == 0:
            avg_reward = sum(rewards[-print_every:]) / print_every
            avg_loss = sum(losses[-min(print_every, len(losses)):]) / min(print_every, len(losses)) if losses else 0
            
            logger.info(f"Episode {episode}/{num_episodes} | "
                      f"Avg Reward: {avg_reward:.3f} | "
                      f"Loss: {avg_loss:.6f} | "
                      f"Epsilon: {agent.epsilon:.3f}")
            
            # Log to tensorboard
            agent.writer.add_scalar('Reward/train', avg_reward, episode)
            agent.writer.add_scalar('Loss/train', avg_loss, episode)
            agent.writer.add_scalar('Epsilon', agent.epsilon, episode)
        
        # Save checkpoints
        if episode % save_every == 0:
            agent.save(f"drl_model_episode_{episode}.pt")
    
    # Save final model
    agent.save("drl_model_final.pt")
    
    # Update stats
    agent.stats['episodes'] = num_episodes
    agent.stats['rewards'].extend(rewards)
    agent.stats['losses'].extend(losses)
    agent.stats['epsilons'].extend(epsilons)
    
    return agent


def evaluate_agent(environment, agent, num_episodes=100):
    """Evaluate the trained agent"""
    logger.info(f"Evaluating agent for {num_episodes} episodes")
    
    rewards = []
    price_accuracies = []
    
    for i in range(num_episodes):
        state = environment.reset()
        action = agent.act(state, eval_mode=True)
        _, reward, _, info = environment.step(action)
        
        rewards.append(reward)
        
        # Calculate price accuracy for sold items
        if info['is_sold'] and info['sold_price'] is not None:
            accuracy = 1 - abs(info['suggested_price'] - info['sold_price']) / info['sold_price']
            price_accuracies.append(accuracy)
    
    # Calculate statistics
    avg_reward = sum(rewards) / num_episodes
    avg_accuracy = sum(price_accuracies) / len(price_accuracies) if price_accuracies else 0
    
    logger.info(f"Evaluation results: Average Reward = {avg_reward:.3f}, Price Accuracy = {avg_accuracy:.3f}")
    
    return {
        'avg_reward': avg_reward,
        'avg_accuracy': avg_accuracy,
        'rewards': rewards,
        'price_accuracies': price_accuracies
    }


def plot_training_results(agent, save_path=None):
    """Plot training results"""
    if not agent.stats['rewards']:
        logger.warning("No training stats available to plot")
        return
    
    # Create figure with subplots
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10, 15))
    
    # Plot rewards
    rewards = agent.stats['rewards']
    window_size = min(50, len(rewards))
    if window_size > 0:
        smoothed_rewards = pd.Series(rewards).rolling(window_size).mean().iloc[window_size-1:].values
        ax1.plot(range(window_size, len(rewards)+1), smoothed_rewards)
        ax1.set_title('Smoothed Rewards')
        ax1.set_xlabel('Episode')
        ax1.set_ylabel('Reward')
        ax1.grid()
    
    # Plot losses
    losses = agent.stats['losses']
    window_size = min(50, len(losses))
    if window_size > 0:
        smoothed_losses = pd.Series(losses).rolling(window_size).mean().iloc[window_size-1:].values
        ax2.plot(range(window_size, len(losses)+1), smoothed_losses)
        ax2.set_title('Smoothed Losses')
        ax2.set_xlabel('Episode')
        ax2.set_ylabel('Loss')
        ax2.grid()
    
    # Plot exploration rate
    epsilons = agent.stats['epsilons']
    if epsilons:
        ax3.plot(epsilons)
        ax3.set_title('Exploration Rate')
        ax3.set_xlabel('Update')
        ax3.set_ylabel('Epsilon')
        ax3.grid()
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path)
        logger.info(f"Training results plot saved to {save_path}")
    else:
        plt.show()
    
    plt.close()


async def main():
    """Main function to demonstrate the DRL model"""
    # Parse command line arguments
    parser = argparse.ArgumentParser(description='Second-hand Item Price Prediction DRL Model')
    parser.add_argument('--category', type=str, default='9355', help='Category ID to train on')
    parser.add_argument('--episodes', type=int, default=1000, help='Number of training episodes')
    parser.add_argument('--eval-episodes', type=int, default=100, help='Number of evaluation episodes')
    parser.add_argument('--model-path', type=str, help='Path to pretrained model for evaluation only')
    parser.add_argument('--mode', choices=['train', 'eval', 'both'], default='both', 
                        help='Mode: train, evaluate, or both')
    
    # For Jupyter notebook, use default arguments
    args = parser.parse_args([])
    
    # Generate synthetic data since no real data is available
    category_id = args.category
    data = create_synthetic_data(num_items=2000, category_id=category_id)
    logger.info(f"Created synthetic dataset with {len(data)} items")
    
    # Create environment
    environment = PriceEnvironment(data, category_id, num_price_levels=15)
    
    # Get state size
    state_columns = [col for col in data.columns 
                     if col not in ['itemId', 'price', 'is_sold', 'category_id', 'days_to_sell']]
    state_size = len(state_columns)
    
    # Create agent
    agent = DRLAgent(
        state_size=state_size, 
        action_size=environment.num_price_levels,
        dueling=True,

SyntaxError: unterminated string literal (detected at line 1) (785248730.py, line 1)

In [32]:
import os
import json
import glob

# Let's check what we have in the workspace directory structure
from metagpt.tools.libs.terminal import Terminal
terminal = Terminal()
result = await terminal.run("find /data/chats/p6wyr/workspace -type d | sort")
print("Directory structure:")
print(result)

# Check if we have any config files
config_files = glob.glob("/data/chats/p6wyr/workspace/config/*.json")
print("\nConfig files:", config_files)

# If config exists, let's examine it
if config_files:
    with open(config_files[0], 'r') as f:
        config = json.load(f)
    print("\nConfiguration:")
    print(json.dumps(config, indent=2))

# Check if we have any data already collected
data_files = glob.glob("/data/chats/p6wyr/workspace/data/raw/category_*/*.csv")
print(f"\nFound {len(data_files)} data files in raw directory")

processed_files = glob.glob("/data/chats/p6wyr/workspace/data/processed/category_*/*.csv")
print(f"Found {len(processed_files)} data files in processed directory")

feature_files = glob.glob("/data/chats/p6wyr/workspace/data/features/category_*/*.csv")
print(f"Found {len(feature_files)} data files in features directory")

# Load a sample data file if available to see the structure
if data_files:
    import pandas as pd
    from metagpt.tools.libs.data_preprocess import get_column_info
    
    sample_file = data_files[0]
    print(f"\nExamining sample file: {sample_file}")
    df = pd.read_csv(sample_file)
    
    column_info = get_column_info(df)
    print("column_info")
    print(column_info)
elif processed_files:
    import pandas as pd
    from metagpt.tools.libs.data_preprocess import get_column_info
    
    sample_file = processed_files[0]
    print(f"\nExamining sample file: {sample_file}")
    df = pd.read_csv(sample_file)
    
    column_info = get_column_info(df)
    print("column_info")
    print(column_info)
elif feature_files:
    import pandas as pd
    from metagpt.tools.libs.data_preprocess import get_column_info
    
    sample_file = feature_files[0]
    print(f"\nExamining sample file: {sample_file}")
    df = pd.read_csv(sample_file)
    
    column_info = get_column_info(df)
    print("column_info")
    print(column_info)
else:
    print("\nNo data files found yet. We need to run the data collection process.")

Directory structure:
/data/chats/p6wyr/workspace
/data/chats/p6wyr/workspace/.git
/data/chats/p6wyr/workspace/.git/branches
/data/chats/p6wyr/workspace/.git/hooks
/data/chats/p6wyr/workspace/.git/info
/data/chats/p6wyr/workspace/.git/lfs
/data/chats/p6wyr/workspace/.git/lfs/tmp
/data/chats/p6wyr/workspace/.git/logs
/data/chats/p6wyr/workspace/.git/logs/refs
/data/chats/p6wyr/workspace/.git/logs/refs/heads
/data/chats/p6wyr/workspace/.git/objects
/data/chats/p6wyr/workspace/.git/objects/37
/data/chats/p6wyr/workspace/.git/objects/40
/data/chats/p6wyr/workspace/.git/objects/68
/data/chats/p6wyr/workspace/.git/objects/80
/data/chats/p6wyr/workspace/.git/objects/info
/data/chats/p6wyr/workspace/.git/objects/pack
/data/chats/p6wyr/workspace/.git/refs
/data/chats/p6wyr/workspace/.git/refs/heads
/data/chats/p6wyr/workspace/.git/refs/tags
/data/chats/p6wyr/workspace/config
/data/chats/p6wyr/workspace/data
/data/chats/p6wyr/workspace/data/cache
/data/chats/p6wyr/workspace/data/cache/api_respons